## Web Search and Scrape
The **Web Search and Scrape** ant demonstrates the use of a Web API secured using an API key with an auth header. 
This sample requires you to create a Web Search (bing) resource in Azure and enter its corresponding API key in [0-AI-settings](0-AI-settings.ipynb)

This demo uses a lot of tokens. If your quota is low, you will get throttled.

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.9.5"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.9.5

## Ensure the Web Search and Scrape assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\Web Search and Scrape" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** environment variable.

"Web Search and Scrape" contains the following files:
```
│   HtmlAgilityPack.dll - Loaded by the notebook, a great open library for c# web scraping
│   HtmlAgilityPackExtensions.dll - Loaded by the notebook, part of this project to do a simple conversion to markdown for less tokens and noise
│   HtmlAgilityPackExtensions.pdb - If you want to debug
│   instructions.md - The prompt
│   manifest.json - The assistant definition
│   
└───OpenAPI
        api.bing.microsoft.com.json - Azure Web Search (crawl operation)
        auth.json - Auth setup for web search
        tool.localhost.json - Local HtmlAgility.HtmlAgilityPackExtensions.ConvertUrlToMarkdown c# function 
```

## Crawl versus ConvertUrlToMarkdown tools

Crawl returns search results with snippets and only has a small summary or excerpt of the page

ConvertUrlToMarkdown fetches the whole page and turns it all into markdown for consumption

## auth.json
AntRunnerLib uses the instructions in this file to create the correct auth header(s) for the API. In this case, a header is added to the web request, **Ocp-Apim-Subscription-Key**, the header's value is supplied by an environment variable named, SEARCH_API_KEY. The environment variable name is user defined and ensuring it has a value is the responsibility of the application. 

```json
{
  "hosts": {
    "api.bing.microsoft.com": {
      "auth_type": "service_http",
      "header_name": "Ocp-Apim-Subscription-Key",
      "header_value_env_var": "SEARCH_API_KEY"
    }
  }
}
```

### Explanation
`AssistantUtility.Create` acts based on the contents of a folder structure following a convention laid out in these notebooks. In this case, it copied the contents of instructions.md into the assistant manifest.json. It also parsed the api.bing.microsoft.com swagger which follows an Open API schema into tool defintions for the assitant.

In [2]:
var assistantId = await AssistantUtility.Create("Web Search and Scrape", config);
Console.WriteLine(assistantId)

asst_8yINMFR3kY6UmS6Qcqh1oaE3


In [3]:
AntLoader.LoadAssembly(@".\AssistantDefinitions\Web Search and Scrape\HtmlAgilityPack.dll");
AntLoader.LoadAssembly(@".\AssistantDefinitions\Web Search and Scrape\HtmlAgilityPackExtensions.dll");

## Run the Assistant

`output.Dialog` shows the conversation.

In [4]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search and Scrape",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.",
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown")


User: Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.

Assistant: I ran this code:
```
from datetime import date

# Get today's date
today_date = date.today()
today_date
```

I called the tool named crawl with these arguments:
```
{"q":"things to do in Atlanta on December 31, 2024","textFormat":"HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta on December 31, 2024"},"webPages":{"value":[{"name":"Atlanta Events in December 2024 - Schedule \u0026amp; Tickets","url":"https://atlanta-ga.events/december/","displayUrl":"https://atlanta-ga.events/december","snippet":"Check Out The Atlanta Events Calendar for December 2024: Current \u0026amp; Upcoming Events \u002B Best \u0026amp; Fun Thing To Do in the City. ... Dec 31, 2024 3:30 AM . Georgia Tech Yellow Jackets Basketball . Events: 3. McCamish Pavilion . Dec 31, 2024 2:30 PM ..."},{"name":"11 Atlanta holiday events happening this weekend | 11alive.com","url":"https://www.11alive.com/article/entertainment/events/11-things-to-do-in-atlanta-dec-26-31/85-46d65687-9cd6-4513-9fcb-5e2012452a26","displayUrl":"https://www.11alive.com/article/entertainment/events/11-things-to-do-in-atlanta-dec-26...","snippet":"Updated: 7:20 AM EST December 26, 2024 ATLANTA \u2014 The holiday season is in full swing, and Atlanta is buzzing with exciting events to enjoy this weekend! ... Date and Location: Dec. 25\u201331; 10 a ..."},{"name":"Events \u0026amp; Things To Do in Atlanta in December 2024 - AllEvents.in","url":"https://allevents.in/atlanta/december","displayUrl":"https://allevents.in/atlanta/december","snippet":"Find events and things to do in december 2024 in Atlanta. Discover parties, concerts, meets,shows, sports, club, reunion, Performance happening in 2024 in Atlanta. Open Menu. Atlanta Change City; ... Tue, 31 Dec Trending events in december in Atlanta. Atlanta Symphony Orchestra: New Year\u0026#39;s Eve"},{"name":"Atlanta Events - December 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-december.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-december.html","snippet":"Things to do in Atlanta, GA in December, including upcoming festivals, Christmas events and other holiday celebrations, lights shows, holiday markets, comedy, musicals, ballet, tours of homes, and more. ... December 31 2024 New Year\u2019s eve event at the Georgia Aquarium, with DJ and dancing, food, and after-hours access to the aquatic exhibits.","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"December 31 events in Atlanta, GA - Eventbrite","url":"https://www.eventbrite.com/d/ga--atlanta/december-31/","displayUrl":"https://www.eventbrite.com/d/ga--atlanta/december-31","snippet":"HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District Atlanta. Tue, Dec 31, 10:00 PM. District Atlanta. Save this event: HUGEL NEW YEARS EVE ... TUESDAY DECEMBER 31 2024 Save this event: NYE at EMBR LOUNGE| 8PM-3AM| TUESDAY DECEMBER 31 2024. THE LEMON DROP - NEW YEARS EVE 2K24. Tue, Dec 31, 9:00 PM. Westside Cultural Arts Center."},{"name":"Events \u0026amp; Things to Do for Christmas in Atlanta - Discover Atlanta","url":"https://discoveratlanta.com/events/seasonal/christmas/","displayUrl":"https://discoveratlanta.com/events/seasonal/christmas","snippet":"The possibilities are endless with so many things to do in Atlanta at Christmas! ... Atlanta Ballet\u2019s The Nutcracker returns this December at Cobb Energy Centre with a larger-than-life production of a winter wonderland and dream-like world of ... 12/31/24. Candlelight Concerts 2024. Pets. 12/30/24 \u2013 1/1/25. Adopt Any Senior Pet for FREE in ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.t6GjYUDzSex1Myxpnsk1ngHaE6\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... New Year\u2019s Eve Celebration at Red Phone Booth 2024 12/31/24. New Year\u2019s Eve Dinner at Signia by Hilton Atlanta 12/31/24 \u2013 1/1/25 ... December 31. Tuesday. Atlanta\u2019s New Year\u2019s Eve Peach Drop ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events Calendar 2024/2025 | Atlanta, GA","url":"https://atlanta-ga.events/","displayUrl":"https://atlanta-ga.events","snippet":"This calendar is your ultimate guide to the best things to do in Atlanta, GA: ... Dec 31, 2024 3:30 AM . Notre Dame Fighting Irish . Events: 3. McCamish Pavilion . Dec 31, 2024 2:30 PM . Georgia Tech Yellow Jackets Basketball ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Discover December Events \u0026amp; Activities in Atlanta, GA - Eventbrite","url":"https://www.eventbrite.com/d/ga--atlanta/december/","displayUrl":"https://www.eventbrite.com/d/ga--atlanta/december","snippet":"Save this event: HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District Atlanta Share this event: HUGEL NEW YEARS EVE | Make The Girls Dance 2024 ... FREE ENTRY W/ RSVP TIL 10PM | TUESDAY DEC 31ST | AT BAMBOO ATL Share this event: NYE 25 | FREE ENTRY W/ RSVP TIL 10PM | TUESDAY DEC 31ST | AT BAMBOO ATL. Almost full. NYE 25 | FREE ENTRY W ..."},{"name":"Cool Things To Do in December in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-december-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-december-in-atlanta","snippet":"Cool Things To Do in December in Atlanta. By Mary Welch. Published November 14, 2023 Last Updated December 2, 2024. December is a great time to be in Atlanta. The list of cool things to do seems endless. There\u2019s unparalleled shopping, great themed entertainment, holiday light displays galore and wonderful restaurants where you can enjoy a ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.B7WMsn5icdXhjRnlj2WcZAHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named multi_tool_use.parallel with these arguments:
```
{"tool_uses":[{"recipient_name":"functions.GetContentFromUrl","parameters":{"url":"https://atlanta-ga.events/december/"}},{"recipient_name":"functions.GetContentFromUrl","parameters":{"url":"https://www.eventbrite.com/d/ga--atlanta/december-31/"}},{"recipient_name":"functions.GetContentFromUrl","parameters":{"url":"https://discoveratlanta.com/events/main/"}}]}
```
and got this result:
```
Error: multi_tool_use.parallel is not a valid tool.
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url":"https://atlanta-ga.events/december/"}
```
and got this result:
```
"JavaScript support required!\r\n\r\n[](https://atlanta-ga.events)[All Events](https://atlanta-ga.events/)[Sports](https://atlanta-ga.events/sports/)[Concerts](https://atlanta-ga.events/concerts/)[Comedy](https://atlanta-ga.events/comedy-shows/)[Broadway](https://atlanta-ga.events/broadway/)[All Shows](https://atlanta-ga.events/shows/)[Family](https://atlanta-ga.events/family-shows/)[Ballet](https://atlanta-ga.events/ballet/)[Opera](https://atlanta-ga.events/opera/)[This Weekend](https://atlanta-ga.events/this-weekend/)[Next Week](https://atlanta-ga.events/next-week/)[Festivals](https://atlanta-ga.events/festivals/)[Venues](https://atlanta-ga.events/entertainment-venues/)* All Events\r\n* Sports\r\n* Concerts\r\n* Comedy\r\n* Theater Broadway All Shows Family Ballet Opera\r\n* More This Weekend Next Week Festivals Venues\r\n\r\n[All Events](https://atlanta-ga.events/)[Sports](https://atlanta-ga.events/sports/)[Concerts](https://atlanta-ga.events/concerts/)[Comedy](https://atlanta-ga.events/comedy-shows/)[Theater](#)* Broadway\r\n* All Shows\r\n* Family\r\n* Ballet\r\n* Opera\r\n\r\n[Broadway](https://atlanta-ga.events/broadway/)[All Shows](https://atlanta-ga.events/shows/)[Family](https://atlanta-ga.events/family-shows/)[Ballet](https://atlanta-ga.events/ballet/)[Opera](https://atlanta-ga.events/opera/)[More](#)* This Weekend\r\n* Next Week\r\n* Festivals\r\n* Venues\r\n\r\n[This Weekend](https://atlanta-ga.events/this-weekend/)[Next Week](https://atlanta-ga.events/next-week/)[Festivals](https://atlanta-ga.events/festivals/)[Venues](https://atlanta-ga.events/entertainment-venues/)* Home\r\n* December\r\n\r\n[Home](https://atlanta-ga.events)# Atlanta Events December 2024\r\n\r\nCheck out the complete schedule of this month\u0026#8217;s events. \u0026#8220;Atlanta Events\u0026#8221; is your guide to epic events: concerts, theater shows, Broadway musicals and plays, opera, ballet, and sporting events to experience while in town. Get ready to discover the best entertainment venues in Atlanta, GA. Find your favorite events, check seating charts, buy tickets, and how to get there. Buy event tickets in advance for perfect seats. We\u2019re sure you will find something that sparks your curiosity!\r\n\r\n**Check out the complete schedule of this month\u0026#8217;s events.**[Atlanta Events](https://atlanta-ga.events/)[concerts](https://atlanta-ga.events/concerts/)[theater shows](https://atlanta-ga.events/shows/)[Broadway](https://atlanta-ga.events/broadway/)[opera](https://atlanta-ga.events/opera/)[ballet](https://atlanta-ga.events/ballet/)[sporting events](https://atlanta-ga.events/sports/)[Jan](https://atlanta-ga.events/january/)[Feb](https://atlanta-ga.events/february/)[Mar](https://atlanta-ga.events/march/)[Apr](https://atlanta-ga.events/april/)[May](https://atlanta-ga.events/may/)[Jun](https://atlanta-ga.events/june/)[Jul](https://atlanta-ga.events/july/)[Aug](https://atlanta-ga.events/august/)[Sep](https://atlanta-ga.events/september/)[Oct](https://atlanta-ga.events/october/)[Nov](https://atlanta-ga.events/november/)## Best Events\r\n\r\n![Georgia Tech Yellow Jackets Basketball](https://atlanta-ga.events/wp-content/uploads/2023/08/Georgia-Tech-Yellow-Jackets-Basketball.jpg)\r\n![location](data:image/svg\u002Bxml;base64,PHN2ZwogICAgICAgICAgICAgICAgICAgICAgICAgIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyIKICAgICAgICAgICAgICAgICAgICAgICAgICB4bWxuc1hsaW5rPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hsaW5rIgogICAgICAgICAgICAgICAgICAgICAgICAgIGZpbGw9IiNmMDAwMDAiCiAgICAgICAgICAgICAgICAgICAgICAgICAgaGVpZ2h0PSIxMHB4IgogICAgICAgICAgICAgICAgICAgICAgICAgIHdpZHRoPSIxMHB4IgogICAgICAgICAgICAgICAgICAgICAgICAgIHZlcnNpb249IjEuMSIKICAgICAgICAgICAgICAgICAgICAgICAgICBpZD0iQ2FwYV8xIgogICAgICAgICAgICAgICAgICAgICAgICAgIHZpZXdCb3g9IjAgMCAyOTcgMjk3IgogICAgICAgICAgICAgICAgICAgICAgICAgIHhtbFNwYWNlPSJwcmVzZXJ2ZSIKICAgICAgICAgICAgICAgICAgICAgICAgPgogICAgICAgICAgICAgICAgICAgICAgICAgIDxwYXRoIGQ9Ik0xNDguNSwwQzg3LjQzLDAsMzcuNzQ3LDQ5LjcwMywzNy43NDcsMTEwLjc5N2MwLDkxLjAyNiw5OS43MjksMTc5LjkwNSwxMDMuOTc2LDE4My42NDUgIGMxLjkzNiwxLjcwNSw0LjM1NiwyLjU1OSw2Ljc3NywyLjU1OWMyLjQyMSwwLDQuODQxLTAuODUzLDYuNzc4LTIuNTU5YzQuMjQ1LTMuNzM5LDEwMy45NzUtOTIuNjE4LDEwMy45NzUtMTgzLjY0NSAgQzI1OS4yNTMsNDkuNzAzLDIwOS41NywwLDE0OC41LDB6IE0xNDguNSw3OS42OTNjMTYuOTY0LDAsMzAuNzY1LDEzLjk1MywzMC43NjUsMzEuMTA0YzAsMTcuMTUxLTEzLjgwMSwzMS4xMDQtMzAuNzY1LDMxLjEwNCAgYy0xNi45NjQsMC0zMC43NjUtMTMuOTUzLTMwLjc2NS0zMS4xMDRDMTE3LjczNSw5My42NDYsMTMxLjUzNiw3OS42OTMsMTQ4LjUsNzkuNjkzeiI\u002BPC9wYXRoPgogICAgICAgICAgICAgICAgICAgICAgICA8L3N2Zz4=)\r\n[Get Tickets](/event/georgia-tech-yellow-jackets-vs-notre-dame-fighting-irish-atlanta-tuesday-december-31-2024-230-pm/)![Zac Brown Band](https://atlanta-ga.events/wp-content/uploads/2024/02/zac-brown-band.jpg)\r\n![location](data:image/svg\u002Bxml;base64,PHN2ZwogICAgICAgICAgICAgICAgICAgICAgICAgIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyIKICAgICAgICAgICAgICAgICAgICAgICAgICB4bWxuc1hsaW5rPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hsaW5rIgogICAgICAgICAgICAgICAgICAgICAgICAgIGZpbGw9IiNmMDAwMDAiCiAgICAgICAgICAgICAgICAgICAgICAgICAgaGVpZ2h0PSIxMHB4IgogICAgICAgICAgICAgICAgICAgICAgICAgIHdpZHRoPSIxMHB4IgogICAgICAgICAgICAgICAgICAgICAgICAgIHZlcnNpb249IjEuMSIKICAgICAgICAgICAgICAgICAgICAgICAgICBpZD0iQ2FwYV8xIgogICAgICAgICAgICAgICAgICAgICAgICAgIHZpZXdCb3g9IjAgMCAyOTcgMjk3IgogICAgICAgICAgICAgICAgICAgICAgICAgIHhtbFNwYWNlPSJwcmVzZXJ2ZSIKICAgICAgICAgICAgICAgICAgICAgICAgPgogICAgICAgICAgICAgICAgICAgICAgICAgIDxwYXRoIGQ9Ik0xNDguNSwwQzg3LjQzLDAsMzcuNzQ3LDQ5LjcwMywzNy43NDcsMTEwLjc5N2MwLDkxLjAyNiw5OS43MjksMTc5LjkwNSwxMDMuOTc2LDE4My42NDUgIGMxLjkzNiwxLjcwNSw0LjM1NiwyLjU1OSw2Ljc3NywyLjU1OWMyLjQyMSwwLDQuODQxLTAuODUzLDYuNzc4LTIuNTU5YzQuMjQ1LTMuNzM5LDEwMy45NzUtOTIuNjE4LDEwMy45NzUtMTgzLjY0NSAgQzI1OS4yNTMsNDkuNzAzLDIwOS41NywwLDE0OC41LDB6IE0xNDguNSw3OS42OTNjMTYuOTY0LDAsMzAuNzY1LDEzLjk1MywzMC43NjUsMzEuMTA0YzAsMTcuMTUxLTEzLjgwMSwzMS4xMDQtMzAuNzY1LDMxLjEwNCAgYy0xNi45NjQsMC0zMC43NjUtMTMuOTUzLTMwLjc2NS0zMS4xMDRDMTE3LjczNSw5My42NDYsMTMxLjUzNiw3OS42OTMsMTQ4LjUsNzkuNjkzeiI\u002BPC9wYXRoPgogICAgICAgICAgICAgICAgICAgICAgICA8L3N2Zz4=)\r\n[Get Tickets](/event/zac-brown-band-atlanta-tuesday-december-31-2024-800-pm/)![Moon Taxi](https://atlanta-ga.events/wp-content/uploads/2024/09/moon-taxi-concert-tickets.jpeg)\r\n![location](data:image/svg\u002Bxml;base64,PHN2ZwogICAgICAgICAgICAgICAgICAgICAgICAgIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyIKICAgICAgICAgICAgICAgICAgICAgICAgICB4bWxuc1hsaW5rPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hsaW5rIgogICAgICAgICAgICAgICAgICAgICAgICAgIGZpbGw9IiNmMDAwMDAiCiAgICAgICAgICAgICAgICAgICAgICAgICAgaGVpZ2h0PSIxMHB4IgogICAgICAgICAgICAgICAgICAgICAgICAgIHdpZHRoPSIxMHB4IgogICAgICAgICAgICAgICAgICAgICAgICAgIHZlcnNpb249IjEuMSIKICAgICAgICAgICAgICAgICAgICAgICAgICBpZD0iQ2FwYV8xIgogICAgICAgICAgICAgICAgICAgICAgICAgIHZpZXdCb3g9IjAgMCAyOTcgMjk3IgogICAgICAgICAgICAgICAgICAgICAgICAgIHhtbFNwYWNlPSJwcmVzZXJ2ZSIKICAgICAgICAgICAgICAgICAgICAgICAgPgogICAgICAgICAgICAgICAgICAgICAgICAgIDxwYXRoIGQ9Ik0xNDguNSwwQzg3LjQzLDAsMzcuNzQ3LDQ5LjcwMywzNy43NDcsMTEwLjc5N2MwLDkxLjAyNiw5OS43MjksMTc5LjkwNSwxMDMuOTc2LDE4My42NDUgIGMxLjkzNiwxLjcwNSw0LjM1NiwyLjU1OSw2Ljc3NywyLjU1OWMyLjQyMSwwLDQuODQxLTAuODUzLDYuNzc4LTIuNTU5YzQuMjQ1LTMuNzM5LDEwMy45NzUtOTIuNjE4LDEwMy45NzUtMTgzLjY0NSAgQzI1OS4yNTMsNDkuNzAzLDIwOS41NywwLDE0OC41LDB6IE0xNDguNSw3OS42OTNjMTYuOTY0LDAsMzAuNzY1LDEzLjk1MywzMC43NjUsMzEuMTA0YzAsMTcuMTUxLTEzLjgwMSwzMS4xMDQtMzAuNzY1LDMxLjEwNCAgYy0xNi45NjQsMC0zMC43NjUtMTMuOTUzLTMwLjc2NS0zMS4xMDRDMTE3LjczNSw5My42NDYsMTMxLjUzNiw3OS42OTMsMTQ4LjUsNzkuNjkzeiI\u002BPC9wYXRoPgogICAgICAgICAgICAgICAgICAgICAgICA8L3N2Zz4=)\r\n[Get Tickets](/event/moon-taxi-atlanta-tuesday-december-31-2024-800-pm/)![Gimme Gimme Disco](https://atlanta-ga.events/wp-content/uploads/2023/03/Gimme-Gimme-Disco.jpg)\r\n![location](data:image/svg\u002Bxml;base64,PHN2ZwogICAgICAgICAgICAgICAgICAgICAgICAgIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyIKICAgICAgICAgICAgICAgICAgICAgICAgICB4bWxuc1hsaW5rPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hsaW5rIgogICAgICAgICAgICAgICAgICAgICAgICAgIGZpbGw9IiNmMDAwMDAiCiAgICAgICAgICAgICAgICAgICAgICAgICAgaGVpZ2h0PSIxMHB4IgogICAgICAgICAgICAgICAgICAgICAgICAgIHdpZHRoPSIxMHB4IgogICAgICAgICAgICAgICAgICAgICAgICAgIHZlcnNpb249IjEuMSIKICAgICAgICAgICAgICAgICAgICAgICAgICBpZD0iQ2FwYV8xIgogICAgICAgICAgICAgICAgICAgICAgICAgIHZpZXdCb3g9IjAgMCAyOTcgMjk3IgogICAgICAgICAgICAgICAgICAgICAgICAgIHhtbFNwYWNlPSJwcmVzZXJ2ZSIKICAgICAgICAgICAgICAgICAgICAgICAgPgogICAgICAgICAgICAgICAgICAgICAgICAgIDxwYXRoIGQ9Ik0xNDguNSwwQzg3LjQzLDAsMzcuNzQ3LDQ5LjcwMywzNy43NDcsMTEwLjc5N2MwLDkxLjAyNiw5OS43MjksMTc5LjkwNSwxMDMuOTc2LDE4My42NDUgIGMxLjkzNiwxLjcwNSw0LjM1NiwyLjU1OSw2Ljc3NywyLjU1OWMyLjQyMSwwLDQuODQxLTAuODUzLDYuNzc4LTIuNTU5YzQuMjQ1LTMuNzM5LDEwMy45NzUtOTIuNjE4LDEwMy45NzUtMTgzLjY0NSAgQzI1OS4yNTMsNDkuNzAzLDIwOS41NywwLDE0OC41LDB6IE0xNDguNSw3OS42OTNjMTYuOTY0LDAsMzAuNzY1LDEzLjk1MywzMC43NjUsMzEuMTA0YzAsMTcuMTUxLTEzLjgwMSwzMS4xMDQtMzAuNzY1LDMxLjEwNCAgYy0xNi45NjQsMC0zMC43NjUtMTMuOTUzLTMwLjc2NS0zMS4xMDRDMTE3LjczNSw5My42NDYsMTMxLjUzNiw3OS42OTMsMTQ4LjUsNzkuNjkzeiI\u002BPC9wYXRoPgogICAgICAgICAgICAgICAgICAgICAgICA8L3N2Zz4=)\r\n[Get Tickets](/event/gimme-gimme-disco-atlanta-tuesday-december-31-2024-900-pm/)![Big Freedia](https://atlanta-ga.events/wp-content/uploads/2023/03/Big-Freedia.jpg)\r\n![location](data:image/svg\u002Bxml;base64,PHN2ZwogICAgICAgICAgICAgICAgICAgICAgICAgIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyIKICAgICAgICAgICAgICAgICAgICAgICAgICB4bWxuc1hsaW5rPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hsaW5rIgogICAgICAgICAgICAgICAgICAgICAgICAgIGZpbGw9IiNmMDAwMDAiCiAgICAgICAgICAgICAgICAgICAgICAgICAgaGVpZ2h0PSIxMHB4IgogICAgICAgICAgICAgICAgICAgICAgICAgIHdpZHRoPSIxMHB4IgogICAgICAgICAgICAgICAgICAgICAgICAgIHZlcnNpb249IjEuMSIKICAgICAgICAgICAgICAgICAgICAgICAgICBpZD0iQ2FwYV8xIgogICAgICAgICAgICAgICAgICAgICAgICAgIHZpZXdCb3g9IjAgMCAyOTcgMjk3IgogICAgICAgICAgICAgICAgICAgICAgICAgIHhtbFNwYWNlPSJwcmVzZXJ2ZSIKICAgICAgICAgICAgICAgICAgICAgICAgPgogICAgICAgICAgICAgICAgICAgICAgICAgIDxwYXRoIGQ9Ik0xNDguNSwwQzg3LjQzLDAsMzcuNzQ3LDQ5LjcwMywzNy43NDcsMTEwLjc5N2MwLDkxLjAyNiw5OS43MjksMTc5LjkwNSwxMDMuOTc2LDE4My42NDUgIGMxLjkzNiwxLjcwNSw0LjM1NiwyLjU1OSw2Ljc3NywyLjU1OWMyLjQyMSwwLDQuODQxLTAuODUzLDYuNzc4LTIuNTU5YzQuMjQ1LTMuNzM5LDEwMy45NzUtOTIuNjE4LDEwMy45NzUtMTgzLjY0NSAgQzI1OS4yNTMsNDkuNzAzLDIwOS41NywwLDE0OC41LDB6IE0xNDguNSw3OS42OTNjMTYuOTY0LDAsMzAuNzY1LDEzLjk1MywzMC43NjUsMzEuMTA0YzAsMTcuMTUxLTEzLjgwMSwzMS4xMDQtMzAuNzY1LDMxLjEwNCAgYy0xNi45NjQsMC0zMC43NjUtMTMuOTUzLTMwLjc2NS0zMS4xMDRDMTE3LjczNSw5My42NDYsMTMxLjUzNiw3OS42OTMsMTQ4LjUsNzkuNjkzeiI\u002BPC9wYXRoPgogICAgICAgICAgICAgICAgICAgICAgICA8L3N2Zz4=)\r\n[Get Tickets](/event/big-freedia-atlanta-tuesday-december-31-2024-930-pm/)![Daily Bread](https://atlanta-ga.events/wp-content/uploads/2023/09/Daily-Bread.jpg)\r\n![location](data:image/svg\u002Bxml;base64,PHN2ZwogICAgICAgICAgICAgICAgICAgICAgICAgIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyIKICAgICAgICAgICAgICAgICAgICAgICAgICB4bWxuc1hsaW5rPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hsaW5rIgogICAgICAgICAgICAgICAgICAgICAgICAgIGZpbGw9IiNmMDAwMDAiCiAgICAgICAgICAgICAgICAgICAgICAgICAgaGVpZ2h0PSIxMHB4IgogICAgICAgICAgICAgICAgICAgICAgICAgIHdpZHRoPSIxMHB4IgogICAgICAgICAgICAgICAgICAgICAgICAgIHZlcnNpb249IjEuMSIKICAgICAgICAgICAgICAgICAgICAgICAgICBpZD0iQ2FwYV8xIgogICAgICAgICAgICAgICAgICAgICAgICAgIHZpZXdCb3g9IjAgMCAyOTcgMjk3IgogICAgICAgICAgICAgICAgICAgICAgICAgIHhtbFNwYWNlPSJwcmVzZXJ2ZSIKICAgICAgICAgICAgICAgICAgICAgICAgPgogICAgICAgICAgICAgICAgICAgICAgICAgIDxwYXRoIGQ9Ik0xNDguNSwwQzg3LjQzLDAsMzcuNzQ3LDQ5LjcwMywzNy43NDcsMTEwLjc5N2MwLDkxLjAyNiw5OS43MjksMTc5LjkwNSwxMDMuOTc2LDE4My42NDUgIGMxLjkzNiwxLjcwNSw0LjM1NiwyLjU1OSw2Ljc3NywyLjU1OWMyLjQyMSwwLDQuODQxLTAuODUzLDYuNzc4LTIuNTU5YzQuMjQ1LTMuNzM5LDEwMy45NzUtOTIuNjE4LDEwMy45NzUtMTgzLjY0NSAgQzI1OS4yNTMsNDkuNzAzLDIwOS41NywwLDE0OC41LDB6IE0xNDguNSw3OS42OTNjMTYuOTY0LDAsMzAuNzY1LDEzLjk1MywzMC43NjUsMzEuMTA0YzAsMTcuMTUxLTEzLjgwMSwzMS4xMDQtMzAuNzY1LDMxLjEwNCAgYy0xNi45NjQsMC0zMC43NjUtMTMuOTUzLTMwLjc2NS0zMS4xMDRDMTE3LjczNSw5My42NDYsMTMxLjUzNiw3OS42OTMsMTQ4LjUsNzkuNjkzeiI\u002BPC9wYXRoPgogICAgICAgICAgICAgICAgICAgICAgICA8L3N2Zz4=)\r\n[Get Tickets](/event/daily-bread-atlanta-tuesday-december-31-2024-930-pm/)## Schedule \u0026 Tickets\r\n\r\n* Dec 31 2024 2:30 PM Tue Georgia Tech Yellow Jackets vs. Notre Dame Fighting Irish McCamish Pavilion 30332, 965 Fowler St., Atlanta, GA, US Prices from $18 Avg. price ~ $62 Available Tickets: 225 Tickets\r\n* Dec 31 2024 7:00 PM Tue Provenance NYE: Seven Lions, Knife Party \u0026 Grabbitz Atlanta Coliseum 30096, 2075 Market St, Duluth, GA, US Prices from $114 Avg. price ~ $132 6 tickets remaining! Tickets\r\n* Dec 31 2024 8:00 PM Tue Zac Brown Band State Farm Arena - GA | Capacity: 21000 30303, One Philips Drive, Atlanta, GA, US Prices from $73 Avg. price ~ $237 Available Tickets: 1122 Tickets\r\n* Dec 31 2024 8:00 PM Tue Moon Taxi The Tabernacle - GA | Capacity: 2600 30303, 152 Luckie St, Atlanta, GA, US Prices from $65 Avg. price ~ $127 Available Tickets: 116 Tickets\r\n* Dec 31 2024 9:00 PM Tue Juelz - Artist Believe Music Hall 30312, 181 Ralph David Abernathy Blvd, Atlanta, GA, US Prices from $69 Avg. price ~ $79 6 tickets remaining! Tickets\r\n* Dec 31 2024 9:00 PM Tue Gimme Gimme Disco Hell at The Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $49 Avg. price ~ $63 21 tickets remaining! Tickets\r\n* Dec 31 2024 9:00 PM Tue The Players Ball NYE Experience: Thrice Groove Buckhead Theatre | Capacity: 2500 30305, 3110 Roswell Road, Atlanta, GA, US Prices from $44 Avg. price ~ $51 22 tickets remaining! Tickets\r\n* Dec 31 2024 9:30 PM Tue Big Freedia City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $224 Avg. price ~ $258 4 tickets remaining! Tickets\r\n* Dec 31 2024 9:30 PM Tue Daily Bread The Eastern - GA 30312, 800 Old Flat Shoals Road SE, Atlanta, GA, US Prices from $111 Avg. price ~ $158 19 tickets remaining! Tickets\r\n* Dec 31 2024 10:00 PM Tue Hugel - Artist District Atlanta 30324, 269 Armour Dr NE, Atlanta, GA, US Prices from $122 Avg. price ~ $189 7 tickets remaining! Tickets\r\n\r\n![Georgia Tech Yellow Jackets vs. Notre Dame Fighting Irish](https://atlanta-ga.events/wp-content/uploads/2023/08/Georgia-Tech-Yellow-Jackets-Basketball.jpg)\r\n[Georgia Tech Yellow Jackets](https://atlanta-ga.events/tickets/georgia-tech-yellow-jackets/)[Notre Dame Fighting Irish](https://atlanta-ga.events/tickets/notre-dame-fighting-irish/)[McCamish Pavilion](https://atlanta-ga.events/venue/mccamish-pavilion/)[Tickets](https://atlanta-ga.events/event/georgia-tech-yellow-jackets-vs-notre-dame-fighting-irish-atlanta-tuesday-december-31-2024-230-pm/?pid=1282)![Provenance NYE: Seven Lions, Knife Party \u0026 Grabbitz](https://atlanta-ga.events/wp-content/uploads/2023/03/Seven-Lions.jpg)\r\n[Provenance NYE](https://atlanta-ga.events/tickets/provenance-nye/)[Seven Lions](https://atlanta-ga.events/tickets/seven-lions/)[Knife Party](https://atlanta-ga.events/tickets/knife-party/)[Grabbitz](https://atlanta-ga.events/tickets/grabbitz/)[Atlanta Coliseum](https://atlanta-ga.events/venue/atlanta-coliseum/)[Tickets](https://atlanta-ga.events/event/provenance-nye-seven-lions-knife-party-and-grabbitz-duluth-tuesday-december-31-2024-700-pm/?pid=8320)![Zac Brown Band](https://atlanta-ga.events/wp-content/uploads/2024/02/zac-brown-band.jpg)\r\n[Zac Brown Band](https://atlanta-ga.events/tickets/zac-brown-band/)[State Farm Arena - GA](https://atlanta-ga.events/venue/state-farm-arena-ga/)[Tickets](https://atlanta-ga.events/event/zac-brown-band-atlanta-tuesday-december-31-2024-800-pm/?pid=769)![Moon Taxi](https://atlanta-ga.events/wp-content/uploads/2024/09/moon-taxi-concert-tickets.jpeg)\r\n[Moon Taxi](https://atlanta-ga.events/tickets/moon-taxi/)[The Tabernacle - GA](https://atlanta-ga.events/venue/the-tabernacle-ga/)[Tickets](https://atlanta-ga.events/event/moon-taxi-atlanta-tuesday-december-31-2024-800-pm/?pid=7640)![Atlanta Events Schedule](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Believe Music Hall](https://atlanta-ga.events/venue/believe-music-hall/)[Tickets](https://atlanta-ga.events/event/juelz-artist-atlanta-tuesday-december-31-2024-900-pm/?pid=4969)![Gimme Gimme Disco](https://atlanta-ga.events/wp-content/uploads/2023/03/Gimme-Gimme-Disco.jpg)\r\n[Gimme Gimme Disco](https://atlanta-ga.events/tickets/gimme-gimme-disco/)[Hell at The Masquerade - GA](https://atlanta-ga.events/venue/hell-at-the-masquerade-ga/)[Tickets](https://atlanta-ga.events/event/gimme-gimme-disco-atlanta-tuesday-december-31-2024-900-pm/?pid=1469)![Atlanta Events Schedule](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Thrice Groove](https://atlanta-ga.events/tickets/thrice-groove/)[Buckhead Theatre](https://atlanta-ga.events/venue/buckhead-theatre/)[Tickets](https://atlanta-ga.events/event/the-players-ball-nye-experience-thrice-groove-atlanta-tuesday-december-31-2024-900-pm/?pid=7935)![Big Freedia](https://atlanta-ga.events/wp-content/uploads/2023/03/Big-Freedia.jpg)\r\n[Big Freedia](https://atlanta-ga.events/tickets/big-freedia/)[City Winery - Atlanta](https://atlanta-ga.events/venue/city-winery-atlanta/)[Tickets](https://atlanta-ga.events/event/big-freedia-atlanta-tuesday-december-31-2024-930-pm/?pid=2964)![Daily Bread](https://atlanta-ga.events/wp-content/uploads/2023/09/Daily-Bread.jpg)\r\n[Daily Bread](https://atlanta-ga.events/tickets/daily-bread/)[The Eastern - GA](https://atlanta-ga.events/venue/the-eastern-ga/)[Tickets](https://atlanta-ga.events/event/daily-bread-atlanta-tuesday-december-31-2024-930-pm/?pid=4699)![Atlanta Events Schedule](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Hugel - Artist](https://atlanta-ga.events/tickets/hugel-artist/)[District Atlanta](https://atlanta-ga.events/venue/district-atlanta/)[Tickets](https://atlanta-ga.events/event/hugel-artist-atlanta-tuesday-december-31-2024-1000-pm/?pid=2550)### List Of Events\r\n\r\n[Georgia Tech Yellow Jackets Events: 3 McCamish Pavilion Dec 31, 2024 2:30 PM](https://atlanta-ga.events/tickets/georgia-tech-yellow-jackets/)![Georgia Tech Yellow Jackets](https://atlanta-ga.events/wp-content/uploads/2023/08/Georgia-Tech-Yellow-Jackets.jpg)\r\n[Notre Dame Fighting Irish Events: 3 McCamish Pavilion Dec 31, 2024 2:30 PM](https://atlanta-ga.events/tickets/notre-dame-fighting-irish/)![Notre Dame Fighting Irish](https://atlanta-ga.events/wp-content/uploads/2024/02/notre-dame-fighting-irish.jpg)\r\n[Georgia Tech Yellow Jackets Basketball Events: 3 McCamish Pavilion Dec 31, 2024 2:30 PM](https://atlanta-ga.events/tickets/georgia-tech-yellow-jackets-basketball/)![Georgia Tech Yellow Jackets Basketball](https://atlanta-ga.events/wp-content/uploads/2023/08/Georgia-Tech-Yellow-Jackets-Basketball.jpg)\r\n[Notre Dame Fighting Irish Basketball Events: 3 McCamish Pavilion Dec 31, 2024 2:30 PM](https://atlanta-ga.events/tickets/notre-dame-fighting-irish-basketball/)![Notre Dame Fighting Irish Basketball](https://atlanta-ga.events/wp-content/uploads/2024/09/notre-dame-fighting-irish-basketball.jpg)\r\n[Knife Party Events: 2 Atlanta Coliseum Dec 31, 2024 7:00 PM](https://atlanta-ga.events/tickets/knife-party/)![Knife Party](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Seven Lions Events: 2 Atlanta Coliseum Dec 31, 2024 7:00 PM](https://atlanta-ga.events/tickets/seven-lions/)![Seven Lions](https://atlanta-ga.events/wp-content/uploads/2023/03/Seven-Lions.jpg)\r\n[Grabbitz Events: 2 Atlanta Coliseum Dec 31, 2024 7:00 PM](https://atlanta-ga.events/tickets/grabbitz/)![Grabbitz](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Provenance NYE Events: 2 Atlanta Coliseum Dec 31, 2024 7:00 PM](https://atlanta-ga.events/tickets/provenance-nye/)![Provenance NYE](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Moon Taxi Events: 2 The Tabernacle - GA Dec 31, 2024 8:00 PM](https://atlanta-ga.events/tickets/moon-taxi/)![Moon Taxi](https://atlanta-ga.events/wp-content/uploads/2024/09/moon-taxi-concert-tickets.jpeg)\r\n[Zac Brown Band Events: 2 State Farm Arena - GA Dec 31, 2024 8:00 PM](https://atlanta-ga.events/tickets/zac-brown-band/)![Zac Brown Band](https://atlanta-ga.events/wp-content/uploads/2024/02/zac-brown-band.jpg)\r\n[Thrice Groove Events: 2 Buckhead Theatre Dec 31, 2024 9:00 PM](https://atlanta-ga.events/tickets/thrice-groove/)![Thrice Groove](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Juelz - Arist Events: 2 Believe Music Hall Dec 31, 2024 9:00 PM](https://atlanta-ga.events/tickets/juelz-arist/)![Juelz - Arist](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Gimme Gimme Disco Events: 2 Hell at The Masquerade - GA Dec 31, 2024 9:00 PM](https://atlanta-ga.events/tickets/gimme-gimme-disco/)![Gimme Gimme Disco](https://atlanta-ga.events/wp-content/uploads/2023/03/Gimme-Gimme-Disco.jpg)\r\n[Daily Bread Events: 2 The Eastern - GA Dec 31, 2024 9:30 PM](https://atlanta-ga.events/tickets/daily-bread/)![Daily Bread](https://atlanta-ga.events/wp-content/uploads/2023/09/Daily-Bread.jpg)\r\n[Big Freedia Events: 2 City Winery - Atlanta Dec 31, 2024 9:30 PM](https://atlanta-ga.events/tickets/big-freedia/)![Big Freedia](https://atlanta-ga.events/wp-content/uploads/2023/03/Big-Freedia.jpg)\r\n[Hugel - Artist Events: 2 District Atlanta Dec 31, 2024 10:00 PM](https://atlanta-ga.events/tickets/hugel-artist/)![Hugel - Artist](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n### Concerts Venues\r\n\r\n[529 30316, 529 Flat Shoals Ave SE, Atlanta, GA, US Live Music Events: 1](https://atlanta-ga.events/venue/529/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Aisle 5 30307, 1123 Euclid Ave Ne, Atlanta, GA, US Live Music Events: 15](https://atlanta-ga.events/venue/aisle-5/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Altar at Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Live Music Events: 28](https://atlanta-ga.events/venue/altar-at-masquerade-ga/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Ameris Bank Amphitheatre 30004, 2200 Encore Parkway, Alpharetta, GA, US Live Music Events: 6](https://atlanta-ga.events/venue/ameris-bank-amphitheatre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Atlanta Coliseum 30096, 2075 Market St, Duluth, GA, US Live Music Events: 3](https://atlanta-ga.events/venue/atlanta-coliseum/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Atlanta Symphony Hall 30309, 1280 Peachtree St, Atlanta, GA, US Live Music Events: 72](https://atlanta-ga.events/venue/atlanta-symphony-hall/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[ATL Comedy Theater 30093, 4650 Jimmy Carter Boulevard, Norcross, GA, US Live Music Events: 33](https://atlanta-ga.events/venue/atl-comedy-theater/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Believe Music Hall 30312, 181 Ralph David Abernathy Blvd, Atlanta, GA, US Live Music Events: 18](https://atlanta-ga.events/venue/believe-music-hall/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Boggs Social \u0026 Supply 30310, 1310 White St. SW, Atlanta, GA, US Live Music Events: 3](https://atlanta-ga.events/venue/boggs-social-supply/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Buckhead Theatre 30305, 3110 Roswell Road, Atlanta, GA, US Live Music Events: 46](https://atlanta-ga.events/venue/buckhead-theatre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Byers Theatre at Sandy Springs Performing Arts Center 30328, 1 Galambos Way, Atlanta, GA, US Live Music Events: 37](https://atlanta-ga.events/venue/byers-theatre-at-sandy-springs-performing-arts-center/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Cadence Bank Amphitheatre at Chastain Park 30327, 4469 Stella Drive Northwest, Atlanta, GA, US Live Music Events: 7](https://atlanta-ga.events/venue/cadence-bank-amphitheatre-at-chastain-park/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Center Parc Credit Union Stadium 30315, 755 Hank Aaron Drive, Atlanta, GA, US Live Music Events: 5](https://atlanta-ga.events/venue/center-parc-credit-union-stadium/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Center Stage Theatre 30309, 1374 West Peachtree Street Northwest, Atlanta, GA, US Live Music Events: 28](https://atlanta-ga.events/venue/center-stage-theatre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[City Winery - Atlanta 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Live Music Events: 112](https://atlanta-ga.events/venue/city-winery-atlanta/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Cobb Energy Performing Arts Centre 30339, 2800 Cobb Galleria Pkwy, Atlanta, GA, US Live Music Events: 50](https://atlanta-ga.events/venue/cobb-energy-performing-arts-centre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Coca-Cola Roxy 30339, 800 Battery Ave SE, Atlanta, GA, US Live Music Events: 32](https://atlanta-ga.events/venue/coca-cola-roxy/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[District Atlanta 30324, 269 Armour Dr NE, Atlanta, GA, US Live Music Events: 7](https://atlanta-ga.events/venue/district-atlanta/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Earl Smith Strand Theatre 30060, 117 North Park Square, Marietta, GA, US Live Music Events: 7](https://atlanta-ga.events/venue/earl-smith-strand-theatre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Eddie\u0027s Attic 30030, 515-b North Mcdonough St., Decatur, GA, US Live Music Events: 6](https://atlanta-ga.events/venue/eddies-attic-2/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Fabulous Fox Theatre - Atlanta 30305, 660 Peachtree St NE, Atlanta, GA, US Live Music Events: 94](https://atlanta-ga.events/venue/fabulous-fox-theatre-atlanta/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[From the Earth Brewing Company 30076, 1570 Holcomb Bridge Rd, Roswell, GA, US Live Music Events: 1](https://atlanta-ga.events/venue/from-the-earth-brewing-company/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Gas South Arena 30097, 6400 Sugarloaf Parkway, Duluth, GA, US Live Music Events: 60](https://atlanta-ga.events/venue/gas-south-arena/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Gas South Theater 30097, 6400 Sugarloaf Parkway, Duluth, GA, US Live Music Events: 27](https://atlanta-ga.events/venue/gas-south-theater/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Heaven Stage at Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Live Music Events: 29](https://atlanta-ga.events/venue/heaven-stage-at-masquerade-ga/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Helium Comedy Club - Atlanta 30005, 3070 Windward Plaza, Alpharetta, GA, US Live Music Events: 75](https://atlanta-ga.events/venue/helium-comedy-club-atlanta/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Hell at The Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Live Music Events: 38](https://atlanta-ga.events/venue/hell-at-the-masquerade-ga/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Lakewood Amphitheatre 30315, 2002 Lakewood Avenue Southeast Live Music Events: 3](https://atlanta-ga.events/venue/lakewood-amphitheatre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Madlife Stage \u0026 Studios 30188, 8722 Main st., Woodstock, GA, US Live Music Events: 3](https://atlanta-ga.events/venue/madlife-stage-studios/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Mercedes-Benz Stadium 30313, Martin Luther King Jr Dr SW \u0026 Northside Dr NW, Atlanta, GA, US Live Music Events: 33](https://atlanta-ga.events/venue/mercedes-benz-stadium/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Music Park at Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Live Music Events: 1](https://atlanta-ga.events/venue/music-park-at-masquerade-ga/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Phase Events 30005, 12150 Morris Rd, Alpharetta, GA, US Live Music Events: 1](https://atlanta-ga.events/venue/phase-events/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n### Theaters\r\n\r\n[Altar at Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Shows \u0026 Events: 28](https://atlanta-ga.events/venue/altar-at-masquerade-ga/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Americasmart 30303, 240 Peachtree Street N.w., Atlanta, GA, US Shows \u0026 Events: 2](https://atlanta-ga.events/venue/americasmart/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Atlanta Symphony Hall 30309, 1280 Peachtree St, Atlanta, GA, US Shows \u0026 Events: 72](https://atlanta-ga.events/venue/atlanta-symphony-hall/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Atlantic Station 30363, 1380 Atlantic Dr , Atlanta, GA, US Shows \u0026 Events: 1](https://atlanta-ga.events/venue/atlantic-station/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Buckhead Theatre 30305, 3110 Roswell Road, Atlanta, GA, US Shows \u0026 Events: 46](https://atlanta-ga.events/venue/buckhead-theatre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Byers Theatre at Sandy Springs Performing Arts Center 30328, 1 Galambos Way, Atlanta, GA, US Shows \u0026 Events: 37](https://atlanta-ga.events/venue/byers-theatre-at-sandy-springs-performing-arts-center/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Center Stage Theatre 30309, 1374 West Peachtree Street Northwest, Atlanta, GA, US Shows \u0026 Events: 28](https://atlanta-ga.events/venue/center-stage-theatre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[City Winery - Atlanta 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Shows \u0026 Events: 112](https://atlanta-ga.events/venue/city-winery-atlanta/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Cobb Energy Performing Arts Centre 30339, 2800 Cobb Galleria Pkwy, Atlanta, GA, US Shows \u0026 Events: 50](https://atlanta-ga.events/venue/cobb-energy-performing-arts-centre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Coca-Cola Roxy 30339, 800 Battery Ave SE, Atlanta, GA, US Shows \u0026 Events: 32](https://atlanta-ga.events/venue/coca-cola-roxy/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Coca-Cola Stage at Alliance Theatre 30309, 1280 Peachtree Street NE, Atlanta, GA, US Shows \u0026 Events: 58](https://atlanta-ga.events/venue/coca-cola-stage-at-alliance-theatre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Earl Smith Strand Theatre 30060, 117 North Park Square, Marietta, GA, US Shows \u0026 Events: 7](https://atlanta-ga.events/venue/earl-smith-strand-theatre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Exhibition Hub Art Center Atlanta 30340, 5660 Buford Hwy NE, Atlanta, GA, US Shows \u0026 Events: 22](https://atlanta-ga.events/venue/exhibition-hub-art-center-atlanta/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Fabulous Fox Theatre - Atlanta 30305, 660 Peachtree St NE, Atlanta, GA, US Shows \u0026 Events: 94](https://atlanta-ga.events/venue/fabulous-fox-theatre-atlanta/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Forsyth County Arts Center 30040, 1150 Dahlonega Hwy, Cumming, GA, US Shows \u0026 Events: 3](https://atlanta-ga.events/venue/forsyth-county-arts-center/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Gas South Arena 30097, 6400 Sugarloaf Parkway, Duluth, GA, US Shows \u0026 Events: 60](https://atlanta-ga.events/venue/gas-south-arena/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Gas South Theater 30097, 6400 Sugarloaf Parkway, Duluth, GA, US Shows \u0026 Events: 27](https://atlanta-ga.events/venue/gas-south-theater/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Gateway Center Arena At College Park 30337, 2330 Convention Center Concourse, Atlanta, GA, US Shows \u0026 Events: 51](https://atlanta-ga.events/venue/gateway-center-arena-at-college-park/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Georgia World Congress Center 30313, 285 International Boulevard, Atlanta, GA, US Shows \u0026 Events: 4](https://atlanta-ga.events/venue/georgia-world-congress-center/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Hall C at Georgia World Congress Center 30313, 285 International Boulevard, Atlanta, GA, US Shows \u0026 Events: 4](https://atlanta-ga.events/venue/hall-c-at-georgia-world-congress-center/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Heaven Stage at Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Shows \u0026 Events: 29](https://atlanta-ga.events/venue/heaven-stage-at-masquerade-ga/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Helium Comedy Club - Atlanta 30005, 3070 Windward Plaza, Alpharetta, GA, US Shows \u0026 Events: 75](https://atlanta-ga.events/venue/helium-comedy-club-atlanta/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Hell at The Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Shows \u0026 Events: 38](https://atlanta-ga.events/venue/hell-at-the-masquerade-ga/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Lake Lanier Islands Resort 30518, 7000 Holiday Road, Sugar Hill, GA, US Shows \u0026 Events: 3](https://atlanta-ga.events/venue/lake-lanier-islands-resort/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Madlife Stage \u0026 Studios 30188, 8722 Main st., Woodstock, GA, US Shows \u0026 Events: 3](https://atlanta-ga.events/venue/madlife-stage-studios/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Sanctuary Nightclub - Atlanta 30305, 3209 Paces Ferry Pl NW, Atlanta, GA, US Shows \u0026 Events: 6](https://atlanta-ga.events/venue/sanctuary-nightclub-atlanta/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[State Farm Arena - GA 30303, One Philips Drive, Atlanta, GA, US Shows \u0026 Events: 64](https://atlanta-ga.events/venue/state-farm-arena-ga/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Terminal West At King Plow Arts Center 30318, 887 West Marietta Studio C, Atlanta, GA, US Shows \u0026 Events: 50](https://atlanta-ga.events/venue/terminal-west-at-king-plow-arts-center/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[The Courtland Grand Hotel 30303, 165 Courtland Street NE, Atlanta, GA, US Shows \u0026 Events: 4](https://atlanta-ga.events/venue/the-courtland-grand-hotel/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[The Loft - GA 30309, 1374 W. Peachtree Street, Atlanta, GA, US Shows \u0026 Events: 9](https://atlanta-ga.events/venue/the-loft-ga/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[The Music Room At Smith\u0027s Olde Bar 30324, 1574 1/2 Piedmont Ave Shows \u0026 Events: 8](https://atlanta-ga.events/venue/the-music-room-at-smiths-olde-bar/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[The Tabernacle - GA 30303, 152 Luckie St, Atlanta, GA, US Shows \u0026 Events: 41](https://atlanta-ga.events/venue/the-tabernacle-ga/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n### Sports Venues\r\n\r\n[Atlanta Motor Speedway 30228, 1500 Tara Place, Hampton, GA, US Sports Events: 6](https://atlanta-ga.events/venue/atlanta-motor-speedway/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Bobby Dodd Stadium 30332, 177 North Avenue NW, Atlanta, GA, US Sports Events: 2](https://atlanta-ga.events/venue/bobby-dodd-stadium/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Center Parc Credit Union Stadium 30315, 755 Hank Aaron Drive, Atlanta, GA, US Sports Events: 5](https://atlanta-ga.events/venue/center-parc-credit-union-stadium/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Center Stage Theatre 30309, 1374 West Peachtree Street Northwest, Atlanta, GA, US Sports Events: 28](https://atlanta-ga.events/venue/center-stage-theatre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Coolray Field 30044, 2500 Buford Drive, Lawrenceville, GA, US Sports Events: 75](https://atlanta-ga.events/venue/coolray-field/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[East Lake Golf Club 30317, 2575 Alston Drive, Se, Atlanta, GA, US Sports Events: 5](https://atlanta-ga.events/venue/east-lake-golf-club/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Fifth Third Stadium 30144, 3200 Busbee Pkwy, Kennesaw, GA, US Sports Events: 1](https://atlanta-ga.events/venue/fifth-third-stadium/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Gas South Arena 30097, 6400 Sugarloaf Parkway, Duluth, GA, US Sports Events: 60](https://atlanta-ga.events/venue/gas-south-arena/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Gateway Center Arena At College Park 30337, 2330 Convention Center Concourse, Atlanta, GA, US Sports Events: 51](https://atlanta-ga.events/venue/gateway-center-arena-at-college-park/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Georgia International Horse Park 30013, 1996 Centennial Olympic Pkwy, Conyers, GA, US Sports Events: 2](https://atlanta-ga.events/venue/georgia-international-horse-park/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Georgia State Convocation Center 30312, 455 Capitol Ave SE, Atlanta, GA, US Sports Events: 10](https://atlanta-ga.events/venue/georgia-state-convocation-center/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[KSU Convocation Center 30144, 590 Cobb Ave. , Kennesaw, GA, US Sports Events: 18](https://atlanta-ga.events/venue/ksu-convocation-center/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[McCamish Pavilion 30332, 965 Fowler St., Atlanta, GA, US Sports Events: 17](https://atlanta-ga.events/venue/mccamish-pavilion/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Mercedes-Benz Stadium 30313, Martin Luther King Jr Dr SW \u0026 Northside Dr NW, Atlanta, GA, US Sports Events: 33](https://atlanta-ga.events/venue/mercedes-benz-stadium/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Russ Chandler Stadium 30318, 255 Ferst Drive Northwest, Atlanta, GA, US Sports Events: 35](https://atlanta-ga.events/venue/russ-chandler-stadium/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Shirley Clements Mewborn Field 30332, 935 Fowler Street, Atlanta, GA, US Sports Events: 4](https://atlanta-ga.events/venue/shirley-clements-mewborn-field/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[State Farm Arena - GA 30303, One Philips Drive, Atlanta, GA, US Sports Events: 64](https://atlanta-ga.events/venue/state-farm-arena-ga/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[TPC Sugarloaf 30097, 2595 Sugarloaf Club Drive, Duluth, GA, US Sports Events: 3](https://atlanta-ga.events/venue/tpc-sugarloaf/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Truist Park 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Sports Events: 87](https://atlanta-ga.events/venue/truist-park/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[](https://atlanta-ga.events)[Contact Us](https://atlanta-ga.events/contact-us/)Stay up to date! Join our newsletter to stay current on all Live Entertainment in Atlanta, plus the occasional special offer, VIP packages, and discount tickets! Unique events and incredible deals are straight to your inbox. Either way, you\u2019ll be the first to know about the best events in Atlanta, GA. \u201CAtlanta events\u201D is part of the \u201CLive Entertainment Guide\u201D collection. We offer an extensive collection of websites providing complete, impartial guides to all the theatrical, musical, sports, and performance arts events in America\u2019s greatest cities.\r\n\r\n[Atlanta events](https://atlanta-ga.events/)[Live Entertainment Guide](https://liveentertainment.guide/)[Sitemap](https://atlanta-ga.events/sitemaps/)[Privacy Policy](https://atlanta-ga.events/privacy-policy/)"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url":"https://www.eventbrite.com/d/ga--atlanta/december-31/"}
```
and got this result:
```
"[upgrade your browser](/support/articles/en_US/Troubleshooting/how-to-troubleshoot-internet-browser-issues)[Eventbrite](https://www.eventbrite.com/)*Eventbrite*[Eventbrite](https://www.eventbrite.com/)*Eventbrite** Find my tickets\r\n* Log In\r\n* Sign Up\r\n* Find EventsCreate EventsSolutionsSolutionsEvent TicketingEvent Marketing PlatformEventbrite AdsPaymentsIndustryIndustryMusicFood \u0026amp; BeveragePerforming ArtsCharity \u0026amp; CausesRetailEvent TypesEvent TypesConcertsClasses \u0026amp; WorkshopsFestivals \u0026amp; FairsConferencesCorporate EventsOnline EventsBlogBlogTips \u0026amp; GuidesNews \u0026amp; TrendsCommunityTools \u0026amp; FeaturesOrganizer Resource HubCreate EventsContact SalesGet StartedHelp CenterHelp CenterFind your ticketsContact your event organizer\r\n\r\n[Find my tickets](https://www.eventbrite.com/signin/signup/?referrer=%2Fmytickets)[Log In](https://www.eventbrite.com/signin/?referrer=%2Fd%2Fga--atlanta%2Fdecember-31%2F)[Sign Up](https://www.eventbrite.com/signin/signup/?referrer=%2Fd%2Fga--atlanta%2Fdecember-31%2F)***** Find Events\r\n* Create EventsSolutionsSolutionsEvent TicketingEvent Marketing PlatformEventbrite AdsPaymentsIndustryIndustryMusicFood \u0026amp; BeveragePerforming ArtsCharity \u0026amp; CausesRetailEvent TypesEvent TypesConcertsClasses \u0026amp; WorkshopsFestivals \u0026amp; FairsConferencesCorporate EventsOnline EventsBlogBlogTips \u0026amp; GuidesNews \u0026amp; TrendsCommunityTools \u0026amp; Features\r\n* Organizer Resource Hub\r\n* Create Events\r\n* Contact Sales\r\n* Get Started\r\n* Help CenterHelp CenterFind your ticketsContact your event organizer\r\n\r\n[Find Events](https://www.eventbrite.com/d/local/events/)************* SolutionsSolutionsEvent TicketingEvent Marketing PlatformEventbrite AdsPayments\r\n* IndustryIndustryMusicFood \u0026amp; BeveragePerforming ArtsCharity \u0026amp; CausesRetail\r\n* Event TypesEvent TypesConcertsClasses \u0026amp; WorkshopsFestivals \u0026amp; FairsConferencesCorporate EventsOnline Events\r\n* BlogBlogTips \u0026amp; GuidesNews \u0026amp; TrendsCommunityTools \u0026amp; Features\r\n\r\n*** Event Ticketing\r\n* Event Marketing Platform\r\n* Eventbrite Ads\r\n* Payments\r\n\r\n[Event Ticketing](https://www.eventbrite.com/l/sell-tickets/)[Event Marketing Platform](https://www.eventbrite.com/l/event-marketing-platform/)[Eventbrite Ads](https://www.eventbrite.com/l/eventbrite-ads/)[Payments](https://www.eventbrite.com/l/event-payment/)*** Music\r\n* Food \u0026amp; Beverage\r\n* Performing Arts\r\n* Charity \u0026amp; Causes\r\n* Retail\r\n\r\n[Music](https://www.eventbrite.com/l/music/)[Food \u0026amp; Beverage](https://www.eventbrite.com/l/food-drink-event-ticketing/)[Performing Arts](https://www.eventbrite.com/l/performing-arts/)[Charity \u0026amp; Causes](https://www.eventbrite.com/l/npo/)[Retail](https://www.eventbrite.com/l/host-retail-events/)*** Concerts\r\n* Classes \u0026amp; Workshops\r\n* Festivals \u0026amp; Fairs\r\n* Conferences\r\n* Corporate Events\r\n* Online Events\r\n\r\n[Concerts](https://www.eventbrite.com/l/music-venues/)[Classes \u0026amp; Workshops](https://www.eventbrite.com/l/create-a-workshop/)[Festivals \u0026amp; Fairs](https://www.eventbrite.com/l/festival-solutions/)[Conferences](https://www.eventbrite.com/l/conferences/)[Corporate Events](https://www.eventbrite.com/l/eventbrite-for-business/)[Online Events](https://www.eventbrite.com/l/virtual-events-platform/)*** Tips \u0026amp; Guides\r\n* News \u0026amp; Trends\r\n* Community\r\n* Tools \u0026amp; Features\r\n\r\n[Tips \u0026amp; Guides](https://www.eventbrite.com/blog/category/tips-and-guides/)[News \u0026amp; Trends](https://www.eventbrite.com/blog/category/news-and-trends/)[Community](https://www.eventbrite.com/blog/category/community/)[Tools \u0026amp; Features](https://www.eventbrite.com/blog/category/tools-and-features/)[Organizer Resource Hub](https://www.eventbrite.com/resources/)[Create Events](https://www.eventbrite.com/organizer/overview/)[Contact Sales](https://www.eventbrite.com/l/contact-eventbrite-sales/)[Get Started](https://www.eventbrite.com/signin/signup/?referrer=/manage/events/create/)********* Help Center\r\n* Find your tickets\r\n* Contact your event organizer\r\n\r\n[Help Center](https://www.eventbrite.com/help/en-us/)[Find your tickets](https://www.eventbrite.com/help/en-us/articles/319355/where-are-my-tickets/)[Contact your event organizer](https://www.eventbrite.com/help/en-us/articles/647151/how-to-contact-the-event-organizer/)***** Find Events\r\n* Create Events\r\n* Help CenterHelp CenterFind your ticketsContact your event organizer\r\n* Find my tickets\r\n* Log In\r\n* Sign Up\r\n\r\n[Find Events](https://www.eventbrite.com/d/local/events/)[Create Events](https://www.eventbrite.com/organizer/overview/)***** Help Center\r\n* Find your tickets\r\n* Contact your event organizer\r\n\r\n[Help Center](https://www.eventbrite.com/help/en-us/)[Find your tickets](https://www.eventbrite.com/help/en-us/articles/319355/where-are-my-tickets/)[Contact your event organizer](https://www.eventbrite.com/help/en-us/articles/647151/how-to-contact-the-event-organizer/)[Find my tickets](https://www.eventbrite.com/signin/signup/?referrer=%2Fmytickets)[Log In](https://www.eventbrite.com/signin/?referrer=%2Fd%2Fga--atlanta%2Fdecember-31%2F)[Sign Up](https://www.eventbrite.com/signin/signup/?referrer=%2Fd%2Fga--atlanta%2Fdecember-31%2F)* Find EventsCreate EventsSolutionsSolutionsEvent TicketingEvent Marketing PlatformEventbrite AdsPaymentsIndustryIndustryMusicFood \u0026amp; BeveragePerforming ArtsCharity \u0026amp; CausesRetailEvent TypesEvent TypesConcertsClasses \u0026amp; WorkshopsFestivals \u0026amp; FairsConferencesCorporate EventsOnline EventsBlogBlogTips \u0026amp; GuidesNews \u0026amp; TrendsCommunityTools \u0026amp; FeaturesOrganizer Resource HubCreate EventsContact SalesGet StartedHelp CenterHelp CenterFind your ticketsContact your event organizer\r\n\r\n***** Find Events\r\n* Create EventsSolutionsSolutionsEvent TicketingEvent Marketing PlatformEventbrite AdsPaymentsIndustryIndustryMusicFood \u0026amp; BeveragePerforming ArtsCharity \u0026amp; CausesRetailEvent TypesEvent TypesConcertsClasses \u0026amp; WorkshopsFestivals \u0026amp; FairsConferencesCorporate EventsOnline EventsBlogBlogTips \u0026amp; GuidesNews \u0026amp; TrendsCommunityTools \u0026amp; Features\r\n* Organizer Resource Hub\r\n* Create Events\r\n* Contact Sales\r\n* Get Started\r\n* Help CenterHelp CenterFind your ticketsContact your event organizer\r\n\r\n[Find Events](https://www.eventbrite.com/d/local/events/)************* SolutionsSolutionsEvent TicketingEvent Marketing PlatformEventbrite AdsPayments\r\n* IndustryIndustryMusicFood \u0026amp; BeveragePerforming ArtsCharity \u0026amp; CausesRetail\r\n* Event TypesEvent TypesConcertsClasses \u0026amp; WorkshopsFestivals \u0026amp; FairsConferencesCorporate EventsOnline Events\r\n* BlogBlogTips \u0026amp; GuidesNews \u0026amp; TrendsCommunityTools \u0026amp; Features\r\n\r\n*** Event Ticketing\r\n* Event Marketing Platform\r\n* Eventbrite Ads\r\n* Payments\r\n\r\n[Event Ticketing](https://www.eventbrite.com/l/sell-tickets/)[Event Marketing Platform](https://www.eventbrite.com/l/event-marketing-platform/)[Eventbrite Ads](https://www.eventbrite.com/l/eventbrite-ads/)[Payments](https://www.eventbrite.com/l/event-payment/)*** Music\r\n* Food \u0026amp; Beverage\r\n* Performing Arts\r\n* Charity \u0026amp; Causes\r\n* Retail\r\n\r\n[Music](https://www.eventbrite.com/l/music/)[Food \u0026amp; Beverage](https://www.eventbrite.com/l/food-drink-event-ticketing/)[Performing Arts](https://www.eventbrite.com/l/performing-arts/)[Charity \u0026amp; Causes](https://www.eventbrite.com/l/npo/)[Retail](https://www.eventbrite.com/l/host-retail-events/)*** Concerts\r\n* Classes \u0026amp; Workshops\r\n* Festivals \u0026amp; Fairs\r\n* Conferences\r\n* Corporate Events\r\n* Online Events\r\n\r\n[Concerts](https://www.eventbrite.com/l/music-venues/)[Classes \u0026amp; Workshops](https://www.eventbrite.com/l/create-a-workshop/)[Festivals \u0026amp; Fairs](https://www.eventbrite.com/l/festival-solutions/)[Conferences](https://www.eventbrite.com/l/conferences/)[Corporate Events](https://www.eventbrite.com/l/eventbrite-for-business/)[Online Events](https://www.eventbrite.com/l/virtual-events-platform/)*** Tips \u0026amp; Guides\r\n* News \u0026amp; Trends\r\n* Community\r\n* Tools \u0026amp; Features\r\n\r\n[Tips \u0026amp; Guides](https://www.eventbrite.com/blog/category/tips-and-guides/)[News \u0026amp; Trends](https://www.eventbrite.com/blog/category/news-and-trends/)[Community](https://www.eventbrite.com/blog/category/community/)[Tools \u0026amp; Features](https://www.eventbrite.com/blog/category/tools-and-features/)[Organizer Resource Hub](https://www.eventbrite.com/resources/)[Create Events](https://www.eventbrite.com/organizer/overview/)[Contact Sales](https://www.eventbrite.com/l/contact-eventbrite-sales/)[Get Started](https://www.eventbrite.com/signin/signup/?referrer=/manage/events/create/)********* Help Center\r\n* Find your tickets\r\n* Contact your event organizer\r\n\r\n[Help Center](https://www.eventbrite.com/help/en-us/)[Find your tickets](https://www.eventbrite.com/help/en-us/articles/319355/where-are-my-tickets/)[Contact your event organizer](https://www.eventbrite.com/help/en-us/articles/647151/how-to-contact-the-event-organizer/)## NYE events are here. \uD83C\uDF89\r\n\r\n[Find your perfect plan.](/b/ga--atlanta/holiday/new-years-eve/)1. Home/\r\n2. United States/\r\n3. Atlanta/\r\n4. December 31 Events\r\n\r\n[Home](/)[United States](/d/united-states/events/)[Atlanta](/d/ga--atlanta/events/)# December 31 events in Atlanta, GA\r\n\r\nFilters\r\n\r\n* Business\r\n* Food \u0026amp; Drink\r\n* Health\r\n* Music\r\n\r\n[Business](/d/ga--atlanta/business--events/)**[Food \u0026amp; Drink](/d/ga--atlanta/food-and-drink--events/)**[Health](/d/ga--atlanta/health--events/)**[Music](/d/ga--atlanta/music--events/)*** Today\r\n* Tomorrow\r\n* This weekend\r\n* Pick a date...\r\n\r\n* Free\r\n* Paid\r\n\r\n* Class\r\n* Conference\r\n* Festival\r\n* Party\r\n\r\n****Search for online events\r\n\r\n* English\r\n* German\r\n* Spanish\r\n* French\r\n\r\n********* U.S. Dollar\r\n* Canadian Dollar\r\n* Euro\r\n* British Pound\r\n\r\n*************** Sales end soonJUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!Today at 9:00 PMBelieve Music HallFrom $0.00Believe Music Hall17.3k followersPromotedSave this event: JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!Share this event: JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!Promoted event actionsSales end soonJUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!Today at 9:00 PMBelieve Music HallFrom $0.00PromotedShare this event: JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!Save this event: JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!Promoted event actions\r\n* Sales end soonNYE 2025 THE BLACK OUTToday at 9:00 PM990 Brady AveFrom $50.00@OFFICIALOPIUMATLANTA265 followersPromotedSave this event: NYE 2025 THE BLACK OUTShare this event: NYE 2025 THE BLACK OUTPromoted event actionsSales end soonNYE 2025 THE BLACK OUTToday at 9:00 PM990 Brady AveFrom $50.00PromotedShare this event: NYE 2025 THE BLACK OUTSave this event: NYE 2025 THE BLACK OUTPromoted event actions\r\n* Almost fullTimes Square in ATL | New Years EveToday at 9:00 PMDOMAINE ATLFrom $0.002 for 1 dealDOMAINE ATL1.4k followersPromotedSave this event: Times Square in ATL | New Years EveShare this event: Times Square in ATL | New Years EvePromoted event actionsAlmost fullTimes Square in ATL | New Years EveToday at 9:00 PMDOMAINE ATLFrom $0.002 for 1 dealPromotedShare this event: Times Square in ATL | New Years EveSave this event: Times Square in ATL | New Years EvePromoted event actions\r\n* Sales end soonHUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District AtlantaToday at 9:00 PMDistrict AtlantaSave this event: HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District AtlantaShare this event: HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District AtlantaSales end soonHUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District AtlantaToday at 9:00 PMDistrict AtlantaShare this event: HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District AtlantaSave this event: HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District Atlanta\r\n* Almost fullNYE CELEBRATION @ DAIQUIRIVILLEToday at 9:00 PMUnderground AtlantaSave this event: NYE CELEBRATION @ DAIQUIRIVILLEShare this event: NYE CELEBRATION @ DAIQUIRIVILLEAlmost fullNYE CELEBRATION @ DAIQUIRIVILLEToday at 9:00 PMUnderground AtlantaShare this event: NYE CELEBRATION @ DAIQUIRIVILLESave this event: NYE CELEBRATION @ DAIQUIRIVILLE\r\n* Going fastPWK NEW YEAR\u2019S LINE UPToday at 9:00 PM1453 Cleveland Ave SWSave this event: PWK NEW YEAR\u2019S LINE UPShare this event: PWK NEW YEAR\u2019S LINE UPGoing fastPWK NEW YEAR\u2019S LINE UPToday at 9:00 PM1453 Cleveland Ave SWShare this event: PWK NEW YEAR\u2019S LINE UPSave this event: PWK NEW YEAR\u2019S LINE UP\r\n* LEMON DROP 2K25 - NEW YEAR\u2019S EVEToday at 8:30 PMWestside Cultural Arts CenterSave this event: LEMON DROP 2K25 - NEW YEAR\u2019S EVEShare this event: LEMON DROP 2K25 - NEW YEAR\u2019S EVELEMON DROP 2K25 - NEW YEAR\u2019S EVEToday at 8:30 PMWestside Cultural Arts CenterShare this event: LEMON DROP 2K25 - NEW YEAR\u2019S EVESave this event: LEMON DROP 2K25 - NEW YEAR\u2019S EVE\r\n* Sales end soonKing of Diamonds New Years Eve Party!Today at 9:00 PMKING OF DIAMONDS ATLANTASave this event: King of Diamonds New Years Eve Party!Share this event: King of Diamonds New Years Eve Party!Sales end soonKing of Diamonds New Years Eve Party!Today at 9:00 PMKING OF DIAMONDS ATLANTAShare this event: King of Diamonds New Years Eve Party!Save this event: King of Diamonds New Years Eve Party!\r\n* Sales end soonTHE LEMON DROP - NEW YEARS EVE 2K24Today at 9:00 PMWestside Cultural Arts CenterSave this event: THE LEMON DROP - NEW YEARS EVE 2K24Share this event: THE LEMON DROP - NEW YEARS EVE 2K24Sales end soonTHE LEMON DROP - NEW YEARS EVE 2K24Today at 9:00 PMWestside Cultural Arts CenterShare this event: THE LEMON DROP - NEW YEARS EVE 2K24Save this event: THE LEMON DROP - NEW YEARS EVE 2K24\r\n* Almost fullTHE GREATEST NYE ON EARTH REVEL ATLANTAToday at 9:00 PMREVEL ATLANTASave this event: THE GREATEST NYE ON EARTH REVEL ATLANTAShare this event: THE GREATEST NYE ON EARTH REVEL ATLANTAAlmost fullTHE GREATEST NYE ON EARTH REVEL ATLANTAToday at 9:00 PMREVEL ATLANTAShare this event: THE GREATEST NYE ON EARTH REVEL ATLANTASave this event: THE GREATEST NYE ON EARTH REVEL ATLANTA\r\n* Going fastNEW YEARS EVE ATLANTA 2025 | BAMBOO ATLANTAToday at 9:00 PMBamboo atlantaSave this event: NEW YEARS EVE ATLANTA 2025 | BAMBOO ATLANTAShare this event: NEW YEARS EVE ATLANTA 2025 | BAMBOO ATLANTAGoing fastNEW YEARS EVE ATLANTA 2025 | BAMBOO ATLANTAToday at 9:00 PMBamboo atlantaShare this event: NEW YEARS EVE ATLANTA 2025 | BAMBOO ATLANTASave this event: NEW YEARS EVE ATLANTA 2025 | BAMBOO ATLANTA\r\n* Almost fullBeer on a Champagne Night: New Year\u0026#x27;s Eve PartyToday at 8:00 PMNew Realm Brewing Co.Save this event: Beer on a Champagne Night: New Year\u0026#x27;s Eve PartyShare this event: Beer on a Champagne Night: New Year\u0026#x27;s Eve PartyAlmost fullBeer on a Champagne Night: New Year\u0026#x27;s Eve PartyToday at 8:00 PMNew Realm Brewing Co.Share this event: Beer on a Champagne Night: New Year\u0026#x27;s Eve PartySave this event: Beer on a Champagne Night: New Year\u0026#x27;s Eve Party\r\n* Havana Club ATL - NYE 2025 PartyToday at 8:00 PMHavana Club ATLSave this event: Havana Club ATL - NYE 2025 PartyShare this event: Havana Club ATL - NYE 2025 PartyHavana Club ATL - NYE 2025 PartyToday at 8:00 PMHavana Club ATLShare this event: Havana Club ATL - NYE 2025 PartySave this event: Havana Club ATL - NYE 2025 Party\r\n* Sales end soonJUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!Today at 9:00 PMBelieve Music HallSave this event: JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!Share this event: JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!Sales end soonJUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!Today at 9:00 PMBelieve Music HallShare this event: JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!Save this event: JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!\r\n* Sales end soonNYE Great Gatsby GalaToday at 8:00 PMBrewDog AtlantaSave this event: NYE Great Gatsby GalaShare this event: NYE Great Gatsby GalaSales end soonNYE Great Gatsby GalaToday at 8:00 PMBrewDog AtlantaShare this event: NYE Great Gatsby GalaSave this event: NYE Great Gatsby Gala\r\n* Sales end soonFREE NEW YEARS EVE PARTY IN ATLANTAToday at 10:00 PMLyfe ATLSave this event: FREE NEW YEARS EVE PARTY IN ATLANTAShare this event: FREE NEW YEARS EVE PARTY IN ATLANTASales end soonFREE NEW YEARS EVE PARTY IN ATLANTAToday at 10:00 PMLyfe ATLShare this event: FREE NEW YEARS EVE PARTY IN ATLANTASave this event: FREE NEW YEARS EVE PARTY IN ATLANTA\r\n* Sales end soonNew Years Eve \u201CBlack Billionaires Ball\u201D (Atlanta)Today at 9:00 PMWestside Cultural Arts Center, 10th Street Northwest, Atlanta, GA, USASave this event: New Years Eve \u201CBlack Billionaires Ball\u201D (Atlanta)Share this event: New Years Eve \u201CBlack Billionaires Ball\u201D (Atlanta)Sales end soonNew Years Eve \u201CBlack Billionaires Ball\u201D (Atlanta)Today at 9:00 PMWestside Cultural Arts Center, 10th Street Northwest, Atlanta, GA, USAShare this event: New Years Eve \u201CBlack Billionaires Ball\u201D (Atlanta)Save this event: New Years Eve \u201CBlack Billionaires Ball\u201D (Atlanta)\r\n* Going fastNEW YEARS EVE BALLToday at 10:00 PM2339 Brannen Rd SESave this event: NEW YEARS EVE BALLShare this event: NEW YEARS EVE BALLGoing fastNEW YEARS EVE BALLToday at 10:00 PM2339 Brannen Rd SEShare this event: NEW YEARS EVE BALLSave this event: NEW YEARS EVE BALL\r\n* Almost fullNYE 25 | FREE ENTRY W/ RSVP TIL 10PM | TUESDAY DEC 31ST | AT BAMBOO ATLToday at 9:00 PMBamboo atlantaSave this event: NYE 25 | FREE ENTRY W/ RSVP TIL 10PM | TUESDAY DEC 31ST | AT BAMBOO ATLShare this event: NYE 25 | FREE ENTRY W/ RSVP TIL 10PM | TUESDAY DEC 31ST | AT BAMBOO ATLAlmost fullNYE 25 | FREE ENTRY W/ RSVP TIL 10PM | TUESDAY DEC 31ST | AT BAMBOO ATLToday at 9:00 PMBamboo atlantaShare this event: NYE 25 | FREE ENTRY W/ RSVP TIL 10PM | TUESDAY DEC 31ST | AT BAMBOO ATLSave this event: NYE 25 | FREE ENTRY W/ RSVP TIL 10PM | TUESDAY DEC 31ST | AT BAMBOO ATL\r\n* Almost fullPEACH FEST | NEW YEARS EVE ATLANTAToday at 8:00 PM150 Central Ave SWSave this event: PEACH FEST | NEW YEARS EVE ATLANTAShare this event: PEACH FEST | NEW YEARS EVE ATLANTAAlmost fullPEACH FEST | NEW YEARS EVE ATLANTAToday at 8:00 PM150 Central Ave SWShare this event: PEACH FEST | NEW YEARS EVE ATLANTASave this event: PEACH FEST | NEW YEARS EVE ATLANTA\r\n* Almost fullMIDNIGHT MIRAGE: New Years Eve @Vision LoungeToday at 9:00 PM2520 Piedmont Rd NESave this event: MIDNIGHT MIRAGE: New Years Eve @Vision LoungeShare this event: MIDNIGHT MIRAGE: New Years Eve @Vision LoungeAlmost fullMIDNIGHT MIRAGE: New Years Eve @Vision LoungeToday at 9:00 PM2520 Piedmont Rd NEShare this event: MIDNIGHT MIRAGE: New Years Eve @Vision LoungeSave this event: MIDNIGHT MIRAGE: New Years Eve @Vision Lounge\r\n* Sales end soonWHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTYToday at 9:00 PMBamboo atlantaSave this event: WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTYShare this event: WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTYSales end soonWHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTYToday at 9:00 PMBamboo atlantaShare this event: WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTYSave this event: WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY\r\n* Sales end soonNYE @ 840Today at 10:00 PM840 Ralph David Abernathy Blvd SWSave this event: NYE @ 840Share this event: NYE @ 840Sales end soonNYE @ 840Today at 10:00 PM840 Ralph David Abernathy Blvd SWShare this event: NYE @ 840Save this event: NYE @ 840\r\n\r\n[](https://www.eventbrite.com/e/juelz-the-15th-annual-world-famous-1-iris-nye-extravaganza-40tics-left-tickets-1069505522809?aff=ebdssbdestsearch)![JUELZ  @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left! primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F889253813%2F1548643990873%2F1%2Foriginal.20241031-214045?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=230\u0026amp;w=460\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=f9d1e02a9b8ba29809b8f1e022dd2ae6)\r\nSales end soon\r\n\r\n[JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!](https://www.eventbrite.com/e/juelz-the-15th-annual-world-famous-1-iris-nye-extravaganza-40tics-left-tickets-1069505522809?aff=ebdssbdestsearch)### JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!\r\n\r\nToday at 9:00 PM\r\n\r\nBelieve Music Hall\r\n\r\nFrom $0.00\r\n\r\nBelieve Music Hall\r\n\r\n**Promoted\r\n\r\n***Save this event: JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!**Share this event: JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!**Promoted event actions*[](https://www.eventbrite.com/e/juelz-the-15th-annual-world-famous-1-iris-nye-extravaganza-40tics-left-tickets-1069505522809?aff=ebdssbdestsearch)[](https://www.eventbrite.com/e/juelz-the-15th-annual-world-famous-1-iris-nye-extravaganza-40tics-left-tickets-1069505522809?aff=ebdssbdestsearch)![JUELZ  @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left! primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F889253813%2F1548643990873%2F1%2Foriginal.20241031-214045?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=230\u0026amp;w=460\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=f9d1e02a9b8ba29809b8f1e022dd2ae6)\r\nSales end soon\r\n\r\n[JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!](https://www.eventbrite.com/e/juelz-the-15th-annual-world-famous-1-iris-nye-extravaganza-40tics-left-tickets-1069505522809?aff=ebdssbdestsearch)### JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!\r\n\r\nToday at 9:00 PM\r\n\r\nBelieve Music Hall\r\n\r\nFrom $0.00\r\n\r\nPromoted\r\n\r\n***Share this event: JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!**Save this event: JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!**Promoted event actions*[](https://www.eventbrite.com/e/juelz-the-15th-annual-world-famous-1-iris-nye-extravaganza-40tics-left-tickets-1069505522809?aff=ebdssbdestsearch)[](https://www.eventbrite.com/e/nye-2025-the-black-out-tickets-1123319722529?aff=ebdssbdestsearch)![NYE 2025 THE BLACK OUT primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F922860103%2F1512321410243%2F1%2Foriginal.20241224-025052?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=230\u0026amp;w=460\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.520744453925\u0026amp;fp-y=0.537937743191\u0026amp;s=8c8b36c18579bf9effc16e48efc1154f)\r\nSales end soon\r\n\r\n[NYE 2025 THE BLACK OUT](https://www.eventbrite.com/e/nye-2025-the-black-out-tickets-1123319722529?aff=ebdssbdestsearch)### NYE 2025 THE BLACK OUT\r\n\r\nToday at 9:00 PM\r\n\r\n990 Brady Ave\r\n\r\nFrom $50.00\r\n\r\n@OFFICIALOPIUMATLANTA\r\n\r\n**Promoted\r\n\r\n***Save this event: NYE 2025 THE BLACK OUT**Share this event: NYE 2025 THE BLACK OUT**Promoted event actions*[](https://www.eventbrite.com/e/nye-2025-the-black-out-tickets-1123319722529?aff=ebdssbdestsearch)[](https://www.eventbrite.com/e/nye-2025-the-black-out-tickets-1123319722529?aff=ebdssbdestsearch)![NYE 2025 THE BLACK OUT primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F922860103%2F1512321410243%2F1%2Foriginal.20241224-025052?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=230\u0026amp;w=460\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.520744453925\u0026amp;fp-y=0.537937743191\u0026amp;s=8c8b36c18579bf9effc16e48efc1154f)\r\nSales end soon\r\n\r\n[NYE 2025 THE BLACK OUT](https://www.eventbrite.com/e/nye-2025-the-black-out-tickets-1123319722529?aff=ebdssbdestsearch)### NYE 2025 THE BLACK OUT\r\n\r\nToday at 9:00 PM\r\n\r\n990 Brady Ave\r\n\r\nFrom $50.00\r\n\r\nPromoted\r\n\r\n***Share this event: NYE 2025 THE BLACK OUT**Save this event: NYE 2025 THE BLACK OUT**Promoted event actions*[](https://www.eventbrite.com/e/nye-2025-the-black-out-tickets-1123319722529?aff=ebdssbdestsearch)[](https://www.eventbrite.com/e/times-square-in-atl-new-years-eve-tickets-1089364381189?aff=ebdssbdestsearch)![Times Square in ATL | New Years Eve primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F924566503%2F68193446877%2F1%2Foriginal.20241229-193011?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=230\u0026amp;w=460\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.518613801453\u0026amp;fp-y=0.970817120623\u0026amp;s=1fb2f5f2499d352a57f285f81c62262c)\r\nAlmost full\r\n\r\n[Times Square in ATL | New Years Eve](https://www.eventbrite.com/e/times-square-in-atl-new-years-eve-tickets-1089364381189?aff=ebdssbdestsearch)### Times Square in ATL | New Years Eve\r\n\r\nToday at 9:00 PM\r\n\r\nDOMAINE ATL\r\n\r\nFrom $0.00\r\n\r\n**2 for 1 deal\r\n\r\nDOMAINE ATL\r\n\r\n**Promoted\r\n\r\n***Save this event: Times Square in ATL | New Years Eve**Share this event: Times Square in ATL | New Years Eve**Promoted event actions*[](https://www.eventbrite.com/e/times-square-in-atl-new-years-eve-tickets-1089364381189?aff=ebdssbdestsearch)[](https://www.eventbrite.com/e/times-square-in-atl-new-years-eve-tickets-1089364381189?aff=ebdssbdestsearch)![Times Square in ATL | New Years Eve primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F924566503%2F68193446877%2F1%2Foriginal.20241229-193011?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=230\u0026amp;w=460\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.518613801453\u0026amp;fp-y=0.970817120623\u0026amp;s=1fb2f5f2499d352a57f285f81c62262c)\r\nAlmost full\r\n\r\n[Times Square in ATL | New Years Eve](https://www.eventbrite.com/e/times-square-in-atl-new-years-eve-tickets-1089364381189?aff=ebdssbdestsearch)### Times Square in ATL | New Years Eve\r\n\r\nToday at 9:00 PM\r\n\r\nDOMAINE ATL\r\n\r\nFrom $0.00\r\n\r\n**2 for 1 deal\r\n\r\nPromoted\r\n\r\n***Share this event: Times Square in ATL | New Years Eve**Save this event: Times Square in ATL | New Years Eve**Promoted event actions*[](https://www.eventbrite.com/e/times-square-in-atl-new-years-eve-tickets-1089364381189?aff=ebdssbdestsearch)[](https://www.eventbrite.com/e/hugel-new-years-eve-make-the-girls-dance-2024-district-atlanta-tickets-1013146738167?aff=ebdssbdestsearch)![HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District Atlanta primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F847414849%2F78855004999%2F1%2Foriginal.20240911-104757?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=03c3b3695114e2467d6971c47e3202fb)\r\nSales end soon\r\n\r\n[HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District Atlanta](https://www.eventbrite.com/e/hugel-new-years-eve-make-the-girls-dance-2024-district-atlanta-tickets-1013146738167?aff=ebdssbdestsearch)### HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District Atlanta\r\n\r\nToday at 9:00 PM\r\n\r\nDistrict Atlanta\r\n\r\n*Save this event: HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District Atlanta**Share this event: HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District Atlanta*[](https://www.eventbrite.com/e/hugel-new-years-eve-make-the-girls-dance-2024-district-atlanta-tickets-1013146738167?aff=ebdssbdestsearch)![HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District Atlanta primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F847414849%2F78855004999%2F1%2Foriginal.20240911-104757?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=03c3b3695114e2467d6971c47e3202fb)\r\nSales end soon\r\n\r\n[HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District Atlanta](https://www.eventbrite.com/e/hugel-new-years-eve-make-the-girls-dance-2024-district-atlanta-tickets-1013146738167?aff=ebdssbdestsearch)### HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District Atlanta\r\n\r\nToday at 9:00 PM\r\n\r\nDistrict Atlanta\r\n\r\n*Share this event: HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District Atlanta**Save this event: HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District Atlanta*[](https://www.eventbrite.com/e/nye-celebration-daiquiriville-tickets-1014766452777?aff=ebdssbdestsearch)![NYE CELEBRATION @ DAIQUIRIVILLE primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F917903283%2F244640390684%2F1%2Foriginal.20241216-013015?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=400\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.0051832460733\u0026amp;fp-y=0.000700636942675\u0026amp;s=5d586d82a4ec084d81dfbb6c37da1794)\r\nAlmost full\r\n\r\n[NYE CELEBRATION @ DAIQUIRIVILLE](https://www.eventbrite.com/e/nye-celebration-daiquiriville-tickets-1014766452777?aff=ebdssbdestsearch)### NYE CELEBRATION @ DAIQUIRIVILLE\r\n\r\nToday at 9:00 PM\r\n\r\nUnderground Atlanta\r\n\r\n*Save this event: NYE CELEBRATION @ DAIQUIRIVILLE**Share this event: NYE CELEBRATION @ DAIQUIRIVILLE*[](https://www.eventbrite.com/e/nye-celebration-daiquiriville-tickets-1014766452777?aff=ebdssbdestsearch)![NYE CELEBRATION @ DAIQUIRIVILLE primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F917903283%2F244640390684%2F1%2Foriginal.20241216-013015?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=400\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.0051832460733\u0026amp;fp-y=0.000700636942675\u0026amp;s=5d586d82a4ec084d81dfbb6c37da1794)\r\nAlmost full\r\n\r\n[NYE CELEBRATION @ DAIQUIRIVILLE](https://www.eventbrite.com/e/nye-celebration-daiquiriville-tickets-1014766452777?aff=ebdssbdestsearch)### NYE CELEBRATION @ DAIQUIRIVILLE\r\n\r\nToday at 9:00 PM\r\n\r\nUnderground Atlanta\r\n\r\n*Share this event: NYE CELEBRATION @ DAIQUIRIVILLE**Save this event: NYE CELEBRATION @ DAIQUIRIVILLE*[](https://www.eventbrite.com/e/pwk-new-years-line-up-tickets-1082674361159?aff=ebdssbdestsearch)![PWK NEW YEAR\u2019S LINE UP primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F922606753%2F192835437811%2F1%2Foriginal.20241223-164754?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=400\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.395287958115\u0026amp;fp-y=0.515923566879\u0026amp;s=c243c4ed94af386d1d2c47d67704d392)\r\nGoing fast\r\n\r\n[PWK NEW YEAR\u2019S LINE UP](https://www.eventbrite.com/e/pwk-new-years-line-up-tickets-1082674361159?aff=ebdssbdestsearch)### PWK NEW YEAR\u2019S LINE UP\r\n\r\nToday at 9:00 PM\r\n\r\n1453 Cleveland Ave SW\r\n\r\n*Save this event: PWK NEW YEAR\u2019S LINE UP**Share this event: PWK NEW YEAR\u2019S LINE UP*[](https://www.eventbrite.com/e/pwk-new-years-line-up-tickets-1082674361159?aff=ebdssbdestsearch)![PWK NEW YEAR\u2019S LINE UP primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F922606753%2F192835437811%2F1%2Foriginal.20241223-164754?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=400\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.395287958115\u0026amp;fp-y=0.515923566879\u0026amp;s=c243c4ed94af386d1d2c47d67704d392)\r\nGoing fast\r\n\r\n[PWK NEW YEAR\u2019S LINE UP](https://www.eventbrite.com/e/pwk-new-years-line-up-tickets-1082674361159?aff=ebdssbdestsearch)### PWK NEW YEAR\u2019S LINE UP\r\n\r\nToday at 9:00 PM\r\n\r\n1453 Cleveland Ave SW\r\n\r\n*Share this event: PWK NEW YEAR\u2019S LINE UP**Save this event: PWK NEW YEAR\u2019S LINE UP*[](https://www.eventbrite.com/e/lemon-drop-2k25-new-years-eve-tickets-1097731306889?aff=ebdssbdestsearch)![LEMON DROP 2K25 - NEW YEAR\u2019S EVE primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F909465433%2F153161972037%2F1%2Foriginal.20241201-220959?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=400\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=5e-05\u0026amp;fp-y=5e-05\u0026amp;s=42c0bf1a0d808120f90dcc15bd62c9c3)\r\n[LEMON DROP 2K25 - NEW YEAR\u2019S EVE](https://www.eventbrite.com/e/lemon-drop-2k25-new-years-eve-tickets-1097731306889?aff=ebdssbdestsearch)### LEMON DROP 2K25 - NEW YEAR\u2019S EVE\r\n\r\nToday at 8:30 PM\r\n\r\nWestside Cultural Arts Center\r\n\r\n*Save this event: LEMON DROP 2K25 - NEW YEAR\u2019S EVE**Share this event: LEMON DROP 2K25 - NEW YEAR\u2019S EVE*[](https://www.eventbrite.com/e/lemon-drop-2k25-new-years-eve-tickets-1097731306889?aff=ebdssbdestsearch)![LEMON DROP 2K25 - NEW YEAR\u2019S EVE primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F909465433%2F153161972037%2F1%2Foriginal.20241201-220959?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=400\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=5e-05\u0026amp;fp-y=5e-05\u0026amp;s=42c0bf1a0d808120f90dcc15bd62c9c3)\r\n[LEMON DROP 2K25 - NEW YEAR\u2019S EVE](https://www.eventbrite.com/e/lemon-drop-2k25-new-years-eve-tickets-1097731306889?aff=ebdssbdestsearch)### LEMON DROP 2K25 - NEW YEAR\u2019S EVE\r\n\r\nToday at 8:30 PM\r\n\r\nWestside Cultural Arts Center\r\n\r\n*Share this event: LEMON DROP 2K25 - NEW YEAR\u2019S EVE**Save this event: LEMON DROP 2K25 - NEW YEAR\u2019S EVE*[](https://www.eventbrite.com/e/king-of-diamonds-new-years-eve-party-tickets-1112942092759?aff=ebdssbdestsearch)![King of Diamonds New Years Eve Party! primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F916522143%2F521482300093%2F1%2Foriginal.20241212-185902?w=200\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;s=736714a81a784dc9056d5619acbbd81e)\r\nSales end soon\r\n\r\n[King of Diamonds New Years Eve Party!](https://www.eventbrite.com/e/king-of-diamonds-new-years-eve-party-tickets-1112942092759?aff=ebdssbdestsearch)### King of Diamonds New Years Eve Party!\r\n\r\nToday at 9:00 PM\r\n\r\nKING OF DIAMONDS ATLANTA\r\n\r\n*Save this event: King of Diamonds New Years Eve Party!**Share this event: King of Diamonds New Years Eve Party!*[](https://www.eventbrite.com/e/king-of-diamonds-new-years-eve-party-tickets-1112942092759?aff=ebdssbdestsearch)![King of Diamonds New Years Eve Party! primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F916522143%2F521482300093%2F1%2Foriginal.20241212-185902?w=200\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;s=736714a81a784dc9056d5619acbbd81e)\r\nSales end soon\r\n\r\n[King of Diamonds New Years Eve Party!](https://www.eventbrite.com/e/king-of-diamonds-new-years-eve-party-tickets-1112942092759?aff=ebdssbdestsearch)### King of Diamonds New Years Eve Party!\r\n\r\nToday at 9:00 PM\r\n\r\nKING OF DIAMONDS ATLANTA\r\n\r\n*Share this event: King of Diamonds New Years Eve Party!**Save this event: King of Diamonds New Years Eve Party!*[](https://www.eventbrite.com/e/the-lemon-drop-new-years-eve-2k24-tickets-1100392255859?aff=ebdssbdestsearch)![THE LEMON DROP - NEW YEARS EVE 2K24 primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F909397693%2F319182551635%2F1%2Foriginal.20241201-191359?h=200\u0026amp;w=400\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;s=0bf50d785c58675ec9c988b8028240db)\r\nSales end soon\r\n\r\n[THE LEMON DROP - NEW YEARS EVE 2K24](https://www.eventbrite.com/e/the-lemon-drop-new-years-eve-2k24-tickets-1100392255859?aff=ebdssbdestsearch)### THE LEMON DROP - NEW YEARS EVE 2K24\r\n\r\nToday at 9:00 PM\r\n\r\nWestside Cultural Arts Center\r\n\r\n*Save this event: THE LEMON DROP - NEW YEARS EVE 2K24**Share this event: THE LEMON DROP - NEW YEARS EVE 2K24*[](https://www.eventbrite.com/e/the-lemon-drop-new-years-eve-2k24-tickets-1100392255859?aff=ebdssbdestsearch)![THE LEMON DROP - NEW YEARS EVE 2K24 primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F909397693%2F319182551635%2F1%2Foriginal.20241201-191359?h=200\u0026amp;w=400\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;s=0bf50d785c58675ec9c988b8028240db)\r\nSales end soon\r\n\r\n[THE LEMON DROP - NEW YEARS EVE 2K24](https://www.eventbrite.com/e/the-lemon-drop-new-years-eve-2k24-tickets-1100392255859?aff=ebdssbdestsearch)### THE LEMON DROP - NEW YEARS EVE 2K24\r\n\r\nToday at 9:00 PM\r\n\r\nWestside Cultural Arts Center\r\n\r\n*Share this event: THE LEMON DROP - NEW YEARS EVE 2K24**Save this event: THE LEMON DROP - NEW YEARS EVE 2K24*[](https://www.eventbrite.com/e/the-greatest-nye-on-earth-revel-atlanta-tickets-1090298114009?aff=ebdssbdestsearch)![THE GREATEST NYE ON EARTH REVEL ATLANTA primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F903438533%2F575490477329%2F1%2Foriginal.20241120-224901?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=400\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=e8592d077d0a4d5fcb4379794ff6a0a7)\r\nAlmost full\r\n\r\n[THE GREATEST NYE ON EARTH REVEL ATLANTA](https://www.eventbrite.com/e/the-greatest-nye-on-earth-revel-atlanta-tickets-1090298114009?aff=ebdssbdestsearch)### THE GREATEST NYE ON EARTH REVEL ATLANTA\r\n\r\nToday at 9:00 PM\r\n\r\nREVEL ATLANTA\r\n\r\n*Save this event: THE GREATEST NYE ON EARTH REVEL ATLANTA**Share this event: THE GREATEST NYE ON EARTH REVEL ATLANTA*[](https://www.eventbrite.com/e/the-greatest-nye-on-earth-revel-atlanta-tickets-1090298114009?aff=ebdssbdestsearch)![THE GREATEST NYE ON EARTH REVEL ATLANTA primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F903438533%2F575490477329%2F1%2Foriginal.20241120-224901?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=400\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=e8592d077d0a4d5fcb4379794ff6a0a7)\r\nAlmost full\r\n\r\n[THE GREATEST NYE ON EARTH REVEL ATLANTA](https://www.eventbrite.com/e/the-greatest-nye-on-earth-revel-atlanta-tickets-1090298114009?aff=ebdssbdestsearch)### THE GREATEST NYE ON EARTH REVEL ATLANTA\r\n\r\nToday at 9:00 PM\r\n\r\nREVEL ATLANTA\r\n\r\n*Share this event: THE GREATEST NYE ON EARTH REVEL ATLANTA**Save this event: THE GREATEST NYE ON EARTH REVEL ATLANTA*[](https://www.eventbrite.com/e/new-years-eve-atlanta-2025-bamboo-atlanta-tickets-1102665866289?aff=ebdssbdestsearch)![NEW YEARS EVE ATLANTA 2025 | BAMBOO ATLANTA primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F910248183%2F186860407339%2F1%2Foriginal.20241202-225936?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=400\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.49053030303\u0026amp;fp-y=0.65953307393\u0026amp;s=dcd07244957fdb73b63c84d02813c1ae)\r\nGoing fast\r\n\r\n[NEW YEARS EVE ATLANTA 2025 | BAMBOO ATLANTA](https://www.eventbrite.com/e/new-years-eve-atlanta-2025-bamboo-atlanta-tickets-1102665866289?aff=ebdssbdestsearch)### NEW YEARS EVE ATLANTA 2025 | BAMBOO ATLANTA\r\n\r\nToday at 9:00 PM\r\n\r\nBamboo atlanta\r\n\r\n*Save this event: NEW YEARS EVE ATLANTA 2025 | BAMBOO ATLANTA**Share this event: NEW YEARS EVE ATLANTA 2025 | BAMBOO ATLANTA*[](https://www.eventbrite.com/e/new-years-eve-atlanta-2025-bamboo-atlanta-tickets-1102665866289?aff=ebdssbdestsearch)![NEW YEARS EVE ATLANTA 2025 | BAMBOO ATLANTA primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F910248183%2F186860407339%2F1%2Foriginal.20241202-225936?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=400\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.49053030303\u0026amp;fp-y=0.65953307393\u0026amp;s=dcd07244957fdb73b63c84d02813c1ae)\r\nGoing fast\r\n\r\n[NEW YEARS EVE ATLANTA 2025 | BAMBOO ATLANTA](https://www.eventbrite.com/e/new-years-eve-atlanta-2025-bamboo-atlanta-tickets-1102665866289?aff=ebdssbdestsearch)### NEW YEARS EVE ATLANTA 2025 | BAMBOO ATLANTA\r\n\r\nToday at 9:00 PM\r\n\r\nBamboo atlanta\r\n\r\n*Share this event: NEW YEARS EVE ATLANTA 2025 | BAMBOO ATLANTA**Save this event: NEW YEARS EVE ATLANTA 2025 | BAMBOO ATLANTA*[](https://www.eventbrite.com/e/beer-on-a-champagne-night-new-years-eve-party-tickets-1089945539449?aff=ebdssbdestsearch)![Beer on a Champagne Night: New Year\u0026#x27;s Eve Party primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F904278483%2F399828225181%2F1%2Foriginal.20241121-234947?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=400\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.0416666666667\u0026amp;fp-y=0.544776119403\u0026amp;s=4ab678d38e0babf06a2499f93092f7a8)\r\nAlmost full\r\n\r\n[Beer on a Champagne Night: New Year\u0026#x27;s Eve Party](https://www.eventbrite.com/e/beer-on-a-champagne-night-new-years-eve-party-tickets-1089945539449?aff=ebdssbdestsearch)### Beer on a Champagne Night: New Year\u0026#x27;s Eve Party\r\n\r\nToday at 8:00 PM\r\n\r\nNew Realm Brewing Co.\r\n\r\n*Save this event: Beer on a Champagne Night: New Year\u0026#x27;s Eve Party**Share this event: Beer on a Champagne Night: New Year\u0026#x27;s Eve Party*[](https://www.eventbrite.com/e/beer-on-a-champagne-night-new-years-eve-party-tickets-1089945539449?aff=ebdssbdestsearch)![Beer on a Champagne Night: New Year\u0026#x27;s Eve Party primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F904278483%2F399828225181%2F1%2Foriginal.20241121-234947?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=400\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.0416666666667\u0026amp;fp-y=0.544776119403\u0026amp;s=4ab678d38e0babf06a2499f93092f7a8)\r\nAlmost full\r\n\r\n[Beer on a Champagne Night: New Year\u0026#x27;s Eve Party](https://www.eventbrite.com/e/beer-on-a-champagne-night-new-years-eve-party-tickets-1089945539449?aff=ebdssbdestsearch)### Beer on a Champagne Night: New Year\u0026#x27;s Eve Party\r\n\r\nToday at 8:00 PM\r\n\r\nNew Realm Brewing Co.\r\n\r\n*Share this event: Beer on a Champagne Night: New Year\u0026#x27;s Eve Party**Save this event: Beer on a Champagne Night: New Year\u0026#x27;s Eve Party*[](https://www.eventbrite.com/e/havana-club-atl-nye-2025-party-tickets-1097373496669?aff=ebdssbdestsearch)![Havana Club ATL - NYE 2025 Party primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F907636203%2F19569498248%2F1%2Foriginal.20241127-163057?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.482954545455\u0026amp;fp-y=0.5\u0026amp;s=61dba24f530a85ef016700fbd166ed61)\r\n[Havana Club ATL - NYE 2025 Party](https://www.eventbrite.com/e/havana-club-atl-nye-2025-party-tickets-1097373496669?aff=ebdssbdestsearch)### Havana Club ATL - NYE 2025 Party\r\n\r\nToday at 8:00 PM\r\n\r\nHavana Club ATL\r\n\r\n*Save this event: Havana Club ATL - NYE 2025 Party**Share this event: Havana Club ATL - NYE 2025 Party*[](https://www.eventbrite.com/e/havana-club-atl-nye-2025-party-tickets-1097373496669?aff=ebdssbdestsearch)![Havana Club ATL - NYE 2025 Party primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F907636203%2F19569498248%2F1%2Foriginal.20241127-163057?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.482954545455\u0026amp;fp-y=0.5\u0026amp;s=61dba24f530a85ef016700fbd166ed61)\r\n[Havana Club ATL - NYE 2025 Party](https://www.eventbrite.com/e/havana-club-atl-nye-2025-party-tickets-1097373496669?aff=ebdssbdestsearch)### Havana Club ATL - NYE 2025 Party\r\n\r\nToday at 8:00 PM\r\n\r\nHavana Club ATL\r\n\r\n*Share this event: Havana Club ATL - NYE 2025 Party**Save this event: Havana Club ATL - NYE 2025 Party*[](https://www.eventbrite.com/e/juelz-the-15th-annual-world-famous-1-iris-nye-extravaganza-40tics-left-tickets-1069505522809?aff=ebdssbdestsearch)![JUELZ  @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left! primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F889253813%2F1548643990873%2F1%2Foriginal.20241031-214045?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=734ed9bdd52b2ce8138858cb9989616a)\r\nSales end soon\r\n\r\n[JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!](https://www.eventbrite.com/e/juelz-the-15th-annual-world-famous-1-iris-nye-extravaganza-40tics-left-tickets-1069505522809?aff=ebdssbdestsearch)### JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!\r\n\r\nToday at 9:00 PM\r\n\r\nBelieve Music Hall\r\n\r\n*Save this event: JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!**Share this event: JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!*[](https://www.eventbrite.com/e/juelz-the-15th-annual-world-famous-1-iris-nye-extravaganza-40tics-left-tickets-1069505522809?aff=ebdssbdestsearch)![JUELZ  @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left! primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F889253813%2F1548643990873%2F1%2Foriginal.20241031-214045?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=734ed9bdd52b2ce8138858cb9989616a)\r\nSales end soon\r\n\r\n[JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!](https://www.eventbrite.com/e/juelz-the-15th-annual-world-famous-1-iris-nye-extravaganza-40tics-left-tickets-1069505522809?aff=ebdssbdestsearch)### JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!\r\n\r\nToday at 9:00 PM\r\n\r\nBelieve Music Hall\r\n\r\n*Share this event: JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!**Save this event: JUELZ @The 15th Annual World Famous #1 IRIS NYE Extravaganza \u0026lt;40tics left!*[](https://www.eventbrite.com/e/nye-great-gatsby-gala-tickets-1112750579939?aff=ebdssbdestsearch)![NYE Great Gatsby Gala primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F916367683%2F1112551372543%2F1%2Foriginal.20241212-153107?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=cc09dc750bcf71cb69ddaa25f489b8a8)\r\nSales end soon\r\n\r\n[NYE Great Gatsby Gala](https://www.eventbrite.com/e/nye-great-gatsby-gala-tickets-1112750579939?aff=ebdssbdestsearch)### NYE Great Gatsby Gala\r\n\r\nToday at 8:00 PM\r\n\r\nBrewDog Atlanta\r\n\r\n*Save this event: NYE Great Gatsby Gala**Share this event: NYE Great Gatsby Gala*[](https://www.eventbrite.com/e/nye-great-gatsby-gala-tickets-1112750579939?aff=ebdssbdestsearch)![NYE Great Gatsby Gala primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F916367683%2F1112551372543%2F1%2Foriginal.20241212-153107?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=cc09dc750bcf71cb69ddaa25f489b8a8)\r\nSales end soon\r\n\r\n[NYE Great Gatsby Gala](https://www.eventbrite.com/e/nye-great-gatsby-gala-tickets-1112750579939?aff=ebdssbdestsearch)### NYE Great Gatsby Gala\r\n\r\nToday at 8:00 PM\r\n\r\nBrewDog Atlanta\r\n\r\n*Share this event: NYE Great Gatsby Gala**Save this event: NYE Great Gatsby Gala*[](https://www.eventbrite.com/e/free-new-years-eve-party-in-atlanta-tickets-1114576591589?aff=ebdssbdestsearch)![FREE NEW YEARS EVE PARTY IN ATLANTA primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F923573453%2F153161972037%2F1%2Foriginal.20241227-004036?h=200\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;s=7231cc3c0ba9d5a250e2a7e7620303b3)\r\nSales end soon\r\n\r\n[FREE NEW YEARS EVE PARTY IN ATLANTA](https://www.eventbrite.com/e/free-new-years-eve-party-in-atlanta-tickets-1114576591589?aff=ebdssbdestsearch)### FREE NEW YEARS EVE PARTY IN ATLANTA\r\n\r\nToday at 10:00 PM\r\n\r\nLyfe ATL\r\n\r\n*Save this event: FREE NEW YEARS EVE PARTY IN ATLANTA**Share this event: FREE NEW YEARS EVE PARTY IN ATLANTA*[](https://www.eventbrite.com/e/free-new-years-eve-party-in-atlanta-tickets-1114576591589?aff=ebdssbdestsearch)![FREE NEW YEARS EVE PARTY IN ATLANTA primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F923573453%2F153161972037%2F1%2Foriginal.20241227-004036?h=200\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;s=7231cc3c0ba9d5a250e2a7e7620303b3)\r\nSales end soon\r\n\r\n[FREE NEW YEARS EVE PARTY IN ATLANTA](https://www.eventbrite.com/e/free-new-years-eve-party-in-atlanta-tickets-1114576591589?aff=ebdssbdestsearch)### FREE NEW YEARS EVE PARTY IN ATLANTA\r\n\r\nToday at 10:00 PM\r\n\r\nLyfe ATL\r\n\r\n*Share this event: FREE NEW YEARS EVE PARTY IN ATLANTA**Save this event: FREE NEW YEARS EVE PARTY IN ATLANTA*[](https://www.eventbrite.com/e/new-years-eve-black-billionaires-ball-atlanta-tickets-1042487617527?aff=ebdssbdestsearch)![New Years Eve \u201CBlack Billionaires Ball\u201D (Atlanta) primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F870195169%2F163760175761%2F1%2Foriginal.20241009-034418?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=6e058054c8b86335ed9970d0b4330023)\r\nSales end soon\r\n\r\n[New Years Eve \u201CBlack Billionaires Ball\u201D (Atlanta)](https://www.eventbrite.com/e/new-years-eve-black-billionaires-ball-atlanta-tickets-1042487617527?aff=ebdssbdestsearch)### New Years Eve \u201CBlack Billionaires Ball\u201D (Atlanta)\r\n\r\nToday at 9:00 PM\r\n\r\nWestside Cultural Arts Center, 10th Street Northwest, Atlanta, GA, USA\r\n\r\n*Save this event: New Years Eve \u201CBlack Billionaires Ball\u201D (Atlanta)**Share this event: New Years Eve \u201CBlack Billionaires Ball\u201D (Atlanta)*[](https://www.eventbrite.com/e/new-years-eve-black-billionaires-ball-atlanta-tickets-1042487617527?aff=ebdssbdestsearch)![New Years Eve \u201CBlack Billionaires Ball\u201D (Atlanta) primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F870195169%2F163760175761%2F1%2Foriginal.20241009-034418?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=6e058054c8b86335ed9970d0b4330023)\r\nSales end soon\r\n\r\n[New Years Eve \u201CBlack Billionaires Ball\u201D (Atlanta)](https://www.eventbrite.com/e/new-years-eve-black-billionaires-ball-atlanta-tickets-1042487617527?aff=ebdssbdestsearch)### New Years Eve \u201CBlack Billionaires Ball\u201D (Atlanta)\r\n\r\nToday at 9:00 PM\r\n\r\nWestside Cultural Arts Center, 10th Street Northwest, Atlanta, GA, USA\r\n\r\n*Share this event: New Years Eve \u201CBlack Billionaires Ball\u201D (Atlanta)**Save this event: New Years Eve \u201CBlack Billionaires Ball\u201D (Atlanta)*[](https://www.eventbrite.com/e/new-years-eve-ball-tickets-1078976911989?aff=ebdssbdestsearch)![NEW YEARS EVE BALL primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F909586733%2F2433003307711%2F1%2Foriginal.20241202-034121?crop=focalpoint\u0026amp;fit=crop\u0026amp;w=200\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=1925d71f19899ebe88e224d154166ac8)\r\nGoing fast\r\n\r\n[NEW YEARS EVE BALL](https://www.eventbrite.com/e/new-years-eve-ball-tickets-1078976911989?aff=ebdssbdestsearch)### NEW YEARS EVE BALL\r\n\r\nToday at 10:00 PM\r\n\r\n2339 Brannen Rd SE\r\n\r\n*Save this event: NEW YEARS EVE BALL**Share this event: NEW YEARS EVE BALL*[](https://www.eventbrite.com/e/new-years-eve-ball-tickets-1078976911989?aff=ebdssbdestsearch)![NEW YEARS EVE BALL primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F909586733%2F2433003307711%2F1%2Foriginal.20241202-034121?crop=focalpoint\u0026amp;fit=crop\u0026amp;w=200\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=1925d71f19899ebe88e224d154166ac8)\r\nGoing fast\r\n\r\n[NEW YEARS EVE BALL](https://www.eventbrite.com/e/new-years-eve-ball-tickets-1078976911989?aff=ebdssbdestsearch)### NEW YEARS EVE BALL\r\n\r\nToday at 10:00 PM\r\n\r\n2339 Brannen Rd SE\r\n\r\n*Share this event: NEW YEARS EVE BALL**Save this event: NEW YEARS EVE BALL*[](https://www.eventbrite.com/e/nye-25-free-entry-w-rsvp-til-10pm-tuesday-dec-31st-at-bamboo-atl-tickets-1105113988689?aff=ebdssbdestsearch)![NYE 25 | FREE ENTRY W/ RSVP TIL 10PM | TUESDAY DEC 31ST | AT BAMBOO ATL primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F911203163%2F218107036300%2F1%2Foriginal.20241204-030451?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=400\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.488658665459\u0026amp;fp-y=0.0330739299611\u0026amp;s=26f9f96b5aa8f69e9a5bcf9cf5597bf2)\r\nAlmost full\r\n\r\n[NYE 25 | FREE ENTRY W/ RSVP TIL 10PM | TUESDAY DEC 31ST | AT BAMBOO ATL](https://www.eventbrite.com/e/nye-25-free-entry-w-rsvp-til-10pm-tuesday-dec-31st-at-bamboo-atl-tickets-1105113988689?aff=ebdssbdestsearch)### NYE 25 | FREE ENTRY W/ RSVP TIL 10PM | TUESDAY DEC 31ST | AT BAMBOO ATL\r\n\r\nToday at 9:00 PM\r\n\r\nBamboo atlanta\r\n\r\n*Save this event: NYE 25 | FREE ENTRY W/ RSVP TIL 10PM | TUESDAY DEC 31ST | AT BAMBOO ATL**Share this event: NYE 25 | FREE ENTRY W/ RSVP TIL 10PM | TUESDAY DEC 31ST | AT BAMBOO ATL*[](https://www.eventbrite.com/e/nye-25-free-entry-w-rsvp-til-10pm-tuesday-dec-31st-at-bamboo-atl-tickets-1105113988689?aff=ebdssbdestsearch)![NYE 25 | FREE ENTRY W/ RSVP TIL 10PM | TUESDAY DEC 31ST | AT BAMBOO ATL primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F911203163%2F218107036300%2F1%2Foriginal.20241204-030451?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=400\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.488658665459\u0026amp;fp-y=0.0330739299611\u0026amp;s=26f9f96b5aa8f69e9a5bcf9cf5597bf2)\r\nAlmost full\r\n\r\n[NYE 25 | FREE ENTRY W/ RSVP TIL 10PM | TUESDAY DEC 31ST | AT BAMBOO ATL](https://www.eventbrite.com/e/nye-25-free-entry-w-rsvp-til-10pm-tuesday-dec-31st-at-bamboo-atl-tickets-1105113988689?aff=ebdssbdestsearch)### NYE 25 | FREE ENTRY W/ RSVP TIL 10PM | TUESDAY DEC 31ST | AT BAMBOO ATL\r\n\r\nToday at 9:00 PM\r\n\r\nBamboo atlanta\r\n\r\n*Share this event: NYE 25 | FREE ENTRY W/ RSVP TIL 10PM | TUESDAY DEC 31ST | AT BAMBOO ATL**Save this event: NYE 25 | FREE ENTRY W/ RSVP TIL 10PM | TUESDAY DEC 31ST | AT BAMBOO ATL*[](https://www.eventbrite.com/e/peach-fest-new-years-eve-atlanta-tickets-1117011203579?aff=ebdssbdestsearch)![PEACH FEST | NEW YEARS EVE ATLANTA primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F633714409%2F186860407339%2F1%2Foriginal.20231102-155054?w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;rect=0%2C628%2C1500%2C750\u0026amp;s=5edc88a2846b197dffa235fadc584019)\r\nAlmost full\r\n\r\n[PEACH FEST | NEW YEARS EVE ATLANTA](https://www.eventbrite.com/e/peach-fest-new-years-eve-atlanta-tickets-1117011203579?aff=ebdssbdestsearch)### PEACH FEST | NEW YEARS EVE ATLANTA\r\n\r\nToday at 8:00 PM\r\n\r\n150 Central Ave SW\r\n\r\n*Save this event: PEACH FEST | NEW YEARS EVE ATLANTA**Share this event: PEACH FEST | NEW YEARS EVE ATLANTA*[](https://www.eventbrite.com/e/peach-fest-new-years-eve-atlanta-tickets-1117011203579?aff=ebdssbdestsearch)![PEACH FEST | NEW YEARS EVE ATLANTA primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F633714409%2F186860407339%2F1%2Foriginal.20231102-155054?w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;rect=0%2C628%2C1500%2C750\u0026amp;s=5edc88a2846b197dffa235fadc584019)\r\nAlmost full\r\n\r\n[PEACH FEST | NEW YEARS EVE ATLANTA](https://www.eventbrite.com/e/peach-fest-new-years-eve-atlanta-tickets-1117011203579?aff=ebdssbdestsearch)### PEACH FEST | NEW YEARS EVE ATLANTA\r\n\r\nToday at 8:00 PM\r\n\r\n150 Central Ave SW\r\n\r\n*Share this event: PEACH FEST | NEW YEARS EVE ATLANTA**Save this event: PEACH FEST | NEW YEARS EVE ATLANTA*[](https://www.eventbrite.com/e/midnight-mirage-new-years-eve-vision-lounge-tickets-1111018900439?aff=ebdssbdestsearch)![MIDNIGHT MIRAGE: New Years Eve @Vision Lounge primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F915393013%2F382110622087%2F1%2Foriginal.20241211-035851?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.489583333333\u0026amp;fp-y=0.764331210191\u0026amp;s=5b3bdf32e8681e2038346f9da251e5c7)\r\nAlmost full\r\n\r\n[MIDNIGHT MIRAGE: New Years Eve @Vision Lounge](https://www.eventbrite.com/e/midnight-mirage-new-years-eve-vision-lounge-tickets-1111018900439?aff=ebdssbdestsearch)### MIDNIGHT MIRAGE: New Years Eve @Vision Lounge\r\n\r\nToday at 9:00 PM\r\n\r\n2520 Piedmont Rd NE\r\n\r\n*Save this event: MIDNIGHT MIRAGE: New Years Eve @Vision Lounge**Share this event: MIDNIGHT MIRAGE: New Years Eve @Vision Lounge*[](https://www.eventbrite.com/e/midnight-mirage-new-years-eve-vision-lounge-tickets-1111018900439?aff=ebdssbdestsearch)![MIDNIGHT MIRAGE: New Years Eve @Vision Lounge primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F915393013%2F382110622087%2F1%2Foriginal.20241211-035851?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.489583333333\u0026amp;fp-y=0.764331210191\u0026amp;s=5b3bdf32e8681e2038346f9da251e5c7)\r\nAlmost full\r\n\r\n[MIDNIGHT MIRAGE: New Years Eve @Vision Lounge](https://www.eventbrite.com/e/midnight-mirage-new-years-eve-vision-lounge-tickets-1111018900439?aff=ebdssbdestsearch)### MIDNIGHT MIRAGE: New Years Eve @Vision Lounge\r\n\r\nToday at 9:00 PM\r\n\r\n2520 Piedmont Rd NE\r\n\r\n*Share this event: MIDNIGHT MIRAGE: New Years Eve @Vision Lounge**Save this event: MIDNIGHT MIRAGE: New Years Eve @Vision Lounge*[](https://www.eventbrite.com/e/what-a-time-to-be-alive-2025-new-years-eve-party-tickets-1097769240349?aff=ebdssbdestsearch)![WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F911204773%2F153161972037%2F1%2Foriginal.20241204-030931?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=400\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.005\u0026amp;fp-y=0.005\u0026amp;s=00150476d13adba94c48dff895477ed4)\r\nSales end soon\r\n\r\n[WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY](https://www.eventbrite.com/e/what-a-time-to-be-alive-2025-new-years-eve-party-tickets-1097769240349?aff=ebdssbdestsearch)### WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY\r\n\r\nToday at 9:00 PM\r\n\r\nBamboo atlanta\r\n\r\n*Save this event: WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY**Share this event: WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY*[](https://www.eventbrite.com/e/what-a-time-to-be-alive-2025-new-years-eve-party-tickets-1097769240349?aff=ebdssbdestsearch)![WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F911204773%2F153161972037%2F1%2Foriginal.20241204-030931?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=400\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.005\u0026amp;fp-y=0.005\u0026amp;s=00150476d13adba94c48dff895477ed4)\r\nSales end soon\r\n\r\n[WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY](https://www.eventbrite.com/e/what-a-time-to-be-alive-2025-new-years-eve-party-tickets-1097769240349?aff=ebdssbdestsearch)### WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY\r\n\r\nToday at 9:00 PM\r\n\r\nBamboo atlanta\r\n\r\n*Share this event: WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY**Save this event: WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY*[](https://www.eventbrite.com/e/nye-840-tickets-1118433206829?aff=ebdssbdestsearch)![NYE  @ 840 primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F920120973%2F1799078899223%2F1%2Foriginal.20241219-035454?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=400\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=f75f0754d8f57caa88a0464eee95664b)\r\nSales end soon\r\n\r\n[NYE @ 840](https://www.eventbrite.com/e/nye-840-tickets-1118433206829?aff=ebdssbdestsearch)### NYE @ 840\r\n\r\nToday at 10:00 PM\r\n\r\n840 Ralph David Abernathy Blvd SW\r\n\r\n*Save this event: NYE @ 840**Share this event: NYE @ 840*[](https://www.eventbrite.com/e/nye-840-tickets-1118433206829?aff=ebdssbdestsearch)![NYE  @ 840 primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F920120973%2F1799078899223%2F1%2Foriginal.20241219-035454?crop=focalpoint\u0026amp;fit=crop\u0026amp;h=200\u0026amp;w=400\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=f75f0754d8f57caa88a0464eee95664b)\r\nSales end soon\r\n\r\n[NYE @ 840](https://www.eventbrite.com/e/nye-840-tickets-1118433206829?aff=ebdssbdestsearch)### NYE @ 840\r\n\r\nToday at 10:00 PM\r\n\r\n840 Ralph David Abernathy Blvd SW\r\n\r\n*Share this event: NYE @ 840**Save this event: NYE @ 840** \r\n* 1 of 32\r\n* \r\n\r\n**[Things to do around Atlanta](/ttd/ga--atlanta/)## Things to do around Atlanta\r\n\r\n[Things to do in Chattanooga](/ttd/tn--chattanooga/)[](/ttd/tn--chattanooga/)[Things to do in Huntsville](/ttd/al--huntsville/)[](/ttd/al--huntsville/)[Things to do in Birmingham](/ttd/al--birmingham/)[](/ttd/al--birmingham/)[Things to do in Montgomery](/ttd/al--montgomery/)[](/ttd/al--montgomery/)[Things to do in Greenville](/ttd/sc--greenville/)[](/ttd/sc--greenville/)[Things to do in Augusta](/ttd/ga--augusta/)[](/ttd/ga--augusta/)[Things to do in Columbus](/ttd/ga--columbus/)[](/ttd/ga--columbus/)[Things to do in Sandy Springs](/ttd/ga--sandy-springs/)[](/ttd/ga--sandy-springs/)[Things to do in Macon](/ttd/ga--macon/)[](/ttd/ga--macon/)[Things to do in Athens](/ttd/ga--athens/)[](/ttd/ga--athens/)[Things to do in East Point](/ttd/ga--east-point/)[](/ttd/ga--east-point/)[Things to do in Woodstock](/ttd/ga--woodstock/)[](/ttd/ga--woodstock/)## Trends in Atlanta\r\n\r\n[Trends in Atlanta](/trending/searches/ga--atlanta/)**[Explore more trends](/trending/searches/ga--atlanta/)**[1. New years eve parties](/d/ga--atlanta/new-years-eve-parties/)### 1. New years eve parties\r\n\r\n[2. December](/d/ga--atlanta/december/)### 2. December\r\n\r\n[3. Nye events](/d/ga--atlanta/nye-events/)### 3. Nye events\r\n\r\n[4. Free christmas toys for children](/d/ga--atlanta/free-christmas-toys-for-children/)### 4. Free christmas toys for children\r\n\r\n[5. New years eve](/d/ga--atlanta/new-years-eve/)### 5. New years eve\r\n\r\n[6. Toy giveaway](/d/ga--atlanta/toy-giveaway/)### 6. Toy giveaway\r\n\r\n[7. Nye party](/d/ga--atlanta/nye-party/)### 7. Nye party\r\n\r\n[8. Comedy shows](/d/ga--atlanta/comedy-shows/)### 8. Comedy shows\r\n\r\n[9. Job fairs](/d/ga--atlanta/job-fairs/)### 9. Job fairs\r\n\r\n[10. January](/d/ga--atlanta/january/)### 10. January\r\n\r\n[11. New years eve concerts](/d/ga--atlanta/new-years-eve-concerts/)### 11. New years eve concerts\r\n\r\n[12. Christmas lights](/d/ga--atlanta/christmas-lights/)### 12. Christmas lights\r\n\r\n[Explore more trends](/trending/searches/ga--atlanta/)## Site Navigation\r\n\r\nUse Eventbrite\r\n\r\n* Create Events\r\n* Pricing\r\n* Event Marketing Platform\r\n* Eventbrite Mobile Ticket App\r\n* Eventbrite Check-In App\r\n* Eventbrite App Marketplace\r\n* Event Registration Software\r\n* Community Guidelines\r\n* FAQs\r\n* Sitemap\r\n\r\n[Create Events](https://www.eventbrite.com/organizer/overview/)[Pricing](https://www.eventbrite.com/organizer/pricing/)[Event Marketing Platform](https://www.eventbrite.com/l/event-marketing-platform/)[Eventbrite Mobile Ticket App](https://www.eventbrite.com/l/eventbrite-app/)[Eventbrite Check-In App](https://www.eventbrite.com/l/organizer-check-in-app/)[Eventbrite App Marketplace](https://www.eventbrite.com/apps/)[Event Registration Software](https://www.eventbrite.com/l/registration-online/)[Community Guidelines](https://www.eventbrite.com/l/community-guidelines/)[FAQs](https://www.eventbrite.com/l/frequently-asked-questions/)[Sitemap](https://www.eventbrite.com/sitemap/)Plan Events\r\n\r\n* Sell Tickets Online\r\n* Event Planning\r\n* Sell Concert Tickets Online\r\n* Event Payment System\r\n* Solutions for Professional Services\r\n* Event Management Software\r\n* Halloween Party Planning\r\n* Virtual Events Platform\r\n* QR Codes for Event Check-In\r\n* Post your event online\r\n\r\n[Sell Tickets Online](https://www.eventbrite.com/l/sell-tickets/)[Event Planning](https://www.eventbrite.com/l/planning-an-event/)[Sell Concert Tickets Online](https://www.eventbrite.com/l/music-venues/)[Event Payment System](https://www.eventbrite.com/l/event-payment/)[Solutions for Professional Services](https://www.eventbrite.com/l/professional-services/)[Event Management Software](https://www.eventbrite.com/l/event-management-software/)[Halloween Party Planning](https://www.eventbrite.com/l/halloween-event-planning/)[Virtual Events Platform](https://www.eventbrite.com/l/virtual-events-platform/)[QR Codes for Event Check-In](https://www.eventbrite.com/l/how-to-use-qr-codes-for-events/)[Post your event online](https://www.eventbrite.com/l/post-events/)Find Events\r\n\r\n* New Orleans Food \u0026amp; Drink Events\r\n* San Francisco Holiday Events\r\n* Tulum Music Events\r\n* Denver Hobby Events\r\n* Atlanta Pop Music Events\r\n* New York Events\r\n* Chicago Events\r\n* Events in Dallas Today\r\n* Los Angeles Events\r\n* Washington Events\r\n\r\n[New Orleans Food \u0026amp; Drink Events](https://www.eventbrite.com/b/la--new-orleans/food-and-drink/)[San Francisco Holiday Events](https://www.eventbrite.com/b/ca--san-francisco/holiday/)[Tulum Music Events](https://www.eventbrite.com/b/mexico--tulum/music/)[Denver Hobby Events](https://www.eventbrite.com/b/co--denver/hobbies/)[Atlanta Pop Music Events](https://www.eventbrite.com/b/ga--atlanta/music/pop/)[New York Events](https://www.eventbrite.com/d/ny--new-york/events/)[Chicago Events](https://www.eventbrite.com/d/il--chicago/events/)[Events in Dallas Today](https://www.eventbrite.com/d/tx--dallas/events--today/)[Los Angeles Events](https://www.eventbrite.com/d/ca--los-angeles/events/)[Washington Events](https://www.eventbrite.com/d/dc--washington/events/)Connect With Us\r\n\r\n* Contact Support\r\n* Contact Sales\r\n* X\r\n* Facebook\r\n* LinkedIn\r\n* Instagram\r\n* TikTok\r\n\r\n[Contact Support](https://www.eventbrite.com/help/en-us/contact-us/)[Contact Sales](https://www.eventbrite.com/l/contact-eventbrite-sales/)[X](https://www.x.com/eventbritehelp)[Facebook](https://www.facebook.com/Eventbrite)[LinkedIn](https://www.linkedin.com/company/eventbrite)[Instagram](https://www.instagram.com/eventbrite)[TikTok](https://www.tiktok.com/@eventbrite)* How It Works\r\n* Pricing\r\n* Contact Support\r\n* About\r\n* Blog\r\n* Help\r\n* Careers\r\n* Press\r\n* Impact\r\n* Investors\r\n* Security\r\n* Developers\r\n* Status\r\n* Terms\r\n* Privacy\r\n* Accessibility\r\n* Cookies\r\n* Manage Cookie Preferences\r\n\r\n[How It Works](https://www.eventbrite.com/how-it-works)[Pricing](https://www.eventbrite.com/organizer/pricing/)[Contact Support](https://www.eventbrite.com/help/en-us/contact-us/)[About](https://www.eventbrite.com/about/)[Blog](https://www.eventbrite.com/blog/)[Help](https://www.eventbrite.com/help/en-us/)[Careers](https://www.eventbritecareers.com/)[Press](https://www.eventbrite.com/blog/press/)[Impact](https://www.eventbrite.com/l/impact/)[Investors](https://investor.eventbrite.com/overview/default.aspx)[Security](https://www.eventbrite.com/security/)[Developers](https://www.eventbrite.com/platform/)[Status](https://www.eventbritestatus.com/)[Terms](https://www.eventbrite.com/l/legalterms/)[Privacy](https://www.eventbrite.com/help/en-us/articles/460838/eventbrite-privacy-policy/)[Accessibility](https://www.eventbrite.com/l/accessibility/)[Cookies](https://www.eventbrite.com/help/en-us/articles/666792/eventbrite-cookie-policy/)[Manage Cookie Preferences](#)**"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url":"https://discoveratlanta.com/events/main/"}
```
and got this result:
```
"[DiscoverAtlanta.com homepage](https://discoveratlanta.com)* Things to Do\r\n* Events\r\n* Plan Your Visit\r\n* Meetings\r\n\r\n[Things to Do](https://discoveratlanta.com/things-to-do/main/)[Events](https://discoveratlanta.com/events/main/)[Plan Your Visit](https://discoveratlanta.com/explore/main/)[Meetings](https://discoveratlanta.com/meetings/main/)* Discover Atlanta Take a Tour Neighborhoods Atlanta\u2019s Food Story Michelin Restaurants Black Travel Guide Local Guides LGBTQ Atlanta\u2019s Stories\r\n* Things to Do 50 Fun Things To Do Attractions Eat Arts \u0026#038; Culture Outdoors History \u0026#038; Heritage Shopping Nightlife Sports This Weekend\r\n* Events Festivals Virtual Events Seasonal \u0026#038; Holidays Submit Your Event\r\n* Plan Your Visit Stay Deals Itineraries Visitor Guide Getting Around View All Hotels\r\n\r\n[Discover Atlanta](https://discoveratlanta.com/)* Take a Tour\r\n* Neighborhoods\r\n* Atlanta\u2019s Food Story\r\n* Michelin Restaurants\r\n* Black Travel Guide\r\n* Local Guides\r\n* LGBTQ\r\n* Atlanta\u2019s Stories\r\n\r\n[Take a Tour](https://discoveratlanta.com/explore/take-a-tour/)[Neighborhoods](https://discoveratlanta.com/explore/neighborhoods/main/)[Atlanta\u2019s Food Story](https://discoveratlanta.com/dining/culinary-scene/)[Michelin Restaurants](https://discoveratlanta.com/michelin-guide-restaurants/)[Black Travel Guide](https://discoveratlanta.com/atl-unguided/)[Local Guides](https://discoveratlanta.com/iamatl/main/)[LGBTQ](https://discoveratlanta.com/explore/lgbt/)[Atlanta\u2019s Stories](https://discoveratlanta.com/stories/main/)[Things to Do](https://discoveratlanta.com/things-to-do/main/)* 50 Fun Things To Do\r\n* Attractions\r\n* Eat\r\n* Arts \u0026#038; Culture\r\n* Outdoors\r\n* History \u0026#038; Heritage\r\n* Shopping\r\n* Nightlife\r\n* Sports\r\n* This Weekend\r\n\r\n[50 Fun Things To Do](https://discoveratlanta.com/50fun/main/)[Attractions](https://discoveratlanta.com/things-to-do/attractions/)[Eat](https://discoveratlanta.com/dining/main/)[Arts \u0026#038; Culture](https://discoveratlanta.com/things-to-do/arts-culture/)[Outdoors](https://discoveratlanta.com/things-to-do/outdoors/)[History \u0026#038; Heritage](https://discoveratlanta.com/things-to-do/history/)[Shopping](https://discoveratlanta.com/things-to-do/shopping/)[Nightlife](https://discoveratlanta.com/things-to-do/nightlife/)[Sports](https://discoveratlanta.com/things-to-do/sports/)[This Weekend](https://discoveratlanta.com/stories/things-to-do/things-to-do-in-atlanta-this-weekend/)[Events](https://discoveratlanta.com/events/main/)* Festivals\r\n* Virtual Events\r\n* Seasonal \u0026#038; Holidays\r\n* Submit Your Event\r\n\r\n[Festivals](https://discoveratlanta.com/events/festivals/)[Virtual Events](https://discoveratlanta.com/events/virtual/)[Seasonal \u0026#038; Holidays](https://discoveratlanta.com/events/seasonal/)[Submit Your Event](https://discoveratlanta.com/add-your-event/)[Plan Your Visit](https://discoveratlanta.com/explore/main/)* Stay\r\n* Deals\r\n* Itineraries\r\n* Visitor Guide\r\n* Getting Around\r\n* View All Hotels\r\n\r\n[Stay](https://discoveratlanta.com/hotels/main/)[Deals](https://discoveratlanta.com/deals/)[Itineraries](https://discoveratlanta.com/explore/itineraries/)[Visitor Guide](https://discoveratlanta.com/explore/visitor-guides/)[Getting Around](https://discoveratlanta.com/explore/transportation/getting-around/)[View All Hotels](https://hotels.discoveratlanta.com/search/)* Meeting Planners Why Atlanta New Developments Convenient Travel Getting Around Meeting Districts Unique Facilities \u0026#038; Venues Key Industries Plan a Meeting Facilities \u0026#038; Venues Hotels Suppliers Submit Request for Proposal (RFP) Destination Services Community Engagement Promote A Meeting Planner\u2019s Timeline City Guides \u0026amp; Brochures Atlanta\u2019s Meetings Stories Contact Us \u2013 Sales\r\n* Travel Pros Tour Services Contact Us\r\n* Newsroom\r\n* Partners Employment Contact Us Become a Member Publications Directions \u0026#038; Parking\r\n\r\n[Meeting Planners](https://discoveratlanta.com/meetings/main/)* Why Atlanta New Developments Convenient Travel Getting Around Meeting Districts Unique Facilities \u0026#038; Venues Key Industries\r\n* Plan a Meeting Facilities \u0026#038; Venues Hotels Suppliers Submit Request for Proposal (RFP) Destination Services Community Engagement\r\n* Promote A Meeting Planner\u2019s Timeline City Guides \u0026amp; Brochures\r\n* Atlanta\u2019s Meetings Stories\r\n* Contact Us \u2013 Sales\r\n\r\n[Why Atlanta](https://discoveratlanta.com/meetings/why-atlanta/)* New Developments\r\n* Convenient Travel\r\n* Getting Around\r\n* Meeting Districts\r\n* Unique Facilities \u0026#038; Venues\r\n* Key Industries\r\n\r\n[New Developments](https://discoveratlanta.com/meetings/why-atlanta/new-developments/)[Convenient Travel](https://discoveratlanta.com/meetings/why-atlanta/convenient-travel/)[Getting Around](https://discoveratlanta.com/meetings/why-atlanta/walkable/)[Meeting Districts](https://discoveratlanta.com/meetings/why-atlanta/meeting-districts/)[Unique Facilities \u0026#038; Venues](https://discoveratlanta.com/meetings/facilities-and-venues/)[Key Industries](https://discoveratlanta.com/meetings/why-atlanta/key-industries/)[Plan a Meeting](https://discoveratlanta.com/meetings/plan/)* Facilities \u0026#038; Venues\r\n* Hotels\r\n* Suppliers\r\n* Submit Request for Proposal (RFP)\r\n* Destination Services\r\n* Community Engagement\r\n\r\n[Facilities \u0026#038; Venues](https://discoveratlanta.com/meetings/facilities-and-venues/search/)[Hotels](https://discoveratlanta.com/meetings/hotels/search/)[Suppliers](https://discoveratlanta.com/meetings/suppliers/search/)[Submit Request for Proposal (RFP)](https://discoveratlanta.com/meetings/rfp/)[Destination Services](https://discoveratlanta.com/meetings/plan/destination-services/)[Community Engagement](https://discoveratlanta.com/meetings/community-engagement)[Promote A Meeting](https://discoveratlanta.com/meetings/promote/)* Planner\u2019s Timeline\r\n* City Guides \u0026amp; Brochures\r\n\r\n[Planner\u2019s Timeline](https://discoveratlanta.com/meetings/timeline/)[City Guides \u0026amp; Brochures](https://discoveratlanta.com/meetings/promote/promotional-materials-and-brochures/)[Atlanta\u2019s Meetings Stories](https://discoveratlanta.com/stories/meetings/)[Contact Us \u2013 Sales](https://discoveratlanta.com/meetings/about/contact-us/)[Travel Pros](https://discoveratlanta.com/travel-tours/)* Tour Services\r\n* Contact Us\r\n\r\n[Tour Services](https://discoveratlanta.com/travel-tours/services/)[Contact Us](https://discoveratlanta.com/travel-tours/contact/)[Newsroom](https://discoveratlanta.com/news/main/)[Partners](https://discoveratlanta.com/acvb/partners/)* Employment\r\n* Contact Us\r\n* Become a Member\r\n* Publications\r\n* Directions \u0026#038; Parking\r\n\r\n[Employment](https://discoveratlanta.com/acvb/employment/)[Contact Us](https://discoveratlanta.com/acvb/contact/)[Become a Member](https://discoveratlanta.com/acvb/become-a-member/)[Publications](https://discoveratlanta.com/publications/)[Directions \u0026#038; Parking](https://discoveratlanta.com/acvb/directions/)[](#events-listing-search)![](https://res.cloudinary.com/atlanta/images/w_191,h_300,c_scale/f_auto,q_auto/v1662573960/newAtlanta.com/events-white-new/events-white-new.png?_i=AA)\r\n# Events\r\n\r\n## Unleash the fun!\r\n\r\nStay up to date on things to do and events in Atlanta today, this week and this weekend. Whether you\u2019re into live music, food and wine, sports or family-friendly entertainment, Atlanta events offer nonstop fun for everyone.\u0026nbsp;\r\n\r\nSearch the Atlanta events calendar below or explore our annual festivals and seasonal events taking place throughout the year. If you\u2019re in ATL now, make sure you see what\u2019s happening today, this week and find things to do this weekend so you don\u2019t miss any one-of-a-kind Atlanta events.\r\n\r\n[annual festivals](https://discoveratlanta.com/events/festivals/)[seasonal events](https://discoveratlanta.com/events/seasonal/)[today](https://discoveratlanta.com/events/all/?date_sort=today)[this week](https://discoveratlanta.com/events/all/?date_sort=this-week)[things to do this weekend](https://discoveratlanta.com/stories/things-to-do-in-atlanta-this-weekend/)Venues and event organizers, submit events here.\r\n\r\n[submit events here](https://discoveratlanta.com/add-your-event/)[Winter Festivals](https://discoveratlanta.com/events/festivals/winter)## Winter Festivals\r\n\r\n[READ MORE](https://discoveratlanta.com/events/festivals/winter)[Holiday Happenings](https://discoveratlanta.com/events/seasonal/christmas/)## Holiday Happenings\r\n\r\n[Find Out More](https://discoveratlanta.com/events/seasonal/christmas/)[Festivals \u0026amp; Annual Events](https://discoveratlanta.com/events/festivals/)## Festivals \u0026amp; Annual Events\r\n\r\n[LEARN MORE](https://discoveratlanta.com/events/festivals/)[Things to do In Atlanta this Weekend](https://discoveratlanta.com/stories/things-to-do/things-to-do-in-atlanta-this-weekend/)## Things to do In Atlanta this Weekend\r\n\r\n[LEARN MORE](https://discoveratlanta.com/stories/things-to-do/things-to-do-in-atlanta-this-weekend/)[Plan An Epic Road Trip To Atlanta](https://discoveratlanta.com/things-to-do/how-to-plan-an-epic-road-trip-to-atlanta/)## Plan An Epic Road Trip To Atlanta\r\n\r\n[Make Your Plans](https://discoveratlanta.com/things-to-do/how-to-plan-an-epic-road-trip-to-atlanta/)## Events in Atlanta This Week\r\n\r\nEvents provide a glimpse into the ATL experience like nothing else, warmly welcoming visitors to the city year-round. Discover events in Atlanta.\r\n\r\n[events in Atlanta](https://discoveratlanta.com/events/main/)[See more events](/events/all)[Deck the Hall at the College Football Hall of FameHoliday12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/deck-the-hall-at-the-college-football-hall-of-fame/)### Deck the Hall at the College Football Hall of Fame\r\n\r\n12/31/24 \u2013 1/1/25\r\n\r\n[Noon Year\u2019s Eve Family CelebrationFamily \u0026amp; Kids12/31/24](https://discoveratlanta.com/event/detail/noon-year-s-eve-family-celebration/)### Noon Year\u2019s Eve Family Celebration\r\n\r\n12/31/24\r\n\r\n[Midnight at ChateauHoliday12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/midnight-at-chateau/)### Midnight at Chateau\r\n\r\n12/31/24 \u2013 1/1/25\r\n\r\n[NYE @ May Peel \u2013 A Champagne StoryHoliday12/31/24](https://discoveratlanta.com/event/detail/nye-may-peel-a-champagne-story/)### NYE @ May Peel \u2013 A Champagne Story\r\n\r\n12/31/24\r\n\r\n[New Year\u2019s Eve Bash at The WhitleyHoliday12/31/24](https://discoveratlanta.com/event/detail/new-year-s-eve-bash-at-the-whitley/)### New Year\u2019s Eve Bash at The Whitley\r\n\r\n12/31/24\r\n\r\n[New Year\u2019s Eve Black-Tie Dinner at The WaldorfDining12/31/24](https://discoveratlanta.com/event/detail/new-year-s-eve-black-tie-dinner-at-the-waldorf/)### New Year\u2019s Eve Black-Tie Dinner at The Waldorf\r\n\r\n12/31/24\r\n\r\n[New Year\u2019s Eve Black-Tie Gala at The WaldorfHoliday12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/new-year-s-eve-black-tie-gala-at-the-waldorf/)### New Year\u2019s Eve Black-Tie Gala at The Waldorf\r\n\r\n12/31/24 \u2013 1/1/25\r\n\r\n[Noon Year\u2019s Eve at LEGO Discovery Center AtlantaFamily \u0026amp; Kids12/31/24](https://discoveratlanta.com/event/detail/noon-year-s-eve-at-lego-discovery-center-atlanta/)### Noon Year\u2019s Eve at LEGO Discovery Center Atlanta\r\n\r\n12/31/24\r\n\r\n[Toast to 2025: Amore e Amore Rings in the New YearHoliday12/31/24](https://discoveratlanta.com/event/detail/toast-to-2025-amore-e-amore-rings-in-the-new-year/)### Toast to 2025: Amore e Amore Rings in the New Year\r\n\r\n12/31/24\r\n\r\n[Celebrate New Year\u2019s Day at Painted PickleHoliday1/1/25](https://discoveratlanta.com/event/detail/celebrate-new-year-s-day-at-painted-pickle/)### Celebrate New Year\u2019s Day at Painted Pickle\r\n\r\n1/1/25\r\n\r\n[See more events](/events/all)## Events in Atlanta This Weekend\r\n\r\nEvents provide a glimpse into the ATL experience like nothing else, warmly welcoming visitors to the city year-round. Discover events in Atlanta.\r\n\r\n[]()![changing img here](https://discoveratlanta.com/wp-content/themes/atlanta/assets/src/gutenberg/blocks/discover-events-highlight/placeholder.JPG)\r\n\r\n\r\n\r\n\r\n[]()\r\n\r\n[Deck the Hall at the College Football Hall of FameHoliday12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/deck-the-hall-at-the-college-football-hall-of-fame/)![Events](https://images.discoveratlanta.com/image/upload/w_800,c_scale,q_auto/v1734546567/EventEntry/qvbnx6yvfyrhtxxehurq.jpg)\r\n### Deck the Hall at the College Football Hall of Fame\r\n\r\nHoliday\r\n\r\n12/31/24 \u2013 1/1/25\r\n\r\n[Noon Year\u2019s Eve Family CelebrationFamily \u0026amp; Kids12/31/24](https://discoveratlanta.com/event/detail/noon-year-s-eve-family-celebration/)![Events](https://images.discoveratlanta.com/image/upload/w_800,c_scale,q_auto/v1734622597/EventEntry/bp9l8tdm5mmozsq49axy.jpg)\r\n### Noon Year\u2019s Eve Family Celebration\r\n\r\nFamily \u0026amp; Kids\r\n\r\n12/31/24\r\n\r\n[Midnight at ChateauHoliday12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/midnight-at-chateau/)![Events](https://images.discoveratlanta.com/image/upload/w_800,c_scale,q_auto/v1734542272/EventEntry/ee1l9tcromkg7tdhohey.jpg)\r\n### Midnight at Chateau\r\n\r\nHoliday\r\n\r\n12/31/24 \u2013 1/1/25\r\n\r\n[NYE @ May Peel \u2013 A Champagne StoryHoliday12/31/24](https://discoveratlanta.com/event/detail/nye-may-peel-a-champagne-story/)![Events](https://images.discoveratlanta.com/image/upload/w_800,c_scale,q_auto/v1734536118/EventEntry/tartv5l8esay4enxh1bh.jpg)\r\n### NYE @ May Peel \u2013 A Champagne Story\r\n\r\nHoliday\r\n\r\n12/31/24\r\n\r\n[New Year\u2019s Eve Bash at The WhitleyHoliday12/31/24](https://discoveratlanta.com/event/detail/new-year-s-eve-bash-at-the-whitley/)![Events](https://images.discoveratlanta.com/image/upload/w_800,c_scale,q_auto/v1731372920/EventEntry/mtombkqoeqfs3zj02teb.jpg)\r\n### New Year\u2019s Eve Bash at The Whitley\r\n\r\nHoliday\r\n\r\n12/31/24\r\n\r\n[New Year\u2019s Eve Black-Tie Dinner at The WaldorfDining12/31/24](https://discoveratlanta.com/event/detail/new-year-s-eve-black-tie-dinner-at-the-waldorf/)![Events](https://images.discoveratlanta.com/image/upload/w_800,c_scale,q_auto/v1734621279/EventEntry/dswxoa43zrqdv1ifjqeq.jpg)\r\n### New Year\u2019s Eve Black-Tie Dinner at The Waldorf\r\n\r\nDining\r\n\r\n12/31/24\r\n\r\n[](https://discoveratlanta.com/event/detail/deck-the-hall-at-the-college-football-hall-of-fame/)![Events](https://images.discoveratlanta.com/image/upload/w_800,c_scale,q_auto/v1734546567/EventEntry/qvbnx6yvfyrhtxxehurq.jpg)\r\n### Deck the Hall at the College Football Hall of Fame\r\n\r\nHoliday\r\n\r\n12/31/24 \u2013 1/1/25\r\n\r\n[](https://discoveratlanta.com/event/detail/noon-year-s-eve-family-celebration/)![Events](https://images.discoveratlanta.com/image/upload/w_800,c_scale,q_auto/v1734622597/EventEntry/bp9l8tdm5mmozsq49axy.jpg)\r\n### Noon Year\u2019s Eve Family Celebration\r\n\r\nFamily \u0026amp; Kids\r\n\r\n12/31/24\r\n\r\n[](https://discoveratlanta.com/event/detail/midnight-at-chateau/)![Events](https://images.discoveratlanta.com/image/upload/w_800,c_scale,q_auto/v1734542272/EventEntry/ee1l9tcromkg7tdhohey.jpg)\r\n### Midnight at Chateau\r\n\r\nHoliday\r\n\r\n12/31/24 \u2013 1/1/25\r\n\r\n[](https://discoveratlanta.com/event/detail/nye-may-peel-a-champagne-story/)![Events](https://images.discoveratlanta.com/image/upload/w_800,c_scale,q_auto/v1734536118/EventEntry/tartv5l8esay4enxh1bh.jpg)\r\n### NYE @ May Peel \u2013 A Champagne Story\r\n\r\nHoliday\r\n\r\n12/31/24\r\n\r\n[](https://discoveratlanta.com/event/detail/new-year-s-eve-bash-at-the-whitley/)![Events](https://images.discoveratlanta.com/image/upload/w_800,c_scale,q_auto/v1731372920/EventEntry/mtombkqoeqfs3zj02teb.jpg)\r\n### New Year\u2019s Eve Bash at The Whitley\r\n\r\nHoliday\r\n\r\n12/31/24\r\n\r\n[](https://discoveratlanta.com/event/detail/new-year-s-eve-black-tie-dinner-at-the-waldorf/)![Events](https://images.discoveratlanta.com/image/upload/w_800,c_scale,q_auto/v1734621279/EventEntry/dswxoa43zrqdv1ifjqeq.jpg)\r\n### New Year\u2019s Eve Black-Tie Dinner at The Waldorf\r\n\r\nDining\r\n\r\n12/31/24\r\n\r\n## Upcoming Atlanta events\r\n\r\n[EXPLORE MORE EVENTS](https://discoveratlanta.com/events/all/)**EXPLORE MORE EVENTS**No events available. Try clearing filters and searching again.\r\n\r\n[Stone Mountain Christmas12/31/24 \u2013 1/5/25](https://discoveratlanta.com/event/detail/stone-mountain-christmas/)![Stone Mountain Christmas](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731510630/EventEntry/cs9byaqulhyyaoohafka.jpg)\r\n[IllumiNights at the Zoo!12/31/24 \u2013 1/19/25](https://discoveratlanta.com/event/detail/illuminights-at-the-zoo/)![IllumiNights at the Zoo!](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1699565286/EventEntry/i0ftuwkioqh5mpuqwtqq.jpg)\r\n[Holidays at Georgia Aquarium12/31/24 \u2013 1/2/25](https://discoveratlanta.com/event/detail/holidays-at-georgia-aquarium/)![Holidays at Georgia Aquarium](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1699978789/EventEntry/l4morfwv7utlhtptmvud.jpg)\r\n[Holiday in the Park12/31/24 \u2013 1/5/25](https://discoveratlanta.com/event/detail/holiday-in-the-park/)![Holiday in the Park](https://res.cloudinary.com/atlanta/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1498181649/EventEntry/qmsdjxqxmoqemum2whmt.jpg)\r\n[Deck the Hall at the College Football Hall of Fame12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/deck-the-hall-at-the-college-football-hall-of-fame/)![Deck the Hall at the College Football Hall of Fame](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734546567/EventEntry/qvbnx6yvfyrhtxxehurq.jpg)\r\n[Atlanta\u2019s New Year\u2019s Eve Peach Drop Festival12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/atlanta-s-new-year-s-eve-peach-drop-festival/)![Atlanta\u0026#039;s New Year\u0026#039;s Eve Peach Drop Festival](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1669996517/EventEntry/wvln4rofrjlcjpilbub1.jpg)\r\n[Alvin Ailey American Dance Theater Live at the Fox2/12/25 \u2013 2/16/25](https://discoveratlanta.com/event/detail/alvin-ailey-american-dance-theater-live-at-the-fox/)![Alvin Ailey American Dance Theater Live at the Fox](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731532374/EventEntry/fuedgtfpemvwupfhjhrv.jpg)\r\n[New Year\u2019s Eve Celebration12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/new-year-s-eve-celebration/)![New Year\u0026#039;s Eve Celebration](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731005676/EventEntry/boc9cvr7vnqacpygwmkv.jpg)\r\n[Noon Year\u2019s Eve Family Celebration12/31/24](https://discoveratlanta.com/event/detail/noon-year-s-eve-family-celebration/)![Noon Year\u0026#039;s Eve Family Celebration](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734622597/EventEntry/bp9l8tdm5mmozsq49axy.jpg)\r\n[Chick-fil-A Peach Bowl1/1/25](https://discoveratlanta.com/event/detail/chick-fil-a-peach-bowl/)![Chick-fil-A Peach Bowl](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1702499768/EventEntry/wqm4kbr99lxseqihty08.jpg)\r\n[AT\u0026#038;T Playoff Playlist Live with Lil Wayne \u0026#038; More!1/18/25 \u2013 1/19/25](https://discoveratlanta.com/event/detail/at-t-playoff-playlist-live-with-lil-wayne-more/)![AT\u0026#038;T Playoff Playlist Live with Lil Wayne \u0026#038; More!](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734539959/EventEntry/vot9ijnqzxmxyqaz19s9.jpg)\r\n[2025 CFP National Championship1/20/25](https://discoveratlanta.com/event/detail/2025-cfp-national-championship/)![2025 CFP National Championship](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734465369/EventEntry/riaxnf4gxtryne9onnxe.jpg)\r\n[Oysterfest2/24/25 \u2013 2/25/25](https://discoveratlanta.com/event/detail/oysterfest/)![Oysterfest](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729799356/EventEntry/l42r9khyl9hox6wc1h0t.jpg)\r\n[Mardi Gras Streetcar Adventure2/22/25](https://discoveratlanta.com/event/detail/mardi-gras-streetcar-adventure/)![Mardi Gras Streetcar Adventure](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1702400087/EventEntry/bh9dlzzijyl4ai0szpdy.jpg)\r\n[WildWoods AGLOW12/31/24 \u2013 2/23/25](https://discoveratlanta.com/event/detail/wildwoods-aglow/)![WildWoods AGLOW](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1668543833/EventEntry/nr03aawebcvgubpqushj.jpg)\r\n[Skate the Station12/31/24 \u2013 1/20/25](https://discoveratlanta.com/event/detail/skate-the-station-2/)![Skate the Station](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731074085/EventEntry/majwv2iiclw0xorelcx5.jpg)\r\n[Garden Lights, Holiday Nights12/31/24 \u2013 1/12/25](https://discoveratlanta.com/event/detail/garden-lights-holiday-nights/)![Garden Lights, Holiday Nights](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1668014394/EventEntry/x4qngfueu0blab1uxxey.jpg)\r\n[$ 3.00 Winter Flicks\u2013Despicable Me 4 at Aurora12/31/24 \u2013 1/2/25](https://discoveratlanta.com/event/detail/3-00-winter-flicks-despicable-me-4-at-aurora/)![$ 3.00 Winter Flicks--Despicable Me 4 at Aurora](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733420198/EventEntry/auyza0uprmq8g10ldj9d.jpg)\r\n[All-Inclusive NYE Party: Get Ready to Party12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/all-inclusive-nye-party-get-ready-to-party/)![All-Inclusive NYE Party: Get Ready to Party](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731956865/EventEntry/q0voillatcqynq0zzmyk.jpg)\r\n[Atlanta New Years Eve Bar Crawl12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/atlanta-new-years-eve-bar-crawl/)![Atlanta New Years Eve Bar Crawl](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733376932/EventEntry/hyli54rajah2pdykrwo2.jpg)\r\n[Champions \u2013 A Celebration of Queen NYE Concert12/31/24](https://discoveratlanta.com/event/detail/champions-a-celebration-of-queen-nye-concert/)![Champions - A Celebration of Queen NYE Concert](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729280087/EventEntry/y0pgtzmte4vzv0memoyq.jpg)\r\n[Chick-fil-A Peach Bowl Fan Night12/31/24](https://discoveratlanta.com/event/detail/chick-fil-a-peach-bowl-fan-night/)![Chick-fil-A Peach Bowl Fan Night](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731700309/EventEntry/skchhcylupk0rqy3ivuj.jpg)\r\n[Escondido 2025 New Year\u2019s Eve Party12/31/24](https://discoveratlanta.com/event/detail/escondido-2025-new-year-s-eve-party/)![Escondido 2025 New Year\u0026#039;s Eve Party](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733265466/EventEntry/fqlvfpstwdb69v4ogcsv.jpg)\r\n[Georgia Tech Men\u2019s Basketball Vs Notre Dame12/31/24](https://discoveratlanta.com/event/detail/georgia-tech-men-s-basketball-vs-notre-dame/)![Georgia Tech Men\u2019s Basketball Vs Notre Dame](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729615389/EventEntry/zzlwwz35r8l0naedrja0.jpg)\r\n[Midnight at Chateau12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/midnight-at-chateau/)![Midnight at Chateau](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734542272/EventEntry/ee1l9tcromkg7tdhohey.jpg)\r\n[NYE @ May Peel \u2013 A Champagne Story12/31/24](https://discoveratlanta.com/event/detail/nye-may-peel-a-champagne-story/)![NYE @ May Peel \u2013 A Champagne Story](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734536118/EventEntry/tartv5l8esay4enxh1bh.jpg)\r\n[NYE: Sear After Dark!12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/nye-sear-after-dark/)![NYE: Sear After Dark!](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734361150/EventEntry/qmepnjlgkjcftge6zclg.jpg)\r\n[New Year\u2019s Eve Celebration at Nobu12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/new-year-s-eve-celebration-at-nobu/)![New Year\u0026#039;s Eve Celebration at Nobu](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729195483/EventEntry/i4gbiwxyuws7wzdc5lrn.jpg)\r\n[New Year\u2019s Eve Celebration at Red Phone Booth 202412/31/24](https://discoveratlanta.com/event/detail/new-year-s-eve-celebration-at-red-phone-booth-2024-2/)![New Year\u0026#039;s Eve Celebration at Red Phone Booth 2024](https://res.cloudinary.com/atlanta/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1599834269/EventEntry/DefaultImages/holiday.jpg)\r\n[New Year\u2019s Eve Dinner at Signia by Hilton Atlanta12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/new-year-s-eve-dinner-at-signia-by-hilton-atlanta/)![New Year\u0026#039;s Eve Dinner at Signia by Hilton Atlanta](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1730752354/EventEntry/ixcmxoubtw3rtlcvfbws.jpg)\r\n[New Years Eve Party 2025 at Buena Vida Tapas Bar12/31/24](https://discoveratlanta.com/event/detail/new-years-eve-party-2025-at-buena-vida-tapas-bar/)![New Years Eve Party 2025 at Buena Vida Tapas Bar](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733338355/EventEntry/tx19zcpm5gdammi6msqd.jpg)\r\n[New Year\u2019s Eve Bash at The Whitley12/31/24](https://discoveratlanta.com/event/detail/new-year-s-eve-bash-at-the-whitley/)![New Year\u2019s Eve Bash at The Whitley](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731372920/EventEntry/mtombkqoeqfs3zj02teb.jpg)\r\n[New Year\u2019s Eve Black-Tie Dinner at The Waldorf12/31/24](https://discoveratlanta.com/event/detail/new-year-s-eve-black-tie-dinner-at-the-waldorf/)![New Year\u2019s Eve Black-Tie Dinner at The Waldorf](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734621279/EventEntry/dswxoa43zrqdv1ifjqeq.jpg)\r\n[New Year\u2019s Eve Black-Tie Gala at The Waldorf12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/new-year-s-eve-black-tie-gala-at-the-waldorf/)![New Year\u2019s Eve Black-Tie Gala at The Waldorf](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734633963/EventEntry/fwhenoca5zs8whpy2vmq.jpg)\r\n[Noon Year\u2019s Eve Balloon Drop12/31/24](https://discoveratlanta.com/event/detail/noon-year-s-eve-balloon-drop/)![Noon Year\u0026#039;s Eve Balloon Drop](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733839576/EventEntry/a327ptjhn6nkzpbh7qzz.jpg)\r\n[Noon Year\u2019s Eve at LEGO Discovery Center Atlanta12/31/24](https://discoveratlanta.com/event/detail/noon-year-s-eve-at-lego-discovery-center-atlanta/)![Noon Year\u0026#039;s Eve at LEGO Discovery Center Atlanta](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734577971/EventEntry/mbgc1yk4glqiyq3zxnxs.jpg)\r\n[Overstory\u2019s New Year\u2019s Eve Celebration12/31/24](https://discoveratlanta.com/event/detail/overstory-s-new-year-s-eve-celebration/)![Overstory\u0026#039;s New Year\u0026#039;s Eve Celebration](https://res.cloudinary.com/atlanta/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1599834376/EventEntry/DefaultImages/nightlife.jpg)\r\n[Peach Bowl Fan Night at Georgia Aquarium12/31/24](https://discoveratlanta.com/event/detail/peach-bowl-fan-night-at-georgia-aquarium/)![Peach Bowl Fan Night at Georgia Aquarium](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733799909/EventEntry/szssgmvhejb1senxk3df.jpg)\r\n[Toast to 2025: Amore e Amore Rings in the New Year12/31/24](https://discoveratlanta.com/event/detail/toast-to-2025-amore-e-amore-rings-in-the-new-year/)![Toast to 2025: Amore e Amore Rings in the New Year](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734541369/EventEntry/aoouncrlkwrd5rdbyt3z.jpg)\r\n[Winter Chalet at InterContinental Buckhead12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/winter-chalet-at-intercontinental-buckhead/)![Winter Chalet at InterContinental Buckhead](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733351858/EventEntry/mfd9445t9vnfrkcouv5j.jpg)\r\n[Zoo Year\u2019s Eve12/31/24](https://discoveratlanta.com/event/detail/zoo-year-s-eve/)![Zoo Year\u0026#039;s Eve](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1730319012/EventEntry/a7yxccj5vtoqj86epj8v.jpg)\r\n[2025 Karaoke 2851/1/25](https://discoveratlanta.com/event/detail/2025-karaoke-285/)![2025 Karaoke 285](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1732315091/EventEntry/jthiaf9rzkuuqnzlofii.jpg)\r\n[Abuelita\u2019s Brunch: Wild Heaven West End1/1/25](https://discoveratlanta.com/event/detail/abuelita-s-brunch-wild-heaven-west-end/)![Abuelita\u2019s Brunch: Wild Heaven West End](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733846250/EventEntry/uzd2bthgonlrmlm64xti.jpg)\r\n[Celebrate New Year\u2019s Day at Painted Pickle1/1/25](https://discoveratlanta.com/event/detail/celebrate-new-year-s-day-at-painted-pickle/)![Celebrate New Year\u2019s Day at Painted Pickle](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734113178/EventEntry/ljfpxg0tjjto96fcehrs.jpg)\r\n[YOGA \u0026#038; SOUND BATH1/1/25](https://discoveratlanta.com/event/detail/yoga-sound-bath/)![YOGA \u0026#038; SOUND BATH](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733778569/EventEntry/l4xqitka5a0b6kqdyea9.jpg)\r\n[Georgia Tech Women\u2019s Basketball vs Syracuse1/2/25](https://discoveratlanta.com/event/detail/georgia-tech-women-s-basketball-vs-syracuse/)![Georgia Tech Women\u2019s Basketball vs Syracuse](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729695761/EventEntry/uqzn4eq3bqelaqdsphg2.jpg)\r\n[Georgia Tech Men\u2019s Basketball Vs Boston College1/4/25](https://discoveratlanta.com/event/detail/georgia-tech-men-s-basketball-vs-boston-college/)![Georgia Tech Men\u2019s Basketball Vs Boston College](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729615508/EventEntry/ynankil3kztugbnfpsnf.jpg)\r\n[Ray Howard Band Presents The Ultimate EWF1/4/25](https://discoveratlanta.com/event/detail/ray-howard-band-presents-the-ultimate-ewf/)![Ray Howard Band Presents The Ultimate EWF](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1728503928/EventEntry/vyd3e1ncktk9fuauaall.jpg)\r\n[Sonic Symphony1/4/25](https://discoveratlanta.com/event/detail/sonic-symphony/)![Sonic Symphony](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731399019/EventEntry/bgm95ivs2pe6bktwyjjz.jpg)\r\n[Carolina Panthers at Atlanta Falcons1/5/25](https://discoveratlanta.com/event/detail/carolina-panthers-at-atlanta-falcons/)![Carolina Panthers at Atlanta Falcons](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n[Thai Omakase with Chef Pattie \u0026#038; Mo\u00EBt Hennessy1/7/25](https://discoveratlanta.com/event/detail/thai-omakase-with-chef-pattie-moet-hennessy/)![Thai Omakase with Chef Pattie \u0026#038; Mo\u00EBt Hennessy](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733856927/EventEntry/m3t3rbtdqqkfg8vj8vnu.jpg)\r\n[\u0026#038; Juliet1/9/25 \u2013 1/12/25](https://discoveratlanta.com/event/detail/juliet/)![\u0026#038; Juliet](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n[2025 DSC Convention \u0026#038; Sporting Expo1/9/25 \u2013 1/12/25](https://discoveratlanta.com/event/detail/2025-dsc-convention-sporting-expo/)![2025 DSC Convention \u0026#038; Sporting Expo](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1721159445/EventEntry/jwpi2cmjkwe2rlhz8yhl.jpg)\r\n[Discover Boating Atlanta Boat Show1/9/25 \u2013 1/12/25](https://discoveratlanta.com/event/detail/discover-boating-atlanta-boat-show/)![Discover Boating Atlanta Boat Show](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1732118753/EventEntry/u4gkgldqq0tensnbazkm.jpg)\r\n[Georgia Tech Women\u2019s Basketball vs Virginia Tech1/9/25](https://discoveratlanta.com/event/detail/georgia-tech-women-s-basketball-vs-virginia-tech/)![Georgia Tech Women\u2019s Basketball vs Virginia Tech](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729695919/EventEntry/ukutgvei3poqnvezxoo4.jpg)\r\n[Donna The Buffalo1/10/25](https://discoveratlanta.com/event/detail/donna-the-buffalo/)![Donna The Buffalo](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1728504537/EventEntry/sqflakdjmunynct4jdue.jpg)\r\n[Portraits \u0026#038; Prosecco1/10/25](https://discoveratlanta.com/event/detail/portraits-prosecco/)![Portraits \u0026#038; Prosecco](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734392978/EventEntry/p322nelbcqucnumnrq1r.jpg)\r\n[Tree Tour: History, Change on the Atlanta Beltline1/11/25](https://discoveratlanta.com/event/detail/tree-tour-history-change-on-the-atlanta-beltline/)![Tree Tour: History, Change on the Atlanta Beltline](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733146311/EventEntry/vmsx2kyhka9ob3cpmtar.jpg)\r\n[NFC Brew Jitsu #3 at Monday Night Garage1/12/25](https://discoveratlanta.com/event/detail/nfc-brew-jitsu-3-at-monday-night-garage/)![NFC Brew Jitsu #3 at Monday Night Garage](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1730933122/EventEntry/rf5r5uzefnam0n9gqzdo.jpg)\r\n[Full Moon Sound Bath x Dear Life Chat1/13/25](https://discoveratlanta.com/event/detail/full-moon-sound-bath-x-dear-life-chat/)![Full Moon Sound Bath x Dear Life Chat](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734572407/EventEntry/nrokwfryoumiluyconai.jpg)\r\n[Georgia Tech Men\u2019s Basketball Vs Clemson1/14/25](https://discoveratlanta.com/event/detail/georgia-tech-men-s-basketball-vs-clemson/)![Georgia Tech Men\u2019s Basketball Vs Clemson](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729615686/EventEntry/lomy5nwruywsvp4sblpw.jpg)\r\n[CFP Taste of the Championship Atlanta1/19/25](https://discoveratlanta.com/event/detail/cfp-taste-of-the-championship-atlanta/)![CFP Taste of the Championship Atlanta](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734464937/EventEntry/vk1qmw7hdetbaupmi5lu.jpg)\r\n[Georgia Tech Women\u2019s Basketball vs Clemson1/19/25](https://discoveratlanta.com/event/detail/georgia-tech-women-s-basketball-vs-clemson/)![Georgia Tech Women\u2019s Basketball vs Clemson](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729696046/EventEntry/hkdj81zh4nsxsktxxegx.jpg)\r\n[The Everly Set1/19/25](https://discoveratlanta.com/event/detail/the-everly-set/)![The Everly Set](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733348717/EventEntry/bl53uthu317vkxynbb3x.jpg)\r\n[College Football Playoff National Championship1/21/25](https://discoveratlanta.com/event/detail/college-football-playoff-national-championship/)![College Football Playoff National Championship](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n[Georgia Tech Men\u2019s Basketball Vs Virginia Tech1/22/25](https://discoveratlanta.com/event/detail/georgia-tech-men-s-basketball-vs-virginia-tech/)![Georgia Tech Men\u2019s Basketball Vs Virginia Tech](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729615833/EventEntry/c5nzrfp4c4fsufbyeoru.jpg)\r\n[Maternal Health Awareness Day 20251/23/25](https://discoveratlanta.com/event/detail/maternal-health-awareness-day-2025/)![Maternal Health Awareness Day 2025](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734013179/EventEntry/ecmitunnqsjzbt1dyxsu.jpg)\r\n[Movies at The Strand: Anastasia (1997)1/23/25](https://discoveratlanta.com/event/detail/movies-at-the-strand-anastasia-1997/)![Movies at The Strand: Anastasia (1997)](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733411491/EventEntry/vdtctmt1fz4yv67yjgtz.jpg)\r\n[FSR Presents: Life Be Lifin\u2019 Starring Monet X Change1/24/25](https://discoveratlanta.com/event/detail/fsr-presents-life-be-lifin-starring-monet-x-change/)![FSR Presents: Life Be Lifin\u0026#039; Starring Monet X Change](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n[First Arab Elbow Course- The Traumatic1/24/25 \u2013 1/26/25](https://discoveratlanta.com/event/detail/first-arab-elbow-course-the-traumatic/)![First Arab Elbow Course- The Traumatic](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1718963670/EventEntry/cjtlswvklqlcf2fttzdm.jpg)\r\n[Jazz at The Strand: The Music of John Coltrane 2021/24/25](https://discoveratlanta.com/event/detail/jazz-at-the-strand-the-music-of-john-coltrane-202/)![Jazz at The Strand: The Music of John Coltrane 202](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733413611/EventEntry/icxos2f3rdegxph5ht5n.jpg)\r\n[Grady CrossTies Gala1/25/25 \u2013 1/26/25](https://discoveratlanta.com/event/detail/grady-crossties-gala/)![Grady CrossTies Gala](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733949365/EventEntry/cx7nry1bi6nc8096zbi5.jpg)\r\n[Heroes Among Us: A Musical Tribute1/25/25](https://discoveratlanta.com/event/detail/heroes-among-us-a-musical-tribute/)![Heroes Among Us: A Musical Tribute](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1725910962/EventEntry/ecrzteqlx0cxy5ssylpb.jpg)\r\n[Indie Game Pop-Up @ CMoA1/25/25](https://discoveratlanta.com/event/detail/indie-game-pop-up-cmoa/)![Indie Game Pop-Up @ CMoA](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734196514/EventEntry/owzuflwpbzfttjotscec.jpg)\r\n[Lunar New Year Celebration1/25/25](https://discoveratlanta.com/event/detail/lunar-new-year-celebration/)![Lunar New Year Celebration](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1722961018/EventEntry/hwviq1drjnc7fbwgkgqv.jpg)\r\n[Magician Rich Crawford Live1/25/25](https://discoveratlanta.com/event/detail/magician-rich-crawford-live/)![Magician Rich Crawford Live](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729660580/EventEntry/yyfckqjnw91p9kjsmghs.jpg)\r\n[What\u2019s Goin\u2019 on at Shorty!1/25/25](https://discoveratlanta.com/event/detail/what-s-goin-on-at-shorty/)![What\u0026#039;s Goin\u0026#039; on at Shorty!](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733237473/EventEntry/jd1wksbzzgfxdwl6gdd1.jpg)\r\n[Georgia Tech Women\u2019s Basketball vs Duke1/26/25](https://discoveratlanta.com/event/detail/georgia-tech-women-s-basketball-vs-duke/)![Georgia Tech Women\u2019s Basketball vs Duke](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729696270/EventEntry/e82rg9uinawhqdi0f5hj.jpg)\r\n[Clue WILL VISIT ATLANTA\u2019S FOX THEATRE1/28/25 \u2013 2/2/25](https://discoveratlanta.com/event/detail/clue-will-visit-atlanta-s-fox-theatre/)![Clue WILL VISIT ATLANTA\u2019S FOX THEATRE](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733950229/EventEntry/evf0lziixzb36jbc50rp.jpg)\r\n[Rainforest Adventures1/28/25 \u2013 3/2/25](https://discoveratlanta.com/event/detail/rainforest-adventures/)![Rainforest Adventures](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1730921950/EventEntry/hxjk0utzvq5hsjcfdhtr.jpg)\r\n[2025 Flourish Family Homeschool Conference1/29/25 \u2013 1/31/25](https://discoveratlanta.com/event/detail/2025-flourish-family-homeschool-conference/)![2025 Flourish Family Homeschool Conference](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1727841207/EventEntry/awzsvj1c37pudhtvsc3i.jpg)\r\n[Lucas Jack LIVE at Eddie\u2019s Attic1/29/25](https://discoveratlanta.com/event/detail/lucas-jack-live-at-eddie-s-attic/)![Lucas Jack LIVE at Eddie\u2019s Attic](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1728936308/EventEntry/zuethiczdysamzacsb6t.jpg)\r\n[Clue The Musical1/30/25 \u2013 2/1/25](https://discoveratlanta.com/event/detail/clue-the-musical/)![Clue The Musical](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n[Rod Wave \u2013 The Last Lap Tour1/30/25](https://discoveratlanta.com/event/detail/rod-wave-the-last-lap-tour/)![Rod Wave - The Last Lap Tour](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734556973/EventEntry/avps9yrapwiezd8ggst9.jpg)\r\n[\u201990s Night at Fernbank1/31/25](https://discoveratlanta.com/event/detail/90s-night-at-fernbank/)![\u0026#039;90s Night at Fernbank](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731937209/EventEntry/ytozp6qy2wuy0hx7jfbq.jpg)\r\n[Groundhog Day Jugglers Festival1/31/25 \u2013 2/2/25](https://discoveratlanta.com/event/detail/groundhog-day-jugglers-festival/)![Groundhog Day Jugglers Festival](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1672845037/EventEntry/l7vzn6xaizyefmqsymtt.jpg)\r\n[Vision Board Workshop1/31/25](https://discoveratlanta.com/event/detail/vision-board-workshop/)![Vision Board Workshop](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731703251/EventEntry/d5gxsux4ocossysy44oh.jpg)\r\n[FLOTAX test2/1/25](https://discoveratlanta.com/event/detail/flotax-test/)![FLOTAX test](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n[Georgia Tech Men\u2019s Basketball Vs Louisville2/1/25](https://discoveratlanta.com/event/detail/georgia-tech-men-s-basketball-vs-louisville/)![Georgia Tech Men\u2019s Basketball Vs Louisville](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729616285/EventEntry/k4lvbsrm6z72tndb9eh1.jpg)\r\n[Hot Wheels Monster Trucks LiveMultiple dates until 02/02/2025](https://discoveratlanta.com/event/detail/hot-wheels-monster-trucks-live/)![Hot Wheels Monster Trucks Live](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1727798851/EventEntry/f0ewizhmzrccwg4wvvya.jpg)\r\n[Hot Chocolate Run Atlanta2/2/25](https://discoveratlanta.com/event/detail/hot-chocolate-run-atlanta/)![Hot Chocolate Run Atlanta](https://res.cloudinary.com/atlanta/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1599834376/EventEntry/DefaultImages/fitness.jpg)\r\n[Mary J Blige The \u2018For My Fans\u201D Tour2/3/25](https://discoveratlanta.com/event/detail/mary-j-blige-the-for-my-fans-tour/)![Mary J Blige The \u0026#039;For My Fans](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1727804704/EventEntry/yhdwwuvigemnaohqorx9.jpg)\r\n[Georgia Tech Women\u2019s Basketball vs SMU2/6/25](https://discoveratlanta.com/event/detail/georgia-tech-women-s-basketball-vs-smu/)![Georgia Tech Women\u2019s Basketball vs SMU](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729696592/EventEntry/rslzial7i8vvoffcewdf.jpg)\r\n[FREE Movie Night: Frozen2/8/25](https://discoveratlanta.com/event/detail/free-movie-night-frozen/)![FREE Movie Night: Frozen](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1722961377/EventEntry/y53fyyfq9msax7zlmcaq.jpg)\r\n[Lauda Musicam Chamber Concert2/8/25](https://discoveratlanta.com/event/detail/lauda-musicam-chamber-concert/)![Lauda Musicam Chamber Concert](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733883863/EventEntry/frupovugcfucqgcxppgd.jpg)\r\n[Matthew Kaminski\u2019s Mondo Hammondo2/8/25](https://discoveratlanta.com/event/detail/matthew-kaminski-s-mondo-hammondo/)![Matthew Kaminski\u0026#039;s Mondo Hammondo](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731046207/EventEntry/yii7r6ro5rgjenof8mnf.jpg)\r\n[A Beautiful Noise: The Neil Diamond Musical2/9/25 \u2013 3/16/25](https://discoveratlanta.com/event/detail/a-beautiful-noise-the-neil-diamond-musical/)![A Beautiful Noise: The Neil Diamond Musical](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n[Georgia Tech Men\u2019s Basketball Vs Stanford2/12/25](https://discoveratlanta.com/event/detail/georgia-tech-men-s-basketball-vs-stanford/)![Georgia Tech Men\u2019s Basketball Vs Stanford](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729616561/EventEntry/e5lvhpyla05ixdtezven.jpg)\r\n[BUSTMultiple days until 03/16/2025](https://discoveratlanta.com/event/detail/bust/)![BUST](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1730142641/EventEntry/bg6bhfxd2uu8fogshdni.jpg)\r\n[Martin Amini2/14/25](https://discoveratlanta.com/event/detail/martin-amini/)![Martin Amini](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1725042684/EventEntry/p9i4euxhpuhlxxppvtbi.jpg)\r\n[TATTOOS, BOOZE, ART, \u0026#038; SHOES 3.02/14/25 \u2013 2/15/25](https://discoveratlanta.com/event/detail/tattoos-booze-art-shoes-3-0/)![TATTOOS, BOOZE, ART, \u0026#038; SHOES 3.0](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734493578/EventEntry/bwzfereitmosyay0ozfh.jpg)\r\n[African-American Folktales2/15/25](https://discoveratlanta.com/event/detail/african-american-folktales/)![African-American Folktales](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1722961481/EventEntry/rlw5bvaoonjk8s6x2p7w.jpg)\r\n[Georgia Tech Men\u2019s Basketball Vs California2/15/25](https://discoveratlanta.com/event/detail/georgia-tech-men-s-basketball-vs-california/)![Georgia Tech Men\u2019s Basketball Vs California](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729616418/EventEntry/qdpzxv9nx1rqjtylbcwq.jpg)\r\n[Trailblazers2/15/25](https://discoveratlanta.com/event/detail/trailblazers/)![Trailblazers](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1719701826/EventEntry/srdiozvtascs30bcftgu.jpg)\r\n[Georgia Tech Women\u2019s Basketball vs Wake Forest2/16/25](https://discoveratlanta.com/event/detail/georgia-tech-women-s-basketball-vs-wake-forest/)![Georgia Tech Women\u2019s Basketball vs Wake Forest](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729696770/EventEntry/udcoeyhw60kos95sk18u.jpg)\r\n[Atlanta Jewish Film Festival2/19/25 \u2013 3/16/25](https://discoveratlanta.com/event/detail/atlanta-jewish-film-festival/)![Atlanta Jewish Film Festival](https://res.cloudinary.com/atlanta/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1599834249/EventEntry/DefaultImages/festivals.jpg)\r\n[Georgia Tech Women\u2019s Basketball vs NC State2/20/25](https://discoveratlanta.com/event/detail/georgia-tech-women-s-basketball-vs-nc-state/)![Georgia Tech Women\u2019s Basketball vs NC State](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733947815/EventEntry/od8ouuezs3p9wwzmb9me.jpg)\r\n[Billy Strings Live at State Farm Arena2/21/25 \u2013 2/22/25](https://discoveratlanta.com/event/detail/billy-strings-live-at-state-farm-arena/)![Billy Strings Live at State Farm Arena](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1704398623/EventEntry/ah0gfgjh5a34qjadh41v.jpg)\r\n[Atlanta Black Expo 20252/22/25 \u2013 2/23/25](https://discoveratlanta.com/event/detail/atlanta-black-expo-2025/)![Atlanta Black Expo 2025](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731633040/EventEntry/i5joh5qutrtgv8jeaofn.jpg)\r\n[QS Discover Master\u2019s and MBA Fair in Atlanta2/22/25](https://discoveratlanta.com/event/detail/qs-discover-master-s-and-mba-fair-in-atlanta/)![QS Discover Master\u0026#039;s and MBA Fair in Atlanta](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734339833/EventEntry/ocractvjza6iat5skkpz.jpg)\r\n[Zoso:The Ultimate Led Zeppelin Experience2/22/25](https://discoveratlanta.com/event/detail/zoso-the-ultimate-led-zeppelin-experience/)![Zoso:The Ultimate Led Zeppelin Experience](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729018134/EventEntry/rcbpphc1whx1ps1acdmp.jpg)\r\n[Georgia Tech Women\u2019s Basketball vs Florida State2/23/25](https://discoveratlanta.com/event/detail/georgia-tech-women-s-basketball-vs-florida-state/)![Georgia Tech Women\u2019s Basketball vs Florida State](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729697091/EventEntry/yczbgw3i1pnwej1uuxz4.jpg)\r\n[National Money Show2/27/25](https://discoveratlanta.com/event/detail/national-money-show/)![National Money Show](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1730068696/EventEntry/vndywdfpgahdkyk0a1te.jpg)\r\n[Cordae \u2013 The Crossroads Tour2/28/25](https://discoveratlanta.com/event/detail/cordae-the-crossroads-tour/)![Cordae - The Crossroads Tour](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734548779/EventEntry/uurcnjtslde3axwcvvgq.jpg)\r\n[School \u0026#038; Group NightCrawler at the Zoo2/28/25](https://discoveratlanta.com/event/detail/school-group-nightcrawler-at-the-zoo/)![School \u0026#038; Group NightCrawler at the Zoo](https://res.cloudinary.com/atlanta/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1599844113/EventEntry/DefaultImages/education.jpg)\r\n[Georgia Tech Men\u2019s Basketball Vs NC State3/1/25](https://discoveratlanta.com/event/detail/georgia-tech-men-s-basketball-vs-nc-state/)![Georgia Tech Men\u2019s Basketball Vs NC State](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729616675/EventEntry/ikl28rlrpz2uetdybizn.jpg)\r\n[Katt Williams Heaven on Earth Tour3/1/25](https://discoveratlanta.com/event/detail/katt-williams-heaven-on-earth-tour/)![Katt Williams Heaven on Earth Tour](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1727797673/EventEntry/ppliqesrtsv2jk4g1gs5.jpg)\r\n[Suwanee Beer Fest Beer Chaser 5K3/1/25](https://discoveratlanta.com/event/detail/suwanee-beer-fest-beer-chaser-5k/)![Suwanee Beer Fest Beer Chaser 5K](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1718386949/EventEntry/bef7ib49alr2lrkwvq5g.jpg)\r\n[Dan Forrest\u2019s Newest Work \u201CCreation\u201D3/2/25](https://discoveratlanta.com/event/detail/dan-forrest-s-newest-work-creation/)![Dan Forrest\u2019s Newest Work  ](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1721763266/EventEntry/obwiwxtsvmjrf4xsk6rn.jpg)\r\n[Georgia Tech Men\u2019s Basketball Vs Miami3/4/25](https://discoveratlanta.com/event/detail/georgia-tech-men-s-basketball-vs-miami/)![Georgia Tech Men\u2019s Basketball Vs Miami](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729616792/EventEntry/ksafbrxm4izhs3fs70jt.jpg)\r\n[Galapagos George, The Little Tortoise that Could3/5/25 \u2013 3/16/25](https://discoveratlanta.com/event/detail/galapagos-george-the-little-tortoise-that-could/)![Galapagos George, The Little Tortoise that Could](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1714597675/EventEntry/zwuuovetp4jrlo7lf2gn.jpg)\r\n[63rd Annual Orchid Show \u2013 \u201CAn Orchid Wonderland\u201D3/7/25 \u2013 3/9/25](https://discoveratlanta.com/event/detail/63rd-annual-orchid-show-an-orchid-wonderland/)![63rd Annual Orchid Show - ](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731618901/EventEntry/kdr0dvtmtqw5odnfmbmn.jpg)\r\n[Martin Lawrence Y\u2019all Know What It Is Tour3/7/25](https://discoveratlanta.com/event/detail/martin-lawrence-y-all-know-what-it-is-tour/)![Martin Lawrence Y\u0026#039;all Know What It Is Tour](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1727797866/EventEntry/juzg5j3hufl0ejllf67g.jpg)\r\n[Blacktop Playground: Improv for Kids3/8/25](https://discoveratlanta.com/event/detail/blacktop-playground-improv-for-kids/)![Blacktop Playground: Improv for Kids](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1722961592/EventEntry/hb78qtz4godbbljigx6z.jpg)\r\n[PACK EXPO 20253/10/25 \u2013 3/12/25](https://discoveratlanta.com/event/detail/pack-expo-2025/)![PACK EXPO 2025](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1685635365/EventEntry/wbmoavffcsbdi6p72hhz.jpg)\r\n[Blake Shelton Friends and Heroes 2025 Tour3/15/25 \u2013 3/16/25](https://discoveratlanta.com/event/detail/blake-shelton-friends-and-heroes-2025-tour/)![Blake Shelton Friends and Heroes 2025 Tour](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1727798057/EventEntry/qsjs4xiqfcxmg13yrgzb.jpg)\r\n[From Screen to Stage3/15/25](https://discoveratlanta.com/event/detail/from-screen-to-stage/)![From Screen to Stage](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1725911104/EventEntry/fyryjxg95r9er9qymwtm.jpg)\r\n[Deftones North American Tour 20253/18/25 \u2013 3/19/25](https://discoveratlanta.com/event/detail/deftones-north-american-tour-2025/)![Deftones North American Tour 2025](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1727798254/EventEntry/y1muus1kadupfvjds7bs.jpg)\r\n[Milo the Magnificent3/19/25 \u2013 3/30/25](https://discoveratlanta.com/event/detail/milo-the-magnificent/)![Milo the Magnificent](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1715095074/EventEntry/segadj3sd1cwiabussqf.jpg)\r\n[J Balvin Live at State Farm Arena3/20/25](https://discoveratlanta.com/event/detail/j-balvin-live-at-state-farm-arena/)![J Balvin Live at State Farm Arena](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734630666/EventEntry/gcjvxwguifniz5ixct4m.jpg)\r\n[Tyler, The Creator \u2013 Chromakopia Tour3/21/25](https://discoveratlanta.com/event/detail/tyler-the-creator-chromakopia-tour/)![Tyler, The Creator - Chromakopia Tour](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734555743/EventEntry/udoorxuvxhkxmxjje1cd.jpg)\r\n[Debussy, Ravel, \u0026#038; Sibelius: A Birthday Celebration3/29/25](https://discoveratlanta.com/event/detail/debussy-ravel-sibelius-a-birthday-celebration/)![Debussy, Ravel, \u0026#038; Sibelius: A Birthday Celebration](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1719701585/EventEntry/cfsofitep9345ycdndzt.jpg)\r\n[It\u2019s Electric!3/29/25](https://discoveratlanta.com/event/detail/it-s-electric/)![It\u2019s Electric!](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1722961771/EventEntry/e99aqcvvstgb3esn1icd.jpg)\r\n[The Millennium Tour 20254/4/25](https://discoveratlanta.com/event/detail/the-millennium-tour-2025/)![The Millennium Tour 2025](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734631195/EventEntry/cqletfwxfxbwykqeymxu.jpg)\r\n[Georgia Tasting Festival 20254/6/25](https://discoveratlanta.com/event/detail/georgia-tasting-festival-2025/)![Georgia Tasting Festival 2025](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733347277/EventEntry/ysqo8jq53n8cvnuiz6lv.jpg)\r\n[Spring Break Magic Show4/9/25](https://discoveratlanta.com/event/detail/spring-break-magic-show/)![Spring Break Magic Show](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1722961984/EventEntry/y6ymqc7rrvfc4mnrqkhg.jpg)\r\n[Kylie Minogue 2025 Tension Tour4/11/25](https://discoveratlanta.com/event/detail/kylie-minogue-2025-tension-tour/)![Kylie Minogue 2025 Tension Tour](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1727970352/EventEntry/n5r0z2or4zqttmtkzkyj.jpg)\r\n[Dr. Dinosaur!4/19/25](https://discoveratlanta.com/event/detail/dr-dinosaur/)![Dr. Dinosaur!](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1722962082/EventEntry/cf8ovr3mdwajqgmgsicz.jpg)\r\n[Move For Grady4/26/25](https://discoveratlanta.com/event/detail/move-for-grady-2/)![Move For Grady](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729705358/EventEntry/m5jlrwp3s6xacaju0y7w.jpg)\r\n[Kendrick Lamar and SZA\u2019s Grand National Tour4/29/25](https://discoveratlanta.com/event/detail/kendrick-lamar-and-sza-s-grand-national-tour/)![Kendrick Lamar and SZA\u0026#039;s Grand National Tour](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1603481265/EventEntry/DefaultImages/Musicconcerts.jpg)\r\n[Pearl Jam \u2013 Dark Matter Tour4/29/25 \u2013 5/1/25](https://discoveratlanta.com/event/detail/pearl-jam-dark-matter-tour/)![Pearl Jam - Dark Matter Tour](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734636991/EventEntry/kexhcxxh56wnbtdl0yas.jpg)\r\n[Fairy God Doctor5/3/25](https://discoveratlanta.com/event/detail/fairy-god-doctor/)![Fairy God Doctor](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1722962169/EventEntry/my9bxjghyi80mnt4mwuq.jpg)\r\n[The Oz Escape: A Wicked Glitch in Atlanta5/3/25](https://discoveratlanta.com/event/detail/the-oz-escape-a-wicked-glitch-in-atlanta/)![The Oz Escape: A Wicked Glitch in Atlanta](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1732001821/EventEntry/ojnx9ycru5ynfuadrhuw.jpg)\r\n[May the 4th Be With You: Sun, Moon, Air, \u0026#038; Space5/4/25](https://discoveratlanta.com/event/detail/may-the-4th-be-with-you-sun-moon-air-space/)![May the 4th Be With You: Sun, Moon, Air, \u0026#038; Space](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1721763532/EventEntry/hknmhvaizbowkmdzmkr4.jpg)\r\n[Fashion Play VI5/6/25](https://discoveratlanta.com/event/detail/fashion-play-vi/)![Fashion Play VI](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1726354349/EventEntry/zzfnxfuc2tgeym2cjll3.jpg)\r\n[A Kaleidoscope of American Music5/10/25](https://discoveratlanta.com/event/detail/a-kaleidoscope-of-american-music/)![A Kaleidoscope of American Music](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1725911245/EventEntry/u7jdlt6zomsaijzn3dym.jpg)\r\n[Atlanta Caribbean Carnival\u21225/10/25 \u2013 5/26/25](https://discoveratlanta.com/event/detail/atlanta-caribbean-carnival-2/)![Atlanta Caribbean Carnival\u2122](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1728321520/EventEntry/u5txbcjlzbqbzm9obt7m.jpg)\r\n[Post Malone \u2013 The Big Stadium Tour5/11/25](https://discoveratlanta.com/event/detail/post-malone-the-big-stadium-tour/)![Post Malone - The Big Stadium Tour](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734637893/EventEntry/d3agytfhtony7gj8hehm.jpg)\r\n[The Firebird5/17/25](https://discoveratlanta.com/event/detail/the-firebird/)![The Firebird](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1719701987/EventEntry/vspg61egxg5ed4izg5kb.jpg)\r\n[Oral History Association 2025 Annual Meeting10/14/25 \u2013 10/19/25](https://discoveratlanta.com/event/detail/oral-history-association-2025-annual-meeting/)![Oral History Association 2025 Annual Meeting](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1717436591/EventEntry/x3glj45aakebg4co7bay.jpg)\r\n[Cirque du Soleil\u2019s LUZIA11/9/25 \u2013 12/15/25](https://discoveratlanta.com/event/detail/cirque-du-soleil-s-luzia/)![Cirque du Soleil\u0026#039;s LUZIA](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729607993/EventEntry/z89ykp7z1y3cmrwloyeq.jpg)\r\n[2026 FIFA World Cup6/11/26 \u2013 7/19/26](https://discoveratlanta.com/event/detail/2026-fifa-world-cup/)![2026 FIFA World Cup](https://res.cloudinary.com/atlanta/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1599834351/EventEntry/DefaultImages/sports.jpg)\r\n[Holiday Bricktacular at LEGO Discovery Center!11/29/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/holiday-bricktacular-at-lego-discovery-center/)![Holiday Bricktacular at LEGO Discovery Center!](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731444310/EventEntry/ahs10viscnx6uk7p3jus.jpg)\r\n[Lenox Square\u2019s Canned Food Drive with ACFB11/29/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/lenox-square-s-canned-food-drive-with-acfb/)![Lenox Square\u2019s Canned Food Drive with ACFB](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1732638488/EventEntry/zynyjrwceczmfowsicd9.jpg)\r\n[Rooftop Dining Igloos12/31/24 \u2013 1/6/25](https://discoveratlanta.com/event/detail/rooftop-dining-igloos/)![Rooftop Dining Igloos](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731523408/EventEntry/urg2xyim0vqoboft0zmf.jpg)\r\n[Westside Sleigh Ride at Westside Motor Lounge11/29/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/westside-sleigh-ride-at-westside-motor-lounge/)![Westside Sleigh Ride at Westside Motor Lounge](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1732649478/EventEntry/wuuojlbzrbunurojdb8v.jpg)\r\n[Cat Eye Creative\u2019s \u201CThe Greatest Gift\u201D Exhibition12/31/24 \u2013 1/5/25](https://discoveratlanta.com/event/detail/cat-eye-creative-s-the-greatest-gift-exhibition/)![Cat Eye Creative\u0026#039;s ](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1732125325/EventEntry/dcg9yxduvw8dpuk1mdlo.jpg)\r\n[Spring HomeSchool Academy with Zoo AtlantaMultiple dates until 05/14/2025](https://discoveratlanta.com/event/detail/spring-homeschool-academy-with-zoo-atlanta/)![Spring HomeSchool Academy with Zoo Atlanta](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1638901209/EventEntry/plimqbinwlm4hgwlpifn.jpg)\r\n[History with Chutzpah3/9/23 \u2013 12/31/24](https://discoveratlanta.com/event/detail/history-with-chutzpah/)![History with Chutzpah](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1678392014/EventEntry/unmctzn1boobsmbm9keq.jpg)\r\n[Candlelight Concerts 20241/1/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/candlelight-concerts-2024/)![Candlelight Concerts 2024](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1603481265/EventEntry/DefaultImages/Musicconcerts.jpg)\r\n[WQEB 97.3 DINE \u0026#038; DONATE, a Food \u0026#038; Clothing Drive1/2/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/wqeb-97-3-dine-donate-a-food-clothing-drive/)![WQEB 97.3 DINE \u0026#038; DONATE, a Food \u0026#038; Clothing Drive](https://res.cloudinary.com/atlanta/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1600954997/EventEntry/DefaultImages/dining.jpg)\r\n[Adopt Any Senior Pet for FREE in 202412/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/adopt-any-senior-pet-for-free-in-2024/)![Adopt Any Senior Pet for FREE in 2024](https://res.cloudinary.com/atlanta/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1599834355/EventEntry/DefaultImages/pets.jpg)\r\n[Free Self-guided Street Art Walking Tours3/5/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/free-self-guided-street-art-walking-tours/)![Free Self-guided Street Art Walking Tours](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1709675802/EventEntry/khfjjtr9loqfsfknymqs.jpg)\r\n[Wing \u0026#038; Rock Fest12/31/24 \u2013 3/23/25](https://discoveratlanta.com/event/detail/wing-rock-fest/)![Wing \u0026#038; Rock Fest](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1723671471/EventEntry/vwjazegztlo66gigfv2z.jpg)\r\n[Spring Festival on PonceMultiple dates until 04/06/2025](https://discoveratlanta.com/event/detail/spring-festival-on-ponce-2/)![Spring Festival on Ponce](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1726831018/EventEntry/frnx5njtk4kgu636e9ri.jpg)\r\n[Pick Your Perks Group Promotion12/31/24 \u2013 5/31/25](https://discoveratlanta.com/event/detail/pick-your-perks-group-promotion/)![Pick Your Perks Group Promotion](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1716993671/EventEntry/lsnfyf8st8xlh5pzqibf.jpg)\r\n[Create Your Own Scented Candle!5/31/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/create-your-own-scented-candle/)![Create Your Own Scented Candle!](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1716910908/EventEntry/tzyggoratqzuata0llrg.jpg)\r\n[Kids Night Out12/31/24 \u2013 2/15/25](https://discoveratlanta.com/event/detail/kids-night-out/)![Kids Night Out](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1715624385/EventEntry/qlfitnf0z1aq1bd2ugaq.jpg)\r\n[Beginner\u2019s Salsa Lessons @ PasoFino Dance Studio12/31/24 \u2013 6/30/25](https://discoveratlanta.com/event/detail/beginner-s-salsa-lessons-pasofino-dance-studio/)![Beginner\u2019s Salsa Lessons @ PasoFino Dance Studio](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1718734218/EventEntry/dfxttpmu9kavha4roq0y.jpg)\r\n[The Fine Colombians: A Steely Dan Tribute12/31/24 \u2013 1/11/25](https://discoveratlanta.com/event/detail/the-fine-colombians-a-steely-dan-tribute-2/)![The Fine Colombians: A Steely Dan Tribute](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1728505138/EventEntry/eakxs1ibmmn5rjocbixs.jpg)\r\n[Titanic: An Immersive Voyage12/31/24 \u2013 1/22/25](https://discoveratlanta.com/event/detail/titanic-an-immersive-voyage/)![Titanic: An Immersive Voyage](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1725270644/EventEntry/fvetr7fvfozljrgzsc1z.jpg)\r\n[The Spirit Of Rush12/31/24 \u2013 1/24/25](https://discoveratlanta.com/event/detail/the-spirit-of-rush-2/)![The Spirit Of Rush](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1728501904/EventEntry/z10ptctgiwsrgbb0letc.jpg)\r\n[Twilight Trek at the ZooMultiple dates until 05/30/2025](https://discoveratlanta.com/event/detail/twilight-trek-at-the-zoo/)![Twilight Trek at the Zoo](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1715626057/EventEntry/nlnmvx8wnggmalpbanmh.jpg)\r\n[Golden Opportunity: Botanical Illustration12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/golden-opportunity-botanical-illustration/)![Golden Opportunity: Botanical Illustration](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1724873191/EventEntry/w6xjce2kd0jcmf3ksk4z.jpg)\r\n[Sacred Breaths12/31/24 \u2013 4/30/25](https://discoveratlanta.com/event/detail/sacred-breaths/)![Sacred Breaths](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1725945664/EventEntry/vmrdizsy2ix9squvz3ob.jpg)\r\n[Georgia O\u2019Keeffe: \u201CMy Two New Yorks\u201D12/31/24 \u2013 2/16/25](https://discoveratlanta.com/event/detail/georgia-o-keeffe-my-two-new-yorks/)![Georgia O\u0026#039;Keeffe: ](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1728936131/EventEntry/zxsennaah25xklwvxkwd.jpg)\r\n[Pizza and Beer CrawlMultiple dates until 04/27/2025](https://discoveratlanta.com/event/detail/pizza-and-beer-crawl/)![Pizza and Beer Crawl](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729015170/EventEntry/w8vxndj0rcjhnesmmav0.jpg)\r\n[Terminus Tours Atlanta \u2013 Private Guided City Tours11/1/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/terminus-tours-atlanta-private-guided-city-tours/)![Terminus Tours Atlanta - Private Guided City Tours](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731293699/EventEntry/ltpl7oqyv3kis4ovh9kg.jpg)\r\n[D-Intensity \u201CHibernation 25\u201D Bootcamp12/31/24 \u2013 3/30/25](https://discoveratlanta.com/event/detail/d-intensity-hibernation-25-bootcamp/)![D-Intensity ](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1728789689/EventEntry/yw0orsyvyzfw3qn2umpd.jpg)\r\n[Winter Lantern Festival12/31/24 \u2013 1/5/25](https://discoveratlanta.com/event/detail/winter-lantern-festival/)![Winter Lantern Festival](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1730320519/EventEntry/xeq73amjilhdwqr30xst.jpg)\r\n[The Dinner Detective Comedy Mystery Dinner ShowMultiple dates until 01/18/2025](https://discoveratlanta.com/event/detail/the-dinner-detective-comedy-mystery-dinner-show-3/)![The Dinner Detective Comedy Mystery Dinner Show](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731515100/EventEntry/hixa8x53rgy493iw2mbr.jpg)\r\n[Rooftop Brunch at 9 Mile Station12/31/24 \u2013 1/11/25](https://discoveratlanta.com/event/detail/rooftop-brunch-at-9-mile-station/)![Rooftop Brunch at 9 Mile Station](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731519701/EventEntry/jur6neeqjm39lvie9psg.jpg)\r\n[Skate City Springs12/31/24 \u2013 1/20/25](https://discoveratlanta.com/event/detail/skate-city-springs/)![Skate City Springs](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731436332/EventEntry/ekeatpkfbebqj9swfuyk.jpg)\r\n[Winter Wonderland at Fernbank12/31/24 \u2013 1/12/25](https://discoveratlanta.com/event/detail/winter-wonderland-at-fernbank/)![Winter Wonderland at Fernbank](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1668539627/EventEntry/x0rsixedeal1zovzdm3v.jpg)\r\n[The Ice Skating Rink Returns to Park Tavern12/31/24 \u2013 2/17/25](https://discoveratlanta.com/event/detail/the-ice-skating-rink-returns-to-park-tavern/)![The Ice Skating Rink Returns to Park Tavern](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1732024073/EventEntry/mkry1qstlh8e6ck0ludc.jpg)\r\n[Winter Rowing Camp Training (12-18y)12/31/24 \u2013 2/1/25](https://discoveratlanta.com/event/detail/winter-rowing-camp-training-12-18y/)![Winter Rowing Camp Training (12-18y)](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1732035183/EventEntry/oajtmqefsfcjoaatvvfx.jpg)\r\n[Midnight at Chateau: A glamorous New Year\u2019s Celebr12/31/24 \u2013 12/31/34](https://discoveratlanta.com/event/detail/midnight-at-chateau-a-glamorous-new-year-s-celebr/)![Midnight at Chateau: A glamorous New Year\u2019s Celebr](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734029239/EventEntry/clwa8hqqluwkxkn9hpj2.jpg)\r\n[Cat in the Hat4/8/25 \u2013 5/25/25](https://discoveratlanta.com/event/detail/cat-in-the-hat/)![Cat in the Hat](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1715796125/EventEntry/xcmkmnekaanup6un7qp5.jpg)\r\n[Editors\u2019 Pick December31Tuesday Stone Mountain Christmas12/31/24 \u2013 1/5/25](https://discoveratlanta.com/event/detail/stone-mountain-christmas/)![Editors Pick icon](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/icon_banner-editorspick.svg)\r\nTuesday\r\n\r\n![Stone Mountain Christmas](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1731510630/EventEntry/cs9byaqulhyyaoohafka.jpg)\r\n![Stone Mountain Christmas](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731510630/EventEntry/cs9byaqulhyyaoohafka.jpg)\r\n[Editors\u2019 Pick December31Tuesday IllumiNights at the Zoo!12/31/24 \u2013 1/19/25](https://discoveratlanta.com/event/detail/illuminights-at-the-zoo/)![Editors Pick icon](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/icon_banner-editorspick.svg)\r\nTuesday\r\n\r\n![IllumiNights at the Zoo!](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1699565286/EventEntry/i0ftuwkioqh5mpuqwtqq.jpg)\r\n![IllumiNights at the Zoo!](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1699565286/EventEntry/i0ftuwkioqh5mpuqwtqq.jpg)\r\n[Editors\u2019 Pick December31Tuesday Holidays at Georgia Aquarium12/31/24 \u2013 1/2/25](https://discoveratlanta.com/event/detail/holidays-at-georgia-aquarium/)![Editors Pick icon](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/icon_banner-editorspick.svg)\r\nTuesday\r\n\r\n![Holidays at Georgia Aquarium](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1699978789/EventEntry/l4morfwv7utlhtptmvud.jpg)\r\n![Holidays at Georgia Aquarium](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1699978789/EventEntry/l4morfwv7utlhtptmvud.jpg)\r\n[Editors\u2019 Pick December31Tuesday Holiday in the Park12/31/24 \u2013 1/5/25](https://discoveratlanta.com/event/detail/holiday-in-the-park/)![Editors Pick icon](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/icon_banner-editorspick.svg)\r\nTuesday\r\n\r\n![Holiday in the Park](https://res.cloudinary.com/atlanta/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1498181649/EventEntry/qmsdjxqxmoqemum2whmt.jpg)\r\n![Holiday in the Park](https://res.cloudinary.com/atlanta/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1498181649/EventEntry/qmsdjxqxmoqemum2whmt.jpg)\r\n[Editors\u2019 Pick December31Tuesday Deck the Hall at the College Football Hall of Fame12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/deck-the-hall-at-the-college-football-hall-of-fame/)![Editors Pick icon](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/icon_banner-editorspick.svg)\r\nTuesday\r\n\r\n![Deck the Hall at the College Football Hall of Fame](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734546567/EventEntry/qvbnx6yvfyrhtxxehurq.jpg)\r\n![Deck the Hall at the College Football Hall of Fame](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734546567/EventEntry/qvbnx6yvfyrhtxxehurq.jpg)\r\n[Editors\u2019 Pick December31Tuesday Atlanta\u2019s New Year\u2019s Eve Peach Drop Festival12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/atlanta-s-new-year-s-eve-peach-drop-festival/)![Editors Pick icon](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/icon_banner-editorspick.svg)\r\nTuesday\r\n\r\n![Atlanta\u0026#039;s New Year\u0026#039;s Eve Peach Drop Festival](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1669996517/EventEntry/wvln4rofrjlcjpilbub1.jpg)\r\n![Atlanta\u0026#039;s New Year\u0026#039;s Eve Peach Drop Festival](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1669996517/EventEntry/wvln4rofrjlcjpilbub1.jpg)\r\n[Editors\u2019 Pick February12Wednesday Alvin Ailey American Dance Theater Live at the Fox2/12/25 \u2013 2/16/25](https://discoveratlanta.com/event/detail/alvin-ailey-american-dance-theater-live-at-the-fox/)![Editors Pick icon](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/icon_banner-editorspick.svg)\r\nWednesday\r\n\r\n![Alvin Ailey American Dance Theater Live at the Fox](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1731532374/EventEntry/fuedgtfpemvwupfhjhrv.jpg)\r\n![Alvin Ailey American Dance Theater Live at the Fox](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731532374/EventEntry/fuedgtfpemvwupfhjhrv.jpg)\r\n[What\u2019s Hot December31Tuesday New Year\u2019s Eve Celebration12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/new-year-s-eve-celebration/)![Whats Hot icon](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/icon_banner-whatshot.svg)\r\nTuesday\r\n\r\n![New Year\u0026#039;s Eve Celebration](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1731005676/EventEntry/boc9cvr7vnqacpygwmkv.jpg)\r\n![New Year\u0026#039;s Eve Celebration](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731005676/EventEntry/boc9cvr7vnqacpygwmkv.jpg)\r\n[What\u2019s Hot December31Tuesday Noon Year\u2019s Eve Family Celebration12/31/24](https://discoveratlanta.com/event/detail/noon-year-s-eve-family-celebration/)![Whats Hot icon](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/icon_banner-whatshot.svg)\r\nTuesday\r\n\r\n![Noon Year\u0026#039;s Eve Family Celebration](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734622597/EventEntry/bp9l8tdm5mmozsq49axy.jpg)\r\n![Noon Year\u0026#039;s Eve Family Celebration](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734622597/EventEntry/bp9l8tdm5mmozsq49axy.jpg)\r\n[What\u2019s Hot January01Wednesday Chick-fil-A Peach Bowl1/1/25](https://discoveratlanta.com/event/detail/chick-fil-a-peach-bowl/)![Whats Hot icon](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/icon_banner-whatshot.svg)\r\nWednesday\r\n\r\n![Chick-fil-A Peach Bowl](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1702499768/EventEntry/wqm4kbr99lxseqihty08.jpg)\r\n![Chick-fil-A Peach Bowl](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1702499768/EventEntry/wqm4kbr99lxseqihty08.jpg)\r\n[What\u2019s Hot January18Saturday AT\u0026#038;T Playoff Playlist Live with Lil Wayne \u0026#038; More!1/18/25 \u2013 1/19/25](https://discoveratlanta.com/event/detail/at-t-playoff-playlist-live-with-lil-wayne-more/)![Whats Hot icon](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/icon_banner-whatshot.svg)\r\nSaturday\r\n\r\n![AT\u0026#038;T Playoff Playlist Live with Lil Wayne \u0026#038; More!](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734539959/EventEntry/vot9ijnqzxmxyqaz19s9.jpg)\r\n![AT\u0026#038;T Playoff Playlist Live with Lil Wayne \u0026#038; More!](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734539959/EventEntry/vot9ijnqzxmxyqaz19s9.jpg)\r\n[What\u2019s Hot January20Monday 2025 CFP National Championship1/20/25](https://discoveratlanta.com/event/detail/2025-cfp-national-championship/)![Whats Hot icon](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/icon_banner-whatshot.svg)\r\nMonday\r\n\r\n![2025 CFP National Championship](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734465369/EventEntry/riaxnf4gxtryne9onnxe.jpg)\r\n![2025 CFP National Championship](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734465369/EventEntry/riaxnf4gxtryne9onnxe.jpg)\r\n[What\u2019s Hot February24Monday Oysterfest2/24/25 \u2013 2/25/25](https://discoveratlanta.com/event/detail/oysterfest/)![Whats Hot icon](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/icon_banner-whatshot.svg)\r\nMonday\r\n\r\n![Oysterfest](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729799356/EventEntry/l42r9khyl9hox6wc1h0t.jpg)\r\n![Oysterfest](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729799356/EventEntry/l42r9khyl9hox6wc1h0t.jpg)\r\n[What\u2019s Hot February22Saturday Mardi Gras Streetcar Adventure2/22/25](https://discoveratlanta.com/event/detail/mardi-gras-streetcar-adventure/)![Whats Hot icon](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/icon_banner-whatshot.svg)\r\nSaturday\r\n\r\n![Mardi Gras Streetcar Adventure](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1702400087/EventEntry/bh9dlzzijyl4ai0szpdy.jpg)\r\n![Mardi Gras Streetcar Adventure](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1702400087/EventEntry/bh9dlzzijyl4ai0szpdy.jpg)\r\n[What\u2019s Hot December31Tuesday WildWoods AGLOW12/31/24 \u2013 2/23/25](https://discoveratlanta.com/event/detail/wildwoods-aglow/)![Whats Hot icon](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/icon_banner-whatshot.svg)\r\nTuesday\r\n\r\n![WildWoods AGLOW](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1668543833/EventEntry/nr03aawebcvgubpqushj.jpg)\r\n![WildWoods AGLOW](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1668543833/EventEntry/nr03aawebcvgubpqushj.jpg)\r\n[What\u2019s Hot December31Tuesday Skate the Station12/31/24 \u2013 1/20/25](https://discoveratlanta.com/event/detail/skate-the-station-2/)![Whats Hot icon](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/icon_banner-whatshot.svg)\r\nTuesday\r\n\r\n![Skate the Station](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1731074085/EventEntry/majwv2iiclw0xorelcx5.jpg)\r\n![Skate the Station](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731074085/EventEntry/majwv2iiclw0xorelcx5.jpg)\r\n[What\u2019s Hot December31Tuesday Garden Lights, Holiday Nights12/31/24 \u2013 1/12/25](https://discoveratlanta.com/event/detail/garden-lights-holiday-nights/)![Whats Hot icon](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/icon_banner-whatshot.svg)\r\nTuesday\r\n\r\n![Garden Lights, Holiday Nights](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1668014394/EventEntry/x4qngfueu0blab1uxxey.jpg)\r\n![Garden Lights, Holiday Nights](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1668014394/EventEntry/x4qngfueu0blab1uxxey.jpg)\r\n[December31Tuesday $ 3.00 Winter Flicks\u2013Despicable Me 4 at Aurora12/31/24 \u2013 1/2/25](https://discoveratlanta.com/event/detail/3-00-winter-flicks-despicable-me-4-at-aurora/)Tuesday\r\n\r\n![$ 3.00 Winter Flicks--Despicable Me 4 at Aurora](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1733420198/EventEntry/auyza0uprmq8g10ldj9d.jpg)\r\n![$ 3.00 Winter Flicks--Despicable Me 4 at Aurora](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733420198/EventEntry/auyza0uprmq8g10ldj9d.jpg)\r\n[December31Tuesday All-Inclusive NYE Party: Get Ready to Party12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/all-inclusive-nye-party-get-ready-to-party/)Tuesday\r\n\r\n![All-Inclusive NYE Party: Get Ready to Party](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1731956865/EventEntry/q0voillatcqynq0zzmyk.jpg)\r\n![All-Inclusive NYE Party: Get Ready to Party](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731956865/EventEntry/q0voillatcqynq0zzmyk.jpg)\r\n[December31Tuesday Atlanta New Years Eve Bar Crawl12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/atlanta-new-years-eve-bar-crawl/)Tuesday\r\n\r\n![Atlanta New Years Eve Bar Crawl](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1733376932/EventEntry/hyli54rajah2pdykrwo2.jpg)\r\n![Atlanta New Years Eve Bar Crawl](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733376932/EventEntry/hyli54rajah2pdykrwo2.jpg)\r\n[December31Tuesday Champions \u2013 A Celebration of Queen NYE Concert12/31/24](https://discoveratlanta.com/event/detail/champions-a-celebration-of-queen-nye-concert/)Tuesday\r\n\r\n![Champions - A Celebration of Queen NYE Concert](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729280087/EventEntry/y0pgtzmte4vzv0memoyq.jpg)\r\n![Champions - A Celebration of Queen NYE Concert](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729280087/EventEntry/y0pgtzmte4vzv0memoyq.jpg)\r\n[December31Tuesday Chick-fil-A Peach Bowl Fan Night12/31/24](https://discoveratlanta.com/event/detail/chick-fil-a-peach-bowl-fan-night/)Tuesday\r\n\r\n![Chick-fil-A Peach Bowl Fan Night](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1731700309/EventEntry/skchhcylupk0rqy3ivuj.jpg)\r\n![Chick-fil-A Peach Bowl Fan Night](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731700309/EventEntry/skchhcylupk0rqy3ivuj.jpg)\r\n[December31Tuesday Escondido 2025 New Year\u2019s Eve Party12/31/24](https://discoveratlanta.com/event/detail/escondido-2025-new-year-s-eve-party/)Tuesday\r\n\r\n![Escondido 2025 New Year\u0026#039;s Eve Party](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1733265466/EventEntry/fqlvfpstwdb69v4ogcsv.jpg)\r\n![Escondido 2025 New Year\u0026#039;s Eve Party](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733265466/EventEntry/fqlvfpstwdb69v4ogcsv.jpg)\r\n[December31Tuesday Georgia Tech Men\u2019s Basketball Vs Notre Dame12/31/24](https://discoveratlanta.com/event/detail/georgia-tech-men-s-basketball-vs-notre-dame/)Tuesday\r\n\r\n![Georgia Tech Men\u2019s Basketball Vs Notre Dame](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729615389/EventEntry/zzlwwz35r8l0naedrja0.jpg)\r\n![Georgia Tech Men\u2019s Basketball Vs Notre Dame](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729615389/EventEntry/zzlwwz35r8l0naedrja0.jpg)\r\n[December31Tuesday Midnight at Chateau12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/midnight-at-chateau/)Tuesday\r\n\r\n![Midnight at Chateau](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734542272/EventEntry/ee1l9tcromkg7tdhohey.jpg)\r\n![Midnight at Chateau](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734542272/EventEntry/ee1l9tcromkg7tdhohey.jpg)\r\n[December31Tuesday NYE @ May Peel \u2013 A Champagne Story12/31/24](https://discoveratlanta.com/event/detail/nye-may-peel-a-champagne-story/)Tuesday\r\n\r\n![NYE @ May Peel \u2013 A Champagne Story](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734536118/EventEntry/tartv5l8esay4enxh1bh.jpg)\r\n![NYE @ May Peel \u2013 A Champagne Story](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734536118/EventEntry/tartv5l8esay4enxh1bh.jpg)\r\n[December31Tuesday NYE: Sear After Dark!12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/nye-sear-after-dark/)Tuesday\r\n\r\n![NYE: Sear After Dark!](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734361150/EventEntry/qmepnjlgkjcftge6zclg.jpg)\r\n![NYE: Sear After Dark!](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734361150/EventEntry/qmepnjlgkjcftge6zclg.jpg)\r\n[December31Tuesday New Year\u2019s Eve Celebration at Nobu12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/new-year-s-eve-celebration-at-nobu/)Tuesday\r\n\r\n![New Year\u0026#039;s Eve Celebration at Nobu](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729195483/EventEntry/i4gbiwxyuws7wzdc5lrn.jpg)\r\n![New Year\u0026#039;s Eve Celebration at Nobu](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729195483/EventEntry/i4gbiwxyuws7wzdc5lrn.jpg)\r\n[December31Tuesday New Year\u2019s Eve Celebration at Red Phone Booth 202412/31/24](https://discoveratlanta.com/event/detail/new-year-s-eve-celebration-at-red-phone-booth-2024-2/)Tuesday\r\n\r\n![New Year\u0026#039;s Eve Celebration at Red Phone Booth 2024](https://res.cloudinary.com/atlanta/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1599834269/EventEntry/DefaultImages/holiday.jpg)\r\n![New Year\u0026#039;s Eve Celebration at Red Phone Booth 2024](https://res.cloudinary.com/atlanta/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1599834269/EventEntry/DefaultImages/holiday.jpg)\r\n[December31Tuesday New Year\u2019s Eve Dinner at Signia by Hilton Atlanta12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/new-year-s-eve-dinner-at-signia-by-hilton-atlanta/)Tuesday\r\n\r\n![New Year\u0026#039;s Eve Dinner at Signia by Hilton Atlanta](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1730752354/EventEntry/ixcmxoubtw3rtlcvfbws.jpg)\r\n![New Year\u0026#039;s Eve Dinner at Signia by Hilton Atlanta](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1730752354/EventEntry/ixcmxoubtw3rtlcvfbws.jpg)\r\n[December31Tuesday New Years Eve Party 2025 at Buena Vida Tapas Bar12/31/24](https://discoveratlanta.com/event/detail/new-years-eve-party-2025-at-buena-vida-tapas-bar/)Tuesday\r\n\r\n![New Years Eve Party 2025 at Buena Vida Tapas Bar](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1733338355/EventEntry/tx19zcpm5gdammi6msqd.jpg)\r\n![New Years Eve Party 2025 at Buena Vida Tapas Bar](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733338355/EventEntry/tx19zcpm5gdammi6msqd.jpg)\r\n[December31Tuesday New Year\u2019s Eve Bash at The Whitley12/31/24](https://discoveratlanta.com/event/detail/new-year-s-eve-bash-at-the-whitley/)Tuesday\r\n\r\n![New Year\u2019s Eve Bash at The Whitley](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1731372920/EventEntry/mtombkqoeqfs3zj02teb.jpg)\r\n![New Year\u2019s Eve Bash at The Whitley](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731372920/EventEntry/mtombkqoeqfs3zj02teb.jpg)\r\n[December31Tuesday New Year\u2019s Eve Black-Tie Dinner at The Waldorf12/31/24](https://discoveratlanta.com/event/detail/new-year-s-eve-black-tie-dinner-at-the-waldorf/)Tuesday\r\n\r\n![New Year\u2019s Eve Black-Tie Dinner at The Waldorf](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734621279/EventEntry/dswxoa43zrqdv1ifjqeq.jpg)\r\n![New Year\u2019s Eve Black-Tie Dinner at The Waldorf](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734621279/EventEntry/dswxoa43zrqdv1ifjqeq.jpg)\r\n[December31Tuesday New Year\u2019s Eve Black-Tie Gala at The Waldorf12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/new-year-s-eve-black-tie-gala-at-the-waldorf/)Tuesday\r\n\r\n![New Year\u2019s Eve Black-Tie Gala at The Waldorf](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734633963/EventEntry/fwhenoca5zs8whpy2vmq.jpg)\r\n![New Year\u2019s Eve Black-Tie Gala at The Waldorf](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734633963/EventEntry/fwhenoca5zs8whpy2vmq.jpg)\r\n[December31Tuesday Noon Year\u2019s Eve Balloon Drop12/31/24](https://discoveratlanta.com/event/detail/noon-year-s-eve-balloon-drop/)Tuesday\r\n\r\n![Noon Year\u0026#039;s Eve Balloon Drop](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1733839576/EventEntry/a327ptjhn6nkzpbh7qzz.jpg)\r\n![Noon Year\u0026#039;s Eve Balloon Drop](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733839576/EventEntry/a327ptjhn6nkzpbh7qzz.jpg)\r\n[December31Tuesday Noon Year\u2019s Eve at LEGO Discovery Center Atlanta12/31/24](https://discoveratlanta.com/event/detail/noon-year-s-eve-at-lego-discovery-center-atlanta/)Tuesday\r\n\r\n![Noon Year\u0026#039;s Eve at LEGO Discovery Center Atlanta](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734577971/EventEntry/mbgc1yk4glqiyq3zxnxs.jpg)\r\n![Noon Year\u0026#039;s Eve at LEGO Discovery Center Atlanta](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734577971/EventEntry/mbgc1yk4glqiyq3zxnxs.jpg)\r\n[December31Tuesday Overstory\u2019s New Year\u2019s Eve Celebration12/31/24](https://discoveratlanta.com/event/detail/overstory-s-new-year-s-eve-celebration/)Tuesday\r\n\r\n![Overstory\u0026#039;s New Year\u0026#039;s Eve Celebration](https://res.cloudinary.com/atlanta/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1599834376/EventEntry/DefaultImages/nightlife.jpg)\r\n![Overstory\u0026#039;s New Year\u0026#039;s Eve Celebration](https://res.cloudinary.com/atlanta/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1599834376/EventEntry/DefaultImages/nightlife.jpg)\r\n[December31Tuesday Peach Bowl Fan Night at Georgia Aquarium12/31/24](https://discoveratlanta.com/event/detail/peach-bowl-fan-night-at-georgia-aquarium/)Tuesday\r\n\r\n![Peach Bowl Fan Night at Georgia Aquarium](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1733799909/EventEntry/szssgmvhejb1senxk3df.jpg)\r\n![Peach Bowl Fan Night at Georgia Aquarium](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733799909/EventEntry/szssgmvhejb1senxk3df.jpg)\r\n[December31Tuesday Toast to 2025: Amore e Amore Rings in the New Year12/31/24](https://discoveratlanta.com/event/detail/toast-to-2025-amore-e-amore-rings-in-the-new-year/)Tuesday\r\n\r\n![Toast to 2025: Amore e Amore Rings in the New Year](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734541369/EventEntry/aoouncrlkwrd5rdbyt3z.jpg)\r\n![Toast to 2025: Amore e Amore Rings in the New Year](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734541369/EventEntry/aoouncrlkwrd5rdbyt3z.jpg)\r\n[December31Tuesday Winter Chalet at InterContinental Buckhead12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/winter-chalet-at-intercontinental-buckhead/)Tuesday\r\n\r\n![Winter Chalet at InterContinental Buckhead](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1733351858/EventEntry/mfd9445t9vnfrkcouv5j.jpg)\r\n![Winter Chalet at InterContinental Buckhead](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733351858/EventEntry/mfd9445t9vnfrkcouv5j.jpg)\r\n[December31Tuesday Zoo Year\u2019s Eve12/31/24](https://discoveratlanta.com/event/detail/zoo-year-s-eve/)Tuesday\r\n\r\n![Zoo Year\u0026#039;s Eve](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1730319012/EventEntry/a7yxccj5vtoqj86epj8v.jpg)\r\n![Zoo Year\u0026#039;s Eve](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1730319012/EventEntry/a7yxccj5vtoqj86epj8v.jpg)\r\n[January01Wednesday 2025 Karaoke 2851/1/25](https://discoveratlanta.com/event/detail/2025-karaoke-285/)Wednesday\r\n\r\n![2025 Karaoke 285](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1732315091/EventEntry/jthiaf9rzkuuqnzlofii.jpg)\r\n![2025 Karaoke 285](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1732315091/EventEntry/jthiaf9rzkuuqnzlofii.jpg)\r\n[January01Wednesday Abuelita\u2019s Brunch: Wild Heaven West End1/1/25](https://discoveratlanta.com/event/detail/abuelita-s-brunch-wild-heaven-west-end/)Wednesday\r\n\r\n![Abuelita\u2019s Brunch: Wild Heaven West End](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1733846250/EventEntry/uzd2bthgonlrmlm64xti.jpg)\r\n![Abuelita\u2019s Brunch: Wild Heaven West End](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733846250/EventEntry/uzd2bthgonlrmlm64xti.jpg)\r\n[January01Wednesday Celebrate New Year\u2019s Day at Painted Pickle1/1/25](https://discoveratlanta.com/event/detail/celebrate-new-year-s-day-at-painted-pickle/)Wednesday\r\n\r\n![Celebrate New Year\u2019s Day at Painted Pickle](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734113178/EventEntry/ljfpxg0tjjto96fcehrs.jpg)\r\n![Celebrate New Year\u2019s Day at Painted Pickle](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734113178/EventEntry/ljfpxg0tjjto96fcehrs.jpg)\r\n[January01Wednesday YOGA \u0026#038; SOUND BATH1/1/25](https://discoveratlanta.com/event/detail/yoga-sound-bath/)Wednesday\r\n\r\n![YOGA \u0026#038; SOUND BATH](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1733778569/EventEntry/l4xqitka5a0b6kqdyea9.jpg)\r\n![YOGA \u0026#038; SOUND BATH](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733778569/EventEntry/l4xqitka5a0b6kqdyea9.jpg)\r\n[January02Thursday Georgia Tech Women\u2019s Basketball vs Syracuse1/2/25](https://discoveratlanta.com/event/detail/georgia-tech-women-s-basketball-vs-syracuse/)Thursday\r\n\r\n![Georgia Tech Women\u2019s Basketball vs Syracuse](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729695761/EventEntry/uqzn4eq3bqelaqdsphg2.jpg)\r\n![Georgia Tech Women\u2019s Basketball vs Syracuse](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729695761/EventEntry/uqzn4eq3bqelaqdsphg2.jpg)\r\n[January04Saturday Georgia Tech Men\u2019s Basketball Vs Boston College1/4/25](https://discoveratlanta.com/event/detail/georgia-tech-men-s-basketball-vs-boston-college/)Saturday\r\n\r\n![Georgia Tech Men\u2019s Basketball Vs Boston College](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729615508/EventEntry/ynankil3kztugbnfpsnf.jpg)\r\n![Georgia Tech Men\u2019s Basketball Vs Boston College](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729615508/EventEntry/ynankil3kztugbnfpsnf.jpg)\r\n[January04Saturday Ray Howard Band Presents The Ultimate EWF1/4/25](https://discoveratlanta.com/event/detail/ray-howard-band-presents-the-ultimate-ewf/)Saturday\r\n\r\n![Ray Howard Band Presents The Ultimate EWF](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1728503928/EventEntry/vyd3e1ncktk9fuauaall.jpg)\r\n![Ray Howard Band Presents The Ultimate EWF](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1728503928/EventEntry/vyd3e1ncktk9fuauaall.jpg)\r\n[January04Saturday Sonic Symphony1/4/25](https://discoveratlanta.com/event/detail/sonic-symphony/)Saturday\r\n\r\n![Sonic Symphony](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1731399019/EventEntry/bgm95ivs2pe6bktwyjjz.jpg)\r\n![Sonic Symphony](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731399019/EventEntry/bgm95ivs2pe6bktwyjjz.jpg)\r\n[January05Sunday Carolina Panthers at Atlanta Falcons1/5/25](https://discoveratlanta.com/event/detail/carolina-panthers-at-atlanta-falcons/)Sunday\r\n\r\n![Carolina Panthers at Atlanta Falcons](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n![Carolina Panthers at Atlanta Falcons](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n[January07Tuesday Thai Omakase with Chef Pattie \u0026#038; Mo\u00EBt Hennessy1/7/25](https://discoveratlanta.com/event/detail/thai-omakase-with-chef-pattie-moet-hennessy/)Tuesday\r\n\r\n![Thai Omakase with Chef Pattie \u0026#038; Mo\u00EBt Hennessy](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1733856927/EventEntry/m3t3rbtdqqkfg8vj8vnu.jpg)\r\n![Thai Omakase with Chef Pattie \u0026#038; Mo\u00EBt Hennessy](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733856927/EventEntry/m3t3rbtdqqkfg8vj8vnu.jpg)\r\n[January12Sunday \u0026#038; Juliet1/9/25 \u2013 1/12/25](https://discoveratlanta.com/event/detail/juliet/)Sunday\r\n\r\n![\u0026#038; Juliet](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n![\u0026#038; Juliet](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n[January09Thursday 2025 DSC Convention \u0026#038; Sporting Expo1/9/25 \u2013 1/12/25](https://discoveratlanta.com/event/detail/2025-dsc-convention-sporting-expo/)Thursday\r\n\r\n![2025 DSC Convention \u0026#038; Sporting Expo](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1721159445/EventEntry/jwpi2cmjkwe2rlhz8yhl.jpg)\r\n![2025 DSC Convention \u0026#038; Sporting Expo](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1721159445/EventEntry/jwpi2cmjkwe2rlhz8yhl.jpg)\r\n[January09Thursday Discover Boating Atlanta Boat Show1/9/25 \u2013 1/12/25](https://discoveratlanta.com/event/detail/discover-boating-atlanta-boat-show/)Thursday\r\n\r\n![Discover Boating Atlanta Boat Show](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1732118753/EventEntry/u4gkgldqq0tensnbazkm.jpg)\r\n![Discover Boating Atlanta Boat Show](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1732118753/EventEntry/u4gkgldqq0tensnbazkm.jpg)\r\n[January09Thursday Georgia Tech Women\u2019s Basketball vs Virginia Tech1/9/25](https://discoveratlanta.com/event/detail/georgia-tech-women-s-basketball-vs-virginia-tech/)Thursday\r\n\r\n![Georgia Tech Women\u2019s Basketball vs Virginia Tech](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729695919/EventEntry/ukutgvei3poqnvezxoo4.jpg)\r\n![Georgia Tech Women\u2019s Basketball vs Virginia Tech](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729695919/EventEntry/ukutgvei3poqnvezxoo4.jpg)\r\n[January10Friday Donna The Buffalo1/10/25](https://discoveratlanta.com/event/detail/donna-the-buffalo/)Friday\r\n\r\n![Donna The Buffalo](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1728504537/EventEntry/sqflakdjmunynct4jdue.jpg)\r\n![Donna The Buffalo](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1728504537/EventEntry/sqflakdjmunynct4jdue.jpg)\r\n[January10Friday Portraits \u0026#038; Prosecco1/10/25](https://discoveratlanta.com/event/detail/portraits-prosecco/)Friday\r\n\r\n![Portraits \u0026#038; Prosecco](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734392978/EventEntry/p322nelbcqucnumnrq1r.jpg)\r\n![Portraits \u0026#038; Prosecco](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734392978/EventEntry/p322nelbcqucnumnrq1r.jpg)\r\n[January11Saturday Tree Tour: History, Change on the Atlanta Beltline1/11/25](https://discoveratlanta.com/event/detail/tree-tour-history-change-on-the-atlanta-beltline/)Saturday\r\n\r\n![Tree Tour: History, Change on the Atlanta Beltline](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1733146311/EventEntry/vmsx2kyhka9ob3cpmtar.jpg)\r\n![Tree Tour: History, Change on the Atlanta Beltline](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733146311/EventEntry/vmsx2kyhka9ob3cpmtar.jpg)\r\n[January12Sunday NFC Brew Jitsu #3 at Monday Night Garage1/12/25](https://discoveratlanta.com/event/detail/nfc-brew-jitsu-3-at-monday-night-garage/)Sunday\r\n\r\n![NFC Brew Jitsu #3 at Monday Night Garage](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1730933122/EventEntry/rf5r5uzefnam0n9gqzdo.jpg)\r\n![NFC Brew Jitsu #3 at Monday Night Garage](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1730933122/EventEntry/rf5r5uzefnam0n9gqzdo.jpg)\r\n[January13Monday Full Moon Sound Bath x Dear Life Chat1/13/25](https://discoveratlanta.com/event/detail/full-moon-sound-bath-x-dear-life-chat/)Monday\r\n\r\n![Full Moon Sound Bath x Dear Life Chat](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734572407/EventEntry/nrokwfryoumiluyconai.jpg)\r\n![Full Moon Sound Bath x Dear Life Chat](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734572407/EventEntry/nrokwfryoumiluyconai.jpg)\r\n[January14Tuesday Georgia Tech Men\u2019s Basketball Vs Clemson1/14/25](https://discoveratlanta.com/event/detail/georgia-tech-men-s-basketball-vs-clemson/)Tuesday\r\n\r\n![Georgia Tech Men\u2019s Basketball Vs Clemson](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729615686/EventEntry/lomy5nwruywsvp4sblpw.jpg)\r\n![Georgia Tech Men\u2019s Basketball Vs Clemson](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729615686/EventEntry/lomy5nwruywsvp4sblpw.jpg)\r\n[January19Sunday CFP Taste of the Championship Atlanta1/19/25](https://discoveratlanta.com/event/detail/cfp-taste-of-the-championship-atlanta/)Sunday\r\n\r\n![CFP Taste of the Championship Atlanta](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734464937/EventEntry/vk1qmw7hdetbaupmi5lu.jpg)\r\n![CFP Taste of the Championship Atlanta](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734464937/EventEntry/vk1qmw7hdetbaupmi5lu.jpg)\r\n[January19Sunday Georgia Tech Women\u2019s Basketball vs Clemson1/19/25](https://discoveratlanta.com/event/detail/georgia-tech-women-s-basketball-vs-clemson/)Sunday\r\n\r\n![Georgia Tech Women\u2019s Basketball vs Clemson](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729696046/EventEntry/hkdj81zh4nsxsktxxegx.jpg)\r\n![Georgia Tech Women\u2019s Basketball vs Clemson](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729696046/EventEntry/hkdj81zh4nsxsktxxegx.jpg)\r\n[January19Sunday The Everly Set1/19/25](https://discoveratlanta.com/event/detail/the-everly-set/)Sunday\r\n\r\n![The Everly Set](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1733348717/EventEntry/bl53uthu317vkxynbb3x.jpg)\r\n![The Everly Set](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733348717/EventEntry/bl53uthu317vkxynbb3x.jpg)\r\n[January21Tuesday College Football Playoff National Championship1/21/25](https://discoveratlanta.com/event/detail/college-football-playoff-national-championship/)Tuesday\r\n\r\n![College Football Playoff National Championship](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n![College Football Playoff National Championship](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n[January22Wednesday Georgia Tech Men\u2019s Basketball Vs Virginia Tech1/22/25](https://discoveratlanta.com/event/detail/georgia-tech-men-s-basketball-vs-virginia-tech/)Wednesday\r\n\r\n![Georgia Tech Men\u2019s Basketball Vs Virginia Tech](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729615833/EventEntry/c5nzrfp4c4fsufbyeoru.jpg)\r\n![Georgia Tech Men\u2019s Basketball Vs Virginia Tech](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729615833/EventEntry/c5nzrfp4c4fsufbyeoru.jpg)\r\n[January23Thursday Maternal Health Awareness Day 20251/23/25](https://discoveratlanta.com/event/detail/maternal-health-awareness-day-2025/)Thursday\r\n\r\n![Maternal Health Awareness Day 2025](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734013179/EventEntry/ecmitunnqsjzbt1dyxsu.jpg)\r\n![Maternal Health Awareness Day 2025](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734013179/EventEntry/ecmitunnqsjzbt1dyxsu.jpg)\r\n[January23Thursday Movies at The Strand: Anastasia (1997)1/23/25](https://discoveratlanta.com/event/detail/movies-at-the-strand-anastasia-1997/)Thursday\r\n\r\n![Movies at The Strand: Anastasia (1997)](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1733411491/EventEntry/vdtctmt1fz4yv67yjgtz.jpg)\r\n![Movies at The Strand: Anastasia (1997)](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733411491/EventEntry/vdtctmt1fz4yv67yjgtz.jpg)\r\n[January24Friday FSR Presents: Life Be Lifin\u2019 Starring Monet X Change1/24/25](https://discoveratlanta.com/event/detail/fsr-presents-life-be-lifin-starring-monet-x-change/)Friday\r\n\r\n![FSR Presents: Life Be Lifin\u0026#039; Starring Monet X Change](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n![FSR Presents: Life Be Lifin\u0026#039; Starring Monet X Change](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n[January24Friday First Arab Elbow Course- The Traumatic1/24/25 \u2013 1/26/25](https://discoveratlanta.com/event/detail/first-arab-elbow-course-the-traumatic/)Friday\r\n\r\n![First Arab Elbow Course- The Traumatic](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1718963670/EventEntry/cjtlswvklqlcf2fttzdm.jpg)\r\n![First Arab Elbow Course- The Traumatic](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1718963670/EventEntry/cjtlswvklqlcf2fttzdm.jpg)\r\n[January24Friday Jazz at The Strand: The Music of John Coltrane 2021/24/25](https://discoveratlanta.com/event/detail/jazz-at-the-strand-the-music-of-john-coltrane-202/)Friday\r\n\r\n![Jazz at The Strand: The Music of John Coltrane 202](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1733413611/EventEntry/icxos2f3rdegxph5ht5n.jpg)\r\n![Jazz at The Strand: The Music of John Coltrane 202](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733413611/EventEntry/icxos2f3rdegxph5ht5n.jpg)\r\n[January25Saturday Grady CrossTies Gala1/25/25 \u2013 1/26/25](https://discoveratlanta.com/event/detail/grady-crossties-gala/)Saturday\r\n\r\n![Grady CrossTies Gala](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1733949365/EventEntry/cx7nry1bi6nc8096zbi5.jpg)\r\n![Grady CrossTies Gala](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733949365/EventEntry/cx7nry1bi6nc8096zbi5.jpg)\r\n[January25Saturday Heroes Among Us: A Musical Tribute1/25/25](https://discoveratlanta.com/event/detail/heroes-among-us-a-musical-tribute/)Saturday\r\n\r\n![Heroes Among Us: A Musical Tribute](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1725910962/EventEntry/ecrzteqlx0cxy5ssylpb.jpg)\r\n![Heroes Among Us: A Musical Tribute](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1725910962/EventEntry/ecrzteqlx0cxy5ssylpb.jpg)\r\n[January25Saturday Indie Game Pop-Up @ CMoA1/25/25](https://discoveratlanta.com/event/detail/indie-game-pop-up-cmoa/)Saturday\r\n\r\n![Indie Game Pop-Up @ CMoA](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734196514/EventEntry/owzuflwpbzfttjotscec.jpg)\r\n![Indie Game Pop-Up @ CMoA](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734196514/EventEntry/owzuflwpbzfttjotscec.jpg)\r\n[January25Saturday Lunar New Year Celebration1/25/25](https://discoveratlanta.com/event/detail/lunar-new-year-celebration/)Saturday\r\n\r\n![Lunar New Year Celebration](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1722961018/EventEntry/hwviq1drjnc7fbwgkgqv.jpg)\r\n![Lunar New Year Celebration](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1722961018/EventEntry/hwviq1drjnc7fbwgkgqv.jpg)\r\n[January25Saturday Magician Rich Crawford Live1/25/25](https://discoveratlanta.com/event/detail/magician-rich-crawford-live/)Saturday\r\n\r\n![Magician Rich Crawford Live](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729660580/EventEntry/yyfckqjnw91p9kjsmghs.jpg)\r\n![Magician Rich Crawford Live](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729660580/EventEntry/yyfckqjnw91p9kjsmghs.jpg)\r\n[January25Saturday What\u2019s Goin\u2019 on at Shorty!1/25/25](https://discoveratlanta.com/event/detail/what-s-goin-on-at-shorty/)Saturday\r\n\r\n![What\u0026#039;s Goin\u0026#039; on at Shorty!](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1733237473/EventEntry/jd1wksbzzgfxdwl6gdd1.jpg)\r\n![What\u0026#039;s Goin\u0026#039; on at Shorty!](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733237473/EventEntry/jd1wksbzzgfxdwl6gdd1.jpg)\r\n[January26Sunday Georgia Tech Women\u2019s Basketball vs Duke1/26/25](https://discoveratlanta.com/event/detail/georgia-tech-women-s-basketball-vs-duke/)Sunday\r\n\r\n![Georgia Tech Women\u2019s Basketball vs Duke](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729696270/EventEntry/e82rg9uinawhqdi0f5hj.jpg)\r\n![Georgia Tech Women\u2019s Basketball vs Duke](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729696270/EventEntry/e82rg9uinawhqdi0f5hj.jpg)\r\n[January28Tuesday Clue WILL VISIT ATLANTA\u2019S FOX THEATRE1/28/25 \u2013 2/2/25](https://discoveratlanta.com/event/detail/clue-will-visit-atlanta-s-fox-theatre/)Tuesday\r\n\r\n![Clue WILL VISIT ATLANTA\u2019S FOX THEATRE](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1733950229/EventEntry/evf0lziixzb36jbc50rp.jpg)\r\n![Clue WILL VISIT ATLANTA\u2019S FOX THEATRE](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733950229/EventEntry/evf0lziixzb36jbc50rp.jpg)\r\n[January28Tuesday Rainforest Adventures1/28/25 \u2013 3/2/25](https://discoveratlanta.com/event/detail/rainforest-adventures/)Tuesday\r\n\r\n![Rainforest Adventures](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1730921950/EventEntry/hxjk0utzvq5hsjcfdhtr.jpg)\r\n![Rainforest Adventures](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1730921950/EventEntry/hxjk0utzvq5hsjcfdhtr.jpg)\r\n[January29Wednesday 2025 Flourish Family Homeschool Conference1/29/25 \u2013 1/31/25](https://discoveratlanta.com/event/detail/2025-flourish-family-homeschool-conference/)Wednesday\r\n\r\n![2025 Flourish Family Homeschool Conference](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1727841207/EventEntry/awzsvj1c37pudhtvsc3i.jpg)\r\n![2025 Flourish Family Homeschool Conference](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1727841207/EventEntry/awzsvj1c37pudhtvsc3i.jpg)\r\n[January29Wednesday Lucas Jack LIVE at Eddie\u2019s Attic1/29/25](https://discoveratlanta.com/event/detail/lucas-jack-live-at-eddie-s-attic/)Wednesday\r\n\r\n![Lucas Jack LIVE at Eddie\u2019s Attic](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1728936308/EventEntry/zuethiczdysamzacsb6t.jpg)\r\n![Lucas Jack LIVE at Eddie\u2019s Attic](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1728936308/EventEntry/zuethiczdysamzacsb6t.jpg)\r\n[February01Saturday Clue The Musical1/30/25 \u2013 2/1/25](https://discoveratlanta.com/event/detail/clue-the-musical/)Saturday\r\n\r\n![Clue The Musical](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n![Clue The Musical](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n[January30Thursday Rod Wave \u2013 The Last Lap Tour1/30/25](https://discoveratlanta.com/event/detail/rod-wave-the-last-lap-tour/)Thursday\r\n\r\n![Rod Wave - The Last Lap Tour](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734556973/EventEntry/avps9yrapwiezd8ggst9.jpg)\r\n![Rod Wave - The Last Lap Tour](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734556973/EventEntry/avps9yrapwiezd8ggst9.jpg)\r\n[January31Friday \u201990s Night at Fernbank1/31/25](https://discoveratlanta.com/event/detail/90s-night-at-fernbank/)Friday\r\n\r\n![\u0026#039;90s Night at Fernbank](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1731937209/EventEntry/ytozp6qy2wuy0hx7jfbq.jpg)\r\n![\u0026#039;90s Night at Fernbank](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731937209/EventEntry/ytozp6qy2wuy0hx7jfbq.jpg)\r\n[January31Friday Groundhog Day Jugglers Festival1/31/25 \u2013 2/2/25](https://discoveratlanta.com/event/detail/groundhog-day-jugglers-festival/)Friday\r\n\r\n![Groundhog Day Jugglers Festival](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1672845037/EventEntry/l7vzn6xaizyefmqsymtt.jpg)\r\n![Groundhog Day Jugglers Festival](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1672845037/EventEntry/l7vzn6xaizyefmqsymtt.jpg)\r\n[January31Friday Vision Board Workshop1/31/25](https://discoveratlanta.com/event/detail/vision-board-workshop/)Friday\r\n\r\n![Vision Board Workshop](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1731703251/EventEntry/d5gxsux4ocossysy44oh.jpg)\r\n![Vision Board Workshop](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731703251/EventEntry/d5gxsux4ocossysy44oh.jpg)\r\n[February01Saturday FLOTAX test2/1/25](https://discoveratlanta.com/event/detail/flotax-test/)Saturday\r\n\r\n![FLOTAX test](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n![FLOTAX test](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n[February01Saturday Georgia Tech Men\u2019s Basketball Vs Louisville2/1/25](https://discoveratlanta.com/event/detail/georgia-tech-men-s-basketball-vs-louisville/)Saturday\r\n\r\n![Georgia Tech Men\u2019s Basketball Vs Louisville](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729616285/EventEntry/k4lvbsrm6z72tndb9eh1.jpg)\r\n![Georgia Tech Men\u2019s Basketball Vs Louisville](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729616285/EventEntry/k4lvbsrm6z72tndb9eh1.jpg)\r\n[February01Saturday Hot Wheels Monster Trucks LiveMultiple dates until 02/02/2025](https://discoveratlanta.com/event/detail/hot-wheels-monster-trucks-live/)Saturday\r\n\r\n![Hot Wheels Monster Trucks Live](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1727798851/EventEntry/f0ewizhmzrccwg4wvvya.jpg)\r\n![Hot Wheels Monster Trucks Live](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1727798851/EventEntry/f0ewizhmzrccwg4wvvya.jpg)\r\n[February02Sunday Hot Chocolate Run Atlanta2/2/25](https://discoveratlanta.com/event/detail/hot-chocolate-run-atlanta/)Sunday\r\n\r\n![Hot Chocolate Run Atlanta](https://res.cloudinary.com/atlanta/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1599834376/EventEntry/DefaultImages/fitness.jpg)\r\n![Hot Chocolate Run Atlanta](https://res.cloudinary.com/atlanta/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1599834376/EventEntry/DefaultImages/fitness.jpg)\r\n[February03Monday Mary J Blige The \u2018For My Fans\u201D Tour2/3/25](https://discoveratlanta.com/event/detail/mary-j-blige-the-for-my-fans-tour/)Monday\r\n\r\n![Mary J Blige The \u0026#039;For My Fans](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1727804704/EventEntry/yhdwwuvigemnaohqorx9.jpg)\r\n![Mary J Blige The \u0026#039;For My Fans](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1727804704/EventEntry/yhdwwuvigemnaohqorx9.jpg)\r\n[February06Thursday Georgia Tech Women\u2019s Basketball vs SMU2/6/25](https://discoveratlanta.com/event/detail/georgia-tech-women-s-basketball-vs-smu/)Thursday\r\n\r\n![Georgia Tech Women\u2019s Basketball vs SMU](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729696592/EventEntry/rslzial7i8vvoffcewdf.jpg)\r\n![Georgia Tech Women\u2019s Basketball vs SMU](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729696592/EventEntry/rslzial7i8vvoffcewdf.jpg)\r\n[February08Saturday FREE Movie Night: Frozen2/8/25](https://discoveratlanta.com/event/detail/free-movie-night-frozen/)Saturday\r\n\r\n![FREE Movie Night: Frozen](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1722961377/EventEntry/y53fyyfq9msax7zlmcaq.jpg)\r\n![FREE Movie Night: Frozen](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1722961377/EventEntry/y53fyyfq9msax7zlmcaq.jpg)\r\n[February08Saturday Lauda Musicam Chamber Concert2/8/25](https://discoveratlanta.com/event/detail/lauda-musicam-chamber-concert/)Saturday\r\n\r\n![Lauda Musicam Chamber Concert](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1733883863/EventEntry/frupovugcfucqgcxppgd.jpg)\r\n![Lauda Musicam Chamber Concert](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733883863/EventEntry/frupovugcfucqgcxppgd.jpg)\r\n[February08Saturday Matthew Kaminski\u2019s Mondo Hammondo2/8/25](https://discoveratlanta.com/event/detail/matthew-kaminski-s-mondo-hammondo/)Saturday\r\n\r\n![Matthew Kaminski\u0026#039;s Mondo Hammondo](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1731046207/EventEntry/yii7r6ro5rgjenof8mnf.jpg)\r\n![Matthew Kaminski\u0026#039;s Mondo Hammondo](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731046207/EventEntry/yii7r6ro5rgjenof8mnf.jpg)\r\n[March12Wednesday A Beautiful Noise: The Neil Diamond Musical2/9/25 \u2013 3/16/25](https://discoveratlanta.com/event/detail/a-beautiful-noise-the-neil-diamond-musical/)Wednesday\r\n\r\n![A Beautiful Noise: The Neil Diamond Musical](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n![A Beautiful Noise: The Neil Diamond Musical](https://discoveratlanta.com/wp-content/themes/atlanta/assets/images/logo_background_square.png)\r\n[February12Wednesday Georgia Tech Men\u2019s Basketball Vs Stanford2/12/25](https://discoveratlanta.com/event/detail/georgia-tech-men-s-basketball-vs-stanford/)Wednesday\r\n\r\n![Georgia Tech Men\u2019s Basketball Vs Stanford](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729616561/EventEntry/e5lvhpyla05ixdtezven.jpg)\r\n![Georgia Tech Men\u2019s Basketball Vs Stanford](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729616561/EventEntry/e5lvhpyla05ixdtezven.jpg)\r\n[February13Thursday BUSTMultiple days until 03/16/2025](https://discoveratlanta.com/event/detail/bust/)Thursday\r\n\r\n![BUST](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1730142641/EventEntry/bg6bhfxd2uu8fogshdni.jpg)\r\n![BUST](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1730142641/EventEntry/bg6bhfxd2uu8fogshdni.jpg)\r\n[February14Friday Martin Amini2/14/25](https://discoveratlanta.com/event/detail/martin-amini/)Friday\r\n\r\n![Martin Amini](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1725042684/EventEntry/p9i4euxhpuhlxxppvtbi.jpg)\r\n![Martin Amini](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1725042684/EventEntry/p9i4euxhpuhlxxppvtbi.jpg)\r\n[February14Friday TATTOOS, BOOZE, ART, \u0026#038; SHOES 3.02/14/25 \u2013 2/15/25](https://discoveratlanta.com/event/detail/tattoos-booze-art-shoes-3-0/)Friday\r\n\r\n![TATTOOS, BOOZE, ART, \u0026#038; SHOES 3.0](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734493578/EventEntry/bwzfereitmosyay0ozfh.jpg)\r\n![TATTOOS, BOOZE, ART, \u0026#038; SHOES 3.0](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734493578/EventEntry/bwzfereitmosyay0ozfh.jpg)\r\n[February15Saturday African-American Folktales2/15/25](https://discoveratlanta.com/event/detail/african-american-folktales/)Saturday\r\n\r\n![African-American Folktales](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1722961481/EventEntry/rlw5bvaoonjk8s6x2p7w.jpg)\r\n![African-American Folktales](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1722961481/EventEntry/rlw5bvaoonjk8s6x2p7w.jpg)\r\n[February15Saturday Georgia Tech Men\u2019s Basketball Vs California2/15/25](https://discoveratlanta.com/event/detail/georgia-tech-men-s-basketball-vs-california/)Saturday\r\n\r\n![Georgia Tech Men\u2019s Basketball Vs California](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729616418/EventEntry/qdpzxv9nx1rqjtylbcwq.jpg)\r\n![Georgia Tech Men\u2019s Basketball Vs California](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729616418/EventEntry/qdpzxv9nx1rqjtylbcwq.jpg)\r\n[February15Saturday Trailblazers2/15/25](https://discoveratlanta.com/event/detail/trailblazers/)Saturday\r\n\r\n![Trailblazers](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1719701826/EventEntry/srdiozvtascs30bcftgu.jpg)\r\n![Trailblazers](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1719701826/EventEntry/srdiozvtascs30bcftgu.jpg)\r\n[February16Sunday Georgia Tech Women\u2019s Basketball vs Wake Forest2/16/25](https://discoveratlanta.com/event/detail/georgia-tech-women-s-basketball-vs-wake-forest/)Sunday\r\n\r\n![Georgia Tech Women\u2019s Basketball vs Wake Forest](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729696770/EventEntry/udcoeyhw60kos95sk18u.jpg)\r\n![Georgia Tech Women\u2019s Basketball vs Wake Forest](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729696770/EventEntry/udcoeyhw60kos95sk18u.jpg)\r\n[February19Wednesday Atlanta Jewish Film Festival2/19/25 \u2013 3/16/25](https://discoveratlanta.com/event/detail/atlanta-jewish-film-festival/)Wednesday\r\n\r\n![Atlanta Jewish Film Festival](https://res.cloudinary.com/atlanta/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1599834249/EventEntry/DefaultImages/festivals.jpg)\r\n![Atlanta Jewish Film Festival](https://res.cloudinary.com/atlanta/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1599834249/EventEntry/DefaultImages/festivals.jpg)\r\n[February20Thursday Georgia Tech Women\u2019s Basketball vs NC State2/20/25](https://discoveratlanta.com/event/detail/georgia-tech-women-s-basketball-vs-nc-state/)Thursday\r\n\r\n![Georgia Tech Women\u2019s Basketball vs NC State](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1733947815/EventEntry/od8ouuezs3p9wwzmb9me.jpg)\r\n![Georgia Tech Women\u2019s Basketball vs NC State](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733947815/EventEntry/od8ouuezs3p9wwzmb9me.jpg)\r\n[February21Friday Billy Strings Live at State Farm Arena2/21/25 \u2013 2/22/25](https://discoveratlanta.com/event/detail/billy-strings-live-at-state-farm-arena/)Friday\r\n\r\n![Billy Strings Live at State Farm Arena](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1704398623/EventEntry/ah0gfgjh5a34qjadh41v.jpg)\r\n![Billy Strings Live at State Farm Arena](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1704398623/EventEntry/ah0gfgjh5a34qjadh41v.jpg)\r\n[February22Saturday Atlanta Black Expo 20252/22/25 \u2013 2/23/25](https://discoveratlanta.com/event/detail/atlanta-black-expo-2025/)Saturday\r\n\r\n![Atlanta Black Expo 2025](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1731633040/EventEntry/i5joh5qutrtgv8jeaofn.jpg)\r\n![Atlanta Black Expo 2025](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731633040/EventEntry/i5joh5qutrtgv8jeaofn.jpg)\r\n[February22Saturday QS Discover Master\u2019s and MBA Fair in Atlanta2/22/25](https://discoveratlanta.com/event/detail/qs-discover-master-s-and-mba-fair-in-atlanta/)Saturday\r\n\r\n![QS Discover Master\u0026#039;s and MBA Fair in Atlanta](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734339833/EventEntry/ocractvjza6iat5skkpz.jpg)\r\n![QS Discover Master\u0026#039;s and MBA Fair in Atlanta](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734339833/EventEntry/ocractvjza6iat5skkpz.jpg)\r\n[February22Saturday Zoso:The Ultimate Led Zeppelin Experience2/22/25](https://discoveratlanta.com/event/detail/zoso-the-ultimate-led-zeppelin-experience/)Saturday\r\n\r\n![Zoso:The Ultimate Led Zeppelin Experience](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729018134/EventEntry/rcbpphc1whx1ps1acdmp.jpg)\r\n![Zoso:The Ultimate Led Zeppelin Experience](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729018134/EventEntry/rcbpphc1whx1ps1acdmp.jpg)\r\n[February23Sunday Georgia Tech Women\u2019s Basketball vs Florida State2/23/25](https://discoveratlanta.com/event/detail/georgia-tech-women-s-basketball-vs-florida-state/)Sunday\r\n\r\n![Georgia Tech Women\u2019s Basketball vs Florida State](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729697091/EventEntry/yczbgw3i1pnwej1uuxz4.jpg)\r\n![Georgia Tech Women\u2019s Basketball vs Florida State](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729697091/EventEntry/yczbgw3i1pnwej1uuxz4.jpg)\r\n[February27Thursday National Money Show2/27/25](https://discoveratlanta.com/event/detail/national-money-show/)Thursday\r\n\r\n![National Money Show](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1730068696/EventEntry/vndywdfpgahdkyk0a1te.jpg)\r\n![National Money Show](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1730068696/EventEntry/vndywdfpgahdkyk0a1te.jpg)\r\n[February28Friday Cordae \u2013 The Crossroads Tour2/28/25](https://discoveratlanta.com/event/detail/cordae-the-crossroads-tour/)Friday\r\n\r\n![Cordae - The Crossroads Tour](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734548779/EventEntry/uurcnjtslde3axwcvvgq.jpg)\r\n![Cordae - The Crossroads Tour](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734548779/EventEntry/uurcnjtslde3axwcvvgq.jpg)\r\n[February28Friday School \u0026#038; Group NightCrawler at the Zoo2/28/25](https://discoveratlanta.com/event/detail/school-group-nightcrawler-at-the-zoo/)Friday\r\n\r\n![School \u0026#038; Group NightCrawler at the Zoo](https://res.cloudinary.com/atlanta/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1599844113/EventEntry/DefaultImages/education.jpg)\r\n![School \u0026#038; Group NightCrawler at the Zoo](https://res.cloudinary.com/atlanta/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1599844113/EventEntry/DefaultImages/education.jpg)\r\n[March01Saturday Georgia Tech Men\u2019s Basketball Vs NC State3/1/25](https://discoveratlanta.com/event/detail/georgia-tech-men-s-basketball-vs-nc-state/)Saturday\r\n\r\n![Georgia Tech Men\u2019s Basketball Vs NC State](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729616675/EventEntry/ikl28rlrpz2uetdybizn.jpg)\r\n![Georgia Tech Men\u2019s Basketball Vs NC State](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729616675/EventEntry/ikl28rlrpz2uetdybizn.jpg)\r\n[March01Saturday Katt Williams Heaven on Earth Tour3/1/25](https://discoveratlanta.com/event/detail/katt-williams-heaven-on-earth-tour/)Saturday\r\n\r\n![Katt Williams Heaven on Earth Tour](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1727797673/EventEntry/ppliqesrtsv2jk4g1gs5.jpg)\r\n![Katt Williams Heaven on Earth Tour](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1727797673/EventEntry/ppliqesrtsv2jk4g1gs5.jpg)\r\n[March01Saturday Suwanee Beer Fest Beer Chaser 5K3/1/25](https://discoveratlanta.com/event/detail/suwanee-beer-fest-beer-chaser-5k/)Saturday\r\n\r\n![Suwanee Beer Fest Beer Chaser 5K](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1718386949/EventEntry/bef7ib49alr2lrkwvq5g.jpg)\r\n![Suwanee Beer Fest Beer Chaser 5K](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1718386949/EventEntry/bef7ib49alr2lrkwvq5g.jpg)\r\n[March02Sunday Dan Forrest\u2019s Newest Work \u201CCreation\u201D3/2/25](https://discoveratlanta.com/event/detail/dan-forrest-s-newest-work-creation/)Sunday\r\n\r\n![Dan Forrest\u2019s Newest Work  ](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1721763266/EventEntry/obwiwxtsvmjrf4xsk6rn.jpg)\r\n![Dan Forrest\u2019s Newest Work  ](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1721763266/EventEntry/obwiwxtsvmjrf4xsk6rn.jpg)\r\n[March04Tuesday Georgia Tech Men\u2019s Basketball Vs Miami3/4/25](https://discoveratlanta.com/event/detail/georgia-tech-men-s-basketball-vs-miami/)Tuesday\r\n\r\n![Georgia Tech Men\u2019s Basketball Vs Miami](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729616792/EventEntry/ksafbrxm4izhs3fs70jt.jpg)\r\n![Georgia Tech Men\u2019s Basketball Vs Miami](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729616792/EventEntry/ksafbrxm4izhs3fs70jt.jpg)\r\n[March05Wednesday Galapagos George, The Little Tortoise that Could3/5/25 \u2013 3/16/25](https://discoveratlanta.com/event/detail/galapagos-george-the-little-tortoise-that-could/)Wednesday\r\n\r\n![Galapagos George, The Little Tortoise that Could](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1714597675/EventEntry/zwuuovetp4jrlo7lf2gn.jpg)\r\n![Galapagos George, The Little Tortoise that Could](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1714597675/EventEntry/zwuuovetp4jrlo7lf2gn.jpg)\r\n[March07Friday 63rd Annual Orchid Show \u2013 \u201CAn Orchid Wonderland\u201D3/7/25 \u2013 3/9/25](https://discoveratlanta.com/event/detail/63rd-annual-orchid-show-an-orchid-wonderland/)Friday\r\n\r\n![63rd Annual Orchid Show - ](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1731618901/EventEntry/kdr0dvtmtqw5odnfmbmn.jpg)\r\n![63rd Annual Orchid Show - ](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731618901/EventEntry/kdr0dvtmtqw5odnfmbmn.jpg)\r\n[March07Friday Martin Lawrence Y\u2019all Know What It Is Tour3/7/25](https://discoveratlanta.com/event/detail/martin-lawrence-y-all-know-what-it-is-tour/)Friday\r\n\r\n![Martin Lawrence Y\u0026#039;all Know What It Is Tour](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1727797866/EventEntry/juzg5j3hufl0ejllf67g.jpg)\r\n![Martin Lawrence Y\u0026#039;all Know What It Is Tour](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1727797866/EventEntry/juzg5j3hufl0ejllf67g.jpg)\r\n[March08Saturday Blacktop Playground: Improv for Kids3/8/25](https://discoveratlanta.com/event/detail/blacktop-playground-improv-for-kids/)Saturday\r\n\r\n![Blacktop Playground: Improv for Kids](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1722961592/EventEntry/hb78qtz4godbbljigx6z.jpg)\r\n![Blacktop Playground: Improv for Kids](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1722961592/EventEntry/hb78qtz4godbbljigx6z.jpg)\r\n[March10Monday PACK EXPO 20253/10/25 \u2013 3/12/25](https://discoveratlanta.com/event/detail/pack-expo-2025/)Monday\r\n\r\n![PACK EXPO 2025](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1685635365/EventEntry/wbmoavffcsbdi6p72hhz.jpg)\r\n![PACK EXPO 2025](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1685635365/EventEntry/wbmoavffcsbdi6p72hhz.jpg)\r\n[March15Saturday Blake Shelton Friends and Heroes 2025 Tour3/15/25 \u2013 3/16/25](https://discoveratlanta.com/event/detail/blake-shelton-friends-and-heroes-2025-tour/)Saturday\r\n\r\n![Blake Shelton Friends and Heroes 2025 Tour](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1727798057/EventEntry/qsjs4xiqfcxmg13yrgzb.jpg)\r\n![Blake Shelton Friends and Heroes 2025 Tour](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1727798057/EventEntry/qsjs4xiqfcxmg13yrgzb.jpg)\r\n[March15Saturday From Screen to Stage3/15/25](https://discoveratlanta.com/event/detail/from-screen-to-stage/)Saturday\r\n\r\n![From Screen to Stage](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1725911104/EventEntry/fyryjxg95r9er9qymwtm.jpg)\r\n![From Screen to Stage](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1725911104/EventEntry/fyryjxg95r9er9qymwtm.jpg)\r\n[March18Tuesday Deftones North American Tour 20253/18/25 \u2013 3/19/25](https://discoveratlanta.com/event/detail/deftones-north-american-tour-2025/)Tuesday\r\n\r\n![Deftones North American Tour 2025](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1727798254/EventEntry/y1muus1kadupfvjds7bs.jpg)\r\n![Deftones North American Tour 2025](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1727798254/EventEntry/y1muus1kadupfvjds7bs.jpg)\r\n[March19Wednesday Milo the Magnificent3/19/25 \u2013 3/30/25](https://discoveratlanta.com/event/detail/milo-the-magnificent/)Wednesday\r\n\r\n![Milo the Magnificent](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1715095074/EventEntry/segadj3sd1cwiabussqf.jpg)\r\n![Milo the Magnificent](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1715095074/EventEntry/segadj3sd1cwiabussqf.jpg)\r\n[March20Thursday J Balvin Live at State Farm Arena3/20/25](https://discoveratlanta.com/event/detail/j-balvin-live-at-state-farm-arena/)Thursday\r\n\r\n![J Balvin Live at State Farm Arena](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734630666/EventEntry/gcjvxwguifniz5ixct4m.jpg)\r\n![J Balvin Live at State Farm Arena](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734630666/EventEntry/gcjvxwguifniz5ixct4m.jpg)\r\n[March21Friday Tyler, The Creator \u2013 Chromakopia Tour3/21/25](https://discoveratlanta.com/event/detail/tyler-the-creator-chromakopia-tour/)Friday\r\n\r\n![Tyler, The Creator - Chromakopia Tour](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734555743/EventEntry/udoorxuvxhkxmxjje1cd.jpg)\r\n![Tyler, The Creator - Chromakopia Tour](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734555743/EventEntry/udoorxuvxhkxmxjje1cd.jpg)\r\n[March29Saturday Debussy, Ravel, \u0026#038; Sibelius: A Birthday Celebration3/29/25](https://discoveratlanta.com/event/detail/debussy-ravel-sibelius-a-birthday-celebration/)Saturday\r\n\r\n![Debussy, Ravel, \u0026#038; Sibelius: A Birthday Celebration](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1719701585/EventEntry/cfsofitep9345ycdndzt.jpg)\r\n![Debussy, Ravel, \u0026#038; Sibelius: A Birthday Celebration](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1719701585/EventEntry/cfsofitep9345ycdndzt.jpg)\r\n[March29Saturday It\u2019s Electric!3/29/25](https://discoveratlanta.com/event/detail/it-s-electric/)Saturday\r\n\r\n![It\u2019s Electric!](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1722961771/EventEntry/e99aqcvvstgb3esn1icd.jpg)\r\n![It\u2019s Electric!](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1722961771/EventEntry/e99aqcvvstgb3esn1icd.jpg)\r\n[April04Friday The Millennium Tour 20254/4/25](https://discoveratlanta.com/event/detail/the-millennium-tour-2025/)Friday\r\n\r\n![The Millennium Tour 2025](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734631195/EventEntry/cqletfwxfxbwykqeymxu.jpg)\r\n![The Millennium Tour 2025](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734631195/EventEntry/cqletfwxfxbwykqeymxu.jpg)\r\n[April06Sunday Georgia Tasting Festival 20254/6/25](https://discoveratlanta.com/event/detail/georgia-tasting-festival-2025/)Sunday\r\n\r\n![Georgia Tasting Festival 2025](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1733347277/EventEntry/ysqo8jq53n8cvnuiz6lv.jpg)\r\n![Georgia Tasting Festival 2025](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1733347277/EventEntry/ysqo8jq53n8cvnuiz6lv.jpg)\r\n[April09Wednesday Spring Break Magic Show4/9/25](https://discoveratlanta.com/event/detail/spring-break-magic-show/)Wednesday\r\n\r\n![Spring Break Magic Show](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1722961984/EventEntry/y6ymqc7rrvfc4mnrqkhg.jpg)\r\n![Spring Break Magic Show](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1722961984/EventEntry/y6ymqc7rrvfc4mnrqkhg.jpg)\r\n[April11Friday Kylie Minogue 2025 Tension Tour4/11/25](https://discoveratlanta.com/event/detail/kylie-minogue-2025-tension-tour/)Friday\r\n\r\n![Kylie Minogue 2025 Tension Tour](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1727970352/EventEntry/n5r0z2or4zqttmtkzkyj.jpg)\r\n![Kylie Minogue 2025 Tension Tour](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1727970352/EventEntry/n5r0z2or4zqttmtkzkyj.jpg)\r\n[April19Saturday Dr. Dinosaur!4/19/25](https://discoveratlanta.com/event/detail/dr-dinosaur/)Saturday\r\n\r\n![Dr. Dinosaur!](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1722962082/EventEntry/cf8ovr3mdwajqgmgsicz.jpg)\r\n![Dr. Dinosaur!](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1722962082/EventEntry/cf8ovr3mdwajqgmgsicz.jpg)\r\n[April26Saturday Move For Grady4/26/25](https://discoveratlanta.com/event/detail/move-for-grady-2/)Saturday\r\n\r\n![Move For Grady](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729705358/EventEntry/m5jlrwp3s6xacaju0y7w.jpg)\r\n![Move For Grady](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729705358/EventEntry/m5jlrwp3s6xacaju0y7w.jpg)\r\n[April29Tuesday Kendrick Lamar and SZA\u2019s Grand National Tour4/29/25](https://discoveratlanta.com/event/detail/kendrick-lamar-and-sza-s-grand-national-tour/)Tuesday\r\n\r\n![Kendrick Lamar and SZA\u0026#039;s Grand National Tour](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1603481265/EventEntry/DefaultImages/Musicconcerts.jpg)\r\n![Kendrick Lamar and SZA\u0026#039;s Grand National Tour](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1603481265/EventEntry/DefaultImages/Musicconcerts.jpg)\r\n[April29Tuesday Pearl Jam \u2013 Dark Matter Tour4/29/25 \u2013 5/1/25](https://discoveratlanta.com/event/detail/pearl-jam-dark-matter-tour/)Tuesday\r\n\r\n![Pearl Jam - Dark Matter Tour](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734636991/EventEntry/kexhcxxh56wnbtdl0yas.jpg)\r\n![Pearl Jam - Dark Matter Tour](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734636991/EventEntry/kexhcxxh56wnbtdl0yas.jpg)\r\n[May03Saturday Fairy God Doctor5/3/25](https://discoveratlanta.com/event/detail/fairy-god-doctor/)Saturday\r\n\r\n![Fairy God Doctor](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1722962169/EventEntry/my9bxjghyi80mnt4mwuq.jpg)\r\n![Fairy God Doctor](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1722962169/EventEntry/my9bxjghyi80mnt4mwuq.jpg)\r\n[May03Saturday The Oz Escape: A Wicked Glitch in Atlanta5/3/25](https://discoveratlanta.com/event/detail/the-oz-escape-a-wicked-glitch-in-atlanta/)Saturday\r\n\r\n![The Oz Escape: A Wicked Glitch in Atlanta](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1732001821/EventEntry/ojnx9ycru5ynfuadrhuw.jpg)\r\n![The Oz Escape: A Wicked Glitch in Atlanta](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1732001821/EventEntry/ojnx9ycru5ynfuadrhuw.jpg)\r\n[May04Sunday May the 4th Be With You: Sun, Moon, Air, \u0026#038; Space5/4/25](https://discoveratlanta.com/event/detail/may-the-4th-be-with-you-sun-moon-air-space/)Sunday\r\n\r\n![May the 4th Be With You: Sun, Moon, Air, \u0026#038; Space](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1721763532/EventEntry/hknmhvaizbowkmdzmkr4.jpg)\r\n![May the 4th Be With You: Sun, Moon, Air, \u0026#038; Space](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1721763532/EventEntry/hknmhvaizbowkmdzmkr4.jpg)\r\n[May06Tuesday Fashion Play VI5/6/25](https://discoveratlanta.com/event/detail/fashion-play-vi/)Tuesday\r\n\r\n![Fashion Play VI](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1726354349/EventEntry/zzfnxfuc2tgeym2cjll3.jpg)\r\n![Fashion Play VI](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1726354349/EventEntry/zzfnxfuc2tgeym2cjll3.jpg)\r\n[May10Saturday A Kaleidoscope of American Music5/10/25](https://discoveratlanta.com/event/detail/a-kaleidoscope-of-american-music/)Saturday\r\n\r\n![A Kaleidoscope of American Music](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1725911245/EventEntry/u7jdlt6zomsaijzn3dym.jpg)\r\n![A Kaleidoscope of American Music](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1725911245/EventEntry/u7jdlt6zomsaijzn3dym.jpg)\r\n[May10Saturday Atlanta Caribbean Carnival\u21225/10/25 \u2013 5/26/25](https://discoveratlanta.com/event/detail/atlanta-caribbean-carnival-2/)Saturday\r\n\r\n![Atlanta Caribbean Carnival\u2122](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1728321520/EventEntry/u5txbcjlzbqbzm9obt7m.jpg)\r\n![Atlanta Caribbean Carnival\u2122](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1728321520/EventEntry/u5txbcjlzbqbzm9obt7m.jpg)\r\n[May11Sunday Post Malone \u2013 The Big Stadium Tour5/11/25](https://discoveratlanta.com/event/detail/post-malone-the-big-stadium-tour/)Sunday\r\n\r\n![Post Malone - The Big Stadium Tour](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734637893/EventEntry/d3agytfhtony7gj8hehm.jpg)\r\n![Post Malone - The Big Stadium Tour](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734637893/EventEntry/d3agytfhtony7gj8hehm.jpg)\r\n[May17Saturday The Firebird5/17/25](https://discoveratlanta.com/event/detail/the-firebird/)Saturday\r\n\r\n![The Firebird](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1719701987/EventEntry/vspg61egxg5ed4izg5kb.jpg)\r\n![The Firebird](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1719701987/EventEntry/vspg61egxg5ed4izg5kb.jpg)\r\n[October14Tuesday Oral History Association 2025 Annual Meeting10/14/25 \u2013 10/19/25](https://discoveratlanta.com/event/detail/oral-history-association-2025-annual-meeting/)Tuesday\r\n\r\n![Oral History Association 2025 Annual Meeting](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1717436591/EventEntry/x3glj45aakebg4co7bay.jpg)\r\n![Oral History Association 2025 Annual Meeting](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1717436591/EventEntry/x3glj45aakebg4co7bay.jpg)\r\n[November09Sunday Cirque du Soleil\u2019s LUZIA11/9/25 \u2013 12/15/25](https://discoveratlanta.com/event/detail/cirque-du-soleil-s-luzia/)Sunday\r\n\r\n![Cirque du Soleil\u0026#039;s LUZIA](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729607993/EventEntry/z89ykp7z1y3cmrwloyeq.jpg)\r\n![Cirque du Soleil\u0026#039;s LUZIA](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729607993/EventEntry/z89ykp7z1y3cmrwloyeq.jpg)\r\n[June11Thursday 2026 FIFA World Cup6/11/26 \u2013 7/19/26](https://discoveratlanta.com/event/detail/2026-fifa-world-cup/)Thursday\r\n\r\n![2026 FIFA World Cup](https://res.cloudinary.com/atlanta/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1599834351/EventEntry/DefaultImages/sports.jpg)\r\n![2026 FIFA World Cup](https://res.cloudinary.com/atlanta/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1599834351/EventEntry/DefaultImages/sports.jpg)\r\n[December31Tuesday Holiday Bricktacular at LEGO Discovery Center!11/29/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/holiday-bricktacular-at-lego-discovery-center/)Tuesday\r\n\r\n![Holiday Bricktacular at LEGO Discovery Center!](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1731444310/EventEntry/ahs10viscnx6uk7p3jus.jpg)\r\n![Holiday Bricktacular at LEGO Discovery Center!](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731444310/EventEntry/ahs10viscnx6uk7p3jus.jpg)\r\n[December31Tuesday Lenox Square\u2019s Canned Food Drive with ACFB11/29/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/lenox-square-s-canned-food-drive-with-acfb/)Tuesday\r\n\r\n![Lenox Square\u2019s Canned Food Drive with ACFB](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1732638488/EventEntry/zynyjrwceczmfowsicd9.jpg)\r\n![Lenox Square\u2019s Canned Food Drive with ACFB](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1732638488/EventEntry/zynyjrwceczmfowsicd9.jpg)\r\n[December31Tuesday Rooftop Dining Igloos12/31/24 \u2013 1/6/25](https://discoveratlanta.com/event/detail/rooftop-dining-igloos/)Tuesday\r\n\r\n![Rooftop Dining Igloos](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1731523408/EventEntry/urg2xyim0vqoboft0zmf.jpg)\r\n![Rooftop Dining Igloos](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731523408/EventEntry/urg2xyim0vqoboft0zmf.jpg)\r\n[December31Tuesday Westside Sleigh Ride at Westside Motor Lounge11/29/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/westside-sleigh-ride-at-westside-motor-lounge/)Tuesday\r\n\r\n![Westside Sleigh Ride at Westside Motor Lounge](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1732649478/EventEntry/wuuojlbzrbunurojdb8v.jpg)\r\n![Westside Sleigh Ride at Westside Motor Lounge](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1732649478/EventEntry/wuuojlbzrbunurojdb8v.jpg)\r\n[December31Tuesday Cat Eye Creative\u2019s \u201CThe Greatest Gift\u201D Exhibition12/31/24 \u2013 1/5/25](https://discoveratlanta.com/event/detail/cat-eye-creative-s-the-greatest-gift-exhibition/)Tuesday\r\n\r\n![Cat Eye Creative\u0026#039;s ](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1732125325/EventEntry/dcg9yxduvw8dpuk1mdlo.jpg)\r\n![Cat Eye Creative\u0026#039;s ](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1732125325/EventEntry/dcg9yxduvw8dpuk1mdlo.jpg)\r\n[February12Wednesday Spring HomeSchool Academy with Zoo AtlantaMultiple dates until 05/14/2025](https://discoveratlanta.com/event/detail/spring-homeschool-academy-with-zoo-atlanta/)Wednesday\r\n\r\n![Spring HomeSchool Academy with Zoo Atlanta](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1638901209/EventEntry/plimqbinwlm4hgwlpifn.jpg)\r\n![Spring HomeSchool Academy with Zoo Atlanta](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1638901209/EventEntry/plimqbinwlm4hgwlpifn.jpg)\r\n[December31Tuesday History with Chutzpah3/9/23 \u2013 12/31/24](https://discoveratlanta.com/event/detail/history-with-chutzpah/)Tuesday\r\n\r\n![History with Chutzpah](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1678392014/EventEntry/unmctzn1boobsmbm9keq.jpg)\r\n![History with Chutzpah](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1678392014/EventEntry/unmctzn1boobsmbm9keq.jpg)\r\n[December31Tuesday Candlelight Concerts 20241/1/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/candlelight-concerts-2024/)Tuesday\r\n\r\n![Candlelight Concerts 2024](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1603481265/EventEntry/DefaultImages/Musicconcerts.jpg)\r\n![Candlelight Concerts 2024](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1603481265/EventEntry/DefaultImages/Musicconcerts.jpg)\r\n[December31Tuesday WQEB 97.3 DINE \u0026#038; DONATE, a Food \u0026#038; Clothing Drive1/2/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/wqeb-97-3-dine-donate-a-food-clothing-drive/)Tuesday\r\n\r\n![WQEB 97.3 DINE \u0026#038; DONATE, a Food \u0026#038; Clothing Drive](https://res.cloudinary.com/atlanta/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1600954997/EventEntry/DefaultImages/dining.jpg)\r\n![WQEB 97.3 DINE \u0026#038; DONATE, a Food \u0026#038; Clothing Drive](https://res.cloudinary.com/atlanta/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1600954997/EventEntry/DefaultImages/dining.jpg)\r\n[December31Tuesday Adopt Any Senior Pet for FREE in 202412/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/adopt-any-senior-pet-for-free-in-2024/)Tuesday\r\n\r\n![Adopt Any Senior Pet for FREE in 2024](https://res.cloudinary.com/atlanta/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1599834355/EventEntry/DefaultImages/pets.jpg)\r\n![Adopt Any Senior Pet for FREE in 2024](https://res.cloudinary.com/atlanta/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1599834355/EventEntry/DefaultImages/pets.jpg)\r\n[December31Tuesday Free Self-guided Street Art Walking Tours3/5/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/free-self-guided-street-art-walking-tours/)Tuesday\r\n\r\n![Free Self-guided Street Art Walking Tours](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1709675802/EventEntry/khfjjtr9loqfsfknymqs.jpg)\r\n![Free Self-guided Street Art Walking Tours](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1709675802/EventEntry/khfjjtr9loqfsfknymqs.jpg)\r\n[March22Saturday Wing \u0026#038; Rock Fest12/31/24 \u2013 3/23/25](https://discoveratlanta.com/event/detail/wing-rock-fest/)Saturday\r\n\r\n![Wing \u0026#038; Rock Fest](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1723671471/EventEntry/vwjazegztlo66gigfv2z.jpg)\r\n![Wing \u0026#038; Rock Fest](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1723671471/EventEntry/vwjazegztlo66gigfv2z.jpg)\r\n[April05Saturday Spring Festival on PonceMultiple dates until 04/06/2025](https://discoveratlanta.com/event/detail/spring-festival-on-ponce-2/)Saturday\r\n\r\n![Spring Festival on Ponce](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1726831018/EventEntry/frnx5njtk4kgu636e9ri.jpg)\r\n![Spring Festival on Ponce](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1726831018/EventEntry/frnx5njtk4kgu636e9ri.jpg)\r\n[December31Tuesday Pick Your Perks Group Promotion12/31/24 \u2013 5/31/25](https://discoveratlanta.com/event/detail/pick-your-perks-group-promotion/)Tuesday\r\n\r\n![Pick Your Perks Group Promotion](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1716993671/EventEntry/lsnfyf8st8xlh5pzqibf.jpg)\r\n![Pick Your Perks Group Promotion](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1716993671/EventEntry/lsnfyf8st8xlh5pzqibf.jpg)\r\n[December31Tuesday Create Your Own Scented Candle!5/31/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/create-your-own-scented-candle/)Tuesday\r\n\r\n![Create Your Own Scented Candle!](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1716910908/EventEntry/tzyggoratqzuata0llrg.jpg)\r\n![Create Your Own Scented Candle!](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1716910908/EventEntry/tzyggoratqzuata0llrg.jpg)\r\n[February15Saturday Kids Night Out12/31/24 \u2013 2/15/25](https://discoveratlanta.com/event/detail/kids-night-out/)Saturday\r\n\r\n![Kids Night Out](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1715624385/EventEntry/qlfitnf0z1aq1bd2ugaq.jpg)\r\n![Kids Night Out](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1715624385/EventEntry/qlfitnf0z1aq1bd2ugaq.jpg)\r\n[December31Tuesday Beginner\u2019s Salsa Lessons @ PasoFino Dance Studio12/31/24 \u2013 6/30/25](https://discoveratlanta.com/event/detail/beginner-s-salsa-lessons-pasofino-dance-studio/)Tuesday\r\n\r\n![Beginner\u2019s Salsa Lessons @ PasoFino Dance Studio](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1718734218/EventEntry/dfxttpmu9kavha4roq0y.jpg)\r\n![Beginner\u2019s Salsa Lessons @ PasoFino Dance Studio](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1718734218/EventEntry/dfxttpmu9kavha4roq0y.jpg)\r\n[January11Saturday The Fine Colombians: A Steely Dan Tribute12/31/24 \u2013 1/11/25](https://discoveratlanta.com/event/detail/the-fine-colombians-a-steely-dan-tribute-2/)Saturday\r\n\r\n![The Fine Colombians: A Steely Dan Tribute](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1728505138/EventEntry/eakxs1ibmmn5rjocbixs.jpg)\r\n![The Fine Colombians: A Steely Dan Tribute](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1728505138/EventEntry/eakxs1ibmmn5rjocbixs.jpg)\r\n[December31Tuesday Titanic: An Immersive Voyage12/31/24 \u2013 1/22/25](https://discoveratlanta.com/event/detail/titanic-an-immersive-voyage/)Tuesday\r\n\r\n![Titanic: An Immersive Voyage](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1725270644/EventEntry/fvetr7fvfozljrgzsc1z.jpg)\r\n![Titanic: An Immersive Voyage](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1725270644/EventEntry/fvetr7fvfozljrgzsc1z.jpg)\r\n[January24Friday The Spirit Of Rush12/31/24 \u2013 1/24/25](https://discoveratlanta.com/event/detail/the-spirit-of-rush-2/)Friday\r\n\r\n![The Spirit Of Rush](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1728501904/EventEntry/z10ptctgiwsrgbb0letc.jpg)\r\n![The Spirit Of Rush](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1728501904/EventEntry/z10ptctgiwsrgbb0letc.jpg)\r\n[March01Saturday Twilight Trek at the ZooMultiple dates until 05/30/2025](https://discoveratlanta.com/event/detail/twilight-trek-at-the-zoo/)Saturday\r\n\r\n![Twilight Trek at the Zoo](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1715626057/EventEntry/nlnmvx8wnggmalpbanmh.jpg)\r\n![Twilight Trek at the Zoo](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1715626057/EventEntry/nlnmvx8wnggmalpbanmh.jpg)\r\n[December31Tuesday Golden Opportunity: Botanical Illustration12/31/24 \u2013 1/1/25](https://discoveratlanta.com/event/detail/golden-opportunity-botanical-illustration/)Tuesday\r\n\r\n![Golden Opportunity: Botanical Illustration](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1724873191/EventEntry/w6xjce2kd0jcmf3ksk4z.jpg)\r\n![Golden Opportunity: Botanical Illustration](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1724873191/EventEntry/w6xjce2kd0jcmf3ksk4z.jpg)\r\n[December31Tuesday Sacred Breaths12/31/24 \u2013 4/30/25](https://discoveratlanta.com/event/detail/sacred-breaths/)Tuesday\r\n\r\n![Sacred Breaths](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1725945664/EventEntry/vmrdizsy2ix9squvz3ob.jpg)\r\n![Sacred Breaths](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1725945664/EventEntry/vmrdizsy2ix9squvz3ob.jpg)\r\n[December31Tuesday Georgia O\u2019Keeffe: \u201CMy Two New Yorks\u201D12/31/24 \u2013 2/16/25](https://discoveratlanta.com/event/detail/georgia-o-keeffe-my-two-new-yorks/)Tuesday\r\n\r\n![Georgia O\u0026#039;Keeffe: ](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1728936131/EventEntry/zxsennaah25xklwvxkwd.jpg)\r\n![Georgia O\u0026#039;Keeffe: ](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1728936131/EventEntry/zxsennaah25xklwvxkwd.jpg)\r\n[March16Sunday Pizza and Beer CrawlMultiple dates until 04/27/2025](https://discoveratlanta.com/event/detail/pizza-and-beer-crawl/)Sunday\r\n\r\n![Pizza and Beer Crawl](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1729015170/EventEntry/w8vxndj0rcjhnesmmav0.jpg)\r\n![Pizza and Beer Crawl](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1729015170/EventEntry/w8vxndj0rcjhnesmmav0.jpg)\r\n[December31Tuesday Terminus Tours Atlanta \u2013 Private Guided City Tours11/1/24 \u2013 12/31/24](https://discoveratlanta.com/event/detail/terminus-tours-atlanta-private-guided-city-tours/)Tuesday\r\n\r\n![Terminus Tours Atlanta - Private Guided City Tours](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1731293699/EventEntry/ltpl7oqyv3kis4ovh9kg.jpg)\r\n![Terminus Tours Atlanta - Private Guided City Tours](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731293699/EventEntry/ltpl7oqyv3kis4ovh9kg.jpg)\r\n[December31Tuesday D-Intensity \u201CHibernation 25\u201D Bootcamp12/31/24 \u2013 3/30/25](https://discoveratlanta.com/event/detail/d-intensity-hibernation-25-bootcamp/)Tuesday\r\n\r\n![D-Intensity ](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1728789689/EventEntry/yw0orsyvyzfw3qn2umpd.jpg)\r\n![D-Intensity ](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1728789689/EventEntry/yw0orsyvyzfw3qn2umpd.jpg)\r\n[December31Tuesday Winter Lantern Festival12/31/24 \u2013 1/5/25](https://discoveratlanta.com/event/detail/winter-lantern-festival/)Tuesday\r\n\r\n![Winter Lantern Festival](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1730320519/EventEntry/xeq73amjilhdwqr30xst.jpg)\r\n![Winter Lantern Festival](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1730320519/EventEntry/xeq73amjilhdwqr30xst.jpg)\r\n[January04Saturday The Dinner Detective Comedy Mystery Dinner ShowMultiple dates until 01/18/2025](https://discoveratlanta.com/event/detail/the-dinner-detective-comedy-mystery-dinner-show-3/)Saturday\r\n\r\n![The Dinner Detective Comedy Mystery Dinner Show](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1731515100/EventEntry/hixa8x53rgy493iw2mbr.jpg)\r\n![The Dinner Detective Comedy Mystery Dinner Show](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731515100/EventEntry/hixa8x53rgy493iw2mbr.jpg)\r\n[December31Tuesday Rooftop Brunch at 9 Mile Station12/31/24 \u2013 1/11/25](https://discoveratlanta.com/event/detail/rooftop-brunch-at-9-mile-station/)Tuesday\r\n\r\n![Rooftop Brunch at 9 Mile Station](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1731519701/EventEntry/jur6neeqjm39lvie9psg.jpg)\r\n![Rooftop Brunch at 9 Mile Station](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731519701/EventEntry/jur6neeqjm39lvie9psg.jpg)\r\n[December31Tuesday Skate City Springs12/31/24 \u2013 1/20/25](https://discoveratlanta.com/event/detail/skate-city-springs/)Tuesday\r\n\r\n![Skate City Springs](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1731436332/EventEntry/ekeatpkfbebqj9swfuyk.jpg)\r\n![Skate City Springs](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1731436332/EventEntry/ekeatpkfbebqj9swfuyk.jpg)\r\n[December31Tuesday Winter Wonderland at Fernbank12/31/24 \u2013 1/12/25](https://discoveratlanta.com/event/detail/winter-wonderland-at-fernbank/)Tuesday\r\n\r\n![Winter Wonderland at Fernbank](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1668539627/EventEntry/x0rsixedeal1zovzdm3v.jpg)\r\n![Winter Wonderland at Fernbank](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1668539627/EventEntry/x0rsixedeal1zovzdm3v.jpg)\r\n[December31Tuesday The Ice Skating Rink Returns to Park Tavern12/31/24 \u2013 2/17/25](https://discoveratlanta.com/event/detail/the-ice-skating-rink-returns-to-park-tavern/)Tuesday\r\n\r\n![The Ice Skating Rink Returns to Park Tavern](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1732024073/EventEntry/mkry1qstlh8e6ck0ludc.jpg)\r\n![The Ice Skating Rink Returns to Park Tavern](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1732024073/EventEntry/mkry1qstlh8e6ck0ludc.jpg)\r\n[December31Tuesday Winter Rowing Camp Training (12-18y)12/31/24 \u2013 2/1/25](https://discoveratlanta.com/event/detail/winter-rowing-camp-training-12-18y/)Tuesday\r\n\r\n![Winter Rowing Camp Training (12-18y)](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1732035183/EventEntry/oajtmqefsfcjoaatvvfx.jpg)\r\n![Winter Rowing Camp Training (12-18y)](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1732035183/EventEntry/oajtmqefsfcjoaatvvfx.jpg)\r\n[December31Tuesday Midnight at Chateau: A glamorous New Year\u2019s Celebr12/31/24 \u2013 12/31/34](https://discoveratlanta.com/event/detail/midnight-at-chateau-a-glamorous-new-year-s-celebr/)Tuesday\r\n\r\n![Midnight at Chateau: A glamorous New Year\u2019s Celebr](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1734029239/EventEntry/clwa8hqqluwkxkn9hpj2.jpg)\r\n![Midnight at Chateau: A glamorous New Year\u2019s Celebr](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1734029239/EventEntry/clwa8hqqluwkxkn9hpj2.jpg)\r\n[April08Tuesday Cat in the Hat4/8/25 \u2013 5/25/25](https://discoveratlanta.com/event/detail/cat-in-the-hat/)Tuesday\r\n\r\n![Cat in the Hat](https://images.discoveratlanta.com/image/upload/w_1600,h_900,c_fill,g_auto,q_auto/v1715796125/EventEntry/xcmkmnekaanup6un7qp5.jpg)\r\n![Cat in the Hat](https://images.discoveratlanta.com/image/upload/w_400,h_400,c_fill,g_auto,q_auto/v1715796125/EventEntry/xcmkmnekaanup6un7qp5.jpg)\r\n## Discover ATL Events\r\n\r\n\r\n\r\n[View All Events](/events/all)## Discover More\r\n\r\nGet to know Atlanta, and discover the best things to do around the city.\r\n\r\n[EventsBounce through the incredible events happening around Atlanta today, tomorrow, this week or this month. Upcoming Events](https://discoveratlanta.com/events/all/)### Upcoming Events\r\n\r\n[Sports and OutdoorsSouth America\u2019s premier international soccer competition is coming to the United States and Atlanta gets to kick off the party. Welcome to Atlanta, Copa Am\u00E9rica 2024! Atlanta has been selected as one of the 14 host cities for Copa America 2024 across the country. Experience Copa America in Atlanta](https://discoveratlanta.com/stories/things-to-do/experience-copa-america-in-atlanta/)### Experience Copa America in Atlanta\r\n\r\n[NewscredCity sidewalks, busy sidewalks\u2026 \u2018Tis the season for shopping. With so many opportunities to shop local, support Black-owned businesses, discover Black fashion designers and artists and explore Atlanta retail havens virtually and in-person, you are sure to find the perfect gift for everyone on your list. Atlanta Gift Guide 2020: Where to Get the Best Gifts while supporting Local Businesses in Atlanta](https://discoveratlanta.com/blog/atlanta-gift-guide-2020-where-to-get-the-best-gifts-while-supporting-local-businesses-in-atlanta)### Atlanta Gift Guide 2020: Where to Get the Best Gifts while supporting Local Businesses in Atlanta\r\n\r\n[DiningAtlanta, Georgia seafood restaurants feature fresh flavors from around the world, as well as New Orleans-style Cajun and Creole cuisine. Seafood](https://discoveratlanta.com/dining/seafood-restaurants/)### Seafood\r\n\r\n[Things to DoFrom down and dirty barbecue and secret bars to neighborhood nightlife and one-of-a-kind experiences, let me be your guide and give you a taste of my favorite spots around the A. Scenes from the A](https://discoveratlanta.com/things-to-do/scenes-from-the-a/)### Scenes from the A\r\n\r\n[Culinary Scene\u00BFQu\u00E9 platillos definen a Atlanta? Me vienen a la mente los platillos sure\u00F1os, pero eso no es todo. Comida T\u00EDpica de Atlanta](https://discoveratlanta.com/stories/eat/comida-tipica-de-atlanta/)### Comida T\u00EDpica de Atlanta\r\n\r\n## Sign up for email updates\r\n\r\n* Discover Atlanta\r\n* Things to Do\r\n* Events\r\n* Plan Your Visit\r\n* Meeting Planners\r\n* Travel Pros\r\n* Newsroom\r\n* Partners\r\n* About Us\r\n* Become a Member\r\n* Employment\r\n* Contact Us\r\n\r\n[Discover Atlanta](https://discoveratlanta.com/)[Things to Do](https://discoveratlanta.com/things-to-do/main/)[Events](https://discoveratlanta.com/events/main/)[Plan Your Visit](https://discoveratlanta.com/explore/main/)[Meeting Planners](https://discoveratlanta.com/meetings/main/)[Travel Pros](https://discoveratlanta.com/travel-tours/)[Newsroom](https://discoveratlanta.com/news/main/)[Partners](https://discoveratlanta.com/acvb/partners/)[About Us](https://discoveratlanta.com/acvb/)[Become a Member](https://discoveratlanta.com/acvb/become-a-member/)[Employment](https://discoveratlanta.com/acvb/employment/)[Contact Us](https://discoveratlanta.com/acvb/contact/)[DiscoverAtlanta.com homepage](https://discoveratlanta.com)Established in 1913, Atlanta Convention \u0026 Visitors Bureau is the official destination marketing organization for the city and serves to favorably impact Atlanta\u0027s economy through conventions and tourism.\r\n\r\n## Atlanta Convention \u0026 Visitors Bureau\r\n\r\n233 Peachtree Street Suite #1400 Atlanta, GA 30303\r\n\r\n(800)ATLANTA (404)521-6600\r\n\r\n[(800)ATLANTA](tel:800-285-2682)[(404)521-6600](tel:404-521-6600)[](https://www.facebook.com/discoveratlanta)[](https://twitter.com/#!/discoveratlanta)[](https://www.instagram.com/discoveratlanta)[](https://www.youtube.com/visitatlanta)[](https://www.pinterest.com/discoveratlanta/)* Site Map\r\n* Privacy Policy\r\n* Terms of Use\r\n* UGC Agreement\r\n\r\n[Site Map](https://discoveratlanta.com/sitemap/)[Privacy Policy](https://discoveratlanta.com/privacy-policy/)[Terms of Use](https://discoveratlanta.com/terms-of-use/)[UGC Agreement](https://discoveratlanta.com/ugc-agreement/)Atlanta Convention \u0026 Visitors Bureau.\r\n\r\nAll Rights Reserved. Copyright \u00A92024\r\n\r\n![](/wp-content/themes/atlanta/assets/images/wtp-icons/PlanYourTrip.png)\r\n![](/wp-content/themes/atlanta/assets/images/wtp-icons/close-btn.png)\r\n"
```

Here are some exciting events happening in Atlanta today, December 31, 2024:

1. **Georgia Tech Yellow Jackets vs. Notre Dame Fighting Irish Basketball Game**
   - **Time:** 2:30 PM
   - **Location:** McCamish Pavilion, 965 Fowler St., Atlanta, GA
   - **Tickets:** [Available Here](https://atlanta-ga.events/event/georgia-tech-yellow-jackets-vs-notre-dame-fighting-irish-atlanta-tuesday-december-31-2024-230-pm/)

2. **Provenance NYE: Seven Lions, Knife Party & Grabbitz**
   - **Time:** 7:00 PM
   - **Location:** Atlanta Coliseum, 2075 Market St, Duluth, GA
   - **Tickets:** [Available Here](https://atlanta-ga.events/event/provenance-nye-seven-lions-knife-party-and-grabbitz-duluth-tuesday-december-31-2024-700-pm/?pid=8320)

3. **Zac Brown Band Concert**
   - **Time:** 8:00 PM
   - **Location:** State Farm Arena, One Philips Drive, Atlanta, GA
   - **Tickets:** [Available Here](https://atlanta-ga.events/event/zac-brown-band-atlanta-tuesday-december-31-2024-800-pm/?pid=769)

4. **Moon Taxi Concert**
   - **Time:** 8:00 PM
   - **Location:** The Tabernacle, 152 Luckie St, Atlanta, GA
   - **Tickets:** [Available Here](https://atlanta-ga.events/event/moon-taxi-atlanta-tuesday-december-31-2024-800-pm/?pid=7640)

5. **Big Freedia Concert**
   - **Time:** 9:30 PM
   - **Location:** City Winery, 675 Ponce De Leon Ave, Atlanta, GA
   - **Tickets:** [Available Here](https://atlanta-ga.events/event/big-freedia-atlanta-tuesday-december-31-2024-930-pm/?pid=2964)

6. **Daily Bread Concert**
   - **Time:** 9:30 PM
   - **Location:** The Eastern, 800 Old Flat Shoals Road SE, Atlanta, GA
   - **Tickets:** [Available Here](https://atlanta-ga.events/event/daily-bread-atlanta-tuesday-december-31-2024-930-pm/?pid=4699)

7. **HUGEL New Year's Eve | Make The Girls Dance 2024**
   - **Time:** 10:00 PM
   - **Location:** District Atlanta, 269 Armour Dr NE, Atlanta, GA
   - **Tickets:** [Available Here](https://atlanta-ga.events/event/hugel-artist-atlanta-tuesday-december-31-2024-1000-pm/?pid=2550)

8. **New Year’s Eve Bash at The Whitley**
   - **Time:** 9:00 PM
   - **Location:** The Whitley, Atlanta, GA
   - **Details:** [More Information](https://discoveratlanta.com/event/detail/new-year-s-eve-bash-at-the-whitley/)

9. **Noon Year’s Eve at LEGO Discovery Center Atlanta**
   - **Time:** 9:00 PM
   - **Location:** LEGO Discovery Center, Atlanta, GA
   - **Details:** [More Information](https://discoveratlanta.com/event/detail/noon-year-s-eve-at-lego-discovery-center-atlanta/)

10. **Atlanta’s New Year’s Eve Peach Drop Festival**
    - **Time:** All Day Event
    - **Location:** Various locations in Atlanta, GA
    - **Details:** [More Information](https://discoveratlanta.com/event/detail/atlanta-s-new-year-s-eve-peach-drop-festival/)

These events offer a variety of entertainment options to ring in the New Year in Atlanta. Enjoy your day!


In [5]:
output.LastMessage.DisplayAs("text/markdown");

Here are some exciting events happening in Atlanta today, December 31, 2024:

1. **Georgia Tech Yellow Jackets vs. Notre Dame Fighting Irish Basketball Game**
   - **Time:** 2:30 PM
   - **Location:** McCamish Pavilion, 965 Fowler St., Atlanta, GA
   - **Tickets:** [Available Here](https://atlanta-ga.events/event/georgia-tech-yellow-jackets-vs-notre-dame-fighting-irish-atlanta-tuesday-december-31-2024-230-pm/)

2. **Provenance NYE: Seven Lions, Knife Party & Grabbitz**
   - **Time:** 7:00 PM
   - **Location:** Atlanta Coliseum, 2075 Market St, Duluth, GA
   - **Tickets:** [Available Here](https://atlanta-ga.events/event/provenance-nye-seven-lions-knife-party-and-grabbitz-duluth-tuesday-december-31-2024-700-pm/?pid=8320)

3. **Zac Brown Band Concert**
   - **Time:** 8:00 PM
   - **Location:** State Farm Arena, One Philips Drive, Atlanta, GA
   - **Tickets:** [Available Here](https://atlanta-ga.events/event/zac-brown-band-atlanta-tuesday-december-31-2024-800-pm/?pid=769)

4. **Moon Taxi Concert**
   - **Time:** 8:00 PM
   - **Location:** The Tabernacle, 152 Luckie St, Atlanta, GA
   - **Tickets:** [Available Here](https://atlanta-ga.events/event/moon-taxi-atlanta-tuesday-december-31-2024-800-pm/?pid=7640)

5. **Big Freedia Concert**
   - **Time:** 9:30 PM
   - **Location:** City Winery, 675 Ponce De Leon Ave, Atlanta, GA
   - **Tickets:** [Available Here](https://atlanta-ga.events/event/big-freedia-atlanta-tuesday-december-31-2024-930-pm/?pid=2964)

6. **Daily Bread Concert**
   - **Time:** 9:30 PM
   - **Location:** The Eastern, 800 Old Flat Shoals Road SE, Atlanta, GA
   - **Tickets:** [Available Here](https://atlanta-ga.events/event/daily-bread-atlanta-tuesday-december-31-2024-930-pm/?pid=4699)

7. **HUGEL New Year's Eve | Make The Girls Dance 2024**
   - **Time:** 10:00 PM
   - **Location:** District Atlanta, 269 Armour Dr NE, Atlanta, GA
   - **Tickets:** [Available Here](https://atlanta-ga.events/event/hugel-artist-atlanta-tuesday-december-31-2024-1000-pm/?pid=2550)

8. **New Year’s Eve Bash at The Whitley**
   - **Time:** 9:00 PM
   - **Location:** The Whitley, Atlanta, GA
   - **Details:** [More Information](https://discoveratlanta.com/event/detail/new-year-s-eve-bash-at-the-whitley/)

9. **Noon Year’s Eve at LEGO Discovery Center Atlanta**
   - **Time:** 9:00 PM
   - **Location:** LEGO Discovery Center, Atlanta, GA
   - **Details:** [More Information](https://discoveratlanta.com/event/detail/noon-year-s-eve-at-lego-discovery-center-atlanta/)

10. **Atlanta’s New Year’s Eve Peach Drop Festival**
    - **Time:** All Day Event
    - **Location:** Various locations in Atlanta, GA
    - **Details:** [More Information](https://discoveratlanta.com/event/detail/atlanta-s-new-year-s-eve-peach-drop-festival/)

These events offer a variety of entertainment options to ring in the New Year in Atlanta. Enjoy your day!

In [6]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search and Scrape",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends. Be careful to not include events in the past in your final answer. You should perform four searches at once.",
};
var output2 = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output2.Dialog.DisplayAs("text/markdown");


User: Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends. Be careful to not include events in the past in your final answer. You should perform four searches at once.

Assistant: I ran this code:
```
from datetime import datetime, timedelta

# Get today's date
today = datetime.today()

# Calculate the next four weekends (Saturday and Sunday)
weekends = []
for i in range(4):
    saturday = today + timedelta(days=(5 - today.weekday()) % 7 + i * 7)
    sunday = saturday + timedelta(days=1)
    weekends.append((saturday, sunday))

weekends
```

I called the tool named crawl with these arguments:
```
{"q": "events in Atlanta January 4 2025", "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"events in Atlanta January 4 2025"},"webPages":{"value":[{"name":"Atlanta Events January 2025: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/january/","displayUrl":"https://atlanta-ga.events/january","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for January 2025. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month."},{"name":"Atlanta Events \u2013 January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","snippet":"Events From January 1 To The Weekend Of January 4-5 2025. \u2192 See December events. \u2192 For concerts this week, see concerts in Atlanta in January \u2013 IllumiNights, November 14 2024 - January 19 2025 Annual after-hours walk-through holiday lights show at Zoo Atlanta, with a mile of illuminated animal lanterns.","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Events \u0026amp; Things To Do in Atlanta in 2025 - AllEvents.in","url":"https://allevents.in/atlanta/january","displayUrl":"https://allevents.in/atlanta/january","snippet":"Find events and things to do in January 2025 in Atlanta. Discover parties, concerts, meets, shows, sports, club, reunion, Performance happening in January 2024 in Atlanta. ... Make this year an incredible one by starting it from exploring events in January 2025 in Atlanta. Find your interests, register or buy tickets for all the upcoming events ..."},{"name":"Discover 2025 Events \u0026amp; Activities in Atlanta, GA | Eventbrite","url":"https://www.eventbrite.com/d/ga--atlanta/2025/","displayUrl":"https://www.eventbrite.com/d/ga--atlanta/2025","snippet":"Share this event: 2025 Atlanta Travel \u0026amp; Adventure Show Save this event: 2025 Atlanta Travel \u0026amp; Adventure Show. ELEVATE : IGNITE CONNECTIONS FOR 2025. Sat, Jan 11, 5:00 PM ... NuNu\u0026#39;s Disco Birthday Brunch Jan 5th 2025 Save this event: NuNu\u0026#39;s Disco Birthday Brunch Jan 5th 2025. Almost full. NYEATL.COM: The VIBE NYE 2025 Celebration at THE TWELVE ..."},{"name":"Atlanta in 2025 | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/2025","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/2025","snippet":"Concerts in Atlanta in 2025. Find tickets to all live music, concerts, tour dates and festivals in and around Atlanta. Currently there are 968 upcoming events. Filter by artist. All Your favorite artists. Filter by date. All Tonight ... Saturday 04 January 2025 \u2013 Saturday 04 January 2025; SalgirahArts Festival: Expressions of Shukran 2025."},{"name":"Atlanta Events Calendar 2024/2025 | Atlanta, GA","url":"https://atlanta-ga.events/","displayUrl":"https://atlanta-ga.events","snippet":"Atlanta Events 2024/2025. Discover the calendar of all current and upcoming events in Atlanta, GA. ... Jan 4, 2025. Category. Search Events Calendar. Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec. Best Events. Schedule \u0026amp; Tickets. List Of Events. Venues. This ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events | Explore Georgia","url":"https://exploregeorgia.org/atlanta/events","displayUrl":"https://exploregeorgia.org/atlanta/events","snippet":"Events in Atlanta. Georgia\u0026#39;s capital city hosts events throughout the year that draw attendees from around the world. Browse the ideas and Atlanta events below to help plan your trip. ... January 4, 2025 - May 15, 2025 Adventure Cubs. Zoo Atlanta. View all dates. Atlanta. Saturday, January 4. January 4, 2025 - May 24, 2025","thumbnailUrl":"https://www.bing.com/th?id=OIP.qcVhz7_S17NbT6s7cPa8nQHaEi\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Concerts in January 2025","url":"https://atlanta.theater/concerts/january/","displayUrl":"https://atlanta.theater/concerts/january","snippet":"Jan 4, 2025 12:00 PM . Canopy . Concerts: 1. Altar at Masquerade - GA . Jan 4, 2025 7:00 PM . Da\u0026#39;Vinchi ... Join our newsletter now to stay current on all theater events in Atlanta, plus the occasional special offer and discount tickets! Unique events and incredible deals are straight to your inbox.","thumbnailUrl":"https://www.bing.com/th?id=OIP.ER6x2TrzIBYiwFYRDnnIMQHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta in January | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","snippet":"Concerts in Atlanta in January. Find tickets to all live music, concerts, tour dates and festivals in and around Atlanta. Currently there are 64 upcoming events. Filter by artist. All Your favorite artists. Filter by date. All Tonight ... Thursday 02 January 2025; Joe Alterman. Eddie\u0026#39;s Attic, Decatur, GA, US . Friday 03 January 2025;"},{"name":"Featured Events in Atlanta in January 2025(December Updated)","url":"https://us.trip.com/events/2607417-2025-01-atlanta-collection/","displayUrl":"https://us.trip.com/events/2607417-2025-01-atlanta-collection","snippet":"Searching for events in Atlanta in January 2025? Trip.com has curated a list of all the events you might like in Atlanta, including live performances, concerts, festivals, and exhibitions. There\u0026#39;s something for everyone. Find and register for popular events in your area now!"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "events in Atlanta January 11 2025", "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"events in Atlanta January 11 2025"},"webPages":{"value":[{"name":"Atlanta Events January 2025: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/january/","displayUrl":"https://atlanta-ga.events/january","snippet":"Atlanta Events January 2025. Check out the complete schedule of this month\u2019s events. \u201CAtlanta Events\u201D is your guide to epic events: ... Prices from $11 Avg. price ~ $50. Available Tickets: 587. Tickets; Jan. 04. 2025. 8:00 PM Sat. SalgirahArts Festival: Expressions of Shukran.","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events \u2013 January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","snippet":"Atlanta\u2019s January event highlights include the various Martin Luther King Day celebrations and parade. Other things to do in January include wine and food festivals, and the Cathedral Antiques Show and Tour of Homes. ... January 11 2025 Comedy show at Atlanta Symphony Hall. More details \u2013 Last Podcast on the Left: JK Ultra Tour, January 11 ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"January 11 events in Atlanta, GA - Eventbrite","url":"https://www.eventbrite.com/d/ga--atlanta/january-11/","displayUrl":"https://www.eventbrite.com/d/ga--atlanta/january-11","snippet":"Save this event: 2025 Atlanta Travel \u0026amp; Adventure Show Share this event: 2025 Atlanta Travel \u0026amp; Adventure Show. 2025 Atlanta Travel \u0026amp; Adventure Show. ... Sat, Jan 11, 4:00 PM. Atlanta\u0026#39;s Original Uptown Comedy Corner (225) Save this event: Saturday Shenanigans Open Mic Comedy Show, ..."},{"name":"Events \u0026amp; Things To Do in Atlanta in 2025 - AllEvents.in","url":"https://allevents.in/atlanta/january","displayUrl":"https://allevents.in/atlanta/january","snippet":"Make this year an incredible one by starting it from exploring events in January 2025 in Atlanta. Find your interests, register or buy tickets for all the upcoming events that you love. Added to your interest Add to your interest ... 11 interested. Atlanta Career Fair Hilton Mon, 13 Jan 313 interested. A Tribute to Quincy Jones City Winery ..."},{"name":"Discover 2025 Events \u0026amp; Activities in Atlanta, GA | Eventbrite","url":"https://www.eventbrite.com/d/ga--atlanta/2025/","displayUrl":"https://www.eventbrite.com/d/ga--atlanta/2025","snippet":"Share this event: 2025 Atlanta Travel \u0026amp; Adventure Show Save this event: 2025 Atlanta Travel \u0026amp; Adventure Show. ELEVATE : IGNITE CONNECTIONS FOR 2025. Sat, Jan 11, 5:00 PM. The Ritz-Carlton, Atlanta. Save this event: ELEVATE : IGNITE CONNECTIONS FOR 2025 Share this event: ELEVATE : IGNITE CONNECTIONS FOR 2025."},{"name":"Cool Things To Do in January in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta","snippet":"The event is happening Jan. 25 and 26, Feb. 1, 2, 8 and 9 from 4-9 p.m. The summit skyride is happening from 10 a.m. to 5 p.m. Martin Luther King Jr. Day on Jan. 20 Atlanta is the perfect place to honor Dr. King (Photo courtesy of The King Center) The King Center plans events from Jan. 7-20 to honor Martin Luther King Jr.","thumbnailUrl":"https://www.bing.com/th?id=OIP.aU9EcatRnz4MDkIKyYbkQgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta in 2025 | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/2025","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/2025","snippet":"Concerts in Atlanta in 2025. Find tickets to all live music, concerts, tour dates and festivals in and around Atlanta. Currently there are 968 upcoming events. Filter by artist. All Your favorite artists. Filter by date. All Tonight ... Saturday 11 January 2025; IN-ZO and Mersiv Tripp St., Daggz, Supertask, DUFFREY, ..."},{"name":"Atlanta in January | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","snippet":"Concerts in Atlanta in January. Find tickets to all live music, concerts, tour dates and festivals in and around Atlanta. Currently there are 64 upcoming events. Filter by artist. All Your favorite artists. Filter by date. All ... Saturday 11 January 2025; IN-ZO Tripp St., Daggz, and Supertask. The Eastern, Atlanta, GA, US ."},{"name":"Concerts In Atlanta, January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-january.html","snippet":"Concerts On Sunday January 12 2025 \u2013 Coucheron \u002B The Four Seasons at Atlanta Symphony Hall. More details \u2013 Maggie Koerner at Eddie\u2019s Attic, Decatur. More details. Concerts On Monday January 13 2025. Concerts today to be confirmed. Concerts On Tuesday January 14 2025 \u2013 L\u2019Imp\u00E9ratrice at The Eastern. More details. Concerts On Wednesday ..."},{"name":"Featured Events in Atlanta in January 2025(December Updated)","url":"https://us.trip.com/events/2607417-2025-01-atlanta-collection/","displayUrl":"https://us.trip.com/events/2607417-2025-01-atlanta-collection","snippet":"Searching for events in Atlanta in January 2025? Trip.com has curated a list of all the events you might like in Atlanta, including live performances, concerts, festivals, and exhibitions. There\u0026#39;s something for everyone. Find and register for popular events in your area now!"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "events in Atlanta January 18 2025", "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"events in Atlanta January 18 2025"},"webPages":{"value":[{"name":"Atlanta Events January 2025: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/january/","displayUrl":"https://atlanta-ga.events/january","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for January 2025. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month."},{"name":"Events \u0026amp; Things To Do in Atlanta in 2025 - AllEvents.in","url":"https://allevents.in/atlanta/january","displayUrl":"https://allevents.in/atlanta/january","snippet":"Find events and things to do in January 2025 in Atlanta. Discover parties, concerts, meets, shows, sports, club, reunion, Performance happening in January 2024 in Atlanta. ... Variety Playhouse, Atlanta: 18 Jan 2025: Electric Avenue at Variety Playhouse: Variety Playhouse, Atlanta: 18 Jan 2025: Fulton Lee at The Loft at Center Stage:"},{"name":"Atlanta Events \u2013 January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","snippet":"Events From January 13 To Martin Luther King Day Weekend, January 18-20 2025. \u2192 For concerts this week, see concerts in Atlanta in January \u2013 IllumiNights, November 14 2024 - January 19 2025 Annual after-hours walk-through holiday lights show at Zoo Atlanta, with a mile of illuminated animal lanterns. More details","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta in January | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","snippet":"Concerts in Atlanta in January. Find tickets to all live music, concerts, tour dates and festivals in and around Atlanta. Currently there are 64 upcoming events. Filter by artist. All Your favorite artists. Filter by date. All ... Saturday 18 January 2025; Fulton Lee. The Loft, Atlanta, GA, US ."},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... Multiple dates until 01/18/2025. Atlanta\u2019s Youth Basketball League 12/29/24 \u2013 12/30/24. Festive Features ... January 18. Saturday. AT\u0026amp;T Playoff Playlist Live with Lil Wayne \u0026amp; More! 1/18/25 \u2013 1/19/25 ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Featured Events in Atlanta in January 2025(December Updated)","url":"https://us.trip.com/events/2607417-2025-01-atlanta-collection/","displayUrl":"https://us.trip.com/events/2607417-2025-01-atlanta-collection","snippet":"Searching for events in Atlanta in January 2025? Trip.com has curated a list of all the events you might like in Atlanta, including live performances, concerts, festivals, and exhibitions. There\u0026#39;s something for everyone. Find and register for popular events in your area now!"},{"name":"Atlanta January 2025: Events, Concerts, Clubs \u0026amp; Things to Do - Experism","url":"https://www.experism.com/atlanta/january","displayUrl":"https://www.experism.com/atlanta/january","snippet":"To be held at Bold Monk Brewing Company, the participants will taste 55 cask-conditioned ales from famous local and international craft brewers. The event returns Saturday, January 31, 2026, from 2:30 PM to 6:00 PM at 1737 Ellsworth Industrial Blvd NW, Atlanta, GA 30318. Tickets for the event are available for purchase at $48.00 each.","thumbnailUrl":"https://www.bing.com/th?id=OIP.WKXWyOtbF2Q-oMmg6Y-HiQHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events Calendar 2024/2025 | Atlanta, GA","url":"https://atlanta-ga.events/","displayUrl":"https://atlanta-ga.events","snippet":"Find the Best Events in Atlanta, GA: Concerts, Theater \u0026amp; Comedy Shows, Sporting Events, Opera Performances, Music Festivals, \u0026amp; more. ... Jan 4, 2025. Category. Search Events Calendar. Jan Feb Mar Apr May Jun Jul Aug Sep ... 18 tickets remaining! Tickets; Dec. 31. 2024. 3:30 AM Tue.","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta in 2025 | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/2025","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/2025","snippet":"Concerts in Atlanta in 2025. Find tickets to all live music, concerts, tour dates and festivals in and around Atlanta. ... Saturday 04 January 2025 \u2013 Saturday 04 January 2025; SalgirahArts Festival: Expressions of Shukran 2025. Gas South Theater ... Tue 18 Feb 2025 The Masquerade - Purgatory Atlanta, GA, US. J.R. Carroll ..."},{"name":"Atlanta Concerts in January 2025","url":"https://atlanta.theater/concerts/january/","displayUrl":"https://atlanta.theater/concerts/january","snippet":"Explore the schedule of all concerts in Atlanta, GA. Find the best live music events this month, and buy concert tickets today! ... Jan 4, 2025 12:00 PM . The Sufistics . Concerts: 3. Gas South Theater . Jan 4, 2025 12:00 PM . Canopy ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.ER6x2TrzIBYiwFYRDnnIMQHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "events in Atlanta January 25 2025", "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"events in Atlanta January 25 2025"},"webPages":{"value":[{"name":"Atlanta Events January 2025: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/january/","displayUrl":"https://atlanta-ga.events/january","snippet":"Atlanta Events January 2025. ... Prices from $25 Avg. price ~ $34. 42 tickets remaining! Tickets; Jan. 04. 2025. 7:00 PM Sat. Da\u0026#39;Vinchi. ATL Comedy Theater. 30093, 4650 Jimmy Carter Boulevard, Norcross, GA, US ... Either way, you\u2019ll be the first to know about the best events in Atlanta, GA.","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events \u2013 January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","snippet":"\u2013 Run With the Dogs, January 25 2025 Annual 5K and pet races in Decatur to benefit the Decatur Bulldogs Boosters. More details \u2013 Spring Gardening Symposium \u0026amp; Plant Sale, January 25 2025 Special event at the Atlanta Botanical Garden, with speakers and plant sale. More details \u2013 Gals on the Go, January 25 2025 Podcast live show at Center Stage.","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Events \u0026amp; Things To Do in Atlanta in 2025 - AllEvents.in","url":"https://allevents.in/atlanta/january","displayUrl":"https://allevents.in/atlanta/january","snippet":"Find events and things to do in January 2025 in Atlanta. Discover parties, concerts, meets, shows, sports, club, reunion, Performance happening in January 2024 in Atlanta. ... Make this year an incredible one by starting it from exploring events in January 2025 in Atlanta. Find your interests, register or buy tickets for all the upcoming events ..."},{"name":"Atlanta in 2025 | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/2025","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/2025","snippet":"Concerts in Atlanta in 2025. Find tickets to all live music, concerts, tour dates and festivals in and around Atlanta. Currently there are 968 upcoming events. Filter by artist. All Your favorite artists. Filter by date. All Tonight ... Sat 25 Jan 2025 Capitol Theatre Macon, GA, US. Home; About us; Get the app; Blog; Jobs; Support;"},{"name":"Concerts 2025 events in Atlanta, GA - Eventbrite","url":"https://www.eventbrite.com/d/ga--atlanta/concerts-2025/","displayUrl":"https://www.eventbrite.com/d/ga--atlanta/concerts-2025","snippet":"DJ DIESEL aka SHAQ @ Believe Music Hall | Sat, Jan 25th! Sat, Jan 25, 9:00 PM. Believe Music Hall. Save this event: DJ DIESEL aka SHAQ @ Believe Music Hall | Sat, Jan 25th! ... Save this event: 2025 CATHEDRAL SINGERS CONCERT Share this event: ... Save this event: Atlanta Speed Dating | Ages 36-48 | Singles Night | SpeedAtlanta Share this event: ..."},{"name":"Featured Events in Atlanta in January 2025(December Updated)","url":"https://us.trip.com/events/2607417-2025-01-atlanta-collection/","displayUrl":"https://us.trip.com/events/2607417-2025-01-atlanta-collection","snippet":"Searching for events in Atlanta in January 2025? Trip.com has curated a list of all the events you might like in Atlanta, including live performances, concerts, festivals, and exhibitions. There\u0026#39;s something for everyone. Find and register for popular events in your area now!"},{"name":"Atlanta in January | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","snippet":"Concerts in Atlanta in January. Find tickets to all live music, concerts, tour dates and festivals in and around Atlanta. Currently there are 64 upcoming events. Filter by artist. All Your favorite artists. Filter by date. All Tonight ... Sat 25 Jan 2025 Buckhead Theatre Atlanta, GA, US. Cool Company. Fri 22 ..."},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... Atlanta Black Expo 2025 2/22/25 \u2013 2/23/25. QS Discover Master\u2019s and MBA Fair in Atlanta 2/22/25. ... January 25. Saturday. Heroes Among Us: A Musical Tribute 1/25/25. January 25. Saturday.","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Concerts In Atlanta, January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-january.html","snippet":"Concerts On Sunday January 12 2025 \u2013 Coucheron \u002B The Four Seasons at Atlanta Symphony Hall. More details \u2013 Maggie Koerner at Eddie\u2019s Attic, Decatur. More details. Concerts On Monday January 13 2025. Concerts today to be confirmed. Concerts On Tuesday January 14 2025 \u2013 L\u2019Imp\u00E9ratrice at The Eastern. More details. Concerts On Wednesday ..."},{"name":"Featured Events in Atlanta in January 2025(December Updated)","url":"https://www.trip.com/events/2607417-2025-01-atlanta-collection/?page=3","displayUrl":"https://www.trip.com/events/2607417-2025-01-atlanta-collection/?page=3","snippet":"Experience the electrifying Malaa Alter Ego Tour at Tabernacle in Atlanta on January 25, 2025. Located at 152 Luckie Street, Atlanta, GA, 30303, this event promises an unforgettable night of music and excitement. Don\u0026#39;t miss the chance to witness an incredible performance by Malaa in the vibrant city of Atlanta."}]}}
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://atlanta-ga.events/january/"}
```
and got this result:
```
"JavaScript support required!\r\n\r\n[](https://atlanta-ga.events)[All Events](https://atlanta-ga.events/)[Sports](https://atlanta-ga.events/sports/)[Concerts](https://atlanta-ga.events/concerts/)[Comedy](https://atlanta-ga.events/comedy-shows/)[Broadway](https://atlanta-ga.events/broadway/)[All Shows](https://atlanta-ga.events/shows/)[Family](https://atlanta-ga.events/family-shows/)[Ballet](https://atlanta-ga.events/ballet/)[Opera](https://atlanta-ga.events/opera/)[This Weekend](https://atlanta-ga.events/this-weekend/)[Next Week](https://atlanta-ga.events/next-week/)[Festivals](https://atlanta-ga.events/festivals/)[Venues](https://atlanta-ga.events/entertainment-venues/)* All Events\r\n* Sports\r\n* Concerts\r\n* Comedy\r\n* Theater Broadway All Shows Family Ballet Opera\r\n* More This Weekend Next Week Festivals Venues\r\n\r\n[All Events](https://atlanta-ga.events/)[Sports](https://atlanta-ga.events/sports/)[Concerts](https://atlanta-ga.events/concerts/)[Comedy](https://atlanta-ga.events/comedy-shows/)[Theater](#)* Broadway\r\n* All Shows\r\n* Family\r\n* Ballet\r\n* Opera\r\n\r\n[Broadway](https://atlanta-ga.events/broadway/)[All Shows](https://atlanta-ga.events/shows/)[Family](https://atlanta-ga.events/family-shows/)[Ballet](https://atlanta-ga.events/ballet/)[Opera](https://atlanta-ga.events/opera/)[More](#)* This Weekend\r\n* Next Week\r\n* Festivals\r\n* Venues\r\n\r\n[This Weekend](https://atlanta-ga.events/this-weekend/)[Next Week](https://atlanta-ga.events/next-week/)[Festivals](https://atlanta-ga.events/festivals/)[Venues](https://atlanta-ga.events/entertainment-venues/)* Home\r\n* January\r\n\r\n[Home](https://atlanta-ga.events)# Atlanta Events January 2025\r\n\r\nCheck out the complete schedule of this month\u0026#8217;s events. \u0026#8220;Atlanta Events\u0026#8221; is your guide to epic events: concerts, theater shows, Broadway musicals and plays, opera, ballet, and sporting events to experience while in town. Get ready to discover the best entertainment venues in Atlanta, GA. Find your favorite events, check seating charts, buy tickets, and how to get there. Buy event tickets in advance for perfect seats. We\u2019re sure you will find something that sparks your curiosity!\r\n\r\n**Check out the complete schedule of this month\u0026#8217;s events.**[Atlanta Events](https://atlanta-ga.events/)[concerts](https://atlanta-ga.events/concerts/)[theater shows](https://atlanta-ga.events/shows/)[Broadway](https://atlanta-ga.events/broadway/)[opera](https://atlanta-ga.events/opera/)[ballet](https://atlanta-ga.events/ballet/)[sporting events](https://atlanta-ga.events/sports/)[Feb](https://atlanta-ga.events/february/)[Mar](https://atlanta-ga.events/march/)[Apr](https://atlanta-ga.events/april/)[May](https://atlanta-ga.events/may/)[Jun](https://atlanta-ga.events/june/)[Jul](https://atlanta-ga.events/july/)[Aug](https://atlanta-ga.events/august/)[Sep](https://atlanta-ga.events/september/)[Oct](https://atlanta-ga.events/october/)[Nov](https://atlanta-ga.events/november/)[Dec](https://atlanta-ga.events/december/)## Schedule \u0026 Tickets\r\n\r\n* Jan 01 2025 1:00 PM Wed Chick-fil-A Peach Bowl - College Football Playoff Quarterfinal: Arizona State Sun Devils vs. Texas Longhorns Mercedes-Benz Stadium | Capacity: 83000 30313, Martin Luther King Jr Dr SW \u0026 Northside Dr NW, Atlanta, GA, US Prices from $30 Avg. price ~ $155 Available Tickets: 3581 Tickets\r\n* Jan 01 2025 8:00 PM Wed A Tribute to Quincy Jones City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $79 Avg. price ~ $91 4 tickets remaining! Tickets\r\n* Jan 02 2025 3:30 AM Thu Titanic: An Immersive Voyage Exhibition Hub Art Center Atlanta 30340, 5660 Buford Hwy NE, Atlanta, GA, US Prices from $61 Avg. price ~ $72 24 tickets remaining! Tickets\r\n* Jan 02 2025 6:00 PM Thu Remnant of a Loss Altar at Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $23 Avg. price ~ $31 16 tickets remaining! Tickets\r\n* Jan 02 2025 7:00 PM Thu Georgia Tech Yellow Jackets Women\u0027s Basketball vs. Syracuse Orange McCamish Pavilion 30332, 965 Fowler St., Atlanta, GA, US Prices from $20 Avg. price ~ $26 Available Tickets: 57 Tickets\r\n* Jan 02 2025 7:00 PM Thu College Park Skyhawks vs. Motor City Cruise Gateway Center Arena At College Park | Capacity: 3500 30337, 2330 Convention Center Concourse, Atlanta, GA, US Prices from $20 Avg. price ~ $73 Available Tickets: 631 Tickets\r\n* Jan 02 2025 7:00 PM Thu Georgia State Panthers vs. South Alabama Jaguars Georgia State Convocation Center 30312, 455 Capitol Ave SE, Atlanta, GA, US Prices from $8 Avg. price ~ $34 43 tickets remaining! Tickets\r\n* Jan 02 2025 8:00 PM Thu Stormy Daniels City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $78 Avg. price ~ $136 26 tickets remaining! Tickets\r\n* Jan 03 2025 3:30 AM Fri Titanic: An Immersive Voyage Exhibition Hub Art Center Atlanta 30340, 5660 Buford Hwy NE, Atlanta, GA, US Prices from $61 Avg. price ~ $72 24 tickets remaining! Tickets\r\n* Jan 03 2025 6:30 PM Fri Raheem DeVaughn City Winery - Atlanta | Capacity: 300 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $178 Avg. price ~ $216 5 tickets remaining! Tickets\r\n* Jan 03 2025 7:00 PM Fri Cast List Altar at Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $28 Avg. price ~ $33 18 tickets remaining! Tickets\r\n* Jan 03 2025 7:30 PM Fri Atlanta Gladiators vs. South Carolina Stingrays Gas South Arena | Capacity: 13100 30097, 6400 Sugarloaf Parkway, Duluth, GA, US Prices from $21 Avg. price ~ $67 Available Tickets: 1996 Tickets\r\n* Jan 03 2025 8:30 PM Fri Best of Atlanta Comedy Showcase Helium Comedy Club - Atlanta 30005, 3070 Windward Plaza, Alpharetta, GA, US Prices from $24 Avg. price ~ $40 43 tickets remaining! Tickets\r\n* Jan 04 2025 3:30 AM Sat Titanic: An Immersive Voyage Exhibition Hub Art Center Atlanta 30340, 5660 Buford Hwy NE, Atlanta, GA, US Prices from $64 Avg. price ~ $77 32 tickets remaining! Tickets\r\n* Jan 04 2025 12:00 PM Sat Georgia Tech Yellow Jackets vs. Boston College Eagles McCamish Pavilion 30332, 965 Fowler St., Atlanta, GA, US Prices from $8 Avg. price ~ $49 Available Tickets: 336 Tickets\r\n* Jan 04 2025 12:00 PM Sat SalgirahArts Festival: Expressions of Shukran Gas South Theater | Capacity: 708 30097, 6400 Sugarloaf Parkway, Duluth, GA, US Prices from $22 Avg. price ~ $43 Available Tickets: 211 Tickets\r\n* Jan 04 2025 5:00 PM Sat Kennesaw State Owls vs. Jacksonville State Gamecocks KSU Convocation Center 30144, 590 Cobb Ave. , Kennesaw, GA, US Prices from $25 Avg. price ~ $33 35 tickets remaining! Tickets\r\n* Jan 04 2025 7:00 PM Sat Georgia State Panthers vs. Louisiana-Lafayette Ragin\u0027 Cajuns Georgia State Convocation Center 30312, 455 Capitol Ave SE, Atlanta, GA, US Prices from $10 Avg. price ~ $33 46 tickets remaining! Tickets\r\n* Jan 04 2025 7:00 PM Sat Canopy Altar at Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $23 Avg. price ~ $31 18 tickets remaining! Tickets\r\n* Jan 04 2025 7:00 PM Sat Kaya\u0027s Embrace Purgatory Stage at Masquerade - GA | Capacity: 300 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Prices from $25 Avg. price ~ $34 42 tickets remaining! Tickets\r\n* Jan 04 2025 7:00 PM Sat Da\u0027Vinchi ATL Comedy Theater 30093, 4650 Jimmy Carter Boulevard, Norcross, GA, US Prices from $59 Avg. price ~ $101 Available Tickets: 60 Tickets\r\n* Jan 04 2025 7:30 PM Sat Georgia Swarm vs. Rochester Knighthawks Gas South Arena | Capacity: 13100 30097, 6400 Sugarloaf Parkway, Duluth, GA, US Prices from $9 Avg. price ~ $48 Available Tickets: 507 Tickets\r\n* Jan 04 2025 8:00 PM Sat SalgirahArts Festival: Expressions of Shukran Gas South Theater | Capacity: 708 30097, 6400 Sugarloaf Parkway, Duluth, GA, US Prices from $28 Avg. price ~ $32 3 tickets remaining! Tickets\r\n* Jan 04 2025 8:00 PM Sat Sonic Symphony Live Cobb Energy Performing Arts Centre | Capacity: 2750 30339, 2800 Cobb Galleria Pkwy, Atlanta, GA, US Prices from $125 Avg. price ~ $338 Available Tickets: 59 Tickets\r\n* Jan 04 2025 8:30 PM Sat Best of Atlanta Comedy Showcase Helium Comedy Club - Atlanta 30005, 3070 Windward Plaza, Alpharetta, GA, US Prices from $24 Avg. price ~ $34 32 tickets remaining! Tickets\r\n* Jan 04 2025 9:00 PM Sat Club 90s: Justin Bieber Night Buckhead Theatre | Capacity: 2500 30305, 3110 Roswell Road, Atlanta, GA, US Prices from $34 Avg. price ~ $40 30 tickets remaining! Tickets\r\n\r\n![Chick-fil-A Peach Bowl - College Football Playoff Quarterfinal: Arizona State Sun Devils vs. Texas Longhorns](https://atlanta-ga.events/wp-content/uploads/2023/05/Chick-fil-A-Peach-Bowl.jpg)\r\n[Chick-fil-A Peach Bowl](https://atlanta-ga.events/tickets/chick-fil-a-peach-bowl/)[Arizona State Sun Devils](https://atlanta-ga.events/tickets/arizona-state-sun-devils/)[Texas Longhorns](https://atlanta-ga.events/tickets/texas-longhorns/)[Mercedes-Benz Stadium](https://atlanta-ga.events/venue/mercedes-benz-stadium/)[Tickets](https://atlanta-ga.events/event/chick-fil-a-peach-bowl-college-football-playoff-quarterfinal-arizona-state-sun-devils-vs-texas-longhorns-atlanta-wednesday-january-1-2025-100-pm/?pid=1702)![Atlanta Events Schedule](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[A Tribute to Quincy Jones](https://atlanta-ga.events/tickets/a-tribute-to-quincy-jones/)[City Winery - Atlanta](https://atlanta-ga.events/venue/city-winery-atlanta/)[Tickets](https://atlanta-ga.events/event/a-tribute-to-quincy-jones-atlanta-wednesday-january-1-2025-800-pm/?pid=8328)![Atlanta Events Schedule](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Titanic: An Immersive Voyage](https://atlanta-ga.events/tickets/titanic-an-immersive-voyage/)[Exhibition Hub Art Center Atlanta](https://atlanta-ga.events/venue/exhibition-hub-art-center-atlanta/)[Tickets](https://atlanta-ga.events/event/titanic-an-immersive-voyage-atlanta-thursday-january-2-2025-tba/?pid=8451)![Atlanta Events Schedule](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Remnant of a Loss](https://atlanta-ga.events/tickets/remnant-of-a-loss/)[Altar at Masquerade - GA](https://atlanta-ga.events/venue/altar-at-masquerade-ga/)[Tickets](https://atlanta-ga.events/event/remnant-of-a-loss-atlanta-thursday-january-2-2025-600-pm/?pid=8096)![Georgia Tech Yellow Jackets Women\u0027s Basketball vs. Syracuse Orange](https://atlanta-ga.events/wp-content/uploads/2024/02/georgia-tech-yellow-jackets-womens-basketball.jpg)\r\n[Georgia Tech Yellow Jackets](https://atlanta-ga.events/tickets/georgia-tech-yellow-jackets/)[Syracuse Orange](https://atlanta-ga.events/tickets/syracuse-orange/)[McCamish Pavilion](https://atlanta-ga.events/venue/mccamish-pavilion/)[Tickets](https://atlanta-ga.events/event/georgia-tech-yellow-jackets-womens-basketball-vs-syracuse-orange-atlanta-thursday-january-2-2025-700-pm/?pid=1318)![College Park Skyhawks vs. Motor City Cruise](https://atlanta-ga.events/wp-content/uploads/2023/09/College-Park-SkyHawks.jpg)\r\n[Motor City Cruise](https://atlanta-ga.events/tickets/motor-city-cruise/)[Gateway Center Arena At College Park](https://atlanta-ga.events/venue/gateway-center-arena-at-college-park/)[Tickets](https://atlanta-ga.events/event/college-park-skyhawks-vs-motor-city-cruise-atlanta-thursday-january-2-2025-700-pm/?pid=1275)![Georgia State Panthers vs. South Alabama Jaguars](https://atlanta-ga.events/wp-content/uploads/2023/08/Georgia-State-Panthers-Basketball.jpg)\r\n[Georgia State Panthers](https://atlanta-ga.events/tickets/georgia-state-panthers/)[South Alabama Jaguars](https://atlanta-ga.events/tickets/south-alabama-jaguars/)[Georgia State Convocation Center](https://atlanta-ga.events/venue/georgia-state-convocation-center/)[Tickets](https://atlanta-ga.events/event/georgia-state-panthers-vs-south-alabama-jaguars-atlanta-thursday-january-2-2025-700-pm/?pid=1302)![Atlanta Events Schedule](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Stormy Daniels](https://atlanta-ga.events/tickets/stormy-daniels/)[City Winery - Atlanta](https://atlanta-ga.events/venue/city-winery-atlanta/)[Tickets](https://atlanta-ga.events/event/stormy-daniels-atlanta-thursday-january-2-2025-800-pm/?pid=8219)![Atlanta Events Schedule](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Titanic: An Immersive Voyage](https://atlanta-ga.events/tickets/titanic-an-immersive-voyage/)[Exhibition Hub Art Center Atlanta](https://atlanta-ga.events/venue/exhibition-hub-art-center-atlanta/)[Tickets](https://atlanta-ga.events/event/titanic-an-immersive-voyage-atlanta-friday-january-3-2025-tba/?pid=8451)![Raheem DeVaughn](https://atlanta-ga.events/wp-content/uploads/2024/08/raheem-devaughn.jpg)\r\n[Raheem DeVaughn](https://atlanta-ga.events/tickets/raheem-devaughn/)[City Winery - Atlanta](https://atlanta-ga.events/venue/city-winery-atlanta/)[Tickets](https://atlanta-ga.events/event/raheem-devaughn-atlanta-friday-january-3-2025-630-pm/?pid=1689)![Atlanta Events Schedule](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Cast List](https://atlanta-ga.events/tickets/cast-list/)[Altar at Masquerade - GA](https://atlanta-ga.events/venue/altar-at-masquerade-ga/)[Tickets](https://atlanta-ga.events/event/cast-list-atlanta-friday-january-3-2025-700-pm/?pid=8190)![Atlanta Gladiators vs. South Carolina Stingrays](https://atlanta-ga.events/wp-content/uploads/2024/02/atlanta-gladiators.jpg)\r\n[Atlanta Gladiators](https://atlanta-ga.events/tickets/atlanta-gladiators/)[South Carolina Stingrays](https://atlanta-ga.events/tickets/south-carolina-stingrays/)[Gas South Arena](https://atlanta-ga.events/venue/gas-south-arena/)[Tickets](https://atlanta-ga.events/event/atlanta-gladiators-vs-south-carolina-stingrays-duluth-friday-january-3-2025-730-pm/?pid=4801)![Best of Atlanta Comedy Showcase](https://atlanta-ga.events/wp-content/uploads/2024/12/best-of-atlanta-comedy-showcase.jpeg)\r\n[Best of Atlanta Comedy Showcase](https://atlanta-ga.events/tickets/best-of-atlanta-comedy-showcase/)[Helium Comedy Club - Atlanta](https://atlanta-ga.events/venue/helium-comedy-club-atlanta/)[Tickets](https://atlanta-ga.events/event/best-of-atlanta-comedy-showcase-alpharetta-friday-january-3-2025-830-pm/?pid=2327)![Atlanta Events Schedule](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Titanic: An Immersive Voyage](https://atlanta-ga.events/tickets/titanic-an-immersive-voyage/)[Exhibition Hub Art Center Atlanta](https://atlanta-ga.events/venue/exhibition-hub-art-center-atlanta/)[Tickets](https://atlanta-ga.events/event/titanic-an-immersive-voyage-atlanta-saturday-january-4-2025-tba/?pid=8451)![Georgia Tech Yellow Jackets vs. Boston College Eagles](https://atlanta-ga.events/wp-content/uploads/2023/08/Georgia-Tech-Yellow-Jackets-Basketball.jpg)\r\n[Georgia Tech Yellow Jackets](https://atlanta-ga.events/tickets/georgia-tech-yellow-jackets/)[Boston College Eagles](https://atlanta-ga.events/tickets/boston-college-eagles/)[McCamish Pavilion](https://atlanta-ga.events/venue/mccamish-pavilion/)[Tickets](https://atlanta-ga.events/event/georgia-tech-yellow-jackets-vs-boston-college-eagles-atlanta-saturday-january-4-2025-1200-pm/?pid=1282)![Atlanta Events Schedule](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Expressions of Shukran](https://atlanta-ga.events/tickets/expressions-of-shukran/)[Gas South Theater](https://atlanta-ga.events/venue/gas-south-theater/)[Tickets](https://atlanta-ga.events/event/salgiraharts-festival-expressions-of-shukran-duluth-saturday-january-4-2025-1200-pm/?pid=8220)![Kennesaw State Owls vs. Jacksonville State Gamecocks](https://atlanta-ga.events/wp-content/uploads/2024/02/kennesaw-state-owls-basketball.jpg)\r\n[Kennesaw State Owls](https://atlanta-ga.events/tickets/kennesaw-state-owls/)[Jacksonville State Gamecocks](https://atlanta-ga.events/tickets/jacksonville-state-gamecocks/)[KSU Convocation Center](https://atlanta-ga.events/venue/ksu-convocation-center/)[Tickets](https://atlanta-ga.events/event/kennesaw-state-owls-vs-jacksonville-state-gamecocks-kennesaw-saturday-january-4-2025-500-pm/?pid=4812)![Georgia State Panthers vs. Louisiana-Lafayette Ragin\u0027 Cajuns](https://atlanta-ga.events/wp-content/uploads/2023/08/Georgia-State-Panthers-Basketball.jpg)\r\n[Georgia State Panthers](https://atlanta-ga.events/tickets/georgia-state-panthers/)[Louisiana-Lafayette Ragin\u0027 Cajuns](https://atlanta-ga.events/tickets/louisiana-lafayette-ragin-cajuns/)[Georgia State Convocation Center](https://atlanta-ga.events/venue/georgia-state-convocation-center/)[Tickets](https://atlanta-ga.events/event/georgia-state-panthers-vs-louisiana-lafayette-ragin-cajuns-atlanta-saturday-january-4-2025-700-pm/?pid=1302)![Atlanta Events Schedule](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Canopy](https://atlanta-ga.events/tickets/canopy/)[Altar at Masquerade - GA](https://atlanta-ga.events/venue/altar-at-masquerade-ga/)[Tickets](https://atlanta-ga.events/event/canopy-atlanta-saturday-january-4-2025-700-pm/?pid=8191)![Atlanta Events Schedule](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Kaya\u0027s Embrace](https://atlanta-ga.events/tickets/kayas-embrace/)[Purgatory Stage at Masquerade - GA](https://atlanta-ga.events/venue/purgatory-stage-at-masquerade-ga/)[Tickets](https://atlanta-ga.events/event/kayas-embrace-atlanta-saturday-january-4-2025-700-pm/?pid=8401)![Atlanta Events Schedule](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Da\u0027Vinchi](https://atlanta-ga.events/tickets/davinchi/)[ATL Comedy Theater](https://atlanta-ga.events/venue/atl-comedy-theater/)[Tickets](https://atlanta-ga.events/event/davinchi-norcross-saturday-january-4-2025-700-pm/?pid=8304)![Georgia Swarm vs. Rochester Knighthawks](https://atlanta-ga.events/wp-content/uploads/2024/02/georgia-swarm.jpg)\r\n[Georgia Swarm](https://atlanta-ga.events/tickets/georgia-swarm/)[Rochester Knighthawks](https://atlanta-ga.events/tickets/rochester-knighthawks/)[Gas South Arena](https://atlanta-ga.events/venue/gas-south-arena/)[Tickets](https://atlanta-ga.events/event/georgia-swarm-vs-rochester-knighthawks-duluth-saturday-january-4-2025-730-pm/?pid=5028)![Atlanta Events Schedule](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Expressions of Shukran](https://atlanta-ga.events/tickets/expressions-of-shukran/)[Gas South Theater](https://atlanta-ga.events/venue/gas-south-theater/)[Tickets](https://atlanta-ga.events/event/salgiraharts-festival-expressions-of-shukran-duluth-saturday-january-4-2025-800-pm/?pid=8220)![Sonic Symphony Live](https://atlanta-ga.events/wp-content/uploads/2023/08/Sonic-Symphony-Live.jpg)\r\n[Sonic Symphony Live](https://atlanta-ga.events/tickets/sonic-symphony-live/)[Cobb Energy Performing Arts Centre](https://atlanta-ga.events/venue/cobb-energy-performing-arts-centre/)[Tickets](https://atlanta-ga.events/event/sonic-symphony-live-atlanta-saturday-january-4-2025-800-pm/?pid=4109)![Best of Atlanta Comedy Showcase](https://atlanta-ga.events/wp-content/uploads/2024/12/best-of-atlanta-comedy-showcase.jpeg)\r\n[Best of Atlanta Comedy Showcase](https://atlanta-ga.events/tickets/best-of-atlanta-comedy-showcase/)[Helium Comedy Club - Atlanta](https://atlanta-ga.events/venue/helium-comedy-club-atlanta/)[Tickets](https://atlanta-ga.events/event/best-of-atlanta-comedy-showcase-alpharetta-saturday-january-4-2025-830-pm/?pid=2327)![Club 90s: Justin Bieber Night](https://atlanta-ga.events/wp-content/uploads/2024/02/club-90s.jpg)\r\n[Club 90s](https://atlanta-ga.events/tickets/club-90s/)[Buckhead Theatre](https://atlanta-ga.events/venue/buckhead-theatre/)[Tickets](https://atlanta-ga.events/event/club-90s-justin-bieber-night-atlanta-saturday-january-4-2025-900-pm/?pid=8384)### List Of Events\r\n\r\n[Arizona State Sun Devils Events: 3 Mercedes-Benz Stadium Jan 1, 2025 1:00 PM](https://atlanta-ga.events/tickets/arizona-state-sun-devils/)![Arizona State Sun Devils](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Texas Longhorns Football Events: 3 Mercedes-Benz Stadium Jan 1, 2025 1:00 PM](https://atlanta-ga.events/tickets/texas-longhorns-football/)![Texas Longhorns Football](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[NCAA Bowl Games Events: 6 Mercedes-Benz Stadium Jan 1, 2025 1:00 PM](https://atlanta-ga.events/tickets/ncaa-bowl-games/)![NCAA Bowl Games](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Texas Longhorns Events: 3 Mercedes-Benz Stadium Jan 1, 2025 1:00 PM](https://atlanta-ga.events/tickets/texas-longhorns/)![Texas Longhorns](https://atlanta-ga.events/wp-content/uploads/2024/12/texas-longhorns-tickets.png)\r\n[College Football Playoffs Events: 6 Mercedes-Benz Stadium Jan 1, 2025 1:00 PM](https://atlanta-ga.events/tickets/college-football-playoffs/)![College Football Playoffs](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Chick-fil-A Peach Bowl Events: 3 Mercedes-Benz Stadium Jan 1, 2025 1:00 PM](https://atlanta-ga.events/tickets/chick-fil-a-peach-bowl/)![Chick-fil-A Peach Bowl](https://atlanta-ga.events/wp-content/uploads/2023/05/Chick-fil-A-Peach-Bowl.jpg)\r\n[Arizona State Sun Devils Football Events: 3 Mercedes-Benz Stadium Jan 1, 2025 1:00 PM](https://atlanta-ga.events/tickets/arizona-state-sun-devils-football/)![Arizona State Sun Devils Football](https://atlanta-ga.events/wp-content/uploads/2024/12/arizona-state-sun-devils-football.jpg)\r\n[A Tribute to Quincy Jones Events: 2 City Winery - Atlanta Jan 1, 2025 8:00 PM](https://atlanta-ga.events/tickets/a-tribute-to-quincy-jones/)![A Tribute to Quincy Jones](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Titanic: An Immersive Voyage Events: 44 Exhibition Hub Art Center Atlanta Jan 2, 2025 3:30 AM](https://atlanta-ga.events/tickets/titanic-an-immersive-voyage/)![Titanic: An Immersive Voyage](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Remnant of a Loss Events: 2 Altar at Masquerade - GA Jan 2, 2025 6:00 PM](https://atlanta-ga.events/tickets/remnant-of-a-loss/)![Remnant of a Loss](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Syracuse Orange Women\u0027s Basketball Events: 3 McCamish Pavilion Jan 2, 2025 7:00 PM](https://atlanta-ga.events/tickets/syracuse-orange-womens-basketball/)![Syracuse Orange Women\u0027s Basketball](https://atlanta-ga.events/wp-content/uploads/2024/09/syracuse-orange-womens-basketball.jpg)\r\n[Georgia Tech Yellow Jackets Events: 21 McCamish Pavilion Jan 2, 2025 7:00 PM](https://atlanta-ga.events/tickets/georgia-tech-yellow-jackets/)![Georgia Tech Yellow Jackets](https://atlanta-ga.events/wp-content/uploads/2023/08/Georgia-Tech-Yellow-Jackets.jpg)\r\n[South Alabama Jaguars Events: 3 Georgia State Convocation Center Jan 2, 2025 7:00 PM](https://atlanta-ga.events/tickets/south-alabama-jaguars/)![South Alabama Jaguars](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Georgia State Panthers Basketball Events: 18 Georgia State Convocation Center Jan 2, 2025 7:00 PM](https://atlanta-ga.events/tickets/georgia-state-panthers-basketball/)![Georgia State Panthers Basketball](https://atlanta-ga.events/wp-content/uploads/2023/08/Georgia-State-Panthers-Basketball.jpg)\r\n[South Alabama Jaguars Basketball Events: 3 Georgia State Convocation Center Jan 2, 2025 7:00 PM](https://atlanta-ga.events/tickets/south-alabama-jaguars-basketball/)![South Alabama Jaguars Basketball](https://atlanta-ga.events/wp-content/uploads/2024/08/south-alabama-jaguars-basketball.jpg)\r\n[Syracuse Orange Events: 3 McCamish Pavilion Jan 2, 2025 7:00 PM](https://atlanta-ga.events/tickets/syracuse-orange/)![Syracuse Orange](https://atlanta-ga.events/wp-content/uploads/2023/08/Syracuse-Orange.jpg)\r\n[College Park SkyHawks Events: 15 Gateway Center Arena At College Park Jan 2, 2025 7:00 PM](https://atlanta-ga.events/tickets/college-park-skyhawks/)![College Park SkyHawks](https://atlanta-ga.events/wp-content/uploads/2023/09/College-Park-SkyHawks.jpg)\r\n[Motor City Cruise Events: 6 Gateway Center Arena At College Park Jan 2, 2025 7:00 PM](https://atlanta-ga.events/tickets/motor-city-cruise/)![Motor City Cruise](https://atlanta-ga.events/wp-content/uploads/2024/09/motor-city-cruise.jpg)\r\n[Georgia Tech Yellow Jackets Women\u0027s Basketball Events: 12 McCamish Pavilion Jan 2, 2025 7:00 PM](https://atlanta-ga.events/tickets/georgia-tech-yellow-jackets-womens-basketball/)![Georgia Tech Yellow Jackets Women\u0027s Basketball](https://atlanta-ga.events/wp-content/uploads/2024/02/georgia-tech-yellow-jackets-womens-basketball.jpg)\r\n[Georgia State Panthers Events: 18 Georgia State Convocation Center Jan 2, 2025 7:00 PM](https://atlanta-ga.events/tickets/georgia-state-panthers/)![Georgia State Panthers](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Stormy Daniels Events: 2 City Winery - Atlanta Jan 2, 2025 8:00 PM](https://atlanta-ga.events/tickets/stormy-daniels/)![Stormy Daniels](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Raheem DeVaughn Events: 10 City Winery - Atlanta Jan 3, 2025 6:30 PM](https://atlanta-ga.events/tickets/raheem-devaughn/)![Raheem DeVaughn](https://atlanta-ga.events/wp-content/uploads/2024/08/raheem-devaughn.jpg)\r\n[Cast List Events: 2 Altar at Masquerade - GA Jan 3, 2025 7:00 PM](https://atlanta-ga.events/tickets/cast-list/)![Cast List](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[South Carolina Stingrays Events: 6 Gas South Arena Jan 3, 2025 7:30 PM](https://atlanta-ga.events/tickets/south-carolina-stingrays/)![South Carolina Stingrays](https://atlanta-ga.events/wp-content/uploads/2024/02/south-carolina-stingrays.jpg)\r\n[Atlanta Gladiators Events: 21 Gas South Arena Jan 3, 2025 7:30 PM](https://atlanta-ga.events/tickets/atlanta-gladiators/)![Atlanta Gladiators](https://atlanta-ga.events/wp-content/uploads/2024/02/atlanta-gladiators.jpg)\r\n[Best of Atlanta Comedy Showcase Events: 4 Helium Comedy Club - Atlanta Jan 3, 2025 8:30 PM](https://atlanta-ga.events/tickets/best-of-atlanta-comedy-showcase/)![Best of Atlanta Comedy Showcase](https://atlanta-ga.events/wp-content/uploads/2024/12/best-of-atlanta-comedy-showcase.jpeg)\r\n[Boston College Eagles Events: 3 McCamish Pavilion Jan 4, 2025 12:00 PM](https://atlanta-ga.events/tickets/boston-college-eagles/)![Boston College Eagles](https://atlanta-ga.events/wp-content/uploads/2023/08/Boston-College-Eagles.jpg)\r\n[Boston College Eagles Basketball Events: 3 McCamish Pavilion Jan 4, 2025 12:00 PM](https://atlanta-ga.events/tickets/boston-college-eagles-basketball/)![Boston College Eagles Basketball](https://atlanta-ga.events/wp-content/uploads/2024/09/boston-college-eagles-basketball.jpg)\r\n[The Sufistics Events: 6 Gas South Theater Jan 4, 2025 12:00 PM](https://atlanta-ga.events/tickets/the-sufistics/)![The Sufistics](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Georgia Tech Yellow Jackets Basketball Events: 9 McCamish Pavilion Jan 4, 2025 12:00 PM](https://atlanta-ga.events/tickets/georgia-tech-yellow-jackets-basketball/)![Georgia Tech Yellow Jackets Basketball](https://atlanta-ga.events/wp-content/uploads/2023/08/Georgia-Tech-Yellow-Jackets-Basketball.jpg)\r\n[Expressions of Shukran Events: 6 Gas South Theater Jan 4, 2025 12:00 PM](https://atlanta-ga.events/tickets/expressions-of-shukran/)![Expressions of Shukran](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Kennesaw State Owls Basketball Events: 12 KSU Convocation Center Jan 4, 2025 5:00 PM](https://atlanta-ga.events/tickets/kennesaw-state-owls-basketball/)![Kennesaw State Owls Basketball](https://atlanta-ga.events/wp-content/uploads/2024/02/kennesaw-state-owls-basketball.jpg)\r\n[Kennesaw State Owls Events: 24 KSU Convocation Center Jan 4, 2025 5:00 PM](https://atlanta-ga.events/tickets/kennesaw-state-owls/)![Kennesaw State Owls](https://atlanta-ga.events/wp-content/uploads/2024/02/kennesaw-state-owls.jpg)\r\n[Jacksonville State Gamecocks Basketball Events: 3 KSU Convocation Center Jan 4, 2025 5:00 PM](https://atlanta-ga.events/tickets/jacksonville-state-gamecocks-basketball/)![Jacksonville State Gamecocks Basketball](https://atlanta-ga.events/wp-content/uploads/2024/08/jacksonville-state-gamecocks-basketball.jpg)\r\n[Jacksonville State Gamecocks Events: 3 KSU Convocation Center Jan 4, 2025 5:00 PM](https://atlanta-ga.events/tickets/jacksonville-state-gamecocks/)![Jacksonville State Gamecocks](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\r\n[Louisiana-Lafayette Ragin\u0027 Cajuns Basketball Events: 3 Georgia State Convocation Center Jan 4, 2025 7:00 PM](https://atlanta-ga.events/tickets/louisiana-lafayette-ragin-cajuns-basketball/)![Louisiana-Lafayette Ragin\u0027 Cajuns Basketball](https://atlanta-ga.events/wp-content/uploads/2024/08/louisiana-lafayette-ragin-cajuns-basketball.jpg)\r\n### Concerts Venues\r\n\r\n[529 30316, 529 Flat Shoals Ave SE, Atlanta, GA, US Concerts: 1](https://atlanta-ga.events/venue/529/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Aisle 5 30307, 1123 Euclid Ave Ne, Atlanta, GA, US Concerts: 15](https://atlanta-ga.events/venue/aisle-5/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Altar at Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Concerts: 28](https://atlanta-ga.events/venue/altar-at-masquerade-ga/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Ameris Bank Amphitheatre 30004, 2200 Encore Parkway, Alpharetta, GA, US Concerts: 6](https://atlanta-ga.events/venue/ameris-bank-amphitheatre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Atlanta Coliseum 30096, 2075 Market St, Duluth, GA, US Concerts: 3](https://atlanta-ga.events/venue/atlanta-coliseum/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Atlanta Symphony Hall 30309, 1280 Peachtree St, Atlanta, GA, US Concerts: 72](https://atlanta-ga.events/venue/atlanta-symphony-hall/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[ATL Comedy Theater 30093, 4650 Jimmy Carter Boulevard, Norcross, GA, US Concerts: 33](https://atlanta-ga.events/venue/atl-comedy-theater/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Believe Music Hall 30312, 181 Ralph David Abernathy Blvd, Atlanta, GA, US Concerts: 18](https://atlanta-ga.events/venue/believe-music-hall/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Boggs Social \u0026 Supply 30310, 1310 White St. SW, Atlanta, GA, US Concerts: 3](https://atlanta-ga.events/venue/boggs-social-supply/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Buckhead Theatre 30305, 3110 Roswell Road, Atlanta, GA, US Concerts: 46](https://atlanta-ga.events/venue/buckhead-theatre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Byers Theatre at Sandy Springs Performing Arts Center 30328, 1 Galambos Way, Atlanta, GA, US Concerts: 37](https://atlanta-ga.events/venue/byers-theatre-at-sandy-springs-performing-arts-center/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Cadence Bank Amphitheatre at Chastain Park 30327, 4469 Stella Drive Northwest, Atlanta, GA, US Concerts: 7](https://atlanta-ga.events/venue/cadence-bank-amphitheatre-at-chastain-park/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Center Parc Credit Union Stadium 30315, 755 Hank Aaron Drive, Atlanta, GA, US Concerts: 5](https://atlanta-ga.events/venue/center-parc-credit-union-stadium/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Center Stage Theatre 30309, 1374 West Peachtree Street Northwest, Atlanta, GA, US Concerts: 28](https://atlanta-ga.events/venue/center-stage-theatre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[City Winery - Atlanta 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Concerts: 112](https://atlanta-ga.events/venue/city-winery-atlanta/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Cobb Energy Performing Arts Centre 30339, 2800 Cobb Galleria Pkwy, Atlanta, GA, US Concerts: 50](https://atlanta-ga.events/venue/cobb-energy-performing-arts-centre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Coca-Cola Roxy 30339, 800 Battery Ave SE, Atlanta, GA, US Concerts: 32](https://atlanta-ga.events/venue/coca-cola-roxy/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[District Atlanta 30324, 269 Armour Dr NE, Atlanta, GA, US Concerts: 7](https://atlanta-ga.events/venue/district-atlanta/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Earl Smith Strand Theatre 30060, 117 North Park Square, Marietta, GA, US Concerts: 7](https://atlanta-ga.events/venue/earl-smith-strand-theatre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Eddie\u0027s Attic 30030, 515-b North Mcdonough St., Decatur, GA, US Concerts: 6](https://atlanta-ga.events/venue/eddies-attic-2/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n### Theaters\r\n\r\n[Altar at Masquerade - GA 30303, 75 MLK Jr Dr SW, Atlanta, GA, US Shows \u0026 Events: 28](https://atlanta-ga.events/venue/altar-at-masquerade-ga/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Americasmart 30303, 240 Peachtree Street N.w., Atlanta, GA, US Shows \u0026 Events: 2](https://atlanta-ga.events/venue/americasmart/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Atlanta Coliseum 30096, 2075 Market St, Duluth, GA, US Shows \u0026 Events: 3](https://atlanta-ga.events/venue/atlanta-coliseum/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Atlanta Symphony Hall 30309, 1280 Peachtree St, Atlanta, GA, US Shows \u0026 Events: 72](https://atlanta-ga.events/venue/atlanta-symphony-hall/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Atlantic Station 30363, 1380 Atlantic Dr , Atlanta, GA, US Shows \u0026 Events: 1](https://atlanta-ga.events/venue/atlantic-station/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[ATL Comedy Theater 30093, 4650 Jimmy Carter Boulevard, Norcross, GA, US Shows \u0026 Events: 33](https://atlanta-ga.events/venue/atl-comedy-theater/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Buckhead Theatre 30305, 3110 Roswell Road, Atlanta, GA, US Shows \u0026 Events: 46](https://atlanta-ga.events/venue/buckhead-theatre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Byers Theatre at Sandy Springs Performing Arts Center 30328, 1 Galambos Way, Atlanta, GA, US Shows \u0026 Events: 37](https://atlanta-ga.events/venue/byers-theatre-at-sandy-springs-performing-arts-center/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Center Stage Theatre 30309, 1374 West Peachtree Street Northwest, Atlanta, GA, US Shows \u0026 Events: 28](https://atlanta-ga.events/venue/center-stage-theatre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[City Winery - Atlanta 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Shows \u0026 Events: 112](https://atlanta-ga.events/venue/city-winery-atlanta/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Cobb Energy Performing Arts Centre 30339, 2800 Cobb Galleria Pkwy, Atlanta, GA, US Shows \u0026 Events: 50](https://atlanta-ga.events/venue/cobb-energy-performing-arts-centre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Coca-Cola Roxy 30339, 800 Battery Ave SE, Atlanta, GA, US Shows \u0026 Events: 32](https://atlanta-ga.events/venue/coca-cola-roxy/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Coca-Cola Stage at Alliance Theatre 30309, 1280 Peachtree Street NE, Atlanta, GA, US Shows \u0026 Events: 58](https://atlanta-ga.events/venue/coca-cola-stage-at-alliance-theatre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Earl Smith Strand Theatre 30060, 117 North Park Square, Marietta, GA, US Shows \u0026 Events: 7](https://atlanta-ga.events/venue/earl-smith-strand-theatre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Exhibition Hub Art Center Atlanta 30340, 5660 Buford Hwy NE, Atlanta, GA, US Shows \u0026 Events: 22](https://atlanta-ga.events/venue/exhibition-hub-art-center-atlanta/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Fabulous Fox Theatre - Atlanta 30305, 660 Peachtree St NE, Atlanta, GA, US Shows \u0026 Events: 94](https://atlanta-ga.events/venue/fabulous-fox-theatre-atlanta/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Forsyth County Arts Center 30040, 1150 Dahlonega Hwy, Cumming, GA, US Shows \u0026 Events: 3](https://atlanta-ga.events/venue/forsyth-county-arts-center/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Gas South Arena 30097, 6400 Sugarloaf Parkway, Duluth, GA, US Shows \u0026 Events: 60](https://atlanta-ga.events/venue/gas-south-arena/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Gas South Theater 30097, 6400 Sugarloaf Parkway, Duluth, GA, US Shows \u0026 Events: 27](https://atlanta-ga.events/venue/gas-south-theater/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Gateway Center Arena At College Park 30337, 2330 Convention Center Concourse, Atlanta, GA, US Shows \u0026 Events: 51](https://atlanta-ga.events/venue/gateway-center-arena-at-college-park/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n### Sports Venues\r\n\r\n[Atlanta Motor Speedway 30228, 1500 Tara Place, Hampton, GA, US Sports Events: 6](https://atlanta-ga.events/venue/atlanta-motor-speedway/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Bobby Dodd Stadium 30332, 177 North Avenue NW, Atlanta, GA, US Sports Events: 2](https://atlanta-ga.events/venue/bobby-dodd-stadium/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Center Parc Credit Union Stadium 30315, 755 Hank Aaron Drive, Atlanta, GA, US Sports Events: 5](https://atlanta-ga.events/venue/center-parc-credit-union-stadium/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Center Stage Theatre 30309, 1374 West Peachtree Street Northwest, Atlanta, GA, US Sports Events: 28](https://atlanta-ga.events/venue/center-stage-theatre/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Coolray Field 30044, 2500 Buford Drive, Lawrenceville, GA, US Sports Events: 75](https://atlanta-ga.events/venue/coolray-field/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[East Lake Golf Club 30317, 2575 Alston Drive, Se, Atlanta, GA, US Sports Events: 5](https://atlanta-ga.events/venue/east-lake-golf-club/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Fifth Third Stadium 30144, 3200 Busbee Pkwy, Kennesaw, GA, US Sports Events: 1](https://atlanta-ga.events/venue/fifth-third-stadium/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Gas South Arena 30097, 6400 Sugarloaf Parkway, Duluth, GA, US Sports Events: 60](https://atlanta-ga.events/venue/gas-south-arena/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Gateway Center Arena At College Park 30337, 2330 Convention Center Concourse, Atlanta, GA, US Sports Events: 51](https://atlanta-ga.events/venue/gateway-center-arena-at-college-park/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Georgia International Horse Park 30013, 1996 Centennial Olympic Pkwy, Conyers, GA, US Sports Events: 2](https://atlanta-ga.events/venue/georgia-international-horse-park/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Georgia State Convocation Center 30312, 455 Capitol Ave SE, Atlanta, GA, US Sports Events: 10](https://atlanta-ga.events/venue/georgia-state-convocation-center/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[KSU Convocation Center 30144, 590 Cobb Ave. , Kennesaw, GA, US Sports Events: 18](https://atlanta-ga.events/venue/ksu-convocation-center/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[McCamish Pavilion 30332, 965 Fowler St., Atlanta, GA, US Sports Events: 17](https://atlanta-ga.events/venue/mccamish-pavilion/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Mercedes-Benz Stadium 30313, Martin Luther King Jr Dr SW \u0026 Northside Dr NW, Atlanta, GA, US Sports Events: 33](https://atlanta-ga.events/venue/mercedes-benz-stadium/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Russ Chandler Stadium 30318, 255 Ferst Drive Northwest, Atlanta, GA, US Sports Events: 35](https://atlanta-ga.events/venue/russ-chandler-stadium/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Shirley Clements Mewborn Field 30332, 935 Fowler Street, Atlanta, GA, US Sports Events: 4](https://atlanta-ga.events/venue/shirley-clements-mewborn-field/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[State Farm Arena - GA 30303, One Philips Drive, Atlanta, GA, US Sports Events: 64](https://atlanta-ga.events/venue/state-farm-arena-ga/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[TPC Sugarloaf 30097, 2595 Sugarloaf Club Drive, Duluth, GA, US Sports Events: 3](https://atlanta-ga.events/venue/tpc-sugarloaf/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[Truist Park 30339, 1100 Circle 75 Pkwy, Atlanta, GA, US Sports Events: 87](https://atlanta-ga.events/venue/truist-park/)![location](https://atlanta-ga.events/wp-content/themes/iwebi-performer-combine/img/location-marker.png)\r\n[](https://atlanta-ga.events)[Contact Us](https://atlanta-ga.events/contact-us/)Stay up to date! Join our newsletter to stay current on all Live Entertainment in Atlanta, plus the occasional special offer, VIP packages, and discount tickets! Unique events and incredible deals are straight to your inbox. Either way, you\u2019ll be the first to know about the best events in Atlanta, GA. \u201CAtlanta events\u201D is part of the \u201CLive Entertainment Guide\u201D collection. We offer an extensive collection of websites providing complete, impartial guides to all the theatrical, musical, sports, and performance arts events in America\u2019s greatest cities.\r\n\r\n[Atlanta events](https://atlanta-ga.events/)[Live Entertainment Guide](https://liveentertainment.guide/)[Sitemap](https://atlanta-ga.events/sitemaps/)[Privacy Policy](https://atlanta-ga.events/privacy-policy/)"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html"}
```
and got this result:
```
"[Go South! Atlanta](https://gosouthatlanta.com/)*Go South!*[See Atlanta](https://gosouthatlanta.com/see-atlanta.html)[See Georgia](https://gosouthatlanta.com/see-georgia.html)[Things To Do](https://gosouthatlanta.com/things-to-do.html)[Visitor Info](https://gosouthatlanta.com/visitor-info.html)[Site Index](https://gosouthatlanta.com/sitemap.html)This site uses cookies. Opt out at any time. More details\r\n\r\n[More details](https://gosouthatlanta.com/cookies.html)Home \u0026rsaquo; Things To Do \u0026rsaquo; January Events\r\n\r\n[Home](https://gosouthatlanta.com/)[Things To Do](https://gosouthatlanta.com/things-to-do.html)[January Events](https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html)# Atlanta Events \u0026ndash; January 2025\r\n\r\nAtlanta is a year-round destination, with almost as many things to do in January as in the warmer months of spring through fall.\r\n\r\nThe weather this month is unpredictable. Atlanta\u0026rsquo;s winters are generally mild, with average highs in the 50s. Colder weather is possible, but January can also bring beautiful, sunny days with temperatures rising into the 70s.\r\n\r\n[weather this month](https://gosouthatlanta.com/visitor-info/atlanta-weather-january.html)On cold days, take advantage of Atlanta\u0026rsquo;s huge range of indoor attractions: museums, art, concerts, the aquarium, shopping, food halls, and so on. On warmer, sunny days, the weather can be just right for wandering through the city\u0026rsquo;s varied neighborhoods; for hiking, whether outside of the city or along the BeltLine; and for other outdoor activities.\r\n\r\n[museums,](https://gosouthatlanta.com/things-to-do/museums-and-historic-houses.html)[art,](https://gosouthatlanta.com/things-to-do/arts-and-culture.html)[concerts,](https://gosouthatlanta.com/things-to-do/events/concerts-in-atlanta.html)Atlanta\u0026rsquo;s January event highlights include the various Martin Luther King Day celebrations and parade. Other things to do in January include wine and food festivals, and the Cathedral Antiques Show and Tour of Homes.\r\n\r\nInterested in a specific type of event? See Atlanta\u0026rsquo;s calendars of upcoming: \u0026ndash; Comedy shows \u0026ndash; Beer and wine festivals \u0026ndash; Music festivals \u0026ndash; Musicals \u0026ndash; Food festivals\r\n\r\n[Comedy shows](https://gosouthatlanta.com/things-to-do/events/atlanta-comedy-shows.html)[Beer and wine festivals](https://gosouthatlanta.com/things-to-do/events/atlanta-beer-and-wine-festivals.html)[Music festivals](https://gosouthatlanta.com/things-to-do/events/atlanta-music-festivals.html)[Musicals](https://gosouthatlanta.com/things-to-do/events/musicals-in-atlanta.html)[Food festivals](https://gosouthatlanta.com/things-to-do/events/atlanta-food-festivals.html)See also: \u0026ndash; Atlanta\u0026rsquo;s festival calendar \u0026ndash; Things to do in Atlanta \u0026ndash; Best time of year to visit Atlanta\r\n\r\n**See also:**[Atlanta\u0026rsquo;s festival calendar](https://gosouthatlanta.com/things-to-do/events/atlanta-festivals.html)[Things to do in Atlanta](https://gosouthatlanta.com/things-to-do.html)[Best time of year to visit Atlanta](https://gosouthatlanta.com/visitor-info/when-to-visit-atlanta.html)!Event dates, venue and other details can change at short notice. Please check all information at the official website before making plans to attend an event.\r\n\r\n**!Event dates, venue and other details can change at short notice. Please check all information at the official website before making plans to attend an event.***Event dates, venue and other details can change at short notice. Please check all information at the official website before making plans to attend an event.*## Events From January 1 To The Weekend Of January 4-5 2025\r\n\r\n\u0026rarr; See December events\r\n\r\n[December events](https://gosouthatlanta.com/things-to-do/events/atlanta-in-december.html)\u0026rarr; For concerts this week, see concerts in Atlanta in January\r\n\r\n*For concerts this week, see concerts in Atlanta in January*[concerts in Atlanta in January](https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-january.html)\u0026ndash; IllumiNights, November 14 2024 - January 19 2025Annual after-hours walk-through holiday lights show at Zoo Atlanta, with a mile of illuminated animal lanterns. More details\r\n\r\n**\u0026ndash; IllumiNights, November 14 2024 - January 19 2025**[More details](https://zooatlanta.org/event/illuminights/)\u0026ndash; Skate the Station, November 15 2024 - January 20 2025Annual outdoor holiday skating at Atlantic Station, with some themed nights. More details\r\n\r\n**\u0026ndash; Skate the Station, November 15 2024 - January 20 2025**[More details](https://atlanticstation.com/event/skate-the-station-2/)\u0026ndash; Garden Lights, Holiday Nights, November 16 2024 - January 12 2025Popular annual holiday lights display at the Atlanta Botanical Garden. More details\r\n\r\n**\u0026ndash; Garden Lights, Holiday Nights, November 16 2024 - January 12 2025**[More details](https://atlantabg.org/calendar/garden-lights-holiday-nights-presented-by-invesco-qqq-2/)\u0026ndash; Resolution Run, January 1 2025New Year\u0026rsquo;s Day 5K run, and family races in Piedmont Park. More details\r\n\r\n**\u0026ndash; Resolution Run, January 1 2025**[More details](https://www.atlantatrackclub.org/2025-resolution-run)\u0026ndash; A Whole Weekend of Out of a Hat Performances: Much Ado About Nothing, A Comedy of Errors, Twelfth Night, January 2-4 2025Part-improvised plays at the Shakespeare Tavern Playhouse. More details\r\n\r\n**\u0026ndash; A Whole Weekend of Out of a Hat Performances: Much Ado About Nothing, A Comedy of Errors, Twelfth Night, January 2-4 2025**[More details](https://www.shakespearetavern.com/on-stage/company/a-whole-weekend-of-out-of-a-hat-performances/)\u0026ndash; Freakin\u0026rsquo; Freezin\u0026rsquo; 5K, January 4 2025Winter 5K at The Monroe/Atlanta BeltLine. More details\r\n\r\n**\u0026ndash; Freakin\u0026rsquo; Freezin\u0026rsquo; 5K, January 4 2025**[More details](https://runsignup.com/Race/GA/Atlanta/FreakinFreezin5K)\u0026ndash; Georgia Bridal Show, January 5 2025Wedding trade show at the Cobb Galleria Centre. More details\r\n\r\n**\u0026ndash; Georgia Bridal Show, January 5 2025**[More details](https://www.georgiabridalshow.com)## Events From January 6 To The Weekend Of January 11-12 2025\r\n\r\n\u0026rarr; For concerts this week, see concerts in Atlanta in January\r\n\r\n*For concerts this week, see concerts in Atlanta in January*[concerts in Atlanta in January](https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-january.html)\u0026ndash; IllumiNights, November 14 2024 - January 19 2025Annual after-hours walk-through holiday lights show at Zoo Atlanta, with a mile of illuminated animal lanterns. More details\r\n\r\n**\u0026ndash; IllumiNights, November 14 2024 - January 19 2025**[More details](https://zooatlanta.org/event/illuminights/)\u0026ndash; Skate the Station, November 15 2024 - January 20 2025Annual outdoor holiday skating at Atlantic Station, with some themed nights. More details\r\n\r\n**\u0026ndash; Skate the Station, November 15 2024 - January 20 2025**[More details](https://atlanticstation.com/event/skate-the-station-2/)\u0026ndash; Garden Lights, Holiday Nights, November 16 2024 - January 12 2025Popular annual holiday lights display at the Atlanta Botanical Garden. More details\r\n\r\n**\u0026ndash; Garden Lights, Holiday Nights, November 16 2024 - January 12 2025**[More details](https://atlantabg.org/calendar/garden-lights-holiday-nights-presented-by-invesco-qqq-2/)\u0026ndash; \u0026amp; Juliet, January 7-12 2025Musical at the Fox Theatre. More details\r\n\r\n**\u0026ndash; \u0026amp; Juliet, January 7-12 2025**[More details](https://www.foxtheatre.org/events/detail/and_juliet)\u0026ndash; Atlanta Boat Show, January 9-12 2025Annual boating trade show at the Georgia World Congress Center. More details\r\n\r\n**\u0026ndash; Atlanta Boat Show, January 9-12 2025**[More details](https://www.atlantaboatshow.com)\u0026ndash; Scott Antique Market, January 9-12 2025Monthly antiques market at the Atlanta Expo Center. More details\r\n\r\n**\u0026ndash; Scott Antique Market, January 9-12 2025**[More details](https://www.scottantiquemarket.com/schedule/)\u0026ndash; Joe Gatto: Let\u0026rsquo;s Get Into It, January 11 2025Comedy show at Atlanta Symphony Hall. More details\r\n\r\n**\u0026ndash; Joe Gatto: Let\u0026rsquo;s Get Into It, January 11 2025**[More details](https://www.aso.org/events/detail/joe-gatto)\u0026ndash; Last Podcast on the Left: JK Ultra Tour, January 11 2025Comedy show at the Coca-Cola Roxy. More details\r\n\r\n**\u0026ndash; Last Podcast on the Left: JK Ultra Tour, January 11 2025**[More details](https://concerts.livenation.com/last-podcast-on-the-left-jk-atlanta-georgia-01-11-2025/event/0E0061581C407C26)\u0026ndash; Atlanta Travel \u0026amp; Adventure Show, January 11-12 2025Annual travel and tourism trade show at the Georgia International Convention Center. More details\r\n\r\n**\u0026ndash; Atlanta Travel \u0026amp; Adventure Show, January 11-12 2025**[More details](https://travelshows.com/shows/atlanta/)\u0026ndash; Love\u0026rsquo;s Labour\u0026rsquo;s Lost, January 11-26 2025Play at the Shakespeare Tavern Playhouse. More details\r\n\r\n**\u0026ndash; Love\u0026rsquo;s Labour\u0026rsquo;s Lost, January 11-26 2025**[More details](https://www.shakespearetavern.com/on-stage/company/loves-labours-lost/)\u0026ndash; Winter RunTheATL, January 12 2025Annual ~15-mile race along the Atlanta BeltLine. More details\r\n\r\n**\u0026ndash; Winter RunTheATL, January 12 2025**[More details](https://www.runtheatl.com/winter-runtheatl/)\u0026ndash; Sarah Silverman: Postmortem, January 12 2025Comedy show at the Cobb Energy Performing Arts Centre. More details\r\n\r\n**\u0026ndash; Sarah Silverman: Postmortem, January 12 2025**[More details](https://www.cobbenergycentre.com/events/detail/sarah-silverman)## Events From January 13 To Martin Luther King Day Weekend, January 18-20 2025\r\n\r\n\u0026rarr; For concerts this week, see concerts in Atlanta in January\r\n\r\n*For concerts this week, see concerts in Atlanta in January*[concerts in Atlanta in January](https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-january.html)\u0026ndash; IllumiNights, November 14 2024 - January 19 2025Annual after-hours walk-through holiday lights show at Zoo Atlanta, with a mile of illuminated animal lanterns. More details\r\n\r\n**\u0026ndash; IllumiNights, November 14 2024 - January 19 2025**[More details](https://zooatlanta.org/event/illuminights/)\u0026ndash; Skate the Station, November 15 2024 - January 20 2025Annual outdoor holiday skating at Atlantic Station, with some themed nights. More details\r\n\r\n**\u0026ndash; Skate the Station, November 15 2024 - January 20 2025**[More details](https://atlanticstation.com/event/skate-the-station-2/)\u0026ndash; Love\u0026rsquo;s Labour\u0026rsquo;s Lost, January 11-26 2025Play at the Shakespeare Tavern Playhouse. More details\r\n\r\n**\u0026ndash; Love\u0026rsquo;s Labour\u0026rsquo;s Lost, January 11-26 2025**[More details](https://www.shakespearetavern.com/on-stage/company/loves-labours-lost/)\u0026ndash; Gimme Please!, January 16 - February 16 2025Children\u0026rsquo;s play at Alliance Theatre. More details\r\n\r\n**\u0026ndash; Gimme Please!, January 16 - February 16 2025**[More details](https://www.alliancetheatre.org/production/2024-25/gimme-please)\u0026ndash; Andrew Santino: Freeze Peach Tour, January 17 2025Comedy show at the Buckhead Theatre. More details\r\n\r\n**\u0026ndash; Andrew Santino: Freeze Peach Tour, January 17 2025**[More details](https://concerts.livenation.com/andrew-santino-freeze-peach-tour-atlanta-georgia-01-17-2025/event/0E0060D3C50860B8)\u0026ndash; Martin Luther King Jr Day: She Shall Not Be Moved, January 18 2025Free special event at the Atlanta History Center to mark MLK Day, featuring the stories of women who shaped the Civil Rights Movement, with children\u0026rsquo;s activities, exhibits, panel discussions and more. More details\r\n\r\n**\u0026ndash; Martin Luther King Jr Day: She Shall Not Be Moved, January 18 2025**[More details](https://www.atlantahistorycenter.com/event/mlk-day/)\u0026ndash; Doctor De Soto, January 18 - March 2 2025Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n**\u0026ndash; Doctor De Soto, January 18 - March 2 2025**[More details](https://www.alliancetheatre.org/production/2024-25/doctor-de-soto)\u0026ndash; MLK Day 5K, January 20 2025Annual 5K run/walk from First Baptist Church of Doraville in celebration of Martin Luther King Day. More details\r\n\r\n**\u0026ndash; MLK Day 5K, January 20 2025**[More details](https://mlkday5k.com)## Events From January 21 To The Weekend Of January 25-26 2025\r\n\r\n\u0026rarr; For concerts this week, see concerts in Atlanta in January\r\n\r\n*For concerts this week, see concerts in Atlanta in January*[concerts in Atlanta in January](https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-january.html)\u0026ndash; Love\u0026rsquo;s Labour\u0026rsquo;s Lost, January 11-26 2025Play at the Shakespeare Tavern Playhouse. More details\r\n\r\n**\u0026ndash; Love\u0026rsquo;s Labour\u0026rsquo;s Lost, January 11-26 2025**[More details](https://www.shakespearetavern.com/on-stage/company/loves-labours-lost/)\u0026ndash; Gimme Please!, January 16 - February 16 2025Children\u0026rsquo;s play at Alliance Theatre. More details\r\n\r\n**\u0026ndash; Gimme Please!, January 16 - February 16 2025**[More details](https://www.alliancetheatre.org/production/2024-25/gimme-please)\u0026ndash; Doctor De Soto, January 18 - March 2 2025Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n**\u0026ndash; Doctor De Soto, January 18 - March 2 2025**[More details](https://www.alliancetheatre.org/production/2024-25/doctor-de-soto)\u0026ndash; Mon\u0026eacute;t X Change, January 23 2025Drag performance at Center Stage. More details\r\n\r\n**\u0026ndash; Mon\u0026eacute;t X Change, January 23 2025**[More details](https://www.centerstage-atlanta.com/events/monet-x-change/)\u0026ndash; Atlanta Camping \u0026amp; RV Show, January 23-26 2025Outdoor lifestyle vehicle trade show at the Atlanta Exposition Center South. More details\r\n\r\n**\u0026ndash; Atlanta Camping \u0026amp; RV Show, January 23-26 2025**[More details](https://atlantarvshow.com)\u0026ndash; Steve Martin \u0026amp; Martin Short: The Dukes of Funnytown!, January 24 2025Comedy show at the Fox Theatre. More details\r\n\r\n**\u0026ndash; Steve Martin \u0026amp; Martin Short: The Dukes of Funnytown!, January 24 2025**[More details](https://www.foxtheatre.org/events/detail/steve-martin-martin-short-2025)\u0026ndash; Cathedral Giving by Design, January 24-25 2025Annual charitable event of The Cathedral of St Philip, with tours of private Atlanta homes, live auction, speakers and other events to benefit Wellspring Living. More details\r\n\r\n**\u0026ndash; Cathedral Giving by Design, January 24-25 2025**[More details](https://www.cathedralgivingbydesign.org)\u0026ndash; Super Freak: The Rick James Story, January 24-25 2025Musical at the Cobb Energy Performing Arts Centre. More details\r\n\r\n**\u0026ndash; Super Freak: The Rick James Story, January 24-25 2025**[More details](https://www.cobbenergycentre.com/events/detail/super-freak-the-rick-james-story-1)\u0026ndash; North Atlanta Home Show, January 24-26 2025Home improvements trade show at the Gas South Convention Center. More details\r\n\r\n**\u0026ndash; North Atlanta Home Show, January 24-26 2025**[More details](https://www.northatlantahomeshow.com)\u0026ndash; Jersey Boys, January 24 - February 23 2025Musical at The Legacy Theater, Tyrone. More details\r\n\r\n**\u0026ndash; Jersey Boys, January 24 - February 23 2025**[More details](https://legacytheater.com/show/doo-wop/)\u0026ndash; Run With the Dogs, January 25 2025Annual 5K and pet races in Decatur to benefit the Decatur Bulldogs Boosters. More details\r\n\r\n**\u0026ndash; Run With the Dogs, January 25 2025**[More details](http://www.rwtd5k.com)\u0026ndash; Spring Gardening Symposium \u0026amp; Plant Sale, January 25 2025Special event at the Atlanta Botanical Garden, with speakers and plant sale. More details\r\n\r\n**\u0026ndash; Spring Gardening Symposium \u0026amp; Plant Sale, January 25 2025**[More details](https://atlantabg.org/calendar/spring-symposium-2025/)\u0026ndash; Gals on the Go, January 25 2025Podcast live show at Center Stage. More details\r\n\r\n**\u0026ndash; Gals on the Go, January 25 2025**[More details](https://www.centerstage-atlanta.com/events/gals-on-the-go/)\u0026ndash; Maria Bamford, January 25 2025Comedy show at the Buckhead Theatre. More details\r\n\r\n**\u0026ndash; Maria Bamford, January 25 2025**[More details](https://concerts.livenation.com/maria-bamford-atlanta-georgia-01-25-2025/event/0E00611B94352720)\u0026ndash; Magician Rich Crawford, January 25 2025Illusion show at Red Clay Music Foundry, Duluth. More details\r\n\r\n**\u0026ndash; Magician Rich Crawford, January 25 2025**[More details](https://eddieowenpresents.freshtix.com/events/richcrawford)\u0026ndash; Lunar New Year Festival, January 25 - February 9 2025Asian-inspired festival at Stone Mountain Park, with drone and light show, lighted parade, cultural craft activities, live performances and other cultural activities. More details\r\n\r\n**\u0026ndash; Lunar New Year Festival, January 25 - February 9 2025**[More details](https://stonemountainpark.com/activity/events/lunar-new-year/)## Events From January 27 To The Weekend Of February 1-2 2025\r\n\r\n\u0026rarr; For concerts this week, see concerts in Atlanta in January\r\n\r\n*For concerts this week, see concerts in Atlanta in January*[concerts in Atlanta in January](https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-january.html)\u0026ndash; Gimme Please!, January 16 - February 16 2025Children\u0026rsquo;s play at Alliance Theatre. More details\r\n\r\n**\u0026ndash; Gimme Please!, January 16 - February 16 2025**[More details](https://www.alliancetheatre.org/production/2024-25/gimme-please)\u0026ndash; Doctor De Soto, January 18 - March 2 2025Children\u0026rsquo;s performance at Alliance Theatre. More details\r\n\r\n**\u0026ndash; Doctor De Soto, January 18 - March 2 2025**[More details](https://www.alliancetheatre.org/production/2024-25/doctor-de-soto)\u0026ndash; Jersey Boys, January 24 - February 23 2025Musical at The Legacy Theater, Tyrone. More details\r\n\r\n**\u0026ndash; Jersey Boys, January 24 - February 23 2025**[More details](https://legacytheater.com/show/doo-wop/)\u0026ndash; Lunar New Year Festival, January 25 - February 9 2025Asian-inspired festival at Stone Mountain Park, with drone and light show, lighted parade, cultural craft activities, live performances and other cultural activities. More details\r\n\r\n**\u0026ndash; Lunar New Year Festival, January 25 - February 9 2025**[More details](https://stonemountainpark.com/activity/events/lunar-new-year/)\u0026ndash; WWE Monday Night RAW, January 27 2025Wrestling event at State Farm Arena. More details\r\n\r\n**\u0026ndash; WWE Monday Night RAW, January 27 2025**[More details](https://www.statefarmarena.com/events/detail/wwe-monday-night-raw-2025)\u0026ndash; Clue, January 28 2025Performance at the Fox Theatre. More details\r\n\r\n**\u0026ndash; Clue, January 28 2025**[More details](https://www.foxtheatre.org/events/detail/clue_musical)\u0026ndash; Ghost: The Musical, January 30 - February 8 2025Musical at Marietta\u0026rsquo;s New Theatre in the Square. More details\r\n\r\n**\u0026ndash; Ghost: The Musical, January 30 - February 8 2025**[More details](https://www.tix.com/ticket-sales/mariettatheatre/5266)\u0026ndash; Shen Yun, January 31 - February 2 2025Dance performance at the Cobb Energy Performing Arts Centre. More details\r\n\r\n**\u0026ndash; Shen Yun, January 31 - February 2 2025**[More details](https://www.cobbenergycentre.com/events/detail/shen-yun-1)\u0026ndash; Andrew Stanley, February 1 2025Comedy show at the Buckhead Theatre. More details\r\n\r\n**\u0026ndash; Andrew Stanley, February 1 2025**[More details](https://concerts.livenation.com/andrew-stanley-800p-comedy-special-taping-atlanta-georgia-02-01-2025/event/0E00611BF7ED694A)\u0026ndash; The Sweet Spot Burlesque: Masquerade Edition, February 1 2025Burlesque performance at Center Stage. More details\r\n\r\n**\u0026ndash; The Sweet Spot Burlesque: Masquerade Edition, February 1 2025**[More details](https://www.centerstage-atlanta.com/events/the-sweet-spot-burlesque-masquerade-edition/)\u0026ndash; Hot Wheels Monster Trucks Live, February 1-2 2025Monster truck show at State Farm Arena. More details\r\n\r\n**\u0026ndash; Hot Wheels Monster Trucks Live, February 1-2 2025**[More details](https://www.statefarmarena.com/events/detail/hot-wheels-monster-trucks-live-glow-n-fire)\u0026ndash; Romeo and Juliet, February 1 - March 2 2025Play at the Shakespeare Tavern Playhouse. More details\r\n\r\n**\u0026ndash; Romeo and Juliet, February 1 - March 2 2025**[More details](https://www.shakespearetavern.com/on-stage/company/romeo-and-juliet/)\u0026ndash; Hot Chocolate Run, February 2 2025Chocolate-themed 5K and 15K runs at Centennial Olympic Park plus pre-event expo and after-party. More details\r\n\r\n**\u0026ndash; Hot Chocolate Run, February 2 2025**[More details](https://hotchocolate15k.com/city/atlanta/)\u0026rarr; See more February events\r\n\r\n[February events](https://gosouthatlanta.com/things-to-do/events/atlanta-in-february.html)\u0026copy; Melanie K Jones 2024\r\n\r\n* Privacy\r\n* Terms\r\n* Contact\r\n\r\n[Privacy](https://gosouthatlanta.com/privacy.html)[Terms](https://gosouthatlanta.com/terms.html)[Contact](https://gosouthatlanta.com/contact.html)"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://allevents.in/atlanta/january"}
```
and got this result:
```
"* Atlanta Change City\r\n* Login\r\n* \r\n* Create an event\r\n* Manage events\r\n* \r\n* Get Event discovery AppNew\r\n* Need help?\r\n\r\n[Atlanta](https://allevents.in/atlanta)[Login](#)Host Control\r\n\r\n[Create an event](https://allevents.in/manage/create.php?ref=topbar-dropdown)[Manage events](https://allevents.in/pages/event-listing?ref=topbar-dropdown)[Get Event discovery AppNew](https://play.google.com/store/apps/details?id=com.amitech.allevents\u0026utm_source=Mobile_topbar\u0026utm_medium=topbar\u0026utm_campaign=topbar)[Need help?](https://allevents.in/pages/support?ref=topbar-dropdown)[](https://allevents.in/atlanta)![All events in City logo](https://allevents.in/img/ae-logo-website-v2.svg)\r\n![Allevents logo with Santa Hat](https://cdn2.allevents.in/transup/a7/a438d4cf8e4ce8a46141fe7c4ffaa8/ae-christmas-logo.webp)\r\n**[Create Event](https://allevents.in/manage/create.php?ref=category-buckets-topbar)**[](https://allevents.in/recommended?ref=category-buckets-topbar)**[Birmingham](https://allevents.in/birmingham-al)[Smyrna](https://allevents.in/smyrna)[Knoxville](https://allevents.in/knoxville)[Greenville](https://allevents.in/greenville-sc)[Chattanooga](https://allevents.in/chattanooga)[Huntsville](https://allevents.in/huntsville)[Decatur](https://allevents.in/decatur)[Marietta](https://allevents.in/marietta)[Montgomery](https://allevents.in/montgomery)[North Druid Hills](https://allevents.in/north%20druid%20hills)* \r\n* \r\n* \r\n* \r\n* \r\n* \r\n* \r\n* \r\n* \r\n* \u002B more\r\n\r\n![United States]()\r\n![Canada]()\r\n![France]()\r\n![Germany]()\r\n![India]()\r\n![Brazil]()\r\n![Australia]()\r\n![Spain]()\r\n![United Kingdom]()\r\n* All\r\n* Music Concerts Parties \u0026 Nightlife Performances Comedy Dance Entertainment\r\n* Fine Arts Theatre Literary Arts Crafts Photography Cooking Art \u0026 Theatre\r\n* Food \u0026 Drinks\r\n* Adventures\r\n* Business\r\n* Festivals\r\n* \u0026#129346; New Year\r\n* Meetups This Weekend Sports Exhibitions Fashion Wellness Yoga Kids Workshops Webinar More\r\n\r\n[All](https://allevents.in/atlanta/all)* Music\r\n* Concerts\r\n* Parties \u0026 Nightlife\r\n* Performances\r\n* Comedy\r\n* Dance\r\n\r\n[Music](https://allevents.in/atlanta/music?ref=category-buckets-bluebar)[Concerts](https://allevents.in/atlanta/concerts?ref=category-buckets-bluebar)[Parties \u0026 Nightlife](https://allevents.in/atlanta/parties?ref=category-buckets-bluebar)[Performances](https://allevents.in/atlanta/performances?ref=category-buckets-bluebar)[Comedy](https://allevents.in/atlanta/comedy?ref=category-buckets-bluebar)[Dance](https://allevents.in/atlanta/dance?ref=category-buckets-bluebar)[Entertainment](https://allevents.in/atlanta/entertainment?ref=category-buckets-bluebar)*** Fine Arts\r\n* Theatre\r\n* Literary Arts\r\n* Crafts\r\n* Photography\r\n* Cooking\r\n\r\n[Fine Arts](https://allevents.in/atlanta/fine-arts?ref=category-buckets-bluebar)[Theatre](https://allevents.in/atlanta/theatre?ref=category-buckets-bluebar)[Literary Arts](https://allevents.in/atlanta/literary-art?ref=category-buckets-bluebar)[Crafts](https://allevents.in/atlanta/crafts?ref=category-buckets-bluebar)[Photography](https://allevents.in/atlanta/photography?ref=category-buckets-bluebar)[Cooking](https://allevents.in/atlanta/cooking?ref=category-buckets-bluebar)[Art \u0026 Theatre](https://allevents.in/atlanta/art?ref=category-buckets-bluebar)**[Food \u0026 Drinks](https://allevents.in/atlanta/food-drinks?ref=category-buckets-bluebar)[Adventures](https://allevents.in/atlanta/trips-adventures?ref=category-buckets-bluebar)[Business](https://allevents.in/atlanta/business?ref=category-buckets-bluebar)[Festivals](https://allevents.in/atlanta/festivals?ref=category-buckets-bluebar)[\u0026#129346; New Year](https://allevents.in/atlanta/new-year?ref=category-buckets-bluebar)* Meetups\r\n* This Weekend\r\n* Sports\r\n* Exhibitions\r\n* Fashion\r\n* Wellness\r\n* Yoga\r\n* Kids\r\n* Workshops\r\n* Webinar\r\n\r\n[Meetups](https://allevents.in/atlanta/meetups?ref=category-buckets-bluebar)[This Weekend](https://allevents.in/atlanta/this-weekend?ref=category-buckets-bluebar)[Sports](https://allevents.in/atlanta/sports?ref=category-buckets-bluebar)[Exhibitions](https://allevents.in/atlanta/exhibitions?ref=category-buckets-bluebar)[Fashion](https://allevents.in/atlanta/fashion?ref=category-buckets-bluebar)[Wellness](https://allevents.in/atlanta/health-wellness?ref=category-buckets-bluebar)[Yoga](https://allevents.in/atlanta/yoga?ref=category-buckets-bluebar)[Kids](https://allevents.in/atlanta/kids?ref=category-buckets-bluebar)[Workshops](https://allevents.in/atlanta/workshops?ref=category-buckets-bluebar)[Webinar](https://allevents.in/atlanta/webinar?ref=category-buckets-bluebar)[More](#)**[](#)[](#)# January 2025 events in Atlanta\r\n\r\n## Activities \u0026 things to do in January 2025 in Atlanta\r\n\r\n![](https://cdn2.allevents.in/transup/d3/eb547db33b41179fc9705d04b60ec7/Group-4-.png)\r\n******[](https://allevents.in/online/ultimate-real-estate-investment-training-and-6-month-mentoring-program/80002773995864?aff=u1gjbz\u0026ref=featured-eventlist)![Featured icon ](https://cdn2.allevents.in/transup/e0/98f47c69394d00951cdb2528a1bca1/fi_2107957featured-star.svg)\r\n![Ultimate Real Estate Investment Training \u0026 6 Month Mentoring Program](https://cdn2.allevents.in/thumbs/thumb676a7607bc0df.jpg)\r\n**### Ultimate Real Estate Investment Training \u0026amp; 6 Month Mentoring Program\r\n\r\n## Trending events in january in Atlanta\r\n\r\n* Featured NYE in the ATL 2025 : New Year\u0027s Eve Atlanta Crowne Plaza Atlanta Perimeter at Ravinia 31 Dec-01 Jan 719 interested USD 100\r\n* \r\n* TAILOR MADE Atlanta New Year\u0027s Eve Black Tie Gala | Everyone FREE Before 11pm w/RSVP Essence Venue 31 Dec-01 Jan 17 interested Free\r\n* New Year\u0027s Eve Atlanta 2025 Crowne Plaza Atlanta Perimeter at Ravinia, an IHG Hotel, Ashford Dunwoody Road Northeast, Atlanta, GA, USA 31 Dec-01 Jan USD 80\r\n* HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District Atlanta District Atlanta 31 Dec-01 Jan USD 55\r\n* TUESDAY CONTENT CLUB Skewed Orbit Studios 21 Jan Onwards USD 31\r\n\r\n**[NYE in the ATL 2025 : New Year\u0027s Eve Atlanta](https://allevents.in/atlanta/nye-in-the-atl-2025-new-years-eve-atlanta/80002059316179?ref=category-buckets)### NYE in the ATL 2025 : New Year\u0027s Eve Atlanta\r\n\r\n******[TAILOR MADE Atlanta New Year\u0027s Eve Black Tie Gala | Everyone FREE Before 11pm w/RSVP](https://allevents.in/atlanta/tailor-made-atlanta-new-years-eve-black-tie-gala-|-everyone-free-before-11pm-w-rsvp/10000944669220067?ref=category-buckets)### TAILOR MADE Atlanta New Year\u0027s Eve Black Tie Gala | Everyone FREE Before 11pm w/RSVP\r\n\r\n******[New Year\u0027s Eve Atlanta 2025](https://allevents.in/atlanta/new-years-eve-atlanta-2025/80001231743751?ref=category-buckets)### New Year\u0027s Eve Atlanta 2025\r\n\r\n****[HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District Atlanta](https://allevents.in/atlanta/hugel-new-years-eve-|-make-the-girls-dance-2024-|-district-atlanta/80002137238102?ref=category-buckets)### HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District Atlanta\r\n\r\n****[TUESDAY CONTENT CLUB](https://allevents.in/atlanta/tuesday-content-club/80008245546310?ref=category-buckets)### TUESDAY CONTENT CLUB\r\n\r\n**## Most popular events in january in Atlanta\r\n\r\n* Hugel at District Atlanta District Atlanta 31 Dec-01 Jan\r\n* \r\n* Last Podcast On The Left at Coca-Cola Roxy Theatre Coca-Cola Roxy Theatre Sat, 11 Jan\r\n* Atlanta Symphony Orchestra: New Year\u0027s Eve Byers Theatre at Sandy Springs Performing Arts Center Tue, 31 Dec 13 interested\r\n* Atlanta Career Fair Hilton Mon, 13 Jan 314 interested\r\n* A Tribute to Quincy Jones City Winery - Atlanta Tue, 31 Dec\r\n\r\n**[Hugel at District Atlanta](https://allevents.in/atlanta/hugel-at-district-atlanta/2400027671732449?ref=category-buckets)### Hugel at District Atlanta\r\n\r\n**[Last Podcast On The Left at Coca-Cola Roxy Theatre](https://allevents.in/atlanta/last-podcast-on-the-left-at-coca-cola-roxy-theatre/2400027672845538?ref=category-buckets)### Last Podcast On The Left at Coca-Cola Roxy Theatre\r\n\r\n**[Atlanta Symphony Orchestra: New Year\u0027s Eve](https://allevents.in/atlanta/atlanta-symphony-orchestra-new-years-eve/2300027487679106?ref=category-buckets)### Atlanta Symphony Orchestra: New Year\u0027s Eve\r\n\r\n****[Atlanta Career Fair](https://allevents.in/atlanta/atlanta-career-fair/10000300752598117?ref=category-buckets)### Atlanta Career Fair\r\n\r\n****[A Tribute to Quincy Jones](https://allevents.in/atlanta/a-tribute-to-quincy-jones/2300027619766845?ref=category-buckets)### A Tribute to Quincy Jones\r\n\r\n## Free events in january in Atlanta\r\n\r\n* NEW YEARS DAY FREE ZUMBA CLASS East Lake Family YMCA, Eva Davis Way Southeast, Atlanta, GA, USA Wed, 01 Jan Free\r\n* \r\n* Free Zumba and Hip Hop Cardio Grant Park Recreation Center 04 Jan Onwards Free\r\n* Taco Tuesday at McCray\u2019s Beltline McCray\u0027s Tavern Beltline 31 Dec Onwards Free\r\n* ATLANTA BEST NIGHT CLUB! CELEBRITY GUEST, NFL, NBA, REVEL NIGHTCLUB!! FRIDAY, SATURDAY, SUNDAY REVEL ATLANTA 03 Jan Onwards Free\r\n* Best Rooftop Party in Atlanta! Suite Lounge with City View Clubs Atl VIP Suite Food Lounge 03 Jan Onwards Free\r\n\r\n**[NEW YEARS DAY FREE ZUMBA CLASS](https://allevents.in/atlanta/new-years-day-free-zumba-class/80002882313305?ref=category-buckets)### NEW YEARS DAY FREE ZUMBA CLASS\r\n\r\n****[Free Zumba and Hip Hop Cardio](https://allevents.in/atlanta/free-zumba-and-hip-hop-cardio/80002645114547?ref=category-buckets)### Free Zumba and Hip Hop Cardio\r\n\r\n****[Taco Tuesday at McCray\u2019s Beltline](https://allevents.in/atlanta/taco-tuesday-at-mccray\u2019s-beltline/80002426164054?ref=category-buckets)### Taco Tuesday at McCray\u2019s Beltline\r\n\r\n****[ATLANTA BEST NIGHT CLUB! CELEBRITY GUEST, NFL, NBA, REVEL NIGHTCLUB!! FRIDAY, SATURDAY, SUNDAY](https://allevents.in/atlanta/atlanta-best-night-club-celebrity-guest-nfl-nba-revel-nightclub-friday-saturday-sunday/80001704849372?ref=category-buckets)### ATLANTA BEST NIGHT CLUB! CELEBRITY GUEST, NFL, NBA, REVEL NIGHTCLUB!! FRIDAY, SATURDAY, SUNDAY\r\n\r\n****[Best Rooftop Party in Atlanta! Suite Lounge with City View Clubs Atl VIP](https://allevents.in/atlanta/best-rooftop-party-in-atlanta-suite-lounge-with-city-view-clubs-atl-vip/80003135700954?ref=category-buckets)### Best Rooftop Party in Atlanta! Suite Lounge with City View Clubs Atl VIP\r\n\r\n**## New-year in Atlanta\r\n\r\n* NEW YEARS DAY FREE ZUMBA CLASS East Lake Family YMCA, Eva Davis Way Southeast, Atlanta, GA, USA Wed, 01 Jan Free\r\n* NYE \u201825 Celebration JACQUEES Perfoming Live @ Slingshot Social Game Club Slingshot Social Game Club Tue, 31 Dec Free\r\n* Atlanta Symphony Orchestra: New Year\u0027s Eve Byers Theatre at Sandy Springs Performing Arts Center Tue, 31 Dec 13 interested\r\n\r\n**[NEW YEARS DAY FREE ZUMBA CLASS](https://allevents.in/atlanta/new-years-day-free-zumba-class/80002882313305?ref=category-buckets)### NEW YEARS DAY FREE ZUMBA CLASS\r\n\r\n****[NYE \u201825 Celebration JACQUEES Perfoming Live @ Slingshot Social Game Club](https://allevents.in/atlanta/nye-\u201825-celebration-jacquees-perfoming-live-slingshot-social-game-club/80003574542992?ref=category-buckets)### NYE \u201825 Celebration JACQUEES Perfoming Live @ Slingshot Social Game Club\r\n\r\n****[Atlanta Symphony Orchestra: New Year\u0027s Eve](https://allevents.in/atlanta/atlanta-symphony-orchestra-new-years-eve/2300027487679106?ref=category-buckets)### Atlanta Symphony Orchestra: New Year\u0027s Eve\r\n\r\n**## Art in Atlanta\r\n\r\n* Atlanta Sexiest BYOB Paint \u0026 Sip Atlanta 03 Jan-22 Feb USD 25\r\n* Last Podcast On The Left at Coca-Cola Roxy Theatre Coca-Cola Roxy Theatre Sat, 11 Jan\r\n* Maria Bamford at Buckhead Theatre Buckhead Theatre Sat, 25 Jan\r\n\r\n**[Atlanta Sexiest BYOB Paint \u0026 Sip](https://allevents.in/atlanta/atlanta-sexiest-byob-paint-and-sip/10000480043772847?ref=category-buckets)### Atlanta Sexiest BYOB Paint \u0026 Sip\r\n\r\n****[Last Podcast On The Left at Coca-Cola Roxy Theatre](https://allevents.in/atlanta/last-podcast-on-the-left-at-coca-cola-roxy-theatre/2400027672845538?ref=category-buckets)### Last Podcast On The Left at Coca-Cola Roxy Theatre\r\n\r\n**[Maria Bamford at Buckhead Theatre](https://allevents.in/atlanta/maria-bamford-at-buckhead-theatre/2400027673415311?ref=category-buckets)### Maria Bamford at Buckhead Theatre\r\n\r\n## Parties in Atlanta\r\n\r\n* Best Rooftop Party in Atlanta! Suite Lounge with City View Clubs Atl VIP Suite Food Lounge 03 Jan Onwards Free\r\n* Intension at The Masquerade - Altar The Masquerade - Altar Fri, 10 Jan\r\n* K*ll Dyll at The Masquerade - Altar The Masquerade - Altar Sun, 12 Jan\r\n\r\n**[Best Rooftop Party in Atlanta! Suite Lounge with City View Clubs Atl VIP](https://allevents.in/atlanta/best-rooftop-party-in-atlanta-suite-lounge-with-city-view-clubs-atl-vip/80003135700954?ref=category-buckets)### Best Rooftop Party in Atlanta! Suite Lounge with City View Clubs Atl VIP\r\n\r\n****[Intension at The Masquerade - Altar](https://allevents.in/atlanta/intension-at-the-masquerade-altar/2400027671894834?ref=category-buckets)### Intension at The Masquerade - Altar\r\n\r\n**[K*ll Dyll at The Masquerade - Altar](https://allevents.in/atlanta/kll-dyll-at-the-masquerade-altar/2400027672645425?ref=category-buckets)### K*ll Dyll at The Masquerade - Altar\r\n\r\n## Music in Atlanta\r\n\r\n* Featured Atlanta Soul-Jazz : Live Soul Jazz Music D.I.O. Studio; located 10 mins from Buckhead Atlanta; Look for orange Dogtopia complex Fri, 03 Jan 288 interested\r\n* Atlanta Symphony Orchestra - Coucheron and The Four Seasons at Atlanta Symphony Hall Atlanta Symphony Hall Thu, 09 Jan\r\n* Travis - The Band Buckhead Theatre Wed, 22 Jan\r\n\r\n**[Atlanta Soul-Jazz : Live Soul Jazz Music](https://allevents.in/atlanta/atlanta-soul-jazz-live-soul-jazz-music/80005571403380?ref=category-buckets)### Atlanta Soul-Jazz : Live Soul Jazz Music\r\n\r\n****[Atlanta Symphony Orchestra - Coucheron and The Four Seasons at Atlanta Symphony Hall](https://allevents.in/atlanta/atlanta-symphony-orchestra-coucheron-and-the-four-seasons-at-atlanta-symphony-hall/2400027630281717?ref=category-buckets)### Atlanta Symphony Orchestra - Coucheron and The Four Seasons at Atlanta Symphony Hall\r\n\r\n**[Travis - The Band](https://allevents.in/atlanta/travis-the-band/2300027179717038?ref=category-buckets)### Travis - The Band\r\n\r\n## Entertainment in Atlanta\r\n\r\n* Free Zumba and Hip Hop Cardio Grant Park Recreation Center 04 Jan Onwards Free\r\n* ATLANTA BEST NIGHT CLUB! CELEBRITY GUEST, NFL, NBA, REVEL NIGHTCLUB!! FRIDAY, SATURDAY, SUNDAY REVEL ATLANTA 03 Jan Onwards Free\r\n* Sweet Honey In The Rock Rialto Center For The Performing Arts Sat, 25 Jan\r\n\r\n**[Free Zumba and Hip Hop Cardio](https://allevents.in/atlanta/free-zumba-and-hip-hop-cardio/80002645114547?ref=category-buckets)### Free Zumba and Hip Hop Cardio\r\n\r\n****[ATLANTA BEST NIGHT CLUB! CELEBRITY GUEST, NFL, NBA, REVEL NIGHTCLUB!! FRIDAY, SATURDAY, SUNDAY](https://allevents.in/atlanta/atlanta-best-night-club-celebrity-guest-nfl-nba-revel-nightclub-friday-saturday-sunday/80001704849372?ref=category-buckets)### ATLANTA BEST NIGHT CLUB! CELEBRITY GUEST, NFL, NBA, REVEL NIGHTCLUB!! FRIDAY, SATURDAY, SUNDAY\r\n\r\n****[Sweet Honey In The Rock](https://allevents.in/atlanta/sweet-honey-in-the-rock/2300027179200638?ref=category-buckets)### Sweet Honey In The Rock\r\n\r\n### Events in January from nearby cities\r\n\r\n## Trending tags in Atlanta\r\n\r\n[entrepreneurship](https://allevents.in/atlanta/entrepreneurship?ref=category-buckets)[marketing](https://allevents.in/atlanta/marketing?ref=category-buckets)[poetry](https://allevents.in/atlanta/poetry?ref=category-buckets)[orchestra](https://allevents.in/atlanta/orchestra?ref=category-buckets)[real-estate](https://allevents.in/atlanta/real-estate?ref=category-buckets)[hookahs](https://allevents.in/atlanta/hookahs?ref=category-buckets)[books](https://allevents.in/atlanta/books?ref=category-buckets)[basketball](https://allevents.in/atlanta/basketball?ref=category-buckets)[painting](https://allevents.in/atlanta/painting?ref=category-buckets)[salsa](https://allevents.in/atlanta/salsa?ref=category-buckets)[employment-opportunities](https://allevents.in/atlanta/employment-opportunities?ref=category-buckets)[fitness](https://allevents.in/atlanta/fitness?ref=category-buckets)[chicken](https://allevents.in/atlanta/chicken?ref=category-buckets)## Explore popular event formats in Atlanta\r\n\r\n## Explore other popular categories in Atlanta\r\n\r\n### Trending events from nearby cities\r\n\r\n### Upcoming Events in January in Atlanta\r\n\r\n**Magic City Hippies at Variety Playhouse****Electric Avenue at Variety Playhouse****Fulton Lee at The Loft at Center Stage****Kathleen Edwards at City Winery - Atlanta****MJ Lenderman at Variety Playhouse****Life Love and Beyond Podcast at City Winery - Atlanta****Louisiana Ragin Cajuns at Georgia State Panthers Mens Basketball at GSU Convocation Center****Marshall Thundering Herd at Georgia State Panthers Mens Basketball at GSU Convocation Center**Customize your experience; only see the events that click with you!\r\n\r\n[Tune my vibe](https://allevents.in/recommended?ref=el-bottom-sugg)**Never miss a single fun event; let us keep you updated!\r\n\r\n### Explore more in Atlanta\r\n\r\n* Wine Events In Atlanta\r\n* Hookahs Events In Atlanta\r\n* Entrepreneurship Events In Atlanta\r\n* Art Events In Atlanta\r\n* Class Events In Atlanta\r\n* Masquerade Events In Atlanta\r\n\r\n[Wine Events In Atlanta](https://allevents.in/atlanta/wine)[Hookahs Events In Atlanta](https://allevents.in/atlanta/hookahs)[Entrepreneurship Events In Atlanta](https://allevents.in/atlanta/entrepreneurship)[Art Events In Atlanta](https://allevents.in/atlanta/art)[Class Events In Atlanta](https://allevents.in/atlanta/class)[Masquerade Events In Atlanta](https://allevents.in/atlanta/masquerade)### Events in January in Nearby Cities\r\n\r\n* Events in January in Birmingham\r\n* Events in January in Smyrna\r\n* Events in January in Knoxville\r\n* Events in January in Greenville\r\n* Events in January in Chattanooga\r\n* Events in January in Huntsville\r\n* Events in January in Decatur\r\n* Events in January in Marietta\r\n\r\n[Events in January in Birmingham](https://allevents.in/birmingham-al/january)[Events in January in Smyrna](https://allevents.in/smyrna/january)[Events in January in Knoxville](https://allevents.in/knoxville/january)[Events in January in Greenville](https://allevents.in/greenville-sc/january)[Events in January in Chattanooga](https://allevents.in/chattanooga/january)[Events in January in Huntsville](https://allevents.in/huntsville/january)[Events in January in Decatur](https://allevents.in/decatur/january)[Events in January in Marietta](https://allevents.in/marietta/january)**Stay tuned with the most relevant events happening around you. Attend, Share \u0026 Influence!\r\n\r\nE.g. Jack is first name and Mandanka is last name.\r\n\r\n*** Business\r\n* Parties\r\n* performances\r\n* Sports\r\n* Festivals\r\n* Workshops\r\n* Music\r\n* Exhibitions\r\n* food-drinks\r\n* health-wellness\r\n* dance\r\n* Fashion\r\n* Art\r\n* fine-arts\r\n* Theatre\r\n* literary-art\r\n* Crafts\r\n* Photography\r\n* Cooking\r\n* comedy\r\n* trips-adventures\r\n* Entertainment\r\n* christmas\r\n* Kids\r\n* Conferences\r\n* yoga\r\n* hackathons\r\n* virtual\r\n* webinar\r\n\r\n************************************************************You are in \uD83E\uDD1F\r\n\r\nLet\u2019s take it to the next level\r\n\r\nReal-time Updates, Real-time Fun\r\n\r\nYour Gateway to local events in your city\r\n\r\n[](https://allevents.in/app?utm_source=web_subscribe_pop-up\u0026ct=web_subscribe_pop-up\u0026utm_campaign=subscribe_pop-up\u0026utm_content=banner)![Download app qr]()\r\n[Get App Now](https://allevents.in/app?utm_source=web_subscribe_pop-up\u0026ct=web_subscribe_pop-up\u0026utm_campaign=subscribe_pop-up\u0026utm_content=button)[](https://allevents.in/app?utm_source=web_subscribe_pop-up\u0026ct=web_subscribe_pop-up\u0026utm_campaign=subscribe_pop-up\u0026utm_content=banner)![download app banner]()\r\n[Get App Now](https://allevents.in/app?utm_source=web_subscribe_pop-up\u0026ct=web_subscribe_pop-up\u0026utm_campaign=subscribe_pop-up\u0026utm_content=button)![](//googleads.g.doubleclick.net/pagead/viewthroughconversion/994326962/?guid=ON\u0026amp;script=0)\r\n## Site Navigation\r\n\r\n### Trending Searches\r\n\r\n**Searches*** Events \u0026 Things To Do In Atlanta This Weekend\r\n* Atlanta Festival Guide 2024\r\n* \u0026#129346; New Year\u0027s Eve Events In Atlanta\r\n* Houston Events\r\n* Atlanta Events\r\n* San Antonio Events\r\n* Las Vegas Events\r\n* Washington Events\r\n* Happy New Year 2025\r\n* Samay Raina Unfiltered - Newark\r\n* REVOLUTION 2025 #1TECHNO BOLLYWOOD NEW YEAR PARTY\r\n* LOVER \u2013 An Eras Tour Tribute Experience, All Ages Show\r\n* A Luxury by Design Experience 80s vs 90s DJ Battle\r\n* New Years Eve at Casa de Vino 57\r\n* Boudoir Boh\u00E9me Masquerade Ball\r\n* New Years Eve Noir and Neon Extravaganza\r\n* Naperville New Year Party\r\n* The Movie Menu Showing The Wood Saturday Jan 18th Evening Show\r\n\r\n[Events \u0026 Things To Do In Atlanta This Weekend](https://allevents.in/atlanta/this-weekend)[Atlanta Festival Guide 2024](https://allevents.in/atlanta/festivals)[\u0026#129346; New Year\u0027s Eve Events In Atlanta](https://allevents.in/atlanta/new-year)[Houston Events](https://allevents.in/houston)[Atlanta Events](https://allevents.in/atlanta)[San Antonio Events](https://allevents.in/san%20antonio)[Las Vegas Events](https://allevents.in/las%20vegas)[Washington Events](https://allevents.in/washington)[Happy New Year 2025](https://allevents.in/san%20jose/happy-new-year-2025/80001505885623)[Samay Raina Unfiltered - Newark](https://allevents.in/elizabeth/samay-raina-unfiltered-newark/80001476967224)[REVOLUTION 2025 #1TECHNO BOLLYWOOD NEW YEAR PARTY](https://allevents.in/dallas/revolution-2025-1techno-bollywood-new-year-party/100001093572638199)[LOVER \u2013 An Eras Tour Tribute Experience, All Ages Show](https://allevents.in/grapevine/lover-\u2013-an-eras-tour-tribute-experience-all-ages-show/80001260281733)[A Luxury by Design Experience 80s vs 90s DJ Battle](https://allevents.in/oak%20brook/a-luxury-by-design-experience-80s-vs-90s-dj-battle/200027284427345)[New Years Eve at Casa de Vino 57](https://allevents.in/augustine/new-years-eve-at-casa-de-vino-57/200027525515842)[Boudoir Boh\u00E9me Masquerade Ball](https://allevents.in/washington/boudoir-boh\u00E9me-masquerade-ball/200027174705140)[New Years Eve Noir and Neon Extravaganza](https://allevents.in/west%20columbia/new-years-eve-noir-and-neon-extravaganza/80002828529585)[Naperville New Year Party](https://allevents.in/naperville/naperville-new-year-party/80009010250754)[The Movie Menu Showing The Wood Saturday Jan 18th Evening Show](https://allevents.in/chicago/the-movie-menu-showing-the-wood-saturday-jan-18th-evening-show/80002821195740)### Host Events\r\n\r\n* Publish Your Events\r\n* Promote Your Events\r\n* Sell Tickets Online\r\n* Host Recorded Events\r\n* Pricing Plans\r\n* Event Manager App\r\n* Rank Your Event\r\n* Event Listing\r\n* Learn Event Marketing\r\n\r\n[Publish Your Events](https://allevents.in/pages/publish-events?ref=footer-n-category-buckets)[Promote Your Events](https://allevents.in/pages/event-marketing?ref=footer-n-category-buckets)[Sell Tickets Online](https://allevents.in/pages/event-ticketing?ref=footer-n-category-buckets)[Host Recorded Events](https://allevents.in/pages/events-on-demand?ref=footer-n-category-buckets)[Pricing Plans](https://allevents.in/pages/pricing?ref=footer-n-category-buckets)[Event Manager App](https://allevents.in/pages/event-manager-app?ref=footer-n-category-buckets)[Rank Your Event](https://allevents.in/pages/google-event-listing?ref=footer-n-category-buckets)[Event Listing](https://allevents.in/pages/event-listing?ref=footer-n-category-buckets)[Learn Event Marketing](https://allevents.in/pages/event-promotion?ref=footer-n-category-buckets)### Discover Events\r\n\r\n* Events for You\r\n* Virtual Events\r\n* Get Event Updates\r\n* Event Discovery App\r\n* Write for Us\r\n\r\n[Events for You](https://allevents.in/recommended?ref=footer-n-category-buckets)[Virtual Events](https://allevents.in/online?ref=footer-n-category-buckets)[Get Event Updates](https://allevents.in/pages/subscribe?ref=footer-n-category-buckets)[Event Discovery App](https://allevents.in/app?ref=footer-n-category-buckets)[Write for Us](https://allevents.in/pages/write-for-us?ref=footer-n-category-buckets)### Explore\r\n\r\n* Event Listing Plugin\r\n* Media Kit\r\n* Affiliate Program\r\n* Support Center\r\n* Blog\r\n\r\n[Event Listing Plugin](https://allevents.in/pages/events-listing-plugin?ref=footer-n-category-buckets)[Media Kit](https://allevents.in/blog/media-kit/?ref=footer-n-category-buckets)[Affiliate Program](https://allevents.in/pages/affiliate-program?ref=footer-n-category-buckets)[Support Center](https://support.allevents.in?ref=footer-n-category-buckets)[Blog](https://allevents.in/blog?ref=footer-n-category-buckets)### #StayHappening\r\n\r\n[](https://play.google.com/store/apps/details?id=com.amitech.allevents)![Allevents on Google Play]()\r\n[](https://itunes.apple.com/app/apple-store/id488116646?pt=505903\u0026ct=footer\u0026mt=8)![Allevents on AppStore]()\r\n[](https://allevents.in/pages/careers?ref=we-are-hiring-footer)![We\u0027re Hiring]()\r\n### Upcoming Events in Atlanta\r\n\r\n* BYOB Spray Paint and Sip - Paint and Sip by Classpop!\u2122\r\n* 1259 Burlesque Show\r\n* Cidesco - Beauty Beyond Borders\r\n* New Years Eve Celebration\r\n* Igloo Experience at Reverence\r\n* Getting LIT(erary) at VHC: Banned Book Club\r\n\r\n[BYOB Spray Paint and Sip - Paint and Sip by Classpop!\u2122](https://allevents.in/atlanta/byob-spray-paint-and-sip-paint-and-sip-by-classpop/100001119455965929)[1259 Burlesque Show](https://allevents.in/atlanta/1259-burlesque-show/80002986680159)[Cidesco - Beauty Beyond Borders](https://allevents.in/atlanta/cidesco-beauty-beyond-borders/100001119332225819)[New Years Eve Celebration](https://allevents.in/atlanta/new-years-eve-celebration/100001127284190359)[Igloo Experience at Reverence](https://allevents.in/atlanta/igloo-experience-at-reverence/200027687586540)[Getting LIT(erary) at VHC: Banned Book Club](https://allevents.in/atlanta/getting-literary-at-vhc-banned-book-club/100001111870678129)* About\r\n* Careers\r\n* Support\r\n* Terms of Service\r\n* Privacy\r\n* Contact Us\r\n\r\n[About](https://allevents.in/about)[Careers](https://allevents.in/pages/careers)[Support](https://support.allevents.in)[Terms of Service](https://allevents.in/conditions)[Privacy](https://allevents.in/privacy)[Contact Us](https://allevents.in/contact)**[allevents.in](https://allevents.in)[Atlanta](https://allevents.in/atlanta)\r\n\r\n[](https://allevents.in)**[](https://facebook.com/allevents.in)**[](https://twitter.com/allevents_in)**[](https://www.instagram.com/allevents.in/)**[](https://www.youtube.com/@allevents-in)**[](https://in.pinterest.com/allevents_in/)**[](https://allevents.in/Atlanta/RSS)**[](#)**\r\n\r\n![](//na.allevents.in/matomo.php?idsite=1)\r\n![](https://www.facebook.com/tr?id=1186492295454388\u0026ev=PageView\u0026noscript=1)\r\n"
```

I called the tool named GetContentFromUrl with these arguments:
```
{"url": "https://www.eventbrite.com/d/ga--atlanta/2025/"}
```
and got this result:
```
"[upgrade your browser](/support/articles/en_US/Troubleshooting/how-to-troubleshoot-internet-browser-issues)[Eventbrite](https://www.eventbrite.com/)*Eventbrite*[Eventbrite](https://www.eventbrite.com/)*Eventbrite** Find my tickets\r\n* Log In\r\n* Sign Up\r\n* Find EventsCreate EventsSolutionsSolutionsEvent TicketingEvent Marketing PlatformEventbrite AdsPaymentsIndustryIndustryMusicFood \u0026amp; BeveragePerforming ArtsCharity \u0026amp; CausesRetailEvent TypesEvent TypesConcertsClasses \u0026amp; WorkshopsFestivals \u0026amp; FairsConferencesCorporate EventsOnline EventsBlogBlogTips \u0026amp; GuidesNews \u0026amp; TrendsCommunityTools \u0026amp; FeaturesOrganizer Resource HubCreate EventsContact SalesGet StartedHelp CenterHelp CenterFind your ticketsContact your event organizer\r\n\r\n[Find my tickets](https://www.eventbrite.com/signin/signup/?referrer=%2Fmytickets)[Log In](https://www.eventbrite.com/signin/?referrer=%2Fd%2Fga--atlanta%2F2025%2F)[Sign Up](https://www.eventbrite.com/signin/signup/?referrer=%2Fd%2Fga--atlanta%2F2025%2F)***** Find Events\r\n* Create EventsSolutionsSolutionsEvent TicketingEvent Marketing PlatformEventbrite AdsPaymentsIndustryIndustryMusicFood \u0026amp; BeveragePerforming ArtsCharity \u0026amp; CausesRetailEvent TypesEvent TypesConcertsClasses \u0026amp; WorkshopsFestivals \u0026amp; FairsConferencesCorporate EventsOnline EventsBlogBlogTips \u0026amp; GuidesNews \u0026amp; TrendsCommunityTools \u0026amp; Features\r\n* Organizer Resource Hub\r\n* Create Events\r\n* Contact Sales\r\n* Get Started\r\n* Help CenterHelp CenterFind your ticketsContact your event organizer\r\n\r\n[Find Events](https://www.eventbrite.com/d/local/events/)************* SolutionsSolutionsEvent TicketingEvent Marketing PlatformEventbrite AdsPayments\r\n* IndustryIndustryMusicFood \u0026amp; BeveragePerforming ArtsCharity \u0026amp; CausesRetail\r\n* Event TypesEvent TypesConcertsClasses \u0026amp; WorkshopsFestivals \u0026amp; FairsConferencesCorporate EventsOnline Events\r\n* BlogBlogTips \u0026amp; GuidesNews \u0026amp; TrendsCommunityTools \u0026amp; Features\r\n\r\n*** Event Ticketing\r\n* Event Marketing Platform\r\n* Eventbrite Ads\r\n* Payments\r\n\r\n[Event Ticketing](https://www.eventbrite.com/l/sell-tickets/)[Event Marketing Platform](https://www.eventbrite.com/l/event-marketing-platform/)[Eventbrite Ads](https://www.eventbrite.com/l/eventbrite-ads/)[Payments](https://www.eventbrite.com/l/event-payment/)*** Music\r\n* Food \u0026amp; Beverage\r\n* Performing Arts\r\n* Charity \u0026amp; Causes\r\n* Retail\r\n\r\n[Music](https://www.eventbrite.com/l/music/)[Food \u0026amp; Beverage](https://www.eventbrite.com/l/food-drink-event-ticketing/)[Performing Arts](https://www.eventbrite.com/l/performing-arts/)[Charity \u0026amp; Causes](https://www.eventbrite.com/l/npo/)[Retail](https://www.eventbrite.com/l/host-retail-events/)*** Concerts\r\n* Classes \u0026amp; Workshops\r\n* Festivals \u0026amp; Fairs\r\n* Conferences\r\n* Corporate Events\r\n* Online Events\r\n\r\n[Concerts](https://www.eventbrite.com/l/music-venues/)[Classes \u0026amp; Workshops](https://www.eventbrite.com/l/create-a-workshop/)[Festivals \u0026amp; Fairs](https://www.eventbrite.com/l/festival-solutions/)[Conferences](https://www.eventbrite.com/l/conferences/)[Corporate Events](https://www.eventbrite.com/l/eventbrite-for-business/)[Online Events](https://www.eventbrite.com/l/virtual-events-platform/)*** Tips \u0026amp; Guides\r\n* News \u0026amp; Trends\r\n* Community\r\n* Tools \u0026amp; Features\r\n\r\n[Tips \u0026amp; Guides](https://www.eventbrite.com/blog/category/tips-and-guides/)[News \u0026amp; Trends](https://www.eventbrite.com/blog/category/news-and-trends/)[Community](https://www.eventbrite.com/blog/category/community/)[Tools \u0026amp; Features](https://www.eventbrite.com/blog/category/tools-and-features/)[Organizer Resource Hub](https://www.eventbrite.com/resources/)[Create Events](https://www.eventbrite.com/organizer/overview/)[Contact Sales](https://www.eventbrite.com/l/contact-eventbrite-sales/)[Get Started](https://www.eventbrite.com/signin/signup/?referrer=/manage/events/create/)********* Help Center\r\n* Find your tickets\r\n* Contact your event organizer\r\n\r\n[Help Center](https://www.eventbrite.com/help/en-us/)[Find your tickets](https://www.eventbrite.com/help/en-us/articles/319355/where-are-my-tickets/)[Contact your event organizer](https://www.eventbrite.com/help/en-us/articles/647151/how-to-contact-the-event-organizer/)***** Find Events\r\n* Create Events\r\n* Help CenterHelp CenterFind your ticketsContact your event organizer\r\n* Find my tickets\r\n* Log In\r\n* Sign Up\r\n\r\n[Find Events](https://www.eventbrite.com/d/local/events/)[Create Events](https://www.eventbrite.com/organizer/overview/)***** Help Center\r\n* Find your tickets\r\n* Contact your event organizer\r\n\r\n[Help Center](https://www.eventbrite.com/help/en-us/)[Find your tickets](https://www.eventbrite.com/help/en-us/articles/319355/where-are-my-tickets/)[Contact your event organizer](https://www.eventbrite.com/help/en-us/articles/647151/how-to-contact-the-event-organizer/)[Find my tickets](https://www.eventbrite.com/signin/signup/?referrer=%2Fmytickets)[Log In](https://www.eventbrite.com/signin/?referrer=%2Fd%2Fga--atlanta%2F2025%2F)[Sign Up](https://www.eventbrite.com/signin/signup/?referrer=%2Fd%2Fga--atlanta%2F2025%2F)* Find EventsCreate EventsSolutionsSolutionsEvent TicketingEvent Marketing PlatformEventbrite AdsPaymentsIndustryIndustryMusicFood \u0026amp; BeveragePerforming ArtsCharity \u0026amp; CausesRetailEvent TypesEvent TypesConcertsClasses \u0026amp; WorkshopsFestivals \u0026amp; FairsConferencesCorporate EventsOnline EventsBlogBlogTips \u0026amp; GuidesNews \u0026amp; TrendsCommunityTools \u0026amp; FeaturesOrganizer Resource HubCreate EventsContact SalesGet StartedHelp CenterHelp CenterFind your ticketsContact your event organizer\r\n\r\n***** Find Events\r\n* Create EventsSolutionsSolutionsEvent TicketingEvent Marketing PlatformEventbrite AdsPaymentsIndustryIndustryMusicFood \u0026amp; BeveragePerforming ArtsCharity \u0026amp; CausesRetailEvent TypesEvent TypesConcertsClasses \u0026amp; WorkshopsFestivals \u0026amp; FairsConferencesCorporate EventsOnline EventsBlogBlogTips \u0026amp; GuidesNews \u0026amp; TrendsCommunityTools \u0026amp; Features\r\n* Organizer Resource Hub\r\n* Create Events\r\n* Contact Sales\r\n* Get Started\r\n* Help CenterHelp CenterFind your ticketsContact your event organizer\r\n\r\n[Find Events](https://www.eventbrite.com/d/local/events/)************* SolutionsSolutionsEvent TicketingEvent Marketing PlatformEventbrite AdsPayments\r\n* IndustryIndustryMusicFood \u0026amp; BeveragePerforming ArtsCharity \u0026amp; CausesRetail\r\n* Event TypesEvent TypesConcertsClasses \u0026amp; WorkshopsFestivals \u0026amp; FairsConferencesCorporate EventsOnline Events\r\n* BlogBlogTips \u0026amp; GuidesNews \u0026amp; TrendsCommunityTools \u0026amp; Features\r\n\r\n*** Event Ticketing\r\n* Event Marketing Platform\r\n* Eventbrite Ads\r\n* Payments\r\n\r\n[Event Ticketing](https://www.eventbrite.com/l/sell-tickets/)[Event Marketing Platform](https://www.eventbrite.com/l/event-marketing-platform/)[Eventbrite Ads](https://www.eventbrite.com/l/eventbrite-ads/)[Payments](https://www.eventbrite.com/l/event-payment/)*** Music\r\n* Food \u0026amp; Beverage\r\n* Performing Arts\r\n* Charity \u0026amp; Causes\r\n* Retail\r\n\r\n[Music](https://www.eventbrite.com/l/music/)[Food \u0026amp; Beverage](https://www.eventbrite.com/l/food-drink-event-ticketing/)[Performing Arts](https://www.eventbrite.com/l/performing-arts/)[Charity \u0026amp; Causes](https://www.eventbrite.com/l/npo/)[Retail](https://www.eventbrite.com/l/host-retail-events/)*** Concerts\r\n* Classes \u0026amp; Workshops\r\n* Festivals \u0026amp; Fairs\r\n* Conferences\r\n* Corporate Events\r\n* Online Events\r\n\r\n[Concerts](https://www.eventbrite.com/l/music-venues/)[Classes \u0026amp; Workshops](https://www.eventbrite.com/l/create-a-workshop/)[Festivals \u0026amp; Fairs](https://www.eventbrite.com/l/festival-solutions/)[Conferences](https://www.eventbrite.com/l/conferences/)[Corporate Events](https://www.eventbrite.com/l/eventbrite-for-business/)[Online Events](https://www.eventbrite.com/l/virtual-events-platform/)*** Tips \u0026amp; Guides\r\n* News \u0026amp; Trends\r\n* Community\r\n* Tools \u0026amp; Features\r\n\r\n[Tips \u0026amp; Guides](https://www.eventbrite.com/blog/category/tips-and-guides/)[News \u0026amp; Trends](https://www.eventbrite.com/blog/category/news-and-trends/)[Community](https://www.eventbrite.com/blog/category/community/)[Tools \u0026amp; Features](https://www.eventbrite.com/blog/category/tools-and-features/)[Organizer Resource Hub](https://www.eventbrite.com/resources/)[Create Events](https://www.eventbrite.com/organizer/overview/)[Contact Sales](https://www.eventbrite.com/l/contact-eventbrite-sales/)[Get Started](https://www.eventbrite.com/signin/signup/?referrer=/manage/events/create/)********* Help Center\r\n* Find your tickets\r\n* Contact your event organizer\r\n\r\n[Help Center](https://www.eventbrite.com/help/en-us/)[Find your tickets](https://www.eventbrite.com/help/en-us/articles/319355/where-are-my-tickets/)[Contact your event organizer](https://www.eventbrite.com/help/en-us/articles/647151/how-to-contact-the-event-organizer/)## NYE events are here. \uD83C\uDF89\r\n\r\n[Find your perfect plan.](/b/ga--atlanta/holiday/new-years-eve/)1. Home/\r\n2. United States/\r\n3. Atlanta/\r\n4. 2025 Events\r\n\r\n[Home](/)[United States](/d/united-states/events/)[Atlanta](/d/ga--atlanta/events/)# 2025 events in Atlanta, GA\r\n\r\nFilters\r\n\r\n* Business\r\n* Food \u0026amp; Drink\r\n* Health\r\n* Music\r\n\r\n[Business](/d/ga--atlanta/business--events/)**[Food \u0026amp; Drink](/d/ga--atlanta/food-and-drink--events/)**[Health](/d/ga--atlanta/health--events/)**[Music](/d/ga--atlanta/music--events/)*** Today\r\n* Tomorrow\r\n* This weekend\r\n* Pick a date...\r\n\r\n* Free\r\n* Paid\r\n\r\n* Class\r\n* Conference\r\n* Festival\r\n* Party\r\n\r\n****Search for online events\r\n\r\n* English\r\n* German\r\n* Spanish\r\n* French\r\n\r\n********* U.S. Dollar\r\n* Canadian Dollar\r\n* Euro\r\n* British Pound\r\n\r\n*************** Vision Board 2025Sun, Jan 12, 12:00 PM3107 Peachtree Rd NE a1Save this event: Vision Board 2025Share this event: Vision Board 2025Vision Board 2025Sun, Jan 12, 12:00 PM3107 Peachtree Rd NE a1Share this event: Vision Board 2025Save this event: Vision Board 2025\r\n* Sales end soonA Toast to 2025: NYE at DrawbarToday at 9:00 PMBellyard, West Midtown Atlanta, a Tribute Portfolio HotelSave this event: A Toast to 2025: NYE at DrawbarShare this event: A Toast to 2025: NYE at DrawbarSales end soonA Toast to 2025: NYE at DrawbarToday at 9:00 PMBellyard, West Midtown Atlanta, a Tribute Portfolio HotelShare this event: A Toast to 2025: NYE at DrawbarSave this event: A Toast to 2025: NYE at Drawbar\r\n* Going fastIn Vision Brunch 2025Saturday at 12:00 PM3699 Lenox Rd NE suite 5Save this event: In Vision Brunch 2025Share this event: In Vision Brunch 2025Going fastIn Vision Brunch 2025Saturday at 12:00 PM3699 Lenox Rd NE suite 5Share this event: In Vision Brunch 2025Save this event: In Vision Brunch 2025\r\n* Havana Club ATL - NYE 2025 PartyToday at 8:00 PMHavana Club ATLSave this event: Havana Club ATL - NYE 2025 PartyShare this event: Havana Club ATL - NYE 2025 PartyHavana Club ATL - NYE 2025 PartyToday at 8:00 PMHavana Club ATLShare this event: Havana Club ATL - NYE 2025 PartySave this event: Havana Club ATL - NYE 2025 Party\r\n* Sales end soonNYE 2025 CELEBRATION PARTYToday at 10:00 PMSebastian Pintxos BarSave this event: NYE 2025 CELEBRATION PARTYShare this event: NYE 2025 CELEBRATION PARTYSales end soonNYE 2025 CELEBRATION PARTYToday at 10:00 PMSebastian Pintxos BarShare this event: NYE 2025 CELEBRATION PARTYSave this event: NYE 2025 CELEBRATION PARTY\r\n* Sales end soon2025 Vision Board PartyFriday at 7:00 PMAC Hotel by Marriott Atlanta Buckhead at Phipps PlazaSave this event: 2025 Vision Board PartyShare this event: 2025 Vision Board PartySales end soon2025 Vision Board PartyFriday at 7:00 PMAC Hotel by Marriott Atlanta Buckhead at Phipps PlazaShare this event: 2025 Vision Board PartySave this event: 2025 Vision Board Party\r\n* From Day One: Atlanta 2025Tue, Jan 28, 8:30 AMGeorgia AquariumSave this event: From Day One: Atlanta 2025Share this event: From Day One: Atlanta 2025From Day One: Atlanta 2025Tue, Jan 28, 8:30 AMGeorgia AquariumShare this event: From Day One: Atlanta 2025Save this event: From Day One: Atlanta 2025\r\n* Sales end soon2025 EVOLUTION: Vision Planning EventSaturday at 2:00 PMAtlantaSave this event: 2025 EVOLUTION: Vision Planning EventShare this event: 2025 EVOLUTION: Vision Planning EventSales end soon2025 EVOLUTION: Vision Planning EventSaturday at 2:00 PMAtlantaShare this event: 2025 EVOLUTION: Vision Planning EventSave this event: 2025 EVOLUTION: Vision Planning Event\r\n* ELEVATE : IGNITE CONNECTIONS FOR 2025Sat, Jan 11, 5:00 PMThe Ritz-Carlton, AtlantaSave this event: ELEVATE : IGNITE CONNECTIONS FOR 2025Share this event: ELEVATE : IGNITE CONNECTIONS FOR 2025ELEVATE : IGNITE CONNECTIONS FOR 2025Sat, Jan 11, 5:00 PMThe Ritz-Carlton, AtlantaShare this event: ELEVATE : IGNITE CONNECTIONS FOR 2025Save this event: ELEVATE : IGNITE CONNECTIONS FOR 2025\r\n* 2025 Atlanta Travel \u0026amp; Adventure ShowGeorgia International Convention Center - HALLS A\u0026amp;BSave this event: 2025 Atlanta Travel \u0026amp; Adventure ShowShare this event: 2025 Atlanta Travel \u0026amp; Adventure Show2025 Atlanta Travel \u0026amp; Adventure ShowGeorgia International Convention Center - HALLS A\u0026amp;BShare this event: 2025 Atlanta Travel \u0026amp; Adventure ShowSave this event: 2025 Atlanta Travel \u0026amp; Adventure Show\r\n* INFERNO (The Official 2025 MOD Party)Fri, Jan 17, 10:00 PMHi-Def Events of AvondaleSave this event: INFERNO (The Official 2025 MOD Party)Share this event: INFERNO (The Official 2025 MOD Party)INFERNO (The Official 2025 MOD Party)Fri, Jan 17, 10:00 PMHi-Def Events of AvondaleShare this event: INFERNO (The Official 2025 MOD Party)Save this event: INFERNO (The Official 2025 MOD Party)\r\n* Insights 2025: Roadmap to OpportunityFri, Jan 24, 8:30 AMThe Georgia Freight DepotSave this event: Insights 2025: Roadmap to OpportunityShare this event: Insights 2025: Roadmap to OpportunityInsights 2025: Roadmap to OpportunityFri, Jan 24, 8:30 AMThe Georgia Freight DepotShare this event: Insights 2025: Roadmap to OpportunitySave this event: Insights 2025: Roadmap to Opportunity\r\n* Sales end soonATL NYE BASH [EVERYONE FREE BEFORE 2025]Today at 10:00 PMLyfe ATLSave this event: ATL NYE BASH [EVERYONE FREE BEFORE 2025]Share this event: ATL NYE BASH [EVERYONE FREE BEFORE 2025]Sales end soonATL NYE BASH [EVERYONE FREE BEFORE 2025]Today at 10:00 PMLyfe ATLShare this event: ATL NYE BASH [EVERYONE FREE BEFORE 2025]Save this event: ATL NYE BASH [EVERYONE FREE BEFORE 2025]\r\n* Ignite 2025Fri, Mar 7, 10:00 AMSonesta Atlanta Northwest Marietta - The BatterySave this event: Ignite 2025Share this event: Ignite 2025Ignite 2025Fri, Mar 7, 10:00 AMSonesta Atlanta Northwest Marietta - The BatteryShare this event: Ignite 2025Save this event: Ignite 2025\r\n* Almost fullNYE 2025 @ El Valle with Miramar ProToday at 10:00 PMEl ValleSave this event: NYE 2025 @ El Valle with Miramar ProShare this event: NYE 2025 @ El Valle with Miramar ProAlmost fullNYE 2025 @ El Valle with Miramar ProToday at 10:00 PMEl ValleShare this event: NYE 2025 @ El Valle with Miramar ProSave this event: NYE 2025 @ El Valle with Miramar Pro\r\n* Sales end soonBLAME IT ON 2025 \u2022 FIRST DAY PARTY OF THE YEARTomorrow at 2:00 PMXPERIENCE SPORTS BAR \u0026amp; LoungeSave this event: BLAME IT ON 2025 \u2022 FIRST DAY PARTY OF THE YEARShare this event: BLAME IT ON 2025 \u2022 FIRST DAY PARTY OF THE YEARSales end soonBLAME IT ON 2025 \u2022 FIRST DAY PARTY OF THE YEARTomorrow at 2:00 PMXPERIENCE SPORTS BAR \u0026amp; LoungeShare this event: BLAME IT ON 2025 \u2022 FIRST DAY PARTY OF THE YEARSave this event: BLAME IT ON 2025 \u2022 FIRST DAY PARTY OF THE YEAR\r\n* 2025 Flourish Family Homeschool ConferenceWed, Jan 29, 5:00 PMNew Life Community CenterSave this event: 2025 Flourish Family Homeschool ConferenceShare this event: 2025 Flourish Family Homeschool Conference2025 Flourish Family Homeschool ConferenceWed, Jan 29, 5:00 PMNew Life Community CenterShare this event: 2025 Flourish Family Homeschool ConferenceSave this event: 2025 Flourish Family Homeschool Conference\r\n* Sales end soonCIRCA NYE| NEW YEAR\u0026#x27;S EVE CELEBRATION 2025Today at 9:00 PMCafe CircaSave this event: CIRCA NYE| NEW YEAR\u0026#x27;S EVE CELEBRATION 2025Share this event: CIRCA NYE| NEW YEAR\u0026#x27;S EVE CELEBRATION 2025Sales end soonCIRCA NYE| NEW YEAR\u0026#x27;S EVE CELEBRATION 2025Today at 9:00 PMCafe CircaShare this event: CIRCA NYE| NEW YEAR\u0026#x27;S EVE CELEBRATION 2025Save this event: CIRCA NYE| NEW YEAR\u0026#x27;S EVE CELEBRATION 2025\r\n* Sales end soonWHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTYToday at 9:00 PMBamboo atlantaSave this event: WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTYShare this event: WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTYSales end soonWHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTYToday at 9:00 PMBamboo atlantaShare this event: WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTYSave this event: WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY\r\n* Ready. Set. Goals 2025Sat, Jan 11, 11:00 AM2781 Main StSave this event: Ready. Set. Goals 2025Share this event: Ready. Set. Goals 2025Ready. Set. Goals 2025Sat, Jan 11, 11:00 AM2781 Main StShare this event: Ready. Set. Goals 2025Save this event: Ready. Set. Goals 2025\r\n\r\n[](https://www.eventbrite.com/e/vision-board-2025-tickets-1095698978139?aff=ebdssbdestsearch)![Vision Board 2025 primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F915641043%2F1336266824853%2F1%2Foriginal.20241211-152720?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.00478286784741\u0026amp;fp-y=0.00022373540856\u0026amp;s=84c27c4fcfa731d82858bfe4c8c6e3fd)\r\n[Vision Board 2025](https://www.eventbrite.com/e/vision-board-2025-tickets-1095698978139?aff=ebdssbdestsearch)### Vision Board 2025\r\n\r\nSun, Jan 12, 12:00 PM\r\n\r\n3107 Peachtree Rd NE a1\r\n\r\n*Save this event: Vision Board 2025**Share this event: Vision Board 2025*[](https://www.eventbrite.com/e/vision-board-2025-tickets-1095698978139?aff=ebdssbdestsearch)![Vision Board 2025 primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F915641043%2F1336266824853%2F1%2Foriginal.20241211-152720?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.00478286784741\u0026amp;fp-y=0.00022373540856\u0026amp;s=84c27c4fcfa731d82858bfe4c8c6e3fd)\r\n[Vision Board 2025](https://www.eventbrite.com/e/vision-board-2025-tickets-1095698978139?aff=ebdssbdestsearch)### Vision Board 2025\r\n\r\nSun, Jan 12, 12:00 PM\r\n\r\n3107 Peachtree Rd NE a1\r\n\r\n*Share this event: Vision Board 2025**Save this event: Vision Board 2025*[](https://www.eventbrite.com/e/a-toast-to-2025-nye-at-drawbar-tickets-1089068666699?aff=ebdssbdestsearch)![A Toast to 2025: NYE at Drawbar primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F906278843%2F550494888477%2F1%2Foriginal.20241125-205619?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.359848484848\u0026amp;fp-y=0.0493527477227\u0026amp;s=90da539f609202863c8a0b87edee6853)\r\nSales end soon\r\n\r\n[A Toast to 2025: NYE at Drawbar](https://www.eventbrite.com/e/a-toast-to-2025-nye-at-drawbar-tickets-1089068666699?aff=ebdssbdestsearch)### A Toast to 2025: NYE at Drawbar\r\n\r\nToday at 9:00 PM\r\n\r\nBellyard, West Midtown Atlanta, a Tribute Portfolio Hotel\r\n\r\n*Save this event: A Toast to 2025: NYE at Drawbar**Share this event: A Toast to 2025: NYE at Drawbar*[](https://www.eventbrite.com/e/a-toast-to-2025-nye-at-drawbar-tickets-1089068666699?aff=ebdssbdestsearch)![A Toast to 2025: NYE at Drawbar primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F906278843%2F550494888477%2F1%2Foriginal.20241125-205619?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.359848484848\u0026amp;fp-y=0.0493527477227\u0026amp;s=90da539f609202863c8a0b87edee6853)\r\nSales end soon\r\n\r\n[A Toast to 2025: NYE at Drawbar](https://www.eventbrite.com/e/a-toast-to-2025-nye-at-drawbar-tickets-1089068666699?aff=ebdssbdestsearch)### A Toast to 2025: NYE at Drawbar\r\n\r\nToday at 9:00 PM\r\n\r\nBellyard, West Midtown Atlanta, a Tribute Portfolio Hotel\r\n\r\n*Share this event: A Toast to 2025: NYE at Drawbar**Save this event: A Toast to 2025: NYE at Drawbar*[](https://www.eventbrite.com/e/in-vision-brunch-2025-tickets-1113209602889?aff=ebdssbdestsearch)![In Vision Brunch 2025 primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F917323843%2F2544154155111%2F1%2Foriginal.20241214-031104?crop=focalpoint\u0026amp;fit=crop\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=6d54288ab35b4080d36225f9ff80ed18)\r\nGoing fast\r\n\r\n[In Vision Brunch 2025](https://www.eventbrite.com/e/in-vision-brunch-2025-tickets-1113209602889?aff=ebdssbdestsearch)### In Vision Brunch 2025\r\n\r\nSaturday at 12:00 PM\r\n\r\n3699 Lenox Rd NE suite 5\r\n\r\n*Save this event: In Vision Brunch 2025**Share this event: In Vision Brunch 2025*[](https://www.eventbrite.com/e/in-vision-brunch-2025-tickets-1113209602889?aff=ebdssbdestsearch)![In Vision Brunch 2025 primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F917323843%2F2544154155111%2F1%2Foriginal.20241214-031104?crop=focalpoint\u0026amp;fit=crop\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=6d54288ab35b4080d36225f9ff80ed18)\r\nGoing fast\r\n\r\n[In Vision Brunch 2025](https://www.eventbrite.com/e/in-vision-brunch-2025-tickets-1113209602889?aff=ebdssbdestsearch)### In Vision Brunch 2025\r\n\r\nSaturday at 12:00 PM\r\n\r\n3699 Lenox Rd NE suite 5\r\n\r\n*Share this event: In Vision Brunch 2025**Save this event: In Vision Brunch 2025*[](https://www.eventbrite.com/e/havana-club-atl-nye-2025-party-tickets-1097373496669?aff=ebdssbdestsearch)![Havana Club ATL - NYE 2025 Party primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F907636203%2F19569498248%2F1%2Foriginal.20241127-163057?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.482954545455\u0026amp;fp-y=0.5\u0026amp;s=61dba24f530a85ef016700fbd166ed61)\r\n[Havana Club ATL - NYE 2025 Party](https://www.eventbrite.com/e/havana-club-atl-nye-2025-party-tickets-1097373496669?aff=ebdssbdestsearch)### Havana Club ATL - NYE 2025 Party\r\n\r\nToday at 8:00 PM\r\n\r\nHavana Club ATL\r\n\r\n*Save this event: Havana Club ATL - NYE 2025 Party**Share this event: Havana Club ATL - NYE 2025 Party*[](https://www.eventbrite.com/e/havana-club-atl-nye-2025-party-tickets-1097373496669?aff=ebdssbdestsearch)![Havana Club ATL - NYE 2025 Party primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F907636203%2F19569498248%2F1%2Foriginal.20241127-163057?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.482954545455\u0026amp;fp-y=0.5\u0026amp;s=61dba24f530a85ef016700fbd166ed61)\r\n[Havana Club ATL - NYE 2025 Party](https://www.eventbrite.com/e/havana-club-atl-nye-2025-party-tickets-1097373496669?aff=ebdssbdestsearch)### Havana Club ATL - NYE 2025 Party\r\n\r\nToday at 8:00 PM\r\n\r\nHavana Club ATL\r\n\r\n*Share this event: Havana Club ATL - NYE 2025 Party**Save this event: Havana Club ATL - NYE 2025 Party*[](https://www.eventbrite.com/e/nye-2025-celebration-party-tickets-1115557254779?aff=ebdssbdestsearch)![NYE 2025  CELEBRATION  PARTY primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F918145583%2F1661149656073%2F1%2Foriginal.20241216-141708?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=c854a3c1e5c8eae356054a82cea8e20d)\r\nSales end soon\r\n\r\n[NYE 2025 CELEBRATION PARTY](https://www.eventbrite.com/e/nye-2025-celebration-party-tickets-1115557254779?aff=ebdssbdestsearch)### NYE 2025 CELEBRATION PARTY\r\n\r\nToday at 10:00 PM\r\n\r\nSebastian Pintxos Bar\r\n\r\n*Save this event: NYE 2025 CELEBRATION PARTY**Share this event: NYE 2025 CELEBRATION PARTY*[](https://www.eventbrite.com/e/nye-2025-celebration-party-tickets-1115557254779?aff=ebdssbdestsearch)![NYE 2025  CELEBRATION  PARTY primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F918145583%2F1661149656073%2F1%2Foriginal.20241216-141708?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=c854a3c1e5c8eae356054a82cea8e20d)\r\nSales end soon\r\n\r\n[NYE 2025 CELEBRATION PARTY](https://www.eventbrite.com/e/nye-2025-celebration-party-tickets-1115557254779?aff=ebdssbdestsearch)### NYE 2025 CELEBRATION PARTY\r\n\r\nToday at 10:00 PM\r\n\r\nSebastian Pintxos Bar\r\n\r\n*Share this event: NYE 2025 CELEBRATION PARTY**Save this event: NYE 2025 CELEBRATION PARTY*[](https://www.eventbrite.com/e/2025-vision-board-party-tickets-1061504050189?aff=ebdssbdestsearch)![2025 Vision Board Party primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F890352593%2F36172503392%2F1%2Foriginal.20241102-212601?crop=focalpoint\u0026amp;fit=crop\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.466277541817\u0026amp;fp-y=0.527446761669\u0026amp;s=de16a9dbc4c593a32b79402a157dd325)\r\nSales end soon\r\n\r\n[2025 Vision Board Party](https://www.eventbrite.com/e/2025-vision-board-party-tickets-1061504050189?aff=ebdssbdestsearch)### 2025 Vision Board Party\r\n\r\nFriday at 7:00 PM\r\n\r\nAC Hotel by Marriott Atlanta Buckhead at Phipps Plaza\r\n\r\n*Save this event: 2025 Vision Board Party**Share this event: 2025 Vision Board Party*[](https://www.eventbrite.com/e/2025-vision-board-party-tickets-1061504050189?aff=ebdssbdestsearch)![2025 Vision Board Party primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F890352593%2F36172503392%2F1%2Foriginal.20241102-212601?crop=focalpoint\u0026amp;fit=crop\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.466277541817\u0026amp;fp-y=0.527446761669\u0026amp;s=de16a9dbc4c593a32b79402a157dd325)\r\nSales end soon\r\n\r\n[2025 Vision Board Party](https://www.eventbrite.com/e/2025-vision-board-party-tickets-1061504050189?aff=ebdssbdestsearch)### 2025 Vision Board Party\r\n\r\nFriday at 7:00 PM\r\n\r\nAC Hotel by Marriott Atlanta Buckhead at Phipps Plaza\r\n\r\n*Share this event: 2025 Vision Board Party**Save this event: 2025 Vision Board Party*[](https://www.eventbrite.com/e/from-day-one-atlanta-2025-tickets-891161677567?aff=ebdssbdestsearch)![From Day One: Atlanta 2025 primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F522566969%2F215963339735%2F1%2Foriginal.jpg?w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;rect=0%2C0%2C2160%2C1080\u0026amp;s=c0ab41f4f4bd65001fa547003d6f65dd)\r\n[From Day One: Atlanta 2025](https://www.eventbrite.com/e/from-day-one-atlanta-2025-tickets-891161677567?aff=ebdssbdestsearch)### From Day One: Atlanta 2025\r\n\r\nTue, Jan 28, 8:30 AM\r\n\r\nGeorgia Aquarium\r\n\r\n*Save this event: From Day One: Atlanta 2025**Share this event: From Day One: Atlanta 2025*[](https://www.eventbrite.com/e/from-day-one-atlanta-2025-tickets-891161677567?aff=ebdssbdestsearch)![From Day One: Atlanta 2025 primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F522566969%2F215963339735%2F1%2Foriginal.jpg?w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;rect=0%2C0%2C2160%2C1080\u0026amp;s=c0ab41f4f4bd65001fa547003d6f65dd)\r\n[From Day One: Atlanta 2025](https://www.eventbrite.com/e/from-day-one-atlanta-2025-tickets-891161677567?aff=ebdssbdestsearch)### From Day One: Atlanta 2025\r\n\r\nTue, Jan 28, 8:30 AM\r\n\r\nGeorgia Aquarium\r\n\r\n*Share this event: From Day One: Atlanta 2025**Save this event: From Day One: Atlanta 2025*[](https://www.eventbrite.com/e/2025-evolution-vision-planning-event-tickets-1069450839249?aff=ebdssbdestsearch)![2025 EVOLUTION: Vision Planning Event primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F895988293%2F136652024770%2F1%2Foriginal.20241110-231653?crop=focalpoint\u0026amp;fit=crop\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.504347826087\u0026amp;fp-y=0.45544705725\u0026amp;s=2d9d767cf9dd815a14883cfd34c616e8)\r\nSales end soon\r\n\r\n[2025 EVOLUTION: Vision Planning Event](https://www.eventbrite.com/e/2025-evolution-vision-planning-event-tickets-1069450839249?aff=ebdssbdestsearch)### 2025 EVOLUTION: Vision Planning Event\r\n\r\nSaturday at 2:00 PM\r\n\r\nAtlanta\r\n\r\n*Save this event: 2025 EVOLUTION: Vision Planning Event**Share this event: 2025 EVOLUTION: Vision Planning Event*[](https://www.eventbrite.com/e/2025-evolution-vision-planning-event-tickets-1069450839249?aff=ebdssbdestsearch)![2025 EVOLUTION: Vision Planning Event primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F895988293%2F136652024770%2F1%2Foriginal.20241110-231653?crop=focalpoint\u0026amp;fit=crop\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.504347826087\u0026amp;fp-y=0.45544705725\u0026amp;s=2d9d767cf9dd815a14883cfd34c616e8)\r\nSales end soon\r\n\r\n[2025 EVOLUTION: Vision Planning Event](https://www.eventbrite.com/e/2025-evolution-vision-planning-event-tickets-1069450839249?aff=ebdssbdestsearch)### 2025 EVOLUTION: Vision Planning Event\r\n\r\nSaturday at 2:00 PM\r\n\r\nAtlanta\r\n\r\n*Share this event: 2025 EVOLUTION: Vision Planning Event**Save this event: 2025 EVOLUTION: Vision Planning Event*[](https://www.eventbrite.com/e/elevate-ignite-connections-for-2025-tickets-1079818278539?aff=ebdssbdestsearch)![ELEVATE : IGNITE CONNECTIONS FOR 2025 primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F896741163%2F753946145463%2F1%2Foriginal.20241111-220751?crop=focalpoint\u0026amp;fit=crop\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=4.98106060606e-05\u0026amp;fp-y=5.2052238806e-05\u0026amp;s=e88f4654495c6a263526b5d4c6cff594)\r\n[ELEVATE : IGNITE CONNECTIONS FOR 2025](https://www.eventbrite.com/e/elevate-ignite-connections-for-2025-tickets-1079818278539?aff=ebdssbdestsearch)### ELEVATE : IGNITE CONNECTIONS FOR 2025\r\n\r\nSat, Jan 11, 5:00 PM\r\n\r\nThe Ritz-Carlton, Atlanta\r\n\r\n*Save this event: ELEVATE : IGNITE CONNECTIONS FOR 2025**Share this event: ELEVATE : IGNITE CONNECTIONS FOR 2025*[](https://www.eventbrite.com/e/elevate-ignite-connections-for-2025-tickets-1079818278539?aff=ebdssbdestsearch)![ELEVATE : IGNITE CONNECTIONS FOR 2025 primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F896741163%2F753946145463%2F1%2Foriginal.20241111-220751?crop=focalpoint\u0026amp;fit=crop\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=4.98106060606e-05\u0026amp;fp-y=5.2052238806e-05\u0026amp;s=e88f4654495c6a263526b5d4c6cff594)\r\n[ELEVATE : IGNITE CONNECTIONS FOR 2025](https://www.eventbrite.com/e/elevate-ignite-connections-for-2025-tickets-1079818278539?aff=ebdssbdestsearch)### ELEVATE : IGNITE CONNECTIONS FOR 2025\r\n\r\nSat, Jan 11, 5:00 PM\r\n\r\nThe Ritz-Carlton, Atlanta\r\n\r\n*Share this event: ELEVATE : IGNITE CONNECTIONS FOR 2025**Save this event: ELEVATE : IGNITE CONNECTIONS FOR 2025*[](https://www.eventbrite.com/e/2025-atlanta-travel-adventure-show-tickets-1002463654777?aff=ebdssbdestsearch)![2025 Atlanta Travel \u0026amp; Adventure Show primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F871864629%2F144231071304%2F1%2Foriginal.20241010-200846?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=ed8feeba3eb7d64b22d47813145eedf4)\r\n[2025 Atlanta Travel \u0026amp; Adventure Show](https://www.eventbrite.com/e/2025-atlanta-travel-adventure-show-tickets-1002463654777?aff=ebdssbdestsearch)### 2025 Atlanta Travel \u0026amp; Adventure Show\r\n\r\nGeorgia International Convention Center - HALLS A\u0026amp;B\r\n\r\n*Save this event: 2025 Atlanta Travel \u0026amp; Adventure Show**Share this event: 2025 Atlanta Travel \u0026amp; Adventure Show*[](https://www.eventbrite.com/e/2025-atlanta-travel-adventure-show-tickets-1002463654777?aff=ebdssbdestsearch)![2025 Atlanta Travel \u0026amp; Adventure Show primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F871864629%2F144231071304%2F1%2Foriginal.20241010-200846?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=ed8feeba3eb7d64b22d47813145eedf4)\r\n[2025 Atlanta Travel \u0026amp; Adventure Show](https://www.eventbrite.com/e/2025-atlanta-travel-adventure-show-tickets-1002463654777?aff=ebdssbdestsearch)### 2025 Atlanta Travel \u0026amp; Adventure Show\r\n\r\nGeorgia International Convention Center - HALLS A\u0026amp;B\r\n\r\n*Share this event: 2025 Atlanta Travel \u0026amp; Adventure Show**Save this event: 2025 Atlanta Travel \u0026amp; Adventure Show*[](https://www.eventbrite.com/e/inferno-the-official-2025-mod-party-tickets-1063997317619?aff=ebdssbdestsearch)![INFERNO (The Official 2025 MOD Party) primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F887953793%2F1602627629533%2F1%2Foriginal.20241030-210622?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.530155642023\u0026amp;fp-y=0.501459143969\u0026amp;s=7cb422ddd70c55f7fc86591cd91cea9e)\r\n[INFERNO (The Official 2025 MOD Party)](https://www.eventbrite.com/e/inferno-the-official-2025-mod-party-tickets-1063997317619?aff=ebdssbdestsearch)### INFERNO (The Official 2025 MOD Party)\r\n\r\nFri, Jan 17, 10:00 PM\r\n\r\nHi-Def Events of Avondale\r\n\r\n*Save this event: INFERNO (The Official 2025 MOD Party)**Share this event: INFERNO (The Official 2025 MOD Party)*[](https://www.eventbrite.com/e/inferno-the-official-2025-mod-party-tickets-1063997317619?aff=ebdssbdestsearch)![INFERNO (The Official 2025 MOD Party) primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F887953793%2F1602627629533%2F1%2Foriginal.20241030-210622?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.530155642023\u0026amp;fp-y=0.501459143969\u0026amp;s=7cb422ddd70c55f7fc86591cd91cea9e)\r\n[INFERNO (The Official 2025 MOD Party)](https://www.eventbrite.com/e/inferno-the-official-2025-mod-party-tickets-1063997317619?aff=ebdssbdestsearch)### INFERNO (The Official 2025 MOD Party)\r\n\r\nFri, Jan 17, 10:00 PM\r\n\r\nHi-Def Events of Avondale\r\n\r\n*Share this event: INFERNO (The Official 2025 MOD Party)**Save this event: INFERNO (The Official 2025 MOD Party)*[](https://www.eventbrite.com/e/insights-2025-roadmap-to-opportunity-tickets-1112919254449?aff=ebdssbdestsearch)![Insights 2025: Roadmap to Opportunity primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F916494053%2F383647855923%2F1%2Foriginal.20241212-181825?crop=focalpoint\u0026amp;fit=crop\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=ca8d11d237c5e27a6d7a2c466a113d13)\r\n[Insights 2025: Roadmap to Opportunity](https://www.eventbrite.com/e/insights-2025-roadmap-to-opportunity-tickets-1112919254449?aff=ebdssbdestsearch)### Insights 2025: Roadmap to Opportunity\r\n\r\nFri, Jan 24, 8:30 AM\r\n\r\nThe Georgia Freight Depot\r\n\r\n*Save this event: Insights 2025: Roadmap to Opportunity**Share this event: Insights 2025: Roadmap to Opportunity*[](https://www.eventbrite.com/e/insights-2025-roadmap-to-opportunity-tickets-1112919254449?aff=ebdssbdestsearch)![Insights 2025: Roadmap to Opportunity primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F916494053%2F383647855923%2F1%2Foriginal.20241212-181825?crop=focalpoint\u0026amp;fit=crop\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=ca8d11d237c5e27a6d7a2c466a113d13)\r\n[Insights 2025: Roadmap to Opportunity](https://www.eventbrite.com/e/insights-2025-roadmap-to-opportunity-tickets-1112919254449?aff=ebdssbdestsearch)### Insights 2025: Roadmap to Opportunity\r\n\r\nFri, Jan 24, 8:30 AM\r\n\r\nThe Georgia Freight Depot\r\n\r\n*Share this event: Insights 2025: Roadmap to Opportunity**Save this event: Insights 2025: Roadmap to Opportunity*[](https://www.eventbrite.com/e/atl-nye-bash-everyone-free-before-2025-tickets-1102666367789?aff=ebdssbdestsearch)![ATL NYE BASH [EVERYONE FREE BEFORE 2025] primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F910239013%2F195816047041%2F1%2Foriginal.20241202-224547?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.505094282238\u0026amp;fp-y=0.808365758755\u0026amp;s=c2055a96cec3e5dbbe3eaf2025d09666)\r\nSales end soon\r\n\r\n[ATL NYE BASH [EVERYONE FREE BEFORE 2025]](https://www.eventbrite.com/e/atl-nye-bash-everyone-free-before-2025-tickets-1102666367789?aff=ebdssbdestsearch)### ATL NYE BASH [EVERYONE FREE BEFORE 2025]\r\n\r\nToday at 10:00 PM\r\n\r\nLyfe ATL\r\n\r\n*Save this event: ATL NYE BASH [EVERYONE FREE BEFORE 2025]**Share this event: ATL NYE BASH [EVERYONE FREE BEFORE 2025]*[](https://www.eventbrite.com/e/atl-nye-bash-everyone-free-before-2025-tickets-1102666367789?aff=ebdssbdestsearch)![ATL NYE BASH [EVERYONE FREE BEFORE 2025] primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F910239013%2F195816047041%2F1%2Foriginal.20241202-224547?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.505094282238\u0026amp;fp-y=0.808365758755\u0026amp;s=c2055a96cec3e5dbbe3eaf2025d09666)\r\nSales end soon\r\n\r\n[ATL NYE BASH [EVERYONE FREE BEFORE 2025]](https://www.eventbrite.com/e/atl-nye-bash-everyone-free-before-2025-tickets-1102666367789?aff=ebdssbdestsearch)### ATL NYE BASH [EVERYONE FREE BEFORE 2025]\r\n\r\nToday at 10:00 PM\r\n\r\nLyfe ATL\r\n\r\n*Share this event: ATL NYE BASH [EVERYONE FREE BEFORE 2025]**Save this event: ATL NYE BASH [EVERYONE FREE BEFORE 2025]*[](https://www.eventbrite.com/e/ignite-2025-tickets-932535457657?aff=ebdssbdestsearch)![Ignite 2025 primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F917090773%2F1692870951603%2F1%2Foriginal.20241213-174719?crop=focalpoint\u0026amp;fit=crop\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.005\u0026amp;fp-y=0.005\u0026amp;s=63e15e730fedcc11b29f5004ce8441bd)\r\n[Ignite 2025](https://www.eventbrite.com/e/ignite-2025-tickets-932535457657?aff=ebdssbdestsearch)### Ignite 2025\r\n\r\nFri, Mar 7, 10:00 AM\r\n\r\nSonesta Atlanta Northwest Marietta - The Battery\r\n\r\n*Save this event: Ignite 2025**Share this event: Ignite 2025*[](https://www.eventbrite.com/e/ignite-2025-tickets-932535457657?aff=ebdssbdestsearch)![Ignite 2025 primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F917090773%2F1692870951603%2F1%2Foriginal.20241213-174719?crop=focalpoint\u0026amp;fit=crop\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.005\u0026amp;fp-y=0.005\u0026amp;s=63e15e730fedcc11b29f5004ce8441bd)\r\n[Ignite 2025](https://www.eventbrite.com/e/ignite-2025-tickets-932535457657?aff=ebdssbdestsearch)### Ignite 2025\r\n\r\nFri, Mar 7, 10:00 AM\r\n\r\nSonesta Atlanta Northwest Marietta - The Battery\r\n\r\n*Share this event: Ignite 2025**Save this event: Ignite 2025*[](https://www.eventbrite.com/e/nye-2025-el-valle-with-miramar-pro-tickets-1105140036599?aff=ebdssbdestsearch)![NYE 2025 @ El Valle with Miramar Pro primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F911214613%2F925200966053%2F1%2Foriginal.20241204-033347?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=76a968fcff164477c1cc46743f2aa0ac)\r\nAlmost full\r\n\r\n[NYE 2025 @ El Valle with Miramar Pro](https://www.eventbrite.com/e/nye-2025-el-valle-with-miramar-pro-tickets-1105140036599?aff=ebdssbdestsearch)### NYE 2025 @ El Valle with Miramar Pro\r\n\r\nToday at 10:00 PM\r\n\r\nEl Valle\r\n\r\n*Save this event: NYE 2025 @ El Valle with Miramar Pro**Share this event: NYE 2025 @ El Valle with Miramar Pro*[](https://www.eventbrite.com/e/nye-2025-el-valle-with-miramar-pro-tickets-1105140036599?aff=ebdssbdestsearch)![NYE 2025 @ El Valle with Miramar Pro primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F911214613%2F925200966053%2F1%2Foriginal.20241204-033347?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=76a968fcff164477c1cc46743f2aa0ac)\r\nAlmost full\r\n\r\n[NYE 2025 @ El Valle with Miramar Pro](https://www.eventbrite.com/e/nye-2025-el-valle-with-miramar-pro-tickets-1105140036599?aff=ebdssbdestsearch)### NYE 2025 @ El Valle with Miramar Pro\r\n\r\nToday at 10:00 PM\r\n\r\nEl Valle\r\n\r\n*Share this event: NYE 2025 @ El Valle with Miramar Pro**Save this event: NYE 2025 @ El Valle with Miramar Pro*[](https://www.eventbrite.com/e/blame-it-on-2025-first-day-party-of-the-year-tickets-1115922998729?aff=ebdssbdestsearch)![BLAME IT ON 2025 \u2022 FIRST DAY PARTY OF THE YEAR primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F918498243%2F491384959225%2F1%2Foriginal.20241216-220338?crop=focalpoint\u0026amp;fit=crop\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.503787878788\u0026amp;fp-y=0.558365758755\u0026amp;s=2ba3cf71cf0190f51fd6f805a68389bb)\r\nSales end soon\r\n\r\n[BLAME IT ON 2025 \u2022 FIRST DAY PARTY OF THE YEAR](https://www.eventbrite.com/e/blame-it-on-2025-first-day-party-of-the-year-tickets-1115922998729?aff=ebdssbdestsearch)### BLAME IT ON 2025 \u2022 FIRST DAY PARTY OF THE YEAR\r\n\r\nTomorrow at 2:00 PM\r\n\r\nXPERIENCE SPORTS BAR \u0026amp; Lounge\r\n\r\n*Save this event: BLAME IT ON 2025 \u2022 FIRST DAY PARTY OF THE YEAR**Share this event: BLAME IT ON 2025 \u2022 FIRST DAY PARTY OF THE YEAR*[](https://www.eventbrite.com/e/blame-it-on-2025-first-day-party-of-the-year-tickets-1115922998729?aff=ebdssbdestsearch)![BLAME IT ON 2025 \u2022 FIRST DAY PARTY OF THE YEAR primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F918498243%2F491384959225%2F1%2Foriginal.20241216-220338?crop=focalpoint\u0026amp;fit=crop\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.503787878788\u0026amp;fp-y=0.558365758755\u0026amp;s=2ba3cf71cf0190f51fd6f805a68389bb)\r\nSales end soon\r\n\r\n[BLAME IT ON 2025 \u2022 FIRST DAY PARTY OF THE YEAR](https://www.eventbrite.com/e/blame-it-on-2025-first-day-party-of-the-year-tickets-1115922998729?aff=ebdssbdestsearch)### BLAME IT ON 2025 \u2022 FIRST DAY PARTY OF THE YEAR\r\n\r\nTomorrow at 2:00 PM\r\n\r\nXPERIENCE SPORTS BAR \u0026amp; Lounge\r\n\r\n*Share this event: BLAME IT ON 2025 \u2022 FIRST DAY PARTY OF THE YEAR**Save this event: BLAME IT ON 2025 \u2022 FIRST DAY PARTY OF THE YEAR*[](https://www.eventbrite.com/e/2025-flourish-family-homeschool-conference-tickets-811724889997?aff=ebdssbdestsearch)![2025 Flourish Family Homeschool Conference primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F915908213%2F1189030595673%2F1%2Foriginal.20241211-210800?crop=focalpoint\u0026amp;fit=crop\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=e0314f006953c869d1462600df77b416)\r\n[2025 Flourish Family Homeschool Conference](https://www.eventbrite.com/e/2025-flourish-family-homeschool-conference-tickets-811724889997?aff=ebdssbdestsearch)### 2025 Flourish Family Homeschool Conference\r\n\r\nWed, Jan 29, 5:00 PM\r\n\r\nNew Life Community Center\r\n\r\n*Save this event: 2025 Flourish Family Homeschool Conference**Share this event: 2025 Flourish Family Homeschool Conference*[](https://www.eventbrite.com/e/2025-flourish-family-homeschool-conference-tickets-811724889997?aff=ebdssbdestsearch)![2025 Flourish Family Homeschool Conference primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F915908213%2F1189030595673%2F1%2Foriginal.20241211-210800?crop=focalpoint\u0026amp;fit=crop\u0026amp;w=512\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.5\u0026amp;fp-y=0.5\u0026amp;s=e0314f006953c869d1462600df77b416)\r\n[2025 Flourish Family Homeschool Conference](https://www.eventbrite.com/e/2025-flourish-family-homeschool-conference-tickets-811724889997?aff=ebdssbdestsearch)### 2025 Flourish Family Homeschool Conference\r\n\r\nWed, Jan 29, 5:00 PM\r\n\r\nNew Life Community Center\r\n\r\n*Share this event: 2025 Flourish Family Homeschool Conference**Save this event: 2025 Flourish Family Homeschool Conference*[](https://www.eventbrite.com/e/circa-nye-new-years-eve-celebration-2025-tickets-1110550278779?aff=ebdssbdestsearch)![CIRCA NYE|  NEW YEAR\u0026#x27;S EVE CELEBRATION 2025 primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F915147423%2F213642457492%2F1%2Foriginal.20241210-202342?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.498106060606\u0026amp;fp-y=0.611940298507\u0026amp;s=2b8fded3c8cde370adfb9158aebfe59a)\r\nSales end soon\r\n\r\n[CIRCA NYE| NEW YEAR\u0026#x27;S EVE CELEBRATION 2025](https://www.eventbrite.com/e/circa-nye-new-years-eve-celebration-2025-tickets-1110550278779?aff=ebdssbdestsearch)### CIRCA NYE| NEW YEAR\u0026#x27;S EVE CELEBRATION 2025\r\n\r\nToday at 9:00 PM\r\n\r\nCafe Circa\r\n\r\n*Save this event: CIRCA NYE| NEW YEAR\u0026#x27;S EVE CELEBRATION 2025**Share this event: CIRCA NYE| NEW YEAR\u0026#x27;S EVE CELEBRATION 2025*[](https://www.eventbrite.com/e/circa-nye-new-years-eve-celebration-2025-tickets-1110550278779?aff=ebdssbdestsearch)![CIRCA NYE|  NEW YEAR\u0026#x27;S EVE CELEBRATION 2025 primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F915147423%2F213642457492%2F1%2Foriginal.20241210-202342?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.498106060606\u0026amp;fp-y=0.611940298507\u0026amp;s=2b8fded3c8cde370adfb9158aebfe59a)\r\nSales end soon\r\n\r\n[CIRCA NYE| NEW YEAR\u0026#x27;S EVE CELEBRATION 2025](https://www.eventbrite.com/e/circa-nye-new-years-eve-celebration-2025-tickets-1110550278779?aff=ebdssbdestsearch)### CIRCA NYE| NEW YEAR\u0026#x27;S EVE CELEBRATION 2025\r\n\r\nToday at 9:00 PM\r\n\r\nCafe Circa\r\n\r\n*Share this event: CIRCA NYE| NEW YEAR\u0026#x27;S EVE CELEBRATION 2025**Save this event: CIRCA NYE| NEW YEAR\u0026#x27;S EVE CELEBRATION 2025*[](https://www.eventbrite.com/e/what-a-time-to-be-alive-2025-new-years-eve-party-tickets-1097769240349?aff=ebdssbdestsearch)![WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F911204773%2F153161972037%2F1%2Foriginal.20241204-030931?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.005\u0026amp;fp-y=0.005\u0026amp;s=b82dee28acb7208f4ebc55d133a6261b)\r\nSales end soon\r\n\r\n[WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY](https://www.eventbrite.com/e/what-a-time-to-be-alive-2025-new-years-eve-party-tickets-1097769240349?aff=ebdssbdestsearch)### WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY\r\n\r\nToday at 9:00 PM\r\n\r\nBamboo atlanta\r\n\r\n*Save this event: WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY**Share this event: WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY*[](https://www.eventbrite.com/e/what-a-time-to-be-alive-2025-new-years-eve-party-tickets-1097769240349?aff=ebdssbdestsearch)![WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F911204773%2F153161972037%2F1%2Foriginal.20241204-030931?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.005\u0026amp;fp-y=0.005\u0026amp;s=b82dee28acb7208f4ebc55d133a6261b)\r\nSales end soon\r\n\r\n[WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY](https://www.eventbrite.com/e/what-a-time-to-be-alive-2025-new-years-eve-party-tickets-1097769240349?aff=ebdssbdestsearch)### WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY\r\n\r\nToday at 9:00 PM\r\n\r\nBamboo atlanta\r\n\r\n*Share this event: WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY**Save this event: WHAT A TIME TO BE ALIVE 2025 - NEW YEAR\u2019S EVE PARTY*[](https://www.eventbrite.com/e/ready-set-goals-2025-tickets-1110782011899?aff=ebdssbdestsearch)![Ready. Set. Goals 2025 primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F915315133%2F2540966365701%2F1%2Foriginal.20241211-004215?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.470725924125\u0026amp;fp-y=0.418287937743\u0026amp;s=89d33afbc23ff84d7ab9ad637e3ed59f)\r\n[Ready. Set. Goals 2025](https://www.eventbrite.com/e/ready-set-goals-2025-tickets-1110782011899?aff=ebdssbdestsearch)### Ready. Set. Goals 2025\r\n\r\nSat, Jan 11, 11:00 AM\r\n\r\n2781 Main St\r\n\r\n*Save this event: Ready. Set. Goals 2025**Share this event: Ready. Set. Goals 2025*[](https://www.eventbrite.com/e/ready-set-goals-2025-tickets-1110782011899?aff=ebdssbdestsearch)![Ready. Set. Goals 2025 primary image](https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F915315133%2F2540966365701%2F1%2Foriginal.20241211-004215?crop=focalpoint\u0026amp;fit=crop\u0026amp;auto=format%2Ccompress\u0026amp;q=75\u0026amp;sharp=10\u0026amp;fp-x=0.470725924125\u0026amp;fp-y=0.418287937743\u0026amp;s=89d33afbc23ff84d7ab9ad637e3ed59f)\r\n[Ready. Set. Goals 2025](https://www.eventbrite.com/e/ready-set-goals-2025-tickets-1110782011899?aff=ebdssbdestsearch)### Ready. Set. Goals 2025\r\n\r\nSat, Jan 11, 11:00 AM\r\n\r\n2781 Main St\r\n\r\n*Share this event: Ready. Set. Goals 2025**Save this event: Ready. Set. Goals 2025** \r\n* 1 of 63\r\n* \r\n\r\n**[Things to do around Atlanta](/ttd/ga--atlanta/)## Things to do around Atlanta\r\n\r\n[Things to do in Chattanooga](/ttd/tn--chattanooga/)[](/ttd/tn--chattanooga/)[Things to do in Huntsville](/ttd/al--huntsville/)[](/ttd/al--huntsville/)[Things to do in Birmingham](/ttd/al--birmingham/)[](/ttd/al--birmingham/)[Things to do in Montgomery](/ttd/al--montgomery/)[](/ttd/al--montgomery/)[Things to do in Greenville](/ttd/sc--greenville/)[](/ttd/sc--greenville/)[Things to do in Augusta](/ttd/ga--augusta/)[](/ttd/ga--augusta/)[Things to do in Columbus](/ttd/ga--columbus/)[](/ttd/ga--columbus/)[Things to do in Sandy Springs](/ttd/ga--sandy-springs/)[](/ttd/ga--sandy-springs/)[Things to do in Macon](/ttd/ga--macon/)[](/ttd/ga--macon/)[Things to do in Athens](/ttd/ga--athens/)[](/ttd/ga--athens/)[Things to do in East Point](/ttd/ga--east-point/)[](/ttd/ga--east-point/)[Things to do in Woodstock](/ttd/ga--woodstock/)[](/ttd/ga--woodstock/)## Trends in Atlanta\r\n\r\n[Trends in Atlanta](/trending/searches/ga--atlanta/)**[Explore more trends](/trending/searches/ga--atlanta/)**[1. New years eve parties](/d/ga--atlanta/new-years-eve-parties/)### 1. New years eve parties\r\n\r\n[2. December](/d/ga--atlanta/december/)### 2. December\r\n\r\n[3. Nye events](/d/ga--atlanta/nye-events/)### 3. Nye events\r\n\r\n[4. Free christmas toys for children](/d/ga--atlanta/free-christmas-toys-for-children/)### 4. Free christmas toys for children\r\n\r\n[5. New years eve](/d/ga--atlanta/new-years-eve/)### 5. New years eve\r\n\r\n[6. Toy giveaway](/d/ga--atlanta/toy-giveaway/)### 6. Toy giveaway\r\n\r\n[7. Nye party](/d/ga--atlanta/nye-party/)### 7. Nye party\r\n\r\n[8. Comedy shows](/d/ga--atlanta/comedy-shows/)### 8. Comedy shows\r\n\r\n[9. Job fairs](/d/ga--atlanta/job-fairs/)### 9. Job fairs\r\n\r\n[10. January](/d/ga--atlanta/january/)### 10. January\r\n\r\n[11. New years eve concerts](/d/ga--atlanta/new-years-eve-concerts/)### 11. New years eve concerts\r\n\r\n[12. Christmas lights](/d/ga--atlanta/christmas-lights/)### 12. Christmas lights\r\n\r\n[Explore more trends](/trending/searches/ga--atlanta/)## Site Navigation\r\n\r\nUse Eventbrite\r\n\r\n* Create Events\r\n* Pricing\r\n* Event Marketing Platform\r\n* Eventbrite Mobile Ticket App\r\n* Eventbrite Check-In App\r\n* Eventbrite App Marketplace\r\n* Event Registration Software\r\n* Community Guidelines\r\n* FAQs\r\n* Sitemap\r\n\r\n[Create Events](https://www.eventbrite.com/organizer/overview/)[Pricing](https://www.eventbrite.com/organizer/pricing/)[Event Marketing Platform](https://www.eventbrite.com/l/event-marketing-platform/)[Eventbrite Mobile Ticket App](https://www.eventbrite.com/l/eventbrite-app/)[Eventbrite Check-In App](https://www.eventbrite.com/l/organizer-check-in-app/)[Eventbrite App Marketplace](https://www.eventbrite.com/apps/)[Event Registration Software](https://www.eventbrite.com/l/registration-online/)[Community Guidelines](https://www.eventbrite.com/l/community-guidelines/)[FAQs](https://www.eventbrite.com/l/frequently-asked-questions/)[Sitemap](https://www.eventbrite.com/sitemap/)Plan Events\r\n\r\n* Sell Tickets Online\r\n* Event Planning\r\n* Sell Concert Tickets Online\r\n* Event Payment System\r\n* Solutions for Professional Services\r\n* Event Management Software\r\n* Halloween Party Planning\r\n* Virtual Events Platform\r\n* QR Codes for Event Check-In\r\n* Post your event online\r\n\r\n[Sell Tickets Online](https://www.eventbrite.com/l/sell-tickets/)[Event Planning](https://www.eventbrite.com/l/planning-an-event/)[Sell Concert Tickets Online](https://www.eventbrite.com/l/music-venues/)[Event Payment System](https://www.eventbrite.com/l/event-payment/)[Solutions for Professional Services](https://www.eventbrite.com/l/professional-services/)[Event Management Software](https://www.eventbrite.com/l/event-management-software/)[Halloween Party Planning](https://www.eventbrite.com/l/halloween-event-planning/)[Virtual Events Platform](https://www.eventbrite.com/l/virtual-events-platform/)[QR Codes for Event Check-In](https://www.eventbrite.com/l/how-to-use-qr-codes-for-events/)[Post your event online](https://www.eventbrite.com/l/post-events/)Find Events\r\n\r\n* New Orleans Food \u0026amp; Drink Events\r\n* San Francisco Holiday Events\r\n* Tulum Music Events\r\n* Denver Hobby Events\r\n* Atlanta Pop Music Events\r\n* New York Events\r\n* Chicago Events\r\n* Events in Dallas Today\r\n* Los Angeles Events\r\n* Washington Events\r\n\r\n[New Orleans Food \u0026amp; Drink Events](https://www.eventbrite.com/b/la--new-orleans/food-and-drink/)[San Francisco Holiday Events](https://www.eventbrite.com/b/ca--san-francisco/holiday/)[Tulum Music Events](https://www.eventbrite.com/b/mexico--tulum/music/)[Denver Hobby Events](https://www.eventbrite.com/b/co--denver/hobbies/)[Atlanta Pop Music Events](https://www.eventbrite.com/b/ga--atlanta/music/pop/)[New York Events](https://www.eventbrite.com/d/ny--new-york/events/)[Chicago Events](https://www.eventbrite.com/d/il--chicago/events/)[Events in Dallas Today](https://www.eventbrite.com/d/tx--dallas/events--today/)[Los Angeles Events](https://www.eventbrite.com/d/ca--los-angeles/events/)[Washington Events](https://www.eventbrite.com/d/dc--washington/events/)Connect With Us\r\n\r\n* Contact Support\r\n* Contact Sales\r\n* X\r\n* Facebook\r\n* LinkedIn\r\n* Instagram\r\n* TikTok\r\n\r\n[Contact Support](https://www.eventbrite.com/help/en-us/contact-us/)[Contact Sales](https://www.eventbrite.com/l/contact-eventbrite-sales/)[X](https://www.x.com/eventbritehelp)[Facebook](https://www.facebook.com/Eventbrite)[LinkedIn](https://www.linkedin.com/company/eventbrite)[Instagram](https://www.instagram.com/eventbrite)[TikTok](https://www.tiktok.com/@eventbrite)* How It Works\r\n* Pricing\r\n* Contact Support\r\n* About\r\n* Blog\r\n* Help\r\n* Careers\r\n* Press\r\n* Impact\r\n* Investors\r\n* Security\r\n* Developers\r\n* Status\r\n* Terms\r\n* Privacy\r\n* Accessibility\r\n* Cookies\r\n* Manage Cookie Preferences\r\n\r\n[How It Works](https://www.eventbrite.com/how-it-works)[Pricing](https://www.eventbrite.com/organizer/pricing/)[Contact Support](https://www.eventbrite.com/help/en-us/contact-us/)[About](https://www.eventbrite.com/about/)[Blog](https://www.eventbrite.com/blog/)[Help](https://www.eventbrite.com/help/en-us/)[Careers](https://www.eventbritecareers.com/)[Press](https://www.eventbrite.com/blog/press/)[Impact](https://www.eventbrite.com/l/impact/)[Investors](https://investor.eventbrite.com/overview/default.aspx)[Security](https://www.eventbrite.com/security/)[Developers](https://www.eventbrite.com/platform/)[Status](https://www.eventbritestatus.com/)[Terms](https://www.eventbrite.com/l/legalterms/)[Privacy](https://www.eventbrite.com/help/en-us/articles/460838/eventbrite-privacy-policy/)[Accessibility](https://www.eventbrite.com/l/accessibility/)[Cookies](https://www.eventbrite.com/help/en-us/articles/666792/eventbrite-cookie-policy/)[Manage Cookie Preferences](#)**"
```

Here are some interesting events you can attend in Atlanta over the next four weekends:

### Weekend of January 4-5, 2025
1. **IllumiNights at Zoo Atlanta**: An annual after-hours walk-through holiday lights show featuring a mile of illuminated animal lanterns. [More details](https://zooatlanta.org/event/illuminights/)
2. **Garden Lights, Holiday Nights at the Atlanta Botanical Garden**: A popular annual holiday lights display that runs until January 12. [More details](https://atlantabg.org/calendar/garden-lights-holiday-nights-presented-by-invesco-qqq-2/)
3. **Skate the Station at Atlantic Station**: Annual outdoor holiday skating with some themed nights, running until January 20. [More details](https://atlanticstation.com/event/skate-the-station-2/)
4. **Expressions of Shukran at Gas South Theater**: A cultural festival celebrating art and gratitude. [More details](https://atlanta-ga.events/event/salgiraharts-festival-expressions-of-shukran-duluth-saturday-january-4-2025-1200-pm/?pid=8220)
5. **Sonic Symphony Live at Cobb Energy Performing Arts Centre**: A live symphonic concert featuring music from Sonic the Hedgehog. [More details](https://atlanta-ga.events/event/sonic-symphony-live-atlanta-saturday-january-4-2025-800-pm/?pid=4109)

### Weekend of January 11-12, 2025
1. **Martin Luther King Day Celebrations**: Various events and a parade to honor Martin Luther King Jr. [More details](https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html)
2. **Last Podcast on the Left: JK Ultra Tour at Atlanta Symphony Hall**: A comedy show on January 11. [More details](https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-january.html)
3. **ELEVATE: Ignite Connections for 2025 at The Ritz-Carlton, Atlanta**: A networking event on January 11. [More details](https://www.eventbrite.com/d/ga--atlanta/2025/)
4. **IN-ZO, Mersiv, Tripp St., Daggz, and Supertask at The Eastern**: A live concert on January 11. [More details](https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025)
5. **Comedy Show at Atlanta Symphony Hall**: Featuring various comedians on January 11. [More details](https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html)

### Weekend of January 18-19, 2025
1. **IllumiNights at Zoo Atlanta**: Last chance to see the holiday lights display which ends on January 19. [More details](https://zooatlanta.org/event/illuminights/)
2. **Electric Avenue at Variety Playhouse**: A live concert on January 18. [More details](https://allevents.in/atlanta/january)
3. **Fulton Lee at The Loft at Center Stage**: A live performance on January 18. [More details](https://allevents.in/atlanta/january)
4. **AT&T Playoff Playlist Live with Lil Wayne & More**: A live music event on January 18-19. [More details](https://discoveratlanta.com/events/main/)
5. **MLK Day Parade**: Part of the Martin Luther King Jr. celebrations. [More details](https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html)

### Weekend of January 25-26, 2025
1. **Run With the Dogs in Decatur**: An annual 5K and pet race event to benefit the Decatur Bulldogs Boosters on January 25. [More details](https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html)
2. **Spring Gardening Symposium & Plant Sale at the Atlanta Botanical Garden**: Featuring speakers and a plant sale on January 25. [More details](https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html)
3. **Gals on the Go Podcast Live Show at Center Stage**: A live podcast recording on January 25. [More details](https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html)
4. **DJ DIESEL aka SHAQ at Believe Music Hall**: A live DJ performance on January 25. [More details](https://www.eventbrite.com/d/ga--atlanta/concerts-2025/)
5. **Heroes Among Us: A Musical Tribute**: A tribute concert on January 25. [More details](https://discoveratlanta.com/events/main/)

These events offer a variety of activities, from concerts and cultural festivals to sports events and holiday celebrations, ensuring there's something for everyone to enjoy.


In [7]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Formatting;

output2.LastMessage.DisplayAs("text/markdown");


Here are some interesting events happening in Atlanta for each of the next four weekends:

### Weekend of October 5-6, 2024
1. **Festival on Ponce** - An arts festival in Olmsted Linear Park with over 125 vendors of fine art, food, drinks, and children’s activities.
2. **Candler Park Fall Fest** - A popular annual festival with live music, art, food, rides, games, and more in Candler Park.
3. **Atlanta Horror Film Festival** - A festival of horror films at the Limelight Theater.
4. **Taste of Tucker** - A food festival along Main Street, Tucker, featuring food and drink from local restaurants and breweries, arts and crafts, music, and more.

### Weekend of October 12-13, 2024
1. **Atlanta Pride Festival** - One of the biggest Pride festivals in the country, held in Piedmont Park, with live performances, parade, marches, marketplace, and more.
2. **Old Fourth Ward Fall Arts Festival** - Featuring local and Georgian artists, demonstrations, live entertainment, art activities, and food in Historic Fourth Ward Park.
3. **Chalktoberfest** - An annual art and beer festival in Marietta Square, featuring professional chalk artists, craft beers, live music, arts and crafts, and food trucks.
4. **Joyce Meyer Live** - A faith event at the Gas South Arena.

### Weekend of October 19-20, 2024
1. **Little 5 Points Halloween Festival & Parade** - A popular annual Halloween celebration with live music, parade, haunted house, ghost tours, and more.
2. **PNC Atlanta 10 Miler & 5K** - Annual runs through Buckhead, starting from Atlantic Station.
3. **Georgia Tech vs. Notre Dame** - A thrilling college football game at Mercedes-Benz Stadium.
4. **Stone Mountain Highland Games and Scottish Festival** - A festival celebrating Scottish heritage at Stone Mountain Park with traditional Scottish athletics, music, dancing, food, and drink.

### Weekend of October 26-27, 2024
1. **ONE Musicfest** - A big annual festival of urban progressive music in Piedmont Park.
2. **Boo at the Zoo** - A Halloween event at Zoo Atlanta with trick or treating, Halloween-themed activities, and more.
3. **Sneaker Con Atlanta** - A sneaker convention at Cobb Galleria Centre celebrating its 15th anniversary with a bigger and better show.
4. **Air Dot Show Tour** - An air show at Atlanta Regional Airport, Peachtree City.

For more details and a comprehensive list of events, you can visit:
- [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)
- [Atlanta GA Events](https://atlanta-ga.events/october/)
- [Atlanta on the Cheap](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)
- [Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)

![Festival on Ponce](https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV&w=80&h=80&c=1&pid=5.1)

Enjoy your weekends in Atlanta!

## Clean Up

In [8]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "Web Search and Scrape");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find Web Search and Scrape");
}

Deleted assistant
